## 1. Methylation

In [284]:
import pandas as pd

In [285]:
raw_methy_df = pd.read_csv('./data/raw_data/CCLE_RRBS_TSS1kb_20181022.txt', sep='\t')
display(raw_methy_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\18103939.py:1: DtypeWarning: Columns (2,618) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_methy_df = pd.read_csv('./data/raw_data/CCLE_RRBS_TSS1kb_20181022.txt', sep='\t')


,locus_id,CpG_sites_hg19,avg_coverage,DMS53_LUNG,SW1116_LARGE_INTESTINE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,...,JHUEM7_ENDOMETRIUM,OE21_OESOPHAGUS,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG
0,SGIP1_1_66998638_66999638,1:66998970;1:66998973;1:66998993;1:66999404;1:...,25.0,0.00000,0.00000,0.75000,0.00000,0.95239,NaN,1.00000,...,NA,NA,NA,NA,NA,NA,0.03448,0.77660,0.06665,0.66667
1,SGIP1_1_66998251_66999251,1:66998970;1:66998973;1:66998993,8.27,0.00000,NaN,NaN,0.00000,0.94443,NaN,NaN,...,NA,NA,NA,NA,NA,NA,0.00000,NaN,NaN,0.66667
2,AZIN2_1_33545713_33546713,1:33546151;1:33546209;1:33546210;1:33546385;1:...,326.58,0.00729,0.03232,0.33160,0.22042,0.01291,0.03444,0.14411,...,NA,NA,NA,NA,NA,NA,0.04821,0.06721,0.11539,0.00046
3,AZIN2_1_33546778_33547778,1:33546783;1:33546788;1:33546795;1:33546797;1:...,480.54,0.22276,0.07784,0.26034,0.32106,0.27485,0.15940,0.34742,...,NA,NA,NA,NA,NA,NA,0.15045,0.34739,0.30208,0.05159
4,AGBL4_1_50489626_50490626,1:50489632;1:50489641;1:50489671;1:50489677;1:...,263.36,0.00000,0.01216,0.98113,0.56622,0.33960,0.43177,0.93363,...,NA,NA,NA,NA,NA,NA,0.01874,0.19714,0.05052,0.54553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21333,SYCE3_22_51001381_51002381,22:51001382;22:51001388;22:51001398;22:5100139...,359.68,0.00000,0.00000,0.36057,0.19895,0.10039,0.11989,0.02138,...,NA,NA,NA,NA,NA,NA,0.00000,0.10934,0.01969,0.02364
21334,RABL2B_22_51222110_51223110,22:51222730,6.38,0.00000,0.00000,0.00000,0.09090,0.57140,0.00000,0.00000,...,NA,NA,NA,NA,NA,NA,0.00000,0.00000,0.00000,0.00000
21335,RPL23AP82_22_51194513_51195513,22:51195083;22:51195404;22:51195430,62.10,0.00000,0.00000,0.07277,0.00263,0.26433,0.04521,0.21740,...,NA,NA,NA,NA,NA,NA,0.08333,0.00000,0.25713,0.00000
21336,RPL23AP82_22_51221156_51222156,22:51222010;22:51222012;22:51222018;22:5122202...,179.18,0.00000,0.00000,0.00000,0.00000,0.00000,0.00274,0.00000,...,NA,NA,NA,NA,NA,NA,0.00000,NaN,0.00000,0.00000


In [286]:
# column locus_id should be separate the content after the  '_'
raw_methy_df['locus_id'] = raw_methy_df['locus_id'].apply(lambda x: x.split('_')[0])
# drop na in the column locus_id
raw_methy_df = raw_methy_df.dropna(subset=['CpG_sites_hg19'])
# remove the column CpG_sites_hg19 and avg_coverage
raw_methy_df = raw_methy_df.drop(columns=['CpG_sites_hg19', 'avg_coverage'])
display(raw_methy_df)

,locus_id,DMS53_LUNG,SW1116_LARGE_INTESTINE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,NCIH1184_LUNG,...,JHUEM7_ENDOMETRIUM,OE21_OESOPHAGUS,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG
0,SGIP1,0.00000,0.00000,0.75000,0.00000,0.95239,NaN,1.00000,NaN,0.11864,...,NA,NA,NA,NA,NA,NA,0.03448,0.77660,0.06665,0.66667
1,SGIP1,0.00000,NaN,NaN,0.00000,0.94443,NaN,NaN,NaN,0.22579,...,NA,NA,NA,NA,NA,NA,0.00000,NaN,NaN,0.66667
2,AZIN2,0.00729,0.03232,0.33160,0.22042,0.01291,0.03444,0.14411,0.02525,0.22553,...,NA,NA,NA,NA,NA,NA,0.04821,0.06721,0.11539,0.00046
3,AZIN2,0.22276,0.07784,0.26034,0.32106,0.27485,0.15940,0.34742,0.28963,0.10735,...,NA,NA,NA,NA,NA,NA,0.15045,0.34739,0.30208,0.05159
4,AGBL4,0.00000,0.01216,0.98113,0.56622,0.33960,0.43177,0.93363,0.13089,0.07346,...,NA,NA,NA,NA,NA,NA,0.01874,0.19714,0.05052,0.54553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21332,CHKB,0.00000,0.00951,0.00552,0.00079,0.04100,0.01047,0.00000,0.00663,0.00000,...,NA,NA,NA,NA,NA,NA,0.00000,0.00000,0.00000,0.00000
21333,SYCE3,0.00000,0.00000,0.36057,0.19895,0.10039,0.11989,0.02138,0.04334,0.00000,...,NA,NA,NA,NA,NA,NA,0.00000,0.10934,0.01969,0.02364
21334,RABL2B,0.00000,0.00000,0.00000,0.09090,0.57140,0.00000,0.00000,0.00000,0.00000,...,NA,NA,NA,NA,NA,NA,0.00000,0.00000,0.00000,0.00000
21335,RPL23AP82,0.00000,0.00000,0.07277,0.00263,0.26433,0.04521,0.21740,0.06030,0.00000,...,NA,NA,NA,NA,NA,NA,0.08333,0.00000,0.25713,0.00000


In [287]:
# Identify columns to convert (all except 'locus_id')
cols_to_convert = raw_methy_df.columns.difference(['locus_id'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_methy_df[cols_to_convert] = raw_methy_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_methy_df = raw_methy_df.fillna(0.0)
display(raw_methy_df)

,locus_id,DMS53_LUNG,SW1116_LARGE_INTESTINE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,NCIH1184_LUNG,...,JHUEM7_ENDOMETRIUM,OE21_OESOPHAGUS,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG
0,SGIP1,0.00000,0.00000,0.75000,0.00000,0.95239,0.00000,1.00000,0.00000,0.11864,...,0.0,0.0,0.0,0.0,0.0,0.0,0.03448,0.77660,0.06665,0.66667
1,SGIP1,0.00000,0.00000,0.00000,0.00000,0.94443,0.00000,0.00000,0.00000,0.22579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.66667
2,AZIN2,0.00729,0.03232,0.33160,0.22042,0.01291,0.03444,0.14411,0.02525,0.22553,...,0.0,0.0,0.0,0.0,0.0,0.0,0.04821,0.06721,0.11539,0.00046
3,AZIN2,0.22276,0.07784,0.26034,0.32106,0.27485,0.15940,0.34742,0.28963,0.10735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.15045,0.34739,0.30208,0.05159
4,AGBL4,0.00000,0.01216,0.98113,0.56622,0.33960,0.43177,0.93363,0.13089,0.07346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01874,0.19714,0.05052,0.54553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21332,CHKB,0.00000,0.00951,0.00552,0.00079,0.04100,0.01047,0.00000,0.00663,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
21333,SYCE3,0.00000,0.00000,0.36057,0.19895,0.10039,0.11989,0.02138,0.04334,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.10934,0.01969,0.02364
21334,RABL2B,0.00000,0.00000,0.00000,0.09090,0.57140,0.00000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
21335,RPL23AP82,0.00000,0.00000,0.07277,0.00263,0.26433,0.04521,0.21740,0.06030,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.08333,0.00000,0.25713,0.00000


In [288]:
# Now perform the groupby mean operation
methy_df = raw_methy_df.groupby('locus_id', as_index=False).mean()
display(methy_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\3346799322.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  methy_df = raw_methy_df.groupby('locus_id', as_index=False).mean()


,locus_id,DMS53_LUNG,SW1116_LARGE_INTESTINE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,NCIH1184_LUNG,...,JHUEM7_ENDOMETRIUM,OE21_OESOPHAGUS,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG
0,A1BG,0.51504,0.50000,0.98288,1.00000,0.55295,0.87010,0.90770,0.86540,0.71970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5000,0.89135,0.28806,0.50752
1,A1BG-AS1,0.59476,0.44440,0.96342,1.00000,0.77274,0.88889,0.97580,0.94286,0.88976,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,1.00000,0.17722,0.92126
2,A2M-AS1,0.00442,0.00645,0.95503,0.92654,0.53876,0.07932,0.95646,0.22245,0.13635,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4363,0.00985,0.02073,0.49632
3,A3GALT2,0.00000,0.16665,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,1.00000,0.00000,0.25580
4,A4GALT,0.00000,0.01401,0.00806,0.22223,0.00504,0.00000,0.76363,0.16626,0.03705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.7493,0.00106,0.08999,0.02828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17175,ZYG11A,0.00000,0.52940,0.83335,0.00000,0.01725,0.00000,0.98000,0.04285,0.03330,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0000,0.32245,0.44000,0.00000
17176,ZYG11B,0.02290,0.00000,0.01404,0.00548,0.00000,0.02240,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.00000,0.00000
17177,ZYX,0.00203,0.00000,0.07789,0.00331,0.00101,0.00000,0.00000,0.01156,0.00031,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.00000,0.00000,0.00273
17178,ZZEF1,0.01010,0.00000,0.03783,0.10369,0.04651,0.12195,0.01603,0.02255,0.02312,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.05190,0.00000,0.00552


In [289]:
bmg_promoter_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_promoter_df = bmg_promoter_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_promoter_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\811266666.py:1: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_promoter_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')


,BioMedGraphica_Conn_ID,HGNC_Symbol
0,BMGC_PM00001,A1BG
1,BMGC_PM00002,NAT2
2,BMGC_PM00003,ADA
3,BMGC_PM00004,CDH2
4,BMGC_PM00005,AKT3
...,...,...
86233,BMGC_PM86234,NaN
86234,BMGC_PM86235,NaN
86235,BMGC_PM86236,NaN
86236,BMGC_PM86237,NaN


In [290]:
# merge the biomedgraphica_id with the raw_methy_df
merged_methy_df = pd.merge(bmg_promoter_df, methy_df, left_on='HGNC_Symbol', right_on='locus_id', how='inner')
merged_methy_df.drop(columns=['HGNC_Symbol', 'locus_id'], inplace=True)
display(merged_methy_df)

,BioMedGraphica_Conn_ID,DMS53_LUNG,SW1116_LARGE_INTESTINE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,NCIH1184_LUNG,...,JHUEM7_ENDOMETRIUM,OE21_OESOPHAGUS,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG
0,BMGC_PM00001,0.51504,0.50000,0.982880,1.00000,0.552950,0.87010,0.90770,0.86540,0.71970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.50000,0.89135,0.28806,0.50752
1,BMGC_PM00003,0.01768,0.00000,0.099330,0.00000,0.045510,0.00000,0.00385,0.00499,0.01187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00306,0.00160,0.00000,0.00375
2,BMGC_PM00004,0.00481,0.85522,0.962970,0.68518,0.111570,0.00000,0.03094,0.00935,0.00457,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00704,0.00862,0.91237
3,BMGC_PM00008,0.04435,1.00000,0.512840,0.45453,0.541650,0.96265,0.82994,0.42857,0.38245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.59091,0.38090,0.63461,0.92726
4,BMGC_PM00012,0.00000,0.00000,0.037610,0.00000,0.000000,0.01279,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17431,BMGC_PM69840,0.93452,0.39385,1.000000,0.67802,0.986100,0.96078,0.96775,1.00000,0.98182,...,0.0,0.0,0.0,0.0,0.0,0.0,1.00000,0.90909,1.00000,0.97222
17432,BMGC_PM69841,0.00000,0.00000,0.350365,0.00000,0.001283,0.00000,0.22449,0.00000,0.25000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
17433,BMGC_PM70228,0.25279,0.51015,0.308140,0.26662,0.820340,0.83620,0.85999,0.65949,0.32076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.38639,0.84845,0.62212,0.75415
17434,BMGC_PM70273,0.03293,0.00000,0.017440,0.01087,0.047620,0.05430,0.09606,0.07547,0.05774,...,0.0,0.0,0.0,0.0,0.0,0.0,0.06818,0.06719,0.11995,0.05687


In [291]:
final_merged_methy_df = pd.merge(bmg_promoter_df, methy_df, left_on='HGNC_Symbol', right_on='locus_id', how='left')
# fill the NaN with 0.0
final_merged_methy_df = final_merged_methy_df.fillna(0.0)
final_merged_methy_df.drop(columns=['HGNC_Symbol', 'locus_id'], inplace=True)
display(final_merged_methy_df)

,BioMedGraphica_Conn_ID,DMS53_LUNG,SW1116_LARGE_INTESTINE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,UMUC3_URINARY_TRACT,HOS_BONE,AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,RVH421_SKIN,NCIH1184_LUNG,...,JHUEM7_ENDOMETRIUM,OE21_OESOPHAGUS,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HOP62_LUNG,EKVX_LUNG,UO31_KIDNEY,SF268_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG
0,BMGC_PM00001,0.51504,0.50000,0.98288,1.00000,0.55295,0.8701,0.90770,0.86540,0.71970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.50000,0.89135,0.28806,0.50752
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
2,BMGC_PM00003,0.01768,0.00000,0.09933,0.00000,0.04551,0.0000,0.00385,0.00499,0.01187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00306,0.00160,0.00000,0.00375
3,BMGC_PM00004,0.00481,0.85522,0.96297,0.68518,0.11157,0.0000,0.03094,0.00935,0.00457,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00704,0.00862,0.91237
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
86234,BMGC_PM86235,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
86235,BMGC_PM86236,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
86236,BMGC_PM86237,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000


## 2. Gene

In [292]:
raw_gene_df = pd.read_csv('./data/raw_data/OmicsCNGene.csv')
display(raw_gene_df)

,Unnamed: 0,RHEB (6009),TIPIN (54962),OR4A47 (403253),NUP133 (55746),CPNE8-AS1 (107984504),PDIA4 (9601),RNU6-899P (106479964),NRXN1-DT (730100),SMIM41 (113523638),...,RN7SKP98 (106480863),RPSAP15 (220885),FAM149B1P1 (388181),RNU6-1020P (106481503),MIR1285-1 (100302218),FXYD5 (53827),RNA5SP154 (100873420),TMEM87B (84910),RNU6-1313P (106480139),RNU6-1158P (106480076)
0,ACH-000628,0.733588,1.273030,1.190365,1.409367,0.734211,0.733588,1.083607,1.043100,0.671598,...,1.409367,0.773267,1.273030,1.059823,0.766085,0.962284,1.009244,1.060017,0.698082,0.698082
1,ACH-000974,0.950736,1.000040,1.048747,1.163154,1.041892,0.950736,1.032830,1.016793,1.041892,...,1.163154,1.070037,1.033814,1.019035,0.950736,0.998635,1.089437,1.020576,0.989170,0.989170
2,ACH-003071,1.125149,0.738668,1.170261,0.779756,1.420911,1.177429,1.132756,0.791989,1.120336,...,0.788941,0.763276,0.714417,1.229201,1.194755,1.434221,0.878627,0.764194,1.047771,1.047771
3,ACH-001338,1.491525,0.988869,0.996618,0.999843,0.998373,1.491525,0.996618,1.005134,0.998373,...,1.542706,0.464602,0.988869,0.996042,1.035373,0.924157,0.946639,0.935294,2.055489,1.946219
4,ACH-001608,1.268255,0.799244,1.378248,0.781063,0.879289,1.268255,1.355953,1.140606,0.790556,...,0.813189,0.472556,0.875332,0.816484,1.279431,0.811873,0.872874,1.147173,1.238517,1.394133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,ACH-001389,1.786448,1.942519,1.124677,0.736300,1.119763,1.786448,0.817688,0.772048,0.775908,...,1.096212,0.367679,0.927629,0.811307,0.798644,0.773894,0.827995,0.785493,1.484038,2.095260
1925,ACH-001835,0.713098,1.058476,1.442339,1.337263,1.029053,0.713098,1.442339,1.521287,1.029053,...,1.368689,0.620956,1.058476,0.684878,1.734274,1.639451,0.685640,1.565325,1.343979,1.343979
1926,ACH-000296,1.156063,0.851183,1.128537,1.610137,0.969262,1.190981,1.128537,1.100498,1.146175,...,1.444669,0.333722,0.851183,1.130857,0.854886,1.053836,0.761691,1.089399,0.775674,0.771612
1927,ACH-001971,2.152319,1.180349,1.208479,1.049469,1.127606,2.152319,0.870848,0.984712,1.127606,...,1.114345,0.956893,1.180349,0.614354,2.044447,0.927493,0.890273,0.984712,1.097954,1.097954


In [293]:
# First, set the "Unnamed: 0" column as the index:
raw_gene_t_df = raw_gene_df.set_index("Unnamed: 0")
# Then transpose the DataFrame:
raw_gene_t_df = raw_gene_t_df.transpose()
# move the index to a column and rename it as the gene_name
raw_gene_t_df.reset_index(inplace=True)
raw_gene_t_df.rename(columns={'index': 'gene_name'}, inplace=True)
# column gene_name should be kept with the content by removing "()" and remove and " " content
raw_gene_t_df['gene_name'] = raw_gene_t_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Optionally, if you want to view the result:
display(raw_gene_t_df)

Unnamed: 0,gene_name,ACH-000628,ACH-000974,ACH-003071,ACH-001338,ACH-001608,ACH-000278,ACH-001350,ACH-000150,ACH-001132,...,ACH-000236,ACH-002177,ACH-002163,ACH-002316,ACH-001039,ACH-001389,ACH-001835,ACH-000296,ACH-001971,ACH-002276
0,RHEB,0.733588,0.950736,1.125149,1.491525,1.268255,1.593313,0.768985,1.678589,1.533440,...,1.732311,0.891360,1.335196,0.988235,0.962098,1.786448,0.713098,1.156063,2.152319,0.995697
1,TIPIN,1.273030,1.000040,0.738668,0.988869,0.799244,1.114384,0.909451,1.142067,0.736635,...,0.983290,0.899519,1.022967,0.989033,1.215076,1.942519,1.058476,0.851183,1.180349,0.964208
2,OR4A47,1.190365,1.048747,1.170261,0.996618,1.378248,1.189679,1.371610,0.971120,1.056310,...,1.224336,0.940104,0.741997,0.995188,0.935269,1.124677,1.442339,1.128537,1.208479,0.988156
3,NUP133,1.409367,1.163154,0.779756,0.999843,0.781063,1.080624,0.946615,1.150672,1.972796,...,0.783025,1.234318,1.035702,0.986356,1.254152,0.736300,1.337263,1.610137,1.049469,1.026052
4,CPNE8-AS1,0.734211,1.041892,1.420911,0.998373,0.879289,1.140088,1.009862,0.809097,1.380724,...,1.124372,0.863252,0.979797,0.976575,0.937251,1.119763,1.029053,0.969262,1.127606,1.005615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38585,FXYD5,0.962284,0.998635,1.434221,0.924157,0.811873,1.030370,0.874268,1.010786,0.962975,...,0.768815,1.123477,0.848746,0.977653,0.965869,0.773894,1.639451,1.053836,0.927493,1.002959
38586,RNA5SP154,1.009244,1.089437,0.878627,0.946639,0.872874,1.074350,1.028733,0.648140,1.073053,...,0.706832,1.158749,0.520117,0.957798,0.947472,0.827995,0.685640,0.761691,0.890273,1.003016
38587,TMEM87B,1.060017,1.020576,0.764194,0.935294,1.147173,1.239277,0.702147,1.166930,1.049231,...,1.071300,0.895312,0.971115,0.972381,0.961541,0.785493,1.565325,1.089399,0.984712,1.018937
38588,RNU6-1313P,0.698082,0.989170,1.047771,2.055489,1.238517,1.097696,0.962941,1.273156,1.217933,...,1.036596,0.909479,0.828142,1.008336,1.267007,1.484038,1.343979,0.775674,1.097954,0.989859


In [294]:
# Identify columns to convert (all except 'Unnamed: 0' and 'gene_name')
cols_to_convert = raw_gene_t_df.columns.difference(['Unnamed: 0', 'gene_name'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_gene_t_df[cols_to_convert] = raw_gene_t_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_gene_t_df = raw_gene_t_df.fillna(0.0)
display(raw_gene_t_df)

Unnamed: 0,gene_name,ACH-000628,ACH-000974,ACH-003071,ACH-001338,ACH-001608,ACH-000278,ACH-001350,ACH-000150,ACH-001132,...,ACH-000236,ACH-002177,ACH-002163,ACH-002316,ACH-001039,ACH-001389,ACH-001835,ACH-000296,ACH-001971,ACH-002276
0,RHEB,0.733588,0.950736,1.125149,1.491525,1.268255,1.593313,0.768985,1.678589,1.533440,...,1.732311,0.891360,1.335196,0.988235,0.962098,1.786448,0.713098,1.156063,2.152319,0.995697
1,TIPIN,1.273030,1.000040,0.738668,0.988869,0.799244,1.114384,0.909451,1.142067,0.736635,...,0.983290,0.899519,1.022967,0.989033,1.215076,1.942519,1.058476,0.851183,1.180349,0.964208
2,OR4A47,1.190365,1.048747,1.170261,0.996618,1.378248,1.189679,1.371610,0.971120,1.056310,...,1.224336,0.940104,0.741997,0.995188,0.935269,1.124677,1.442339,1.128537,1.208479,0.988156
3,NUP133,1.409367,1.163154,0.779756,0.999843,0.781063,1.080624,0.946615,1.150672,1.972796,...,0.783025,1.234318,1.035702,0.986356,1.254152,0.736300,1.337263,1.610137,1.049469,1.026052
4,CPNE8-AS1,0.734211,1.041892,1.420911,0.998373,0.879289,1.140088,1.009862,0.809097,1.380724,...,1.124372,0.863252,0.979797,0.976575,0.937251,1.119763,1.029053,0.969262,1.127606,1.005615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38585,FXYD5,0.962284,0.998635,1.434221,0.924157,0.811873,1.030370,0.874268,1.010786,0.962975,...,0.768815,1.123477,0.848746,0.977653,0.965869,0.773894,1.639451,1.053836,0.927493,1.002959
38586,RNA5SP154,1.009244,1.089437,0.878627,0.946639,0.872874,1.074350,1.028733,0.648140,1.073053,...,0.706832,1.158749,0.520117,0.957798,0.947472,0.827995,0.685640,0.761691,0.890273,1.003016
38587,TMEM87B,1.060017,1.020576,0.764194,0.935294,1.147173,1.239277,0.702147,1.166930,1.049231,...,1.071300,0.895312,0.971115,0.972381,0.961541,0.785493,1.565325,1.089399,0.984712,1.018937
38588,RNU6-1313P,0.698082,0.989170,1.047771,2.055489,1.238517,1.097696,0.962941,1.273156,1.217933,...,1.036596,0.909479,0.828142,1.008336,1.267007,1.484038,1.343979,0.775674,1.097954,0.989859


In [295]:
# groupby the gene_name and calculate the mean
gene_df = raw_gene_t_df.groupby('gene_name', as_index=False).mean()
display(gene_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4156208938.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  gene_df = raw_gene_t_df.groupby('gene_name', as_index=False).mean()


Unnamed: 0,gene_name,ACH-000628,ACH-000974,ACH-003071,ACH-001338,ACH-001608,ACH-000278,ACH-001350,ACH-000150,ACH-001132,...,ACH-000236,ACH-002177,ACH-002163,ACH-002316,ACH-001039,ACH-001389,ACH-001835,ACH-000296,ACH-001971,ACH-002276
0,A1BG,1.268945,0.998635,1.449249,0.974584,0.766355,1.149191,0.814400,1.010786,1.384487,...,1.462479,1.137881,0.825004,0.977653,0.993997,0.732935,1.641815,1.170731,0.926029,1.028622
1,A1BG-AS1,1.268945,0.998635,1.449249,0.974584,0.766355,1.149191,0.814400,1.010786,1.384487,...,1.462479,1.137881,0.825004,0.977653,0.993997,0.732935,1.641815,1.170731,0.926029,1.028622
2,A1CF,0.731812,1.030577,0.765131,0.996227,1.399586,1.174703,1.061718,0.923442,1.053757,...,1.041980,0.611262,0.532489,0.984514,0.949537,0.824844,1.977986,1.060355,0.935096,1.004427
3,A2M,0.700969,1.041507,0.779162,0.834302,0.863143,1.154409,1.352536,1.010928,2.104212,...,1.123299,0.863252,1.034677,0.976575,0.937251,1.122751,1.029053,1.122235,1.127606,1.005615
4,A2M-AS1,0.700969,1.041507,0.779162,0.834302,0.863143,1.154409,1.352536,1.010928,2.104212,...,1.123299,0.863252,1.034677,0.976575,0.937251,1.122751,1.029053,1.122235,1.127606,1.005615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38585,ZYG11B,0.999906,1.021629,1.179644,1.007182,1.615836,1.114022,1.250759,0.662718,0.974934,...,0.703032,1.199221,1.027847,0.968364,0.945624,1.534429,1.300574,1.150561,0.865839,0.996423
38586,ZYX,0.733588,0.950736,1.038878,1.451015,1.268255,1.529940,0.764897,1.831970,1.509656,...,1.732311,0.639230,1.335196,0.988235,0.962098,1.786448,0.713098,0.871164,2.152319,0.995697
38587,ZYXP1,1.338678,1.289464,1.258723,1.002211,0.759560,2.222632,2.029633,0.492568,1.045696,...,1.396531,1.167229,0.939060,0.965529,0.916855,0.836631,1.088331,1.269616,1.108575,0.998080
38588,ZZEF1,0.991158,0.981948,1.442267,0.935950,0.813832,0.572681,0.645097,1.035650,0.482337,...,0.786005,1.545586,1.254272,0.997750,1.051698,0.781627,0.662658,0.360974,0.852137,1.008135


In [296]:
bmg_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_gene_df = bmg_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_gene_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\2858376915.py:1: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')


,BioMedGraphica_Conn_ID,HGNC_Symbol
0,BMGC_GN00001,A1BG
1,BMGC_GN00002,NAT2
2,BMGC_GN00003,ADA
3,BMGC_GN00004,CDH2
4,BMGC_GN00005,AKT3
...,...,...
86233,BMGC_GN86234,NaN
86234,BMGC_GN86235,NaN
86235,BMGC_GN86236,NaN
86236,BMGC_GN86237,NaN


In [297]:
# merge the biomedgraphica_id with the raw_gene_t_df
merged_gene_df = pd.merge(bmg_gene_df, gene_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merged_gene_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merged_gene_df)

,BioMedGraphica_Conn_ID,ACH-000628,ACH-000974,ACH-003071,ACH-001338,ACH-001608,ACH-000278,ACH-001350,ACH-000150,ACH-001132,...,ACH-000236,ACH-002177,ACH-002163,ACH-002316,ACH-001039,ACH-001389,ACH-001835,ACH-000296,ACH-001971,ACH-002276
0,BMGC_GN00001,1.268945,0.998635,1.449249,0.974584,0.766355,1.149191,0.814400,1.010786,1.384487,...,1.462479,1.137881,0.825004,0.977653,0.993997,0.732935,1.641815,1.170731,0.926029,1.028622
1,BMGC_GN00002,0.731485,0.799281,1.156498,1.005014,0.748238,0.611852,0.696108,0.473682,1.032457,...,0.349017,0.890035,0.474526,0.965529,0.589237,1.145237,1.088331,0.498268,1.108575,0.998080
2,BMGC_GN00003,1.882274,1.056340,1.052670,1.430607,1.535008,1.188952,1.832935,1.380597,0.773227,...,2.058987,0.898108,0.977277,1.017972,1.457635,0.726367,1.019664,1.282703,1.600703,0.989567
3,BMGC_GN00004,0.946750,1.056789,1.431942,0.978040,0.932105,1.107280,1.360372,0.912048,1.064972,...,0.731571,0.919491,0.774345,0.972422,0.630750,0.394350,1.315620,0.338280,0.860810,1.010605
4,BMGC_GN00005,1.409367,1.163154,0.803250,0.999843,1.134695,1.080624,0.946615,1.150672,2.083959,...,0.783025,1.234318,1.035702,0.986356,1.254152,0.736300,1.594780,1.164666,1.049469,1.026052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42798,BMGC_GN70228,0.945141,1.048747,1.134761,0.996618,0.722325,0.531759,1.256330,0.835083,0.989560,...,1.224336,0.449821,1.013743,0.995188,0.935269,1.069099,0.816622,1.020159,0.912347,0.988156
42799,BMGC_GN70238,4.304596,12.632883,12.244713,1.533437,4.385596,2.726301,8.075323,1.273156,18.004317,...,1.036596,0.909479,2.239729,0.997750,1.258731,0.781627,4.005427,2.065520,1.526971,3.329526
42800,BMGC_GN70239,4.304596,12.632883,12.244713,1.533437,4.385596,2.726301,8.075323,1.273156,18.004317,...,1.036596,0.909479,2.239729,0.997750,1.258731,0.781627,4.005427,2.065520,1.526971,3.329526
42801,BMGC_GN70273,0.717960,1.041892,1.190471,0.998373,1.032063,0.619187,1.240514,0.632949,1.250405,...,1.093958,1.178416,1.052442,0.976575,1.221468,0.734510,1.029053,1.119342,1.127606,1.005615


In [298]:
final_merged_gene_df = pd.merge(bmg_gene_df, gene_df, left_on='HGNC_Symbol', right_on='gene_name', how='left')
# fill the NaN with -1.0
final_merged_gene_df = final_merged_gene_df.fillna(0.0)
final_merged_gene_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(final_merged_gene_df)

,BioMedGraphica_Conn_ID,ACH-000628,ACH-000974,ACH-003071,ACH-001338,ACH-001608,ACH-000278,ACH-001350,ACH-000150,ACH-001132,...,ACH-000236,ACH-002177,ACH-002163,ACH-002316,ACH-001039,ACH-001389,ACH-001835,ACH-000296,ACH-001971,ACH-002276
0,BMGC_GN00001,1.268945,0.998635,1.449249,0.974584,0.766355,1.149191,0.814400,1.010786,1.384487,...,1.462479,1.137881,0.825004,0.977653,0.993997,0.732935,1.641815,1.170731,0.926029,1.028622
1,BMGC_GN00002,0.731485,0.799281,1.156498,1.005014,0.748238,0.611852,0.696108,0.473682,1.032457,...,0.349017,0.890035,0.474526,0.965529,0.589237,1.145237,1.088331,0.498268,1.108575,0.998080
2,BMGC_GN00003,1.882274,1.056340,1.052670,1.430607,1.535008,1.188952,1.832935,1.380597,0.773227,...,2.058987,0.898108,0.977277,1.017972,1.457635,0.726367,1.019664,1.282703,1.600703,0.989567
3,BMGC_GN00004,0.946750,1.056789,1.431942,0.978040,0.932105,1.107280,1.360372,0.912048,1.064972,...,0.731571,0.919491,0.774345,0.972422,0.630750,0.394350,1.315620,0.338280,0.860810,1.010605
4,BMGC_GN00005,1.409367,1.163154,0.803250,0.999843,1.134695,1.080624,0.946615,1.150672,2.083959,...,0.783025,1.234318,1.035702,0.986356,1.254152,0.736300,1.594780,1.164666,1.049469,1.026052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_GN86234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86234,BMGC_GN86235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86235,BMGC_GN86236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86236,BMGC_GN86237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## 3. Transcript

In [299]:
raw_transcript_df = pd.read_csv('./data/raw_data/OmicsExpressionProteinCodingGenesTPMLogp1BatchCorrected.csv')
display(raw_transcript_df)

,Unnamed: 0,TFE3 (7030),MMADHC (27249),PABPC1L (80336),RELL2 (285613),GPT (2875),GUSB (2990),H4-16 (121504),DDB2 (1643),RPS4X (6191),...,OR2A42 (402317),HOXB7 (3217),NSA2 (10412),UGT2B11 (10720),ZNF747 (65988),NAA60 (79903),MYH4 (4622),MTAP (4507),C14orf93 (60686),NR3C2 (4306)
0,ACH-000873,3.607691,3.531071,4.982285,2.326016,0.607679,5.118549,1.282028,4.512297,10.075438,...,0.042564,3.870581,5.178480,-0.007723,2.005498,4.213717,-0.001724,3.904800,2.190931,0.203910
1,ACH-000860,4.192566,6.865682,4.853288,2.725583,0.197016,5.666426,1.154470,4.306808,10.215240,...,0.031621,-0.005897,6.749713,-0.007723,1.393644,4.334535,-0.001724,4.812949,3.366939,0.370726
2,ACH-000439,3.265517,5.804615,1.777636,4.185835,0.389025,6.593342,0.410244,4.411381,11.644890,...,-0.013226,-0.033868,6.877322,1.596383,3.143557,4.624226,-0.001724,4.122929,2.197136,0.015412
3,ACH-000318,3.705500,5.813170,4.536005,2.290409,0.471169,5.160089,2.325673,5.017968,9.163992,...,-0.013226,3.051092,5.610027,-0.007723,2.334419,3.881219,-0.001724,0.394778,3.217293,0.277760
4,ACH-001142,6.105227,7.051513,4.457555,2.543055,0.022363,4.589966,2.369987,4.105131,10.179188,...,0.053402,3.582671,7.416571,-0.007723,2.294077,4.889203,-0.001724,4.379944,2.355443,0.520338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,ACH-001843,5.343844,6.553367,5.674036,3.390159,0.644076,5.555928,1.037326,4.275462,11.191982,...,0.026960,5.222260,7.411799,0.010848,0.563822,4.885868,0.000741,4.898450,2.716015,0.067286
1669,ACH-002074,5.252185,6.258400,2.872415,1.886574,0.796319,6.534374,3.269016,4.587816,9.946776,...,0.104563,4.270369,6.188434,0.010848,3.447185,5.964889,0.000741,4.181277,3.931034,0.695929
1670,ACH-001164,4.050893,6.248805,2.790849,1.599247,0.663686,5.533158,0.646986,5.091849,10.865960,...,0.026960,0.470043,6.898124,1.385374,2.641063,4.824132,0.022977,3.384695,2.413815,0.221968
1671,ACH-003180,5.161680,7.678250,1.932748,1.463134,0.394206,6.464239,0.360318,4.761168,9.531740,...,0.104563,4.741295,6.250017,0.010848,2.652496,5.139537,0.044994,4.773658,2.686730,0.011768


In [300]:
# First, set the "Unnamed: 0" column as the index:
raw_transcript_df = raw_transcript_df.set_index("Unnamed: 0")
# Then transpose the DataFrame:
raw_transcript_df = raw_transcript_df.transpose()
# move the index to a column and rename it as the gene_name
raw_transcript_df.reset_index(inplace=True)
raw_transcript_df.rename(columns={'index': 'gene_name'}, inplace=True)
# column gene_name should be kept with the content by removing "()" and remove and
raw_transcript_df['gene_name'] = raw_transcript_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Optionally, if you want to view the result:
display(raw_transcript_df)

Unnamed: 0,gene_name,ACH-000873,ACH-000860,ACH-000439,ACH-000318,ACH-001142,ACH-000789,ACH-000544,ACH-000531,ACH-000695,...,ACH-002996,ACH-003061,ACH-001384,ACH-001700,ACH-002470,ACH-001843,ACH-002074,ACH-001164,ACH-003180,ACH-001979
0,TFE3,3.607691,4.192566,3.265517,3.705500,6.105227,3.610912,2.446179,5.681865,4.364434,...,5.586557,4.076275,5.469260,4.482530,3.543934,5.343844,5.252185,4.050893,5.161680,4.986577
1,MMADHC,3.531071,6.865682,5.804615,5.813170,7.051513,6.427401,5.885347,6.781293,6.502812,...,7.616900,6.957816,6.276709,6.061848,6.524889,6.553367,6.258400,6.248805,7.678250,7.612689
2,PABPC1L,4.982285,4.853288,1.777636,4.536005,4.457555,4.756976,2.603382,1.889260,3.187749,...,2.992110,5.160160,4.805626,3.787419,3.395479,5.674036,2.872415,2.790849,1.932748,3.058782
3,RELL2,2.326016,2.725583,4.185835,2.290409,2.543055,4.364494,2.392491,2.070912,3.921761,...,2.219140,3.081301,2.934213,2.397418,0.801529,3.390159,1.886574,1.599247,1.463134,3.065330
4,GPT,0.607679,0.197016,0.389025,0.471169,0.022363,0.929382,0.112811,-0.009340,0.633240,...,0.394206,0.814637,0.437848,0.663686,0.372059,0.644076,0.796319,0.663686,0.394206,0.394206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19133,NAA60,4.213717,4.334535,4.624226,3.881219,4.889203,4.739785,3.552251,4.360793,5.393965,...,3.665560,4.485363,4.352813,5.027732,4.295766,4.885868,5.964889,4.824132,5.139537,5.181626
19134,MYH4,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,...,0.000741,0.066796,0.000741,0.000741,0.000741,0.000741,0.000741,0.022977,0.044994,0.000741
19135,MTAP,3.904800,4.812949,4.122929,0.394778,4.379944,4.911818,3.890611,3.643557,4.974483,...,6.052774,5.392289,3.797140,4.540203,4.702090,4.898450,4.181277,3.384695,4.773658,4.881492
19136,C14orf93,2.190931,3.366939,2.197136,3.217293,2.355443,3.855860,2.318863,1.962670,4.300053,...,2.645150,3.394596,2.325457,3.070859,2.465353,2.716015,3.931034,2.413815,2.686730,2.779229


In [301]:
# Identify columns to convert (all except 'Unnamed: 0' and 'gene_name')
cols_to_convert = raw_transcript_df.columns.difference(['Unnamed: 0', 'gene_name'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_transcript_df[cols_to_convert] = raw_transcript_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_transcript_df = raw_transcript_df.fillna(0.0)
display(raw_transcript_df)

Unnamed: 0,gene_name,ACH-000873,ACH-000860,ACH-000439,ACH-000318,ACH-001142,ACH-000789,ACH-000544,ACH-000531,ACH-000695,...,ACH-002996,ACH-003061,ACH-001384,ACH-001700,ACH-002470,ACH-001843,ACH-002074,ACH-001164,ACH-003180,ACH-001979
0,TFE3,3.607691,4.192566,3.265517,3.705500,6.105227,3.610912,2.446179,5.681865,4.364434,...,5.586557,4.076275,5.469260,4.482530,3.543934,5.343844,5.252185,4.050893,5.161680,4.986577
1,MMADHC,3.531071,6.865682,5.804615,5.813170,7.051513,6.427401,5.885347,6.781293,6.502812,...,7.616900,6.957816,6.276709,6.061848,6.524889,6.553367,6.258400,6.248805,7.678250,7.612689
2,PABPC1L,4.982285,4.853288,1.777636,4.536005,4.457555,4.756976,2.603382,1.889260,3.187749,...,2.992110,5.160160,4.805626,3.787419,3.395479,5.674036,2.872415,2.790849,1.932748,3.058782
3,RELL2,2.326016,2.725583,4.185835,2.290409,2.543055,4.364494,2.392491,2.070912,3.921761,...,2.219140,3.081301,2.934213,2.397418,0.801529,3.390159,1.886574,1.599247,1.463134,3.065330
4,GPT,0.607679,0.197016,0.389025,0.471169,0.022363,0.929382,0.112811,-0.009340,0.633240,...,0.394206,0.814637,0.437848,0.663686,0.372059,0.644076,0.796319,0.663686,0.394206,0.394206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19133,NAA60,4.213717,4.334535,4.624226,3.881219,4.889203,4.739785,3.552251,4.360793,5.393965,...,3.665560,4.485363,4.352813,5.027732,4.295766,4.885868,5.964889,4.824132,5.139537,5.181626
19134,MYH4,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,...,0.000741,0.066796,0.000741,0.000741,0.000741,0.000741,0.000741,0.022977,0.044994,0.000741
19135,MTAP,3.904800,4.812949,4.122929,0.394778,4.379944,4.911818,3.890611,3.643557,4.974483,...,6.052774,5.392289,3.797140,4.540203,4.702090,4.898450,4.181277,3.384695,4.773658,4.881492
19136,C14orf93,2.190931,3.366939,2.197136,3.217293,2.355443,3.855860,2.318863,1.962670,4.300053,...,2.645150,3.394596,2.325457,3.070859,2.465353,2.716015,3.931034,2.413815,2.686730,2.779229


In [302]:
bmg_transcript_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_transcript_df = bmg_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_transcript_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\594098286.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_transcript_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')


,BioMedGraphica_Conn_ID,HGNC_Symbol
0,BMGC_TS000001,ARF5
1,BMGC_TS000002,M6PR
2,BMGC_TS000003,ESRRA
3,BMGC_TS000004,FKBP4
4,BMGC_TS000005,CYP26B1
...,...,...
412034,BMGC_TS412035,LINC02968
412035,BMGC_TS412036,LINC03112
412036,BMGC_TS412037,LINC03112
412037,BMGC_TS412038,LINC03112


In [303]:
# merge the biomedgraphica_id with the raw_transcript_df
merge_transcript_df = pd.merge(bmg_transcript_df, raw_transcript_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merge_transcript_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merge_transcript_df)

,BioMedGraphica_Conn_ID,ACH-000873,ACH-000860,ACH-000439,ACH-000318,ACH-001142,ACH-000789,ACH-000544,ACH-000531,ACH-000695,...,ACH-002996,ACH-003061,ACH-001384,ACH-001700,ACH-002470,ACH-001843,ACH-002074,ACH-001164,ACH-003180,ACH-001979
0,BMGC_TS000001,6.010975,6.650175,5.885916,7.142466,6.720866,8.317689,5.401430,6.562183,7.781027,...,6.265994,7.902718,7.112875,8.403703,6.377124,7.161867,8.339458,7.267401,6.349134,7.053324
1,BMGC_TS000002,7.038597,5.851489,7.032545,6.123710,6.438191,6.674373,5.617127,5.864259,4.408654,...,6.463395,7.781410,6.244496,5.552777,6.305295,6.782487,6.672697,5.816018,5.844664,6.555430
2,BMGC_TS000003,5.463087,4.154380,4.578244,5.522401,3.750699,4.963768,3.523931,3.163117,4.831130,...,3.906757,4.659487,4.521520,4.750653,3.971007,5.337452,3.530706,4.306837,3.879305,3.721688
3,BMGC_TS000004,6.376783,5.864786,6.265494,6.496076,4.836182,8.226768,6.183044,4.384002,6.179894,...,6.368665,7.368284,6.055639,5.348946,6.446293,5.623277,7.118444,5.467079,5.451382,5.297473
4,BMGC_TS000005,2.813257,2.289688,-0.046550,0.735355,0.043941,1.202660,0.641833,0.757766,0.423583,...,1.344332,3.388847,1.874635,0.602110,0.235919,0.221142,0.714818,0.578576,1.966306,0.235919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184492,BMGC_TS411885,0.754822,1.026857,0.998199,1.338890,-0.027065,-0.041679,-0.012595,-0.027065,0.001734,...,2.572651,0.288850,0.055933,0.151145,0.055933,0.083800,0.324180,0.802649,0.055933,1.945652
184493,BMGC_TS411886,0.754822,1.026857,0.998199,1.338890,-0.027065,-0.041679,-0.012595,-0.027065,0.001734,...,2.572651,0.288850,0.055933,0.151145,0.055933,0.083800,0.324180,0.802649,0.055933,1.945652
184494,BMGC_TS411887,0.754822,1.026857,0.998199,1.338890,-0.027065,-0.041679,-0.012595,-0.027065,0.001734,...,2.572651,0.288850,0.055933,0.151145,0.055933,0.083800,0.324180,0.802649,0.055933,1.945652
184495,BMGC_TS412018,3.665326,3.764924,4.628350,3.606514,4.904823,6.962571,2.996201,3.097047,5.627501,...,4.407802,3.795327,3.903598,3.948776,4.658968,5.434939,4.204795,5.267714,3.429109,3.956163


In [304]:
# merge the biomedgraphica_id with the raw_transcript_df
final_merged_transcript_df = pd.merge(bmg_transcript_df, raw_transcript_df, left_on='HGNC_Symbol', right_on='gene_name', how='left')
# fill the NaN with -1.0
final_merged_transcript_df = final_merged_transcript_df.fillna(0.0)
final_merged_transcript_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(final_merged_transcript_df)

,BioMedGraphica_Conn_ID,ACH-000873,ACH-000860,ACH-000439,ACH-000318,ACH-001142,ACH-000789,ACH-000544,ACH-000531,ACH-000695,...,ACH-002996,ACH-003061,ACH-001384,ACH-001700,ACH-002470,ACH-001843,ACH-002074,ACH-001164,ACH-003180,ACH-001979
0,BMGC_TS000001,6.010975,6.650175,5.885916,7.142466,6.720866,8.317689,5.401430,6.562183,7.781027,...,6.265994,7.902718,7.112875,8.403703,6.377124,7.161867,8.339458,7.267401,6.349134,7.053324
1,BMGC_TS000002,7.038597,5.851489,7.032545,6.123710,6.438191,6.674373,5.617127,5.864259,4.408654,...,6.463395,7.781410,6.244496,5.552777,6.305295,6.782487,6.672697,5.816018,5.844664,6.555430
2,BMGC_TS000003,5.463087,4.154380,4.578244,5.522401,3.750699,4.963768,3.523931,3.163117,4.831130,...,3.906757,4.659487,4.521520,4.750653,3.971007,5.337452,3.530706,4.306837,3.879305,3.721688
3,BMGC_TS000004,6.376783,5.864786,6.265494,6.496076,4.836182,8.226768,6.183044,4.384002,6.179894,...,6.368665,7.368284,6.055639,5.348946,6.446293,5.623277,7.118444,5.467079,5.451382,5.297473
4,BMGC_TS000005,2.813257,2.289688,-0.046550,0.735355,0.043941,1.202660,0.641833,0.757766,0.423583,...,1.344332,3.388847,1.874635,0.602110,0.235919,0.221142,0.714818,0.578576,1.966306,0.235919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412034,BMGC_TS412035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
412035,BMGC_TS412036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
412036,BMGC_TS412037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
412037,BMGC_TS412038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## 4. Protein

In [305]:
raw_protein_df = pd.read_csv('./data/raw_data/protein_quant_current_normalized.csv')
raw_protein_df = raw_protein_df.drop(columns=['Protein_Id', 'Gene_Symbol', 'Description', 'Group_ID', 'Uniprot'])
# Also drop columns names contain Peptides
raw_protein_df = raw_protein_df[raw_protein_df.columns.drop(list(raw_protein_df.filter(regex='Peptides')))]
display(raw_protein_df)

,Uniprot_Acc,MDAMB468_BREAST_TenPx01,SH4_SKIN_TenPx01,AU565_BREAST_TenPx01,KMRC1_KIDNEY_TenPx01,CAL51_BREAST_TenPx01,RPMI7951_SKIN_TenPx01,RERFLCMS_LUNG_TenPx01,IGR37_SKIN_TenPx01,VMRCRCW_KIDNEY_TenPx01,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,P55011,2.111348,0.070468,-0.463928,-0.883645,0.788565,-0.912352,0.729167,-0.658213,-1.150653,...,-1.345024,1.811205,-0.921176,-1.045727,0.845574,-0.071063,0.702306,-1.395878,-1.165032,-0.547536
1,P35453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O60341,0.379683,-0.283538,0.191211,-0.595606,0.391243,-0.381329,0.410168,0.066076,-0.324100,...,0.161196,-0.337994,-0.455601,0.392393,0.258380,-0.817245,-0.790010,0.929522,-0.828736,0.735431
3,O00570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.040198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P48431,-0.246367,-0.572753,-0.341305,-0.335211,-0.124474,0.043893,1.882953,-0.616307,-0.533335,...,-4.627606,-1.261703,1.136851,-3.179027,1.424570,2.549687,4.424095,-1.083698,-1.640990,-2.905547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12750,Q9Y258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12751,P20292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.690046,-1.600231,-1.110583,-1.047633,0.023850,-1.393552,4.556818,-0.531699,0.640295
12752,Q9H1C7,-0.313512,0.105086,-0.586815,-0.132940,0.452423,-0.546389,0.298349,0.461098,0.309062,...,-0.393738,0.604400,1.643414,-1.481004,0.408420,-0.177391,0.445977,-2.497558,0.964551,-2.004067
12753,Q99735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
# Identify columns to convert (all except 'Unnamed: 0' and 'protein_name')
cols_to_convert = raw_protein_df.columns.difference(['Uniprot_Acc'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_protein_df[cols_to_convert] = raw_protein_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_protein_df = raw_protein_df.fillna(0.0)
display(raw_protein_df)

,Uniprot_Acc,MDAMB468_BREAST_TenPx01,SH4_SKIN_TenPx01,AU565_BREAST_TenPx01,KMRC1_KIDNEY_TenPx01,CAL51_BREAST_TenPx01,RPMI7951_SKIN_TenPx01,RERFLCMS_LUNG_TenPx01,IGR37_SKIN_TenPx01,VMRCRCW_KIDNEY_TenPx01,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,P55011,2.111348,0.070468,-0.463928,-0.883645,0.788565,-0.912352,0.729167,-0.658213,-1.150653,...,-1.345024,1.811205,-0.921176,-1.045727,0.845574,-0.071063,0.702306,-1.395878,-1.165032,-0.547536
1,P35453,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,O60341,0.379683,-0.283538,0.191211,-0.595606,0.391243,-0.381329,0.410168,0.066076,-0.324100,...,0.161196,-0.337994,-0.455601,0.392393,0.258380,-0.817245,-0.790010,0.929522,-0.828736,0.735431
3,O00570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.040198,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,P48431,-0.246367,-0.572753,-0.341305,-0.335211,-0.124474,0.043893,1.882953,-0.616307,-0.533335,...,-4.627606,-1.261703,1.136851,-3.179027,1.424570,2.549687,4.424095,-1.083698,-1.640990,-2.905547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12750,Q9Y258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12751,P20292,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.690046,-1.600231,-1.110583,-1.047633,0.023850,-1.393552,4.556818,-0.531699,0.640295
12752,Q9H1C7,-0.313512,0.105086,-0.586815,-0.132940,0.452423,-0.546389,0.298349,0.461098,0.309062,...,-0.393738,0.604400,1.643414,-1.481004,0.408420,-0.177391,0.445977,-2.497558,0.964551,-2.004067
12753,Q99735,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [307]:
# groupby the protein_name and calculate the mean
protein_df = raw_protein_df.groupby('Uniprot_Acc', as_index=False).mean()
display(protein_df)


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\3695680667.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  protein_df = raw_protein_df.groupby('Uniprot_Acc', as_index=False).mean()


,Uniprot_Acc,MDAMB468_BREAST_TenPx01,SH4_SKIN_TenPx01,AU565_BREAST_TenPx01,KMRC1_KIDNEY_TenPx01,CAL51_BREAST_TenPx01,RPMI7951_SKIN_TenPx01,RERFLCMS_LUNG_TenPx01,IGR37_SKIN_TenPx01,VMRCRCW_KIDNEY_TenPx01,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,A0AUZ9-2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.035406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,A0AV02-4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,A0AV96,0.358567,-1.112410,0.855575,0.061377,0.284258,-0.485070,0.092390,-0.824035,0.926286,...,-1.347445,2.090889,-1.348085,2.449369,-1.362958,0.348655,-1.442707,0.436273,-1.585523,0.077705
3,A0AVF1,-0.172066,0.339446,-0.181171,-0.341233,-0.059558,0.175596,-0.103205,0.359589,-0.042908,...,0.000000,-0.650165,0.909498,0.761474,-0.181335,-1.032205,-0.684201,-0.770205,0.289067,-1.031203
4,A0AVI2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.458639,0.578876,-0.487621,-2.524540,-0.532270,2.834559,-0.101096,0.621485,3.827484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12750,U3KPW6,-0.421581,0.845537,0.446649,0.085607,0.079110,-1.501110,0.125189,1.085682,-0.523255,...,0.637635,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12751,U3KQ28,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.054038,0.568362,-0.300467,-0.144897,-0.023671,0.088995,0.287703,0.124844,0.354875,0.052102
12752,U3KQ54,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.062826,-0.367516,-0.323326,0.422455,0.617692,0.697067,-0.435225,-0.362489,-0.360115
12753,U3KQB3,-1.372372,-0.648092,0.154645,0.593048,0.268130,0.025445,0.584467,0.035815,0.101186,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [308]:
# merge the biomedgraphica_id with the raw_protein_df
bmg_protein_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
bmg_protein_df = bmg_protein_df[['BioMedGraphica_Conn_ID', 'Uniprot_ID']]
display(bmg_protein_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\1778311738.py:2: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_protein_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')


,BioMedGraphica_Conn_ID,Uniprot_ID
0,BMGC_PT000001,A0A023T787
1,BMGC_PT000002,A0A024QYX0
2,BMGC_PT000003,A0A024QYX5
3,BMGC_PT000004,A0A024QZ37
4,BMGC_PT000005,A0A024QZ45
...,...,...
121414,BMGC_PT121415,NaN
121415,BMGC_PT121416,NaN
121416,BMGC_PT121417,NaN
121417,BMGC_PT121418,NaN


In [309]:
# merge the biomedgraphica_id with the raw_protein_df
merged_protein_df = pd.merge(bmg_protein_df, protein_df, left_on='Uniprot_ID', right_on='Uniprot_Acc', how='inner')
merged_protein_df.drop(columns=['Uniprot_ID', 'Uniprot_Acc'], inplace=True)
display(merged_protein_df)

,BioMedGraphica_Conn_ID,MDAMB468_BREAST_TenPx01,SH4_SKIN_TenPx01,AU565_BREAST_TenPx01,KMRC1_KIDNEY_TenPx01,CAL51_BREAST_TenPx01,RPMI7951_SKIN_TenPx01,RERFLCMS_LUNG_TenPx01,IGR37_SKIN_TenPx01,VMRCRCW_KIDNEY_TenPx01,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,BMGC_PT001337,0.358567,-1.112410,0.855575,0.061377,0.284258,-0.485070,0.092390,-0.824035,0.926286,...,-1.347445,2.090889,-1.348085,2.449369,-1.362958,0.348655,-1.442707,0.436273,-1.585523,0.077705
1,BMGC_PT001347,-0.172066,0.339446,-0.181171,-0.341233,-0.059558,0.175596,-0.103205,0.359589,-0.042908,...,0.000000,-0.650165,0.909498,0.761474,-0.181335,-1.032205,-0.684201,-0.770205,0.289067,-1.031203
2,BMGC_PT001355,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.458639,0.578876,-0.487621,-2.524540,-0.532270,2.834559,-0.101096,0.621485,3.827484
3,BMGC_PT001356,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,BMGC_PT001359,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10611,BMGC_PT048268,0.531404,-0.228307,0.270226,0.041147,0.044493,-0.320486,-0.084089,-0.439713,0.108860,...,-0.114846,0.438270,-0.314925,-0.064443,-0.077137,-0.614710,-0.511348,-0.163492,-0.016835,0.113500
10612,BMGC_PT048269,0.140324,0.629608,-0.387707,-0.218811,0.205440,-0.163033,-0.363271,0.510087,-0.220798,...,-0.187114,-0.394658,0.907714,-0.270577,0.003400,-1.732360,-0.685070,1.224937,0.328577,-0.276705
10613,BMGC_PT048274,0.007468,-0.316567,0.084812,0.161356,0.072855,0.166777,-0.161962,-0.391137,0.215005,...,0.088471,-0.371191,0.785289,-0.158731,0.087618,-0.428187,-0.291988,0.452430,-0.251963,-0.546281
10614,BMGC_PT048275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.019504,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [310]:
# merge the biomedgraphica_id with the raw_protein_df
final_merged_protein_df = pd.merge(bmg_protein_df, protein_df, left_on='Uniprot_ID', right_on='Uniprot_Acc', how='left')
# fill the NaN with 0.0
final_merged_protein_df = final_merged_protein_df.fillna(0.0)
final_merged_protein_df.drop(columns=['Uniprot_ID', 'Uniprot_Acc'], inplace=True)
display(final_merged_protein_df)

,BioMedGraphica_Conn_ID,MDAMB468_BREAST_TenPx01,SH4_SKIN_TenPx01,AU565_BREAST_TenPx01,KMRC1_KIDNEY_TenPx01,CAL51_BREAST_TenPx01,RPMI7951_SKIN_TenPx01,RERFLCMS_LUNG_TenPx01,IGR37_SKIN_TenPx01,VMRCRCW_KIDNEY_TenPx01,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 5. Drug

In [311]:
raw_drug_df = pd.read_csv('./data/raw_data/sanger-dose-response.csv')
display(raw_drug_df)

,DATASET,COSMIC_ID,DRUG_ID,MIN_CONC,MAX_CONC,RMSE_PUBLISHED,Z_SCORE_PUBLISHED,IC50_PUBLISHED,AUC_PUBLISHED,ARXSPAN_ID,DRUG_NAME,BROAD_ID,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
0,GDSC1,683665,1,0.007813,2.0,0.022518,-0.192056,10.977393,0.982116,ACH-002270,ERLOTINIB,BRD-K70401845,0.992788,2.839376e+00,-5.670993,0.514389,0.990834,NaN,0.000034,0.904675
1,GDSC1,684055,1,0.007813,2.0,0.031831,0.505823,23.133991,0.984820,ACH-002104,ERLOTINIB,BRD-K70401845,1.006405,2.864875e-02,-0.186377,0.990054,0.997138,NaN,0.000057,0.028903
2,GDSC1,684057,1,0.007813,2.0,0.026047,1.280750,52.935278,0.985696,ACH-002106,ERLOTINIB,BRD-K70401845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GDSC1,684059,1,0.007813,2.0,0.110056,0.086028,14.774223,0.972701,ACH-002108,ERLOTINIB,BRD-K70401845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GDSC1,684062,1,0.007813,2.0,0.087010,-0.114395,11.926884,0.944463,ACH-002111,ERLOTINIB,BRD-K70401845,0.989580,7.580375e-02,-12.222777,0.894027,0.933185,NaN,0.000623,0.777093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387621,GDSC2,905951,2172,0.009766,10.0,0.143863,0.042524,25.410793,0.877741,ACH-000288,JQ1,"BRD-K54606188, NA",3.929045,3.287745e+11,-0.012190,-3.449953,0.856099,NaN,0.006178,0.292447
387622,GDSC2,906862,2172,0.009766,10.0,0.088631,-2.223819,0.339325,0.510590,ACH-001065,JQ1,"BRD-K54606188, NA",0.935866,4.096968e-01,-2.014115,0.176515,0.585800,-1.073816,0.003198,0.971991
387623,GDSC2,907046,2172,0.009766,10.0,0.114748,-0.578942,7.780877,0.843211,ACH-000930,JQ1,"BRD-K54606188, NA",0.960799,3.695946e-01,-1.161533,0.687792,0.830671,NaN,0.002792,0.795935
387624,GDSC2,749709,2172,0.009766,10.0,0.047625,1.642265,534.688321,0.983634,ACH-000859,JQ1,"BRD-K54606188, NA",0.977260,1.161541e+30,-0.031889,0.900234,0.969870,NaN,0.000879,0.000540


In [312]:
# keep columns ['ARXSPAN_ID', 'DRUG_NAME', 'IC50_PUBLISHED']
raw_drug_df = raw_drug_df[['ARXSPAN_ID', 'DRUG_NAME', 'IC50_PUBLISHED', 'AUC_PUBLISHED']]
# check if there is nan in all of the dataframe
print(raw_drug_df.isnull().sum())
# drop the nan in this dataframe
raw_drug_df = raw_drug_df.dropna().reset_index(drop=True)
# check if there is nan in all of the dataframe again
print(raw_drug_df.isnull().sum())
display(raw_drug_df)

ARXSPAN_ID        8621
DRUG_NAME           74
IC50_PUBLISHED       0
AUC_PUBLISHED        0
dtype: int64
ARXSPAN_ID        0
DRUG_NAME         0
IC50_PUBLISHED    0
AUC_PUBLISHED     0
dtype: int64


,ARXSPAN_ID,DRUG_NAME,IC50_PUBLISHED,AUC_PUBLISHED
0,ACH-002270,ERLOTINIB,10.977393,0.982116
1,ACH-002104,ERLOTINIB,23.133991,0.984820
2,ACH-002106,ERLOTINIB,52.935278,0.985696
3,ACH-002108,ERLOTINIB,14.774223,0.972701
4,ACH-002111,ERLOTINIB,11.926884,0.944463
...,...,...,...,...
379000,ACH-000288,JQ1,25.410793,0.877741
379001,ACH-001065,JQ1,0.339325,0.510590
379002,ACH-000930,JQ1,7.780877,0.843211
379003,ACH-000859,JQ1,534.688321,0.983634


In [313]:
# fetch the drug_name as a independent dataframe
drug_name_df = raw_drug_df[['DRUG_NAME']]
# remove the duplicate in the drug_name_df
drug_name_df = drug_name_df.drop_duplicates().reset_index(drop=True)
display(drug_name_df)

,DRUG_NAME
0,ERLOTINIB
1,SIROLIMUS
2,SUNITINIB
3,PHA-665752
4,MG-132
...,...
392,LJI308
393,AZ6102
394,"GSK591, EPZ015866, GSK3203591"
395,VE-821


In [314]:
bmg_drug_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Drug/BioMedGraphica_Conn_Drug.csv')
display(bmg_drug_df)
bmg_drug_name_df = bmg_drug_df[['BioMedGraphica_Conn_ID', 'PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym']]
# keep BioMedGraphica_Conn_ID and Drug_Name
display(bmg_drug_name_df)

,BioMedGraphica_Conn_ID,BioMedGraphica_ID,PubChem_CID,PubChem_SID,PubChem_Name,CAS_RN,IUPAC_Name,UNII,UNII_Name,NDC,DrugBank_ID,DrugBank_Name,PubChem_Canonical_SMILES,UNII_SMILES,InChI,InChIKEY,PubChem_Synonym
0,BMGC_DG00001,BMG_DG000018,10001648.0,NaN,"12,13-Dihydroxyfumitremorgin C",111427-99-7,"(1R,2S,12S,15S)-1,2-dihydroxy-7-methoxy-12-(2-...",NaN,NaN,NaN,NaN,NaN,CC(=C[C@H]1C2=C([C@@H]([C@@]3(N1C(=O)[C@@H]4CC...,NaN,InChI=1S/C22H25N3O5/c1-11(2)9-16-18-17(13-7-6-...,CPHRCQUGNAGVIB-FWJAZRMLSA-N,"12,13-dihydroxyfumitremorgin C|111427-99-7|12a..."
1,BMGC_DG00002,BMG_DG000025,100028.0,474493534,NaN,3280-71-5,NaN,TN5YFF4JQQ,2-AMINO-4-SEC-BUTYLPHENOL,NaN,NaN,NaN,NaN,C(CC)(C)c1cc(N)c(O)cc1,NaN,XKJRYOAGNVYYIU-UHFFFAOYSA-N,NaN
2,BMGC_DG00003,BMG_DG000029,10002952.0,198966102,NaN,179044-94-1,NaN,U90K6D923K,AMG-3,NaN,NaN,NaN,NaN,c12c([C@]3([C@](C(O1)(C)C)(CC=C(C3)C)[H])[H])c...,NaN,JECXXFXYJAQVAH-WOJBJXKFSA-N,NaN
3,BMGC_DG00004,BMG_DG000037,10004545.0,198937626,NaN,693823-79-9,NaN,HX4T7JEV3D,AMG-076 FREE BASE,NaN,NaN,NaN,NaN,C1(CCCCC1)(C(=O)O)CCN2C[C@@]3([C@H](c4c5cc(ccc...,NaN,WAOZNBXSEVIAGX-MBOZVWFJSA-N,NaN
4,BMGC_DG00005,BMG_DG000045,10006.0,198983861,NaN,461-72-3,NaN,I6208298TA,HYDANTOIN,NaN,NaN,NaN,NaN,C1(NC(=O)NC1)=O,NaN,WJRBRSLFGCUECM-UHFFFAOYSA-N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20913,BMGC_DG20914,BMG_DG269064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19372,Zoxazolamine,NaN,NaN,NaN,NaN,NaN
20914,BMGC_DG20915,BMG_DG269070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19378,Megestrol,NaN,NaN,NaN,NaN,NaN
20915,BMGC_DG20916,BMG_DG269073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19381,AZACYCLONOL,NaN,NaN,NaN,NaN,NaN
20916,BMGC_DG20917,BMG_DG269082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DB19397,AZD8630,NaN,NaN,NaN,NaN,NaN


,BioMedGraphica_Conn_ID,PubChem_Name,IUPAC_Name,UNII_Name,DrugBank_Name,PubChem_Synonym
0,BMGC_DG00001,"12,13-Dihydroxyfumitremorgin C","(1R,2S,12S,15S)-1,2-dihydroxy-7-methoxy-12-(2-...",NaN,NaN,"12,13-dihydroxyfumitremorgin C|111427-99-7|12a..."
1,BMGC_DG00002,NaN,NaN,2-AMINO-4-SEC-BUTYLPHENOL,NaN,NaN
2,BMGC_DG00003,NaN,NaN,AMG-3,NaN,NaN
3,BMGC_DG00004,NaN,NaN,AMG-076 FREE BASE,NaN,NaN
4,BMGC_DG00005,NaN,NaN,HYDANTOIN,NaN,NaN
...,...,...,...,...,...,...
20913,BMGC_DG20914,NaN,NaN,NaN,Zoxazolamine,NaN
20914,BMGC_DG20915,NaN,NaN,NaN,Megestrol,NaN
20915,BMGC_DG20916,NaN,NaN,NaN,AZACYCLONOL,NaN
20916,BMGC_DG20917,NaN,NaN,NaN,AZD8630,NaN


In [315]:
# Combine 'PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym' into one column called 'Drug_Name'
bmg_drug_name_df['Drug_Name'] = bmg_drug_name_df[['PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym']].apply(lambda x: ';'.join(x.dropna().astype(str)), axis=1)
bmg_drug_name_df = bmg_drug_name_df[['BioMedGraphica_Conn_ID', 'Drug_Name']]
display(bmg_drug_name_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\185313899.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bmg_drug_name_df['Drug_Name'] = bmg_drug_name_df[['PubChem_Name', 'IUPAC_Name', 'UNII_Name', 'DrugBank_Name', 'PubChem_Synonym']].apply(lambda x: ';'.join(x.dropna().astype(str)), axis=1)


,BioMedGraphica_Conn_ID,Drug_Name
0,BMGC_DG00001,"12,13-Dihydroxyfumitremorgin C;(1R,2S,12S,15S)..."
1,BMGC_DG00002,2-AMINO-4-SEC-BUTYLPHENOL
2,BMGC_DG00003,AMG-3
3,BMGC_DG00004,AMG-076 FREE BASE
4,BMGC_DG00005,HYDANTOIN
...,...,...
20913,BMGC_DG20914,Zoxazolamine
20914,BMGC_DG20915,Megestrol
20915,BMGC_DG20916,AZACYCLONOL
20916,BMGC_DG20917,AZD8630


In [316]:
# since in the column drug name, there are lots of names split by ";", if this space contain the name in the drug_name_df, then we should match this 2 rows and merge the drug_name_df with the bmg_drug_df
# Fix the drug name processing
# First, check if there are any NaN values in Drug_Name column
print(f"Number of NaN values in Drug_Name column: {bmg_drug_name_df['Drug_Name'].isna().sum()}")

# Fill NaN values with empty string to avoid errors
bmg_drug_name_df['Drug_Name'] = bmg_drug_name_df['Drug_Name'].fillna('')

# Convert to string to ensure split() works on all entries
bmg_drug_name_df['drug_name_list'] = bmg_drug_name_df['Drug_Name'].astype(str).apply(lambda x: [name.strip() for name in x.split(';')] if x else [])

# Similarly for drug_name_df
drug_name_df['drug_name_list'] = drug_name_df['DRUG_NAME'].astype(str).apply(lambda x: [name.strip() for name in x.split(';')] if x else [])

Number of NaN values in Drug_Name column: 0


In [317]:
display(bmg_drug_name_df)
display(drug_name_df)

,BioMedGraphica_Conn_ID,Drug_Name,drug_name_list
0,BMGC_DG00001,"12,13-Dihydroxyfumitremorgin C;(1R,2S,12S,15S)...","[12,13-Dihydroxyfumitremorgin C, (1R,2S,12S,15..."
1,BMGC_DG00002,2-AMINO-4-SEC-BUTYLPHENOL,[2-AMINO-4-SEC-BUTYLPHENOL]
2,BMGC_DG00003,AMG-3,[AMG-3]
3,BMGC_DG00004,AMG-076 FREE BASE,[AMG-076 FREE BASE]
4,BMGC_DG00005,HYDANTOIN,[HYDANTOIN]
...,...,...,...
20913,BMGC_DG20914,Zoxazolamine,[Zoxazolamine]
20914,BMGC_DG20915,Megestrol,[Megestrol]
20915,BMGC_DG20916,AZACYCLONOL,[AZACYCLONOL]
20916,BMGC_DG20917,AZD8630,[AZD8630]


,DRUG_NAME,drug_name_list
0,ERLOTINIB,[ERLOTINIB]
1,SIROLIMUS,[SIROLIMUS]
2,SUNITINIB,[SUNITINIB]
3,PHA-665752,[PHA-665752]
4,MG-132,[MG-132]
...,...,...
392,LJI308,[LJI308]
393,AZ6102,[AZ6102]
394,"GSK591, EPZ015866, GSK3203591","[GSK591, EPZ015866, GSK3203591]"
395,VE-821,[VE-821]


In [318]:
# Create a flattened mapping of drug names to their BMG IDs
drug_name_to_bmg = {}
for idx, row in bmg_drug_name_df.iterrows():
    bmg_id = row['BioMedGraphica_Conn_ID']
    for drug_name in row['drug_name_list']:
        if drug_name:  # Avoid empty strings
            drug_name_to_bmg[drug_name.upper()] = bmg_id

# Create a new mapping from ARXSPAN_ID to BMG_ID based on drug name matches
arxspan_to_bmg = {}
for idx, row in drug_name_df.iterrows():
    arxspan_id = row['DRUG_NAME']
    for drug_name in row['drug_name_list']:
        if drug_name.upper() in drug_name_to_bmg:
            arxspan_to_bmg[arxspan_id] = drug_name_to_bmg[drug_name.upper()]
            break

# Create a mapping dataframe
mapping_df = pd.DataFrame(list(arxspan_to_bmg.items()), columns=['DRUG_NAME', 'BioMedGraphica_Conn_ID'])

# Display how many drug names were successfully matched
print(f"Successfully matched {len(mapping_df)} out of {len(drug_name_df)} drugs")

# Display the first few rows of the mapping
print("\nSample of drug name mappings:")
display(mapping_df)

# Now you can use this mapping to merge with your drug_response_df
# Example:
# merged_df = pd.merge(drug_response_df, mapping_df, on='DRUG_NAME', how='left')


Successfully matched 172 out of 397 drugs

Sample of drug name mappings:


,DRUG_NAME,BioMedGraphica_Conn_ID
0,ERLOTINIB,BMGC_DG04654
1,SIROLIMUS,BMGC_DG11078
2,SUNITINIB,BMGC_DG11707
3,PHA-665752,BMGC_DG00627
4,MG-132,BMGC_DG09927
...,...,...
167,OSIMERTINIB,BMGC_DG15052
168,CEDIRANIB,BMGC_DG17637
169,GDC-0810,BMGC_DG12527
170,SAVOLITINIB,BMGC_DG14332


In [319]:
final_merged_drug_df = mapping_df.copy()
# sort values by the column BioMedGraphica_ID
final_merged_drug_df = final_merged_drug_df.sort_values(by='BioMedGraphica_Conn_ID').reset_index(drop=True)
display(final_merged_drug_df)

,DRUG_NAME,BioMedGraphica_Conn_ID
0,AR-12,BMGC_DG00030
1,PAZOPANIB,BMGC_DG00141
2,SELUMETINIB,BMGC_DG00151
3,AFATINIB,BMGC_DG00254
4,SARACATINIB,BMGC_DG00420
...,...,...
167,ABT-737,BMGC_DG20690
168,DACINOSTAT,BMGC_DG20696
169,PI-103,BMGC_DG20703
170,SNX-2112,BMGC_DG20709


In [320]:
# filter the final drug score dataframe by filtering the 'DRUG_NAME' in the final_merged_drug_df DRUG_NAME
final_drug_df = raw_drug_df[raw_drug_df['DRUG_NAME'].isin(final_merged_drug_df['DRUG_NAME'])].reset_index(drop=True)
display(final_drug_df)

,ARXSPAN_ID,DRUG_NAME,IC50_PUBLISHED,AUC_PUBLISHED
0,ACH-002270,ERLOTINIB,10.977393,0.982116
1,ACH-002104,ERLOTINIB,23.133991,0.984820
2,ACH-002106,ERLOTINIB,52.935278,0.985696
3,ACH-002108,ERLOTINIB,14.774223,0.972701
4,ACH-002111,ERLOTINIB,11.926884,0.944463
...,...,...,...,...
175318,ACH-000288,JQ1,25.410793,0.877741
175319,ACH-001065,JQ1,0.339325,0.510590
175320,ACH-000930,JQ1,7.780877,0.843211
175321,ACH-000859,JQ1,534.688321,0.983634


## 6. CRISPR

In [321]:
# load entity type ['Promoter', 'Gene', 'Transcript', 'Protein'] for bmgc_entity_df
bmgc_entity_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/BioMedGraphica_Conn_Entity.csv')
# check if there is any null values in the bmgc_entity_df
print(bmgc_entity_df.isnull().sum())
# filter out the rows with Type in the list ['promoter', 'gene', 'transcript', 'protein']
bmgc_omics_df = bmgc_entity_df[bmgc_entity_df['Type'].isin(['Promoter', 'Gene', 'Transcript', 'Protein'])].reset_index(drop=True)
# check if there is any null values in the bmgc_omics_df
print(bmgc_omics_df.isnull().sum())
display(bmgc_omics_df)

BioMedGraphica_Conn_ID    0
BioMedGraphica_ID         0
Type                      0
dtype: int64
BioMedGraphica_Conn_ID    0
BioMedGraphica_ID         0
Type                      0
dtype: int64


,BioMedGraphica_Conn_ID,BioMedGraphica_ID,Type
0,BMGC_PM00001,BMG_PM000001,Promoter
1,BMGC_PM00002,BMG_PM000002,Promoter
2,BMGC_PM00003,BMG_PM000003,Promoter
3,BMGC_PM00004,BMG_PM000004,Promoter
4,BMGC_PM00005,BMG_PM000005,Promoter
...,...,...,...
705929,BMGC_PT121415,BMG_PT173974,Protein
705930,BMGC_PT121416,BMG_PT173975,Protein
705931,BMGC_PT121417,BMG_PT173976,Protein
705932,BMGC_PT121418,BMG_PT173977,Protein


In [322]:
# load the relation
bmgc_relation_df = pd.read_csv('./data/BioMedGraphica-Conn/Relation/BioMedGraphica_Conn_Relation.csv')
# check if there is any null values in the bmgc_relation_df
print(bmgc_relation_df.isnull().sum())

# filter our the rows with Relation type in the list ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein', 'Protein-Protein']
bmgc_omics_relation_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein', 'Protein-Protein'])].reset_index(drop=True)
# check if there is any null values in the bmgc_omics_relation_df
print(bmgc_omics_relation_df.isnull().sum())
display(bmgc_omics_relation_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\1745844678.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  bmgc_relation_df = pd.read_csv('./data/BioMedGraphica-Conn/Relation/BioMedGraphica_Conn_Relation.csv')


BioMedGraphica_Conn_ID        0
BioMedGraphica_ID             0
BMGC_From_ID                  0
BMGC_To_ID                    0
BMG_From_ID                   0
BMG_To_ID                     0
Source                    86238
Type                          0
dtype: int64
BioMedGraphica_Conn_ID        0
BioMedGraphica_ID             0
BMGC_From_ID                  0
BMGC_To_ID                    0
BMG_From_ID                   0
BMG_To_ID                     0
Source                    86238
Type                          0
dtype: int64


,BioMedGraphica_Conn_ID,BioMedGraphica_ID,BMGC_From_ID,BMGC_To_ID,BMG_From_ID,BMG_To_ID,Source,Type
0,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMGC_GN00001,BMG_PM000001,BMG_GN000001,NaN,Promoter-Gene
1,BMGC_ED_PMGN00002,BMG_ED_PMGN00002,BMGC_PM00002,BMGC_GN00002,BMG_PM000002,BMG_GN000002,NaN,Promoter-Gene
2,BMGC_ED_PMGN00003,BMG_ED_PMGN00003,BMGC_PM00003,BMGC_GN00003,BMG_PM000003,BMG_GN000003,NaN,Promoter-Gene
3,BMGC_ED_PMGN00004,BMG_ED_PMGN00004,BMGC_PM00004,BMGC_GN00004,BMG_PM000004,BMG_GN000004,NaN,Promoter-Gene
4,BMGC_ED_PMGN00005,BMG_ED_PMGN00005,BMGC_PM00005,BMGC_GN00005,BMG_PM000005,BMG_GN000005,NaN,Promoter-Gene
...,...,...,...,...,...,...,...,...
17151448,BMGC_ED_PTPT16484816,BMG_ED_PTPT16484816,BMGC_PT121419,BMGC_PT117260,BMG_PT173978,BMG_PT169819,KEGG,Protein-Protein
17151449,BMGC_ED_PTPT16484817,BMG_ED_PTPT16484817,BMGC_PT121419,BMGC_PT120142,BMG_PT173978,BMG_PT172701,KEGG,Protein-Protein
17151450,BMGC_ED_PTPT16484818,BMG_ED_PTPT16484818,BMGC_PT121419,BMGC_PT120825,BMG_PT173978,BMG_PT173384,KEGG,Protein-Protein
17151451,BMGC_ED_PTPT16484819,BMG_ED_PTPT16484819,BMGC_PT121419,BMGC_PT120826,BMG_PT173978,BMG_PT173385,KEGG,Protein-Protein


In [323]:
# translation chain converging to the same node
# fetch the promoter, gene, transcript and protein entity alone
promoter_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Promoter'].copy()
gene_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Gene'].copy()
transcript_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Transcript'].copy()
protein_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Protein'].copy()

display(bmgc_omics_relation_df)
# recheck the null values in bmgc_omics_relation_df
print("Null values in bmgc_omics_relation_df:")
print(bmgc_omics_relation_df.isnull().sum())

# fetch the Promoter-Gene, Gene-Transcript, Transcript-Protein relation alone
promoter_gene_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Promoter-Gene'].copy()
gene_transcript_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Gene-Transcript'].copy()
transcript_protein_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Transcript-Protein'].copy()

,BioMedGraphica_Conn_ID,BioMedGraphica_ID,BMGC_From_ID,BMGC_To_ID,BMG_From_ID,BMG_To_ID,Source,Type
0,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMGC_GN00001,BMG_PM000001,BMG_GN000001,NaN,Promoter-Gene
1,BMGC_ED_PMGN00002,BMG_ED_PMGN00002,BMGC_PM00002,BMGC_GN00002,BMG_PM000002,BMG_GN000002,NaN,Promoter-Gene
2,BMGC_ED_PMGN00003,BMG_ED_PMGN00003,BMGC_PM00003,BMGC_GN00003,BMG_PM000003,BMG_GN000003,NaN,Promoter-Gene
3,BMGC_ED_PMGN00004,BMG_ED_PMGN00004,BMGC_PM00004,BMGC_GN00004,BMG_PM000004,BMG_GN000004,NaN,Promoter-Gene
4,BMGC_ED_PMGN00005,BMG_ED_PMGN00005,BMGC_PM00005,BMGC_GN00005,BMG_PM000005,BMG_GN000005,NaN,Promoter-Gene
...,...,...,...,...,...,...,...,...
17151448,BMGC_ED_PTPT16484816,BMG_ED_PTPT16484816,BMGC_PT121419,BMGC_PT117260,BMG_PT173978,BMG_PT169819,KEGG,Protein-Protein
17151449,BMGC_ED_PTPT16484817,BMG_ED_PTPT16484817,BMGC_PT121419,BMGC_PT120142,BMG_PT173978,BMG_PT172701,KEGG,Protein-Protein
17151450,BMGC_ED_PTPT16484818,BMG_ED_PTPT16484818,BMGC_PT121419,BMGC_PT120825,BMG_PT173978,BMG_PT173384,KEGG,Protein-Protein
17151451,BMGC_ED_PTPT16484819,BMG_ED_PTPT16484819,BMGC_PT121419,BMGC_PT120826,BMG_PT173978,BMG_PT173385,KEGG,Protein-Protein


Null values in bmgc_omics_relation_df:
BioMedGraphica_Conn_ID        0
BioMedGraphica_ID             0
BMGC_From_ID                  0
BMGC_To_ID                    0
BMG_From_ID                   0
BMG_To_ID                     0
Source                    86238
Type                          0
dtype: int64


In [324]:
gene_transcript_entity_df = pd.merge(gene_entity_df, gene_transcript_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BioMedGraphica_Conn_ID', right_on='BMGC_From_ID', how='outer')
gene_transcript_protein_entity_df = pd.merge(gene_transcript_entity_df, transcript_protein_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BMGC_To_ID', right_on='BMGC_From_ID', how='outer')
# drop NaN values in BMGC_From_ID_x	BMGC_To_ID_x BMGC_From_ID_y	BMGC_To_ID_y
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.dropna(subset=['BMGC_From_ID_x', 'BMGC_To_ID_x', 'BMGC_From_ID_y', 'BMGC_To_ID_y']).reset_index(drop=True)
# keep the columns ['BioMedGraphica_Conn_ID', 'BMGC_To_ID_x', 'BMGC_To_ID_y'] and rename the columns to ['BMGC_GN_ID', 'BMGC_TS_ID', 'BMGC_PT_ID']
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df[['BioMedGraphica_Conn_ID', 'BMGC_To_ID_y']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID', 'BMGC_To_ID_y': 'BMGC_PT_ID'}).sort_values(by='BMGC_GN_ID').reset_index(drop=True)
# drop duplicates rows in gene_transcript_protein_entity_df
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.drop_duplicates().reset_index(drop=True)
display(gene_transcript_protein_entity_df)

,BMGC_GN_ID,BMGC_PT_ID
0,BMGC_GN00001,BMGC_PT013542
1,BMGC_GN00001,BMGC_PT091702
2,BMGC_GN00002,BMGC_PT070222
3,BMGC_GN00002,BMGC_PT041299
4,BMGC_GN00002,BMGC_PT038889
...,...,...
126009,BMGC_GN70323,BMGC_PT121177
126010,BMGC_GN70327,BMGC_PT121280
126011,BMGC_GN70329,BMGC_PT121282
126012,BMGC_GN76774,BMGC_PT007612


In [325]:
# just copy gene_transcript_protein_entity_df as promoter_gene_transcript_protein_entity_df
bmgc_promoter_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
bmgc_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
promoter_gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.copy()
promoter_gene_df = pd.concat([bmgc_promoter_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PM_ID'}), bmgc_gene_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID'})], axis=1)
promoter_protein_entity_df = pd.merge(promoter_gene_transcript_protein_entity_df, promoter_gene_df, left_on='BMGC_GN_ID', right_on='BMGC_GN_ID', how='left').drop(columns=['BMGC_GN_ID'])
promoter_protein_entity_df = promoter_protein_entity_df[['BMGC_PM_ID', 'BMGC_PT_ID']]
display(promoter_protein_entity_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\390326110.py:2: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmgc_promoter_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\390326110.py:3: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmgc_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')


,BMGC_PM_ID,BMGC_PT_ID
0,BMGC_PM00001,BMGC_PT013542
1,BMGC_PM00001,BMGC_PT091702
2,BMGC_PM00002,BMGC_PT070222
3,BMGC_PM00002,BMGC_PT041299
4,BMGC_PM00002,BMGC_PT038889
...,...,...
126009,BMGC_PM70323,BMGC_PT121177
126010,BMGC_PM70327,BMGC_PT121280
126011,BMGC_PM70329,BMGC_PT121282
126012,BMGC_PM76774,BMGC_PT007612


In [326]:
transcript_protein_entity_df = pd.merge(transcript_entity_df, transcript_protein_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BioMedGraphica_Conn_ID', right_on='BMGC_From_ID', how='outer')
# drop NaN rows in the BMGC_From_ID and	BMGC_To_ID
transcript_protein_entity_df = transcript_protein_entity_df.dropna(subset=['BMGC_From_ID', 'BMGC_To_ID']).reset_index(drop=True)
# keep the columns ['BioMedGraphica_Conn_ID', 'BMGC_To_ID'] and rename the columns to ['BMGC_TS_ID', 'BMGC_PT_ID']
transcript_protein_entity_df = transcript_protein_entity_df[['BioMedGraphica_Conn_ID', 'BMGC_To_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_TS_ID', 'BMGC_To_ID': 'BMGC_PT_ID'}).sort_values(by='BMGC_TS_ID').reset_index(drop=True)
# drop duplicates rows in transcript_protein_entity_df
transcript_protein_entity_df = transcript_protein_entity_df.drop_duplicates().reset_index(drop=True)
display(transcript_protein_entity_df)

,BMGC_TS_ID,BMGC_PT_ID
0,BMGC_TS000001,BMGC_PT017571
1,BMGC_TS000002,BMGC_PT014978
2,BMGC_TS000003,BMGC_PT014403
3,BMGC_TS000004,BMGC_PT017807
4,BMGC_TS000005,BMGC_PT043262
...,...,...
152580,BMGC_TS411885,BMGC_PT121415
152581,BMGC_TS411886,BMGC_PT121416
152582,BMGC_TS411887,BMGC_PT121417
152583,BMGC_TS412018,BMGC_PT121418


In [327]:
# keep the columns ['BioMedGraphica_Conn_ID'] and rename the columns to ['BMGC_PT_ID']
only_protein_entity_df = protein_entity_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PT_ID'}).sort_values(by='BMGC_PT_ID').reset_index(drop=True)
display(only_protein_entity_df)

,BMGC_PT_ID
0,BMGC_PT000001
1,BMGC_PT000002
2,BMGC_PT000003
3,BMGC_PT000004
4,BMGC_PT000005
...,...
121414,BMGC_PT121415
121415,BMGC_PT121416
121416,BMGC_PT121417
121417,BMGC_PT121418


### 6.1 CRISPR top 100 gene entities

In [328]:
import pandas as pd
raw_crispr_df = pd.read_csv('./data/raw_data/CRISPRGeneEffect.csv')
display(raw_crispr_df)

,Unnamed: 0,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.121964,0.042647,0.026102,-0.147614,-0.020425,0.184014,-0.076020,-0.097068,0.182759,...,-0.120536,0.107286,0.166390,0.262275,0.163876,-0.249508,0.053725,0.067986,0.136285,-0.438536
1,ACH-000004,-0.016484,-0.088500,-0.088198,-0.041225,-0.028074,-0.157206,0.280101,-0.106904,0.178125,...,-0.192527,-0.324059,0.230377,0.087609,0.074897,0.054335,-0.330343,0.099067,0.274566,0.001871
2,ACH-000005,-0.184847,0.003300,0.160881,0.086224,-0.149315,-0.253837,0.167011,-0.101209,-0.129827,...,-0.312827,-0.338023,-0.039700,-0.055349,-0.000367,-0.205605,-0.066032,-0.054518,0.035579,-0.150486
3,ACH-000007,-0.071921,-0.113717,0.082872,0.099633,-0.008378,-0.022310,0.014416,-0.184977,-0.173739,...,-0.334843,-0.355499,-0.014183,0.230944,0.044628,-0.081863,-0.390748,-0.036547,-0.273129,-0.382723
4,ACH-000009,-0.019163,-0.134669,0.060323,0.076647,0.078922,-0.100243,0.047559,-0.136988,0.037759,...,-0.299593,-0.194427,-0.027365,0.236591,-0.084224,-0.098271,-0.510495,0.052938,0.018623,-0.258353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,ACH-002926,-0.107883,-0.101501,0.046677,0.096868,-0.118650,-0.065258,-0.064944,-0.175912,-0.044707,...,-0.409541,-0.579376,0.038706,0.338610,-0.174379,-0.040264,-0.239763,0.065168,-0.144088,-0.334424
1174,ACH-002928,-0.287606,-0.179101,-0.135824,-0.046579,-0.249888,-0.097376,0.070385,0.049444,-0.244937,...,-0.095328,-0.440452,0.303500,0.097690,0.151790,0.039994,0.098240,-0.266654,-0.115108,-0.080421
1175,ACH-003012,-0.145878,-0.004000,0.091933,0.191332,-0.065385,-0.134152,0.012353,-0.321012,0.001571,...,-0.280119,-0.662673,-0.180936,0.169997,-0.036326,0.158702,-0.213281,-0.017884,-0.195395,-0.309021
1176,ACH-003177,-0.038435,-0.055660,0.018989,0.117182,0.063397,0.049993,0.175520,-0.085752,0.078435,...,-0.080700,-0.476797,-0.158375,-0.008999,-0.030058,-0.103513,-0.119234,-0.153744,0.032584,-0.536071


In [329]:
# get the column names of the raw_crispr_df aside from the first column and convert this to a list
raw_crispr_df_columns = raw_crispr_df.columns[1:].tolist()
print(raw_crispr_df_columns)

['A1BG (1)', 'A1CF (29974)', 'A2M (2)', 'A2ML1 (144568)', 'A3GALT2 (127550)', 'A4GALT (53947)', 'A4GNT (51146)', 'AAAS (8086)', 'AACS (65985)', 'AADAC (13)', 'AADACL2 (344752)', 'AADACL3 (126767)', 'AADACL4 (343066)', 'AADAT (51166)', 'AAGAB (79719)', 'AAK1 (22848)', 'AAMDC (28971)', 'AAMP (14)', 'AANAT (15)', 'AAR2 (25980)', 'AARD (441376)', 'AARS1 (16)', 'AARS2 (57505)', 'AARSD1 (80755)', 'AASDH (132949)', 'AASDHPPT (60496)', 'AASS (10157)', 'AATF (26574)', 'AATK (9625)', 'ABAT (18)', 'ABCA1 (19)', 'ABCA10 (10349)', 'ABCA12 (26154)', 'ABCA13 (154664)', 'ABCA2 (20)', 'ABCA3 (21)', 'ABCA4 (24)', 'ABCA5 (23461)', 'ABCA6 (23460)', 'ABCA7 (10347)', 'ABCA8 (10351)', 'ABCA9 (10350)', 'ABCB1 (5243)', 'ABCB10 (23456)', 'ABCB11 (8647)', 'ABCB4 (5244)', 'ABCB5 (340273)', 'ABCB6 (10058)', 'ABCB7 (22)', 'ABCB8 (11194)', 'ABCB9 (23457)', 'ABCC1 (4363)', 'ABCC10 (89845)', 'ABCC11 (85320)', 'ABCC12 (94160)', 'ABCC2 (1244)', 'ABCC3 (8714)', 'ABCC4 (10257)', 'ABCC5 (10057)', 'ABCC6 (368)', 'ABCC8 (683

## 6. Coalign the samples

In [330]:
methy_samples = final_merged_methy_df.columns[1:]
gene_samples = final_merged_gene_df.columns[1:]
transcript_samples = final_merged_transcript_df.columns[1:]
protein_samples = final_merged_protein_df.columns[1:]
drug_samples = list(set(list(final_drug_df['ARXSPAN_ID'])))

# print all samples
print(methy_samples)
print(gene_samples)
print(transcript_samples)
print(protein_samples)
print(drug_samples)

Index(['DMS53_LUNG', 'SW1116_LARGE_INTESTINE',
       'P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'UMUC3_URINARY_TRACT',
       'HOS_BONE', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'RVH421_SKIN',
       'NCIH1184_LUNG', 'HCC2157_BREAST',
       ...
       'JHUEM7_ENDOMETRIUM', 'OE21_OESOPHAGUS',
       'MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'HOP62_LUNG', 'EKVX_LUNG',
       'UO31_KIDNEY', 'SF268_CENTRAL_NERVOUS_SYSTEM',
       'SF539_CENTRAL_NERVOUS_SYSTEM', 'SNB75_CENTRAL_NERVOUS_SYSTEM',
       'HOP92_LUNG'],
      dtype='object', length=843)
Index(['ACH-000628', 'ACH-000974', 'ACH-003071', 'ACH-001338', 'ACH-001608',
       'ACH-000278', 'ACH-001350', 'ACH-000150', 'ACH-001132', 'ACH-000324',
       ...
       'ACH-000236', 'ACH-002177', 'ACH-002163', 'ACH-002316', 'ACH-001039',
       'ACH-001389', 'ACH-001835', 'ACH-000296', 'ACH-001971', 'ACH-002276'],
      dtype='object', length=1929)
Index(['ACH-000873', 'ACH-000860',

In [331]:
# cell line names in the protein samples, remove the content after the second "_"
# result = ['_'.join(s.split('_')[:2]) for s in strings]
protein_split_samples = ['_'.join(s.split('_')[:2]) for s in protein_samples]
print(protein_split_samples)

['MDAMB468_BREAST', 'SH4_SKIN', 'AU565_BREAST', 'KMRC1_KIDNEY', 'CAL51_BREAST', 'RPMI7951_SKIN', 'RERFLCMS_LUNG', 'IGR37_SKIN', 'VMRCRCW_KIDNEY', 'HEP3B217_LIVER', 'HEPG2_LIVER', 'SKHEP1_LIVER', 'WM2664_SKIN', 'KMRC20_KIDNEY', 'HS294T_SKIN', 'CAOV3_OVARY', 'CAL120_BREAST', 'JMSU1_URINARY', 'PATU8988T_PANCREAS', 'JIMT1_BREAST', 'COV362_OVARY', 'KPL1_BREAST', 'A375_SKIN', 'A2058_SKIN', 'SW620_LARGE', 'MDST8_LARGE', 'HUH1_LIVER', 'HCT116_LARGE', 'HLF_LIVER', 'HT29_LARGE', 'ABC1_LUNG', 'CAMA1_BREAST', 'RKO_LARGE', 'HT115_LARGE', 'JHH5_LIVER', 'SKMEL3_SKIN', 'NCIH358_LUNG', 'HT1197_URINARY', 'HUH7_LIVER', 'HGC27_STOMACH', 'DMS273_LUNG', 'NCIH1299_LUNG', 'SKMEL28_SKIN', 'CAKI2_KIDNEY', 'HT55_LARGE', 'NCIH1048_LUNG', 'HSC3_UPPER', 'SW403_LARGE', 'BT549_BREAST', 'HCC56_LARGE', 'CAL851_BREAST', 'MDAMB157_BREAST', 'SW48_LARGE', 'IGR39_SKIN', 'MCF7_BREAST', 'BICR6_UPPER', 'WM1799_SKIN', 'IPC298_SKIN', 'NCIH1693_LUNG', 'HCC1954_BREAST', 'TYKNU_OVARY', 'DMS114_LUNG', 'TE1_OESOPHAGUS', 'TCCPAN2_PANC

In [332]:
cell_line_anno_df = pd.read_csv('./data/raw_data/Cell_lines_annotations_20181226.txt', sep='\t')
# drop NaN in the depMapID
cell_line_anno_df = cell_line_anno_df.dropna(subset=['depMapID']).reset_index(drop=True)
# drop NaN in the Pathology
cell_line_anno_df = cell_line_anno_df.dropna(subset=['Pathology']).reset_index(drop=True)
# drop rows if the column 'PATHOLOGIST_ANNOTATION' contains 'benign'
# First handle NaN values in PATHOLOGIST_ANNOTATION
cell_line_anno_df = cell_line_anno_df[~(
    cell_line_anno_df['PATHOLOGIST_ANNOTATION'].fillna('').str.contains('benign', case=False)
)]
display(cell_line_anno_df)

,CCLE_ID,depMapID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,Histology,Hist_Subtype1,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,DMS53_LUNG,ACH-000698,DMS 53,primary,lung,NS,NS,NS,carcinoma,small_cell_carcinoma,...,Waymouth's +10%FBS,NaN,5%DMSO,NaN,450.0,lung_small_cell,lung_small_cell,Lung:SCLC,157.241638,SCLC
1,SW1116_LARGE_INTESTINE,ACH-000489,SW1116,primary,large_intestine,colon,NS,NS,carcinoma,adenocarcinoma,...,L15+10%FBS,NaN,5%DMSO,NaN,123.2,colorectal,colorectal,Colorectal:Carcinoma,94.373192,COAD/READ
2,NCIH1694_LUNG,ACH-000431,NCI-H1694,metastasis,lung,NS,NS,NS,carcinoma,small_cell_carcinoma,...,DMEM:F12 (1:1) + 5% FBS,".005 mg/ml insulin, .01 mg/ml transferrin, 30n...",5% DMSO,NaN,81.2,lung_small_cell,lung_small_cell,Lung:SCLC,109.534987,SCLC
3,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000707,P3HR-1,metastasis,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,Burkitt_lymphoma,...,NaN,NaN,NaN,NaN,NaN,lymphoma_Burkitt,lymphoma_Burkitt,Lymphoma:NH_B_cell,166.456406,DLBC
4,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000509,HuT 78,primary,haematopoietic_and_lymphoid_tissue,skin,NS,NS,lymphoid_neoplasm,mycosis_fungoides-Sezary_syndrome,...,NaN,NaN,NaN,NaN,NaN,lymphoma_other,T-cell_lymphoma_other,Lymphoma:NH_T_cell,118.217569,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,SF539_CENTRAL_NERVOUS_SYSTEM,ACH-000273,SF539,primary,central_nervous_system,brain,NS,NS,glioma,NS,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,105.3,glioma,glioma,CNS:Glioma_HighGrade,107.540583,LGG
1016,SNB75_CENTRAL_NERVOUS_SYSTEM,ACH-000504,SNB75,primary,central_nervous_system,brain,NS,NS,glioma,NS,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,54.0,glioma,glioma,CNS:Glioma_HighGrade,106.364784,GBM
1017,HOP92_LUNG,ACH-000825,HOP-92,primary,lung,NS,NS,NS,carcinoma,large_cell_carcinoma,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,122.2,lung_NSC,lung_NSC,Lung:NSCLC_Others,198.739234,NaN
1018,ACCS_URINARY_TRACT,ACH-001011,ACCS,primary,urinary_tract,bladder,NS,NS,carcinoma,transitional_cell_carcinoma,...,NaN,NaN,NaN,NaN,NaN,urinary_tract,urinary_tract,NaN,NaN,BLCA


In [333]:
print(len(list(set(protein_split_samples))))
# give me the duplicate elements in the protein_split_samples
import collections
rep_list = [item for item, count in collections.Counter(protein_split_samples).items() if count > 1]
# for column names in final_merged_protein_df, if the column name contains the element in the rep_list, collect the column name in rep_col_list
rep_col_list = [col for col in final_merged_protein_df.columns if any(rep in col for rep in rep_list)]
print(rep_col_list)
# display the rep_col_list in the final_merged_protein_df
display(final_merged_protein_df[rep_col_list])

# removed all of the rep_col_list in the final_merged_protein_df
final_merged_protein_df = final_merged_protein_df.drop(columns=rep_col_list)
display(final_merged_protein_df)

375
['CAL120_BREAST_TenPx02', 'SW948_LARGE_INTESTINE_TenPx11', 'SW948_LARGE_INTESTINE_TenPx20', 'CAL120_BREAST_TenPx28', 'HCT15_LARGE_INTESTINE_TenPx30', 'HCT15_LARGE_INTESTINE_TenPx18']


,CAL120_BREAST_TenPx02,SW948_LARGE_INTESTINE_TenPx11,SW948_LARGE_INTESTINE_TenPx20,CAL120_BREAST_TenPx28,HCT15_LARGE_INTESTINE_TenPx30,HCT15_LARGE_INTESTINE_TenPx18
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
121414,0.0,0.0,0.0,0.0,0.0,0.0
121415,0.0,0.0,0.0,0.0,0.0,0.0
121416,0.0,0.0,0.0,0.0,0.0,0.0
121417,0.0,0.0,0.0,0.0,0.0,0.0


,BioMedGraphica_Conn_ID,MDAMB468_BREAST_TenPx01,SH4_SKIN_TenPx01,AU565_BREAST_TenPx01,KMRC1_KIDNEY_TenPx01,CAL51_BREAST_TenPx01,RPMI7951_SKIN_TenPx01,RERFLCMS_LUNG_TenPx01,IGR37_SKIN_TenPx01,VMRCRCW_KIDNEY_TenPx01,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [334]:
# get the overlapped cell lines in methy_samples with cell_line_anno_df by merge on cell line id
# first format the methy_samples as dataframe
methy_samples_df = pd.DataFrame(methy_samples, columns=['CCLE_Name'])
# merge the methy_samples_df with the cell_line_anno_df
merged_methy_samples_df = pd.merge(methy_samples_df, cell_line_anno_df, left_on='CCLE_Name', right_on='CCLE_ID', how='inner')
display(merged_methy_samples_df)

# get the map dictionary from the ccle id to depmap id
methy_map_dict = dict(zip(merged_methy_samples_df['CCLE_Name'], merged_methy_samples_df['depMapID']))
print(methy_map_dict)

,CCLE_Name,CCLE_ID,depMapID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,Histology,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,DMS53_LUNG,DMS53_LUNG,ACH-000698,DMS 53,primary,lung,NS,NS,NS,carcinoma,...,Waymouth's +10%FBS,NaN,5%DMSO,NaN,450.0,lung_small_cell,lung_small_cell,Lung:SCLC,157.241638,SCLC
1,SW1116_LARGE_INTESTINE,SW1116_LARGE_INTESTINE,ACH-000489,SW1116,primary,large_intestine,colon,NS,NS,carcinoma,...,L15+10%FBS,NaN,5%DMSO,NaN,123.2,colorectal,colorectal,Colorectal:Carcinoma,94.373192,COAD/READ
2,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000707,P3HR-1,metastasis,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,...,NaN,NaN,NaN,NaN,NaN,lymphoma_Burkitt,lymphoma_Burkitt,Lymphoma:NH_B_cell,166.456406,DLBC
3,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000509,HuT 78,primary,haematopoietic_and_lymphoid_tissue,skin,NS,NS,lymphoid_neoplasm,...,NaN,NaN,NaN,NaN,NaN,lymphoma_other,T-cell_lymphoma_other,Lymphoma:NH_T_cell,118.217569,NaN
4,UMUC3_URINARY_TRACT,UMUC3_URINARY_TRACT,ACH-000522,UM-UC-3,primary,urinary_tract,bladder,NS,NS,carcinoma,...,EMEM +10%FBS,NaN,5%DMSO,NaN,386.7,urinary_tract,urinary_tract,Bladder:Carcinoma,124.707962,BLCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,UO31_KIDNEY,UO31_KIDNEY,ACH-000428,UO-31,primary,kidney,NS,NS,NS,carcinoma,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,27.8,kidney,kidney,Kidney:Carcinoma,98.686744,KIRC
836,SF268_CENTRAL_NERVOUS_SYSTEM,SF268_CENTRAL_NERVOUS_SYSTEM,ACH-000655,SF268,primary,central_nervous_system,brain,NS,NS,glioma,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,63.0,glioma,glioma,CNS:Glioma_HighGrade,157.809603,GBM
837,SF539_CENTRAL_NERVOUS_SYSTEM,SF539_CENTRAL_NERVOUS_SYSTEM,ACH-000273,SF539,primary,central_nervous_system,brain,NS,NS,glioma,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,105.3,glioma,glioma,CNS:Glioma_HighGrade,107.540583,LGG
838,SNB75_CENTRAL_NERVOUS_SYSTEM,SNB75_CENTRAL_NERVOUS_SYSTEM,ACH-000504,SNB75,primary,central_nervous_system,brain,NS,NS,glioma,...,RPMI-1640+ 5% FBS,NaN,5% DMSO,NaN,54.0,glioma,glioma,CNS:Glioma_HighGrade,106.364784,GBM


{'DMS53_LUNG': 'ACH-000698', 'SW1116_LARGE_INTESTINE': 'ACH-000489', 'P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000707', 'HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000509', 'UMUC3_URINARY_TRACT': 'ACH-000522', 'HOS_BONE': 'ACH-000613', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000557', 'RVH421_SKIN': 'ACH-000614', 'NCIH1184_LUNG': 'ACH-000523', 'HCC2157_BREAST': 'ACH-000691', 'NCIH2227_LUNG': 'ACH-000610', 'SNU449_LIVER': 'ACH-000420', 'NCIH28_PLEURA': 'ACH-000648', 'OV56_OVARY': 'ACH-000091', 'RMUGS_OVARY': 'ACH-000701', 'LN229_CENTRAL_NERVOUS_SYSTEM': 'ACH-000595', 'P31FUJ_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000770', 'RKN_SOFT_TISSUE': 'ACH-000505', 'PATU8988S_PANCREAS': 'ACH-000022', 'NH6_AUTONOMIC_GANGLIA': 'ACH-000203', 'SF126_CENTRAL_NERVOUS_SYSTEM': 'ACH-000609', 'RERFLCAD2_LUNG': 'ACH-000774', 'OUMS23_LARGE_INTESTINE': 'ACH-000296', 'SNGM_ENDOMETRIUM': 'ACH-000974', 'OUMS27_BONE': 'ACH-001150', 'NCIH2347_LUNG': 'ACH-000875', 'SW1990_PANCREAS': 'ACH-000

In [335]:
# get the cleaned cell line names by removed the content after the second "_" in final_merged_protein_df.columns[1:]
original_protein_samples = final_merged_protein_df.columns[1:]
cleaned_protein_samples = ['_'.join(s.split('_')[:2]) for s in original_protein_samples]
# get the cleaned_protein_samples_dict_df from dictionary
cleaned_protein_samples_dict_df = pd.DataFrame({'CCLE_Name': original_protein_samples, 'cleaned_CCLE_Name': cleaned_protein_samples})
# merge the cleaned_protein_samples_df with the cell_line_anno_df
merged_cleaned_protein_samples_df = pd.merge(cleaned_protein_samples_dict_df, cell_line_anno_df, left_on='cleaned_CCLE_Name', right_on='CCLE_ID', how='inner')
display(merged_cleaned_protein_samples_df)
# get the depmap protein id
depmap_protein_samples = merged_cleaned_protein_samples_df['depMapID'].to_list()

# Create the map dictionary for the protein samples
protein_map_dict = dict(zip(original_protein_samples, depmap_protein_samples))
print(protein_map_dict)

,CCLE_Name,cleaned_CCLE_Name,CCLE_ID,depMapID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,MDAMB468_BREAST_TenPx01,MDAMB468_BREAST,MDAMB468_BREAST,ACH-000849,MDA-MB-468,metastasis,breast,NS,NS,NS,...,DMEM+10% FBS,NaN,5% DMSO,NaN,63.5,breast,breast,Breast:Carcinoma,188.252652,BRCA
1,SH4_SKIN_TenPx01,SH4_SKIN,SH4_SKIN,ACH-000441,SH-4,metastasis,skin,NS,NS,NS,...,DMEM+10% FBS,NaN,5%DMSO,NaN,68.9,melanoma,melanoma,Skin:Melanoma,109.541023,SKCM
2,AU565_BREAST_TenPx01,AU565_BREAST,AU565_BREAST,ACH-000248,AU565,metastasis,breast,NS,NS,NS,...,RPMI 1640 + 10%FBS,NaN,5% DMSO,NaN,74.1,breast,breast,Breast:Carcinoma,223.382430,BRCA
3,KMRC1_KIDNEY_TenPx01,KMRC1_KIDNEY,KMRC1_KIDNEY,ACH-000684,KMRC-1,primary,kidney,NS,NS,NS,...,DMEM+10% FBS,NaN,5% DMSO,NaN,46.4,kidney,kidney,Kidney:Carcinoma,204.674772,KIRC
4,CAL51_BREAST_TenPx01,CAL51_BREAST,CAL51_BREAST,ACH-000856,CAL-51,metastasis,breast,NS,NS,NS,...,80% Dulbecco's MEM(4.5g/L glucose) +20% h.i.FBS,NaN,10%DMSO,>60 hrs,26.8,breast,breast,Breast:Carcinoma,206.841626,BRCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,22RV1_PROSTATE_TenPx24,22RV1_PROSTATE,22RV1_PROSTATE,ACH-000956,22Rv1,primary,prostate,NS,NS,NS,...,RPMI-1640+10% FBS,NaN,5%DMSO,NaN,58.4,prostate,prostate,Prostate:Carcinoma,539.830868,PRAD
254,BT20_BREAST_TenPx24,BT20_BREAST,BT20_BREAST,ACH-000536,BT-20,primary,breast,NS,NS,NS,...,NaN,NaN,NaN,NaN,NaN,breast,breast,Breast:Carcinoma,114.140088,BRCA
255,CALU6_LUNG_TenPx24,CALU6_LUNG,CALU6_LUNG,ACH-000264,Calu-6,primary,lung,NS,NS,NS,...,EMEM +10%FBS,NaN,5% DMSO,NaN,78.9,lung_NSC,lung_NSC,Lung:NSCLC_Others,87.777181,LUAD
256,KP4_PANCREAS_TenPx24,KP4_PANCREAS,KP4_PANCREAS,ACH-000265,KP4,metastasis,pancreas,NS,NS,NS,...,IMDM+10%FBS,NaN,5%DMSO,NaN,NaN,pancreas,pancreas,Pancreas:Carcinoma,143.110198,PAAD


{'MDAMB468_BREAST_TenPx01': 'ACH-000849', 'SH4_SKIN_TenPx01': 'ACH-000441', 'AU565_BREAST_TenPx01': 'ACH-000248', 'KMRC1_KIDNEY_TenPx01': 'ACH-000684', 'CAL51_BREAST_TenPx01': 'ACH-000856', 'RPMI7951_SKIN_TenPx01': 'ACH-000348', 'RERFLCMS_LUNG_TenPx01': 'ACH-000062', 'IGR37_SKIN_TenPx01': 'ACH-000650', 'VMRCRCW_KIDNEY_TenPx01': 'ACH-000484', 'HEP3B217_LIVER_TenPx02': 'ACH-000625', 'HEPG2_LIVER_TenPx02': 'ACH-000739', 'SKHEP1_LIVER_TenPx02': 'ACH-000361', 'WM2664_SKIN_TenPx02': 'ACH-001239', 'KMRC20_KIDNEY_TenPx02': 'ACH-000250', 'HS294T_SKIN_TenPx02': 'ACH-000014', 'CAOV3_OVARY_TenPx02': 'ACH-000713', 'JMSU1_URINARY_TRACT_TenPx02': 'ACH-000023', 'PATU8988T_PANCREAS_TenPx03': 'ACH-000711', 'JIMT1_BREAST_TenPx03': 'ACH-000278', 'COV362_OVARY_TenPx03': 'ACH-000028', 'KPL1_BREAST_TenPx03': 'ACH-000219', 'A375_SKIN_TenPx03': 'ACH-000788', 'A2058_SKIN_TenPx03': 'ACH-000475', 'SW620_LARGE_INTESTINE_TenPx03': 'ACH-000393', 'MDST8_LARGE_INTESTINE_TenPx03': 'ACH-000528', 'HUH1_LIVER_TenPx03': 'A

In [336]:
# get the methy_mapped_samples and protein_mapped_samples from merged_methy_samples_df and merged_cleaned_protein_samples_df
methy_mapped_samples = list(merged_methy_samples_df['depMapID'])
protein_mapped_samples = list(merged_cleaned_protein_samples_df['depMapID'])

### 6.1 Aligning over omics

#### 6.1.1 Aligning with intersection

In [337]:
# get the overlapped samples only over omics samples
omics_overlapped_samples = sorted(list(set(methy_mapped_samples) & set(gene_samples) & set(transcript_samples) & set(protein_mapped_samples)))
print("length of omics overlapped samples: ", len(omics_overlapped_samples))
# check the overlapped samples with annotation samples
annotation_samples = list(cell_line_anno_df['depMapID'])
overlapped_omics_annotation_samples = sorted(list(set(omics_overlapped_samples) & set(annotation_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_annotation_samples))

length of omics overlapped samples:  223
length of overlapped omics and annotation samples:  223


#### 6.1.2 Aligning with union on omics and intersection with annotation

In [338]:
# get the union samples only over omics samples
omics_union_samples = sorted(list(set(methy_mapped_samples) | set(gene_samples) | set(transcript_samples) | set(protein_mapped_samples)))
print("length of omics union samples: ", len(omics_union_samples))
# check the intersection of union samples with annotation samples
overlapped_omics_union_annotation_samples = sorted(list(set(omics_union_samples) & set(annotation_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_union_annotation_samples))
print(overlapped_omics_union_annotation_samples)

length of omics union samples:  2007
length of overlapped omics and annotation samples:  985
['ACH-000001', 'ACH-000002', 'ACH-000004', 'ACH-000005', 'ACH-000006', 'ACH-000007', 'ACH-000008', 'ACH-000009', 'ACH-000011', 'ACH-000012', 'ACH-000013', 'ACH-000014', 'ACH-000015', 'ACH-000016', 'ACH-000017', 'ACH-000018', 'ACH-000019', 'ACH-000020', 'ACH-000021', 'ACH-000022', 'ACH-000023', 'ACH-000024', 'ACH-000026', 'ACH-000027', 'ACH-000028', 'ACH-000030', 'ACH-000031', 'ACH-000032', 'ACH-000035', 'ACH-000037', 'ACH-000038', 'ACH-000039', 'ACH-000040', 'ACH-000041', 'ACH-000042', 'ACH-000043', 'ACH-000044', 'ACH-000045', 'ACH-000046', 'ACH-000047', 'ACH-000048', 'ACH-000050', 'ACH-000051', 'ACH-000052', 'ACH-000053', 'ACH-000054', 'ACH-000055', 'ACH-000056', 'ACH-000058', 'ACH-000059', 'ACH-000060', 'ACH-000061', 'ACH-000062', 'ACH-000063', 'ACH-000065', 'ACH-000066', 'ACH-000067', 'ACH-000068', 'ACH-000069', 'ACH-000070', 'ACH-000071', 'ACH-000072', 'ACH-000073', 'ACH-000074', 'ACH-00007

#### 6.1.3 overlapped_omics_union_annotation_samples (cancerous / non-cancerous) Ratio

In [339]:
non_cancerous_samples_df = pd.read_csv('./data/raw_data/cell-lines-in-Non-Cancerous.csv')
display(non_cancerous_samples_df)
# get the depmap id from the non_cancerous_samples_df
non_cancerous_samples = list(non_cancerous_samples_df['Depmap Id'])
# get the overlapped non_cancerous samples with the overlapped_omics_union_annotation_samples
overlapped_non_cancerous_samples = sorted(list(set(overlapped_omics_union_annotation_samples) & set(non_cancerous_samples)))
print("length of overlapped omics and annotation samples: ", len(overlapped_omics_union_annotation_samples))
print("length of overlapped non cancerous samples: ", len(overlapped_non_cancerous_samples))


,Depmap Id,Cell Line,Primary Disease,Tumor Type
0,ACH-002370,NCIBL2122,Non-Cancerous,NaN
1,ACH-003133,ABMT0021,Non-Cancerous,NaN
2,ACH-001207,TIG3TD,Non-Cancerous,NaN
3,ACH-000229,HS822T,Non-Cancerous,Primary
4,ACH-002364,NCIBL1437,Non-Cancerous,NaN
...,...,...,...,...
132,ACH-000214,HS675T,Non-Cancerous,NaN
133,ACH-002368,NCIBL2087,Non-Cancerous,NaN
134,ACH-000850,HS698T,Non-Cancerous,Metastatic
135,ACH-002463,RPE1SS77,Non-Cancerous,NaN


length of overlapped omics and annotation samples:  985
length of overlapped non cancerous samples:  39


### 6.2 Overlapping over DTI

In [340]:
# get the overlapped samples among all samples (methy_mapped_samples, gene_samples, transcript_samples, protein_mapped_samples, drug_samples)
dti_overlapped_samples = sorted(list(set(overlapped_omics_union_annotation_samples) & set(drug_samples)))
print("len(overlapped_samples):", len(dti_overlapped_samples))
# need to confirm zero intersection between the overlapped samples and non_cancerous_samples
dti_overlapped_non_cancerous_samples = sorted(list(set(dti_overlapped_samples) & set(non_cancerous_samples)))
print("len(overlapped_non_cancerous_samples):", len(dti_overlapped_non_cancerous_samples))
# if there are overlapped samples between the dti_overlapped_samples and non_cancerous_samples, then remove these samples from the dti_overlapped_samples
if len(dti_overlapped_non_cancerous_samples) > 0:
    dti_overlapped_samples = sorted(list(set(dti_overlapped_samples) - set(dti_overlapped_non_cancerous_samples)))
# convert the overlapped_samples to the dataframe with annotation
dti_overlapped_samples_df = pd.merge(pd.DataFrame(dti_overlapped_samples, columns=['depMapID']), cell_line_anno_df, on='depMapID', how='inner')
display(dti_overlapped_samples_df)

len(overlapped_samples): 650
len(overlapped_non_cancerous_samples): 1


,depMapID,CCLE_ID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,Histology,Hist_Subtype1,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,ACH-000001,NIHOVCAR3_OVARY,NIH:OVCAR-3,metastasis,ovary,NS,NS,NS,carcinoma,NS,...,RPMI+20%FBS,NaN,5%DMSO,NaN,NaN,ovary,ovary,Ovary:Carcinoma,83.806951,OV
1,ACH-000002,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HL-60,primary,haematopoietic_and_lymphoid_tissue,NS,NS,NS,haematopoietic_neoplasm,acute_myeloid_leukaemia,...,IMDM+20 % FBS,na,5%DMSO,NaN,113.3,AML,AML,Leukemia:AML,81.133673,LAML
2,ACH-000004,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL,primary,haematopoietic_and_lymphoid_tissue,NS,NS,NS,haematopoietic_neoplasm,acute_myeloid_leukaemia,...,RPMI1640+10%FBS,NaN,10%DMSO,36 hrs,38.4,AML,AML,Leukemia:AML,98.185377,LAML
3,ACH-000006,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,MONO-MAC-6,primary,haematopoietic_and_lymphoid_tissue,NS,NS,NS,haematopoietic_neoplasm,acute_myeloid_leukaemia,...,90% RPMI 1640 +10% h.i. FBS,2mM L-glutamine + non-essential amino acids +1...,NaN,~ 60 hrs,55.3,AML,AML,Leukemia:AML,77.065234,LAML
4,ACH-000007,LS513_LARGE_INTESTINE,LS513,primary,large_intestine,caecum,NS,NS,carcinoma,adenocarcinoma,...,NaN,NaN,NaN,NaN,NaN,colorectal,colorectal,Colorectal:Carcinoma,112.349539,COAD/READ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,ACH-001199,SNUC2B_LARGE_INTESTINE,SNU-C2B,primary,large_intestine,caecum,NS,NS,carcinoma,adenocarcinoma,...,NaN,NaN,NaN,NaN,NaN,colorectal,colorectal,NaN,NaN,COAD/READ
645,ACH-001203,SUPHD1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SUP-HD1,primary,haematopoietic_and_lymphoid_tissue,lymph_node,NS,NS,lymphoid_neoplasm,Hodgkin_lymphoma,...,McCOY'S 5A+20% FBS,NaN,5% DMSO,NaN,NaN,lymphoma_Hodgkin,lymphoma_Hodgkin,NaN,NaN,NaN
646,ACH-001233,VMRCLCD_LUNG,VMRC-LCD,primary,lung,NS,NS,NS,carcinoma,adenocarcinoma,...,EMEM+10% FBS,NaN,5% DMSO,NaN,NaN,lung_NSC,lung_NSC,NaN,NaN,LUAD
647,ACH-001306,8305C_THYROID,8305C,primary,thyroid,NS,NS,NS,carcinoma,anaplastic_carcinoma,...,RPMI1640+20%FBS,NaN,10%DMSO,54hrs,121.6,thyroid,thyroid,Thyroid:Carcinoma,200.908236,THCA


In [341]:
# mkdir for processsing folder
import os
if not os.path.exists('./data/process_data'):
    os.makedirs('./data/process_data')

In [342]:
dti_overlapped_samples_df.to_csv('./data/process_data/dti_overlapped_samples.csv', index=False)

In [343]:
print(dti_overlapped_samples_df.columns)

Index(['depMapID', 'CCLE_ID', 'Name', 'Pathology', 'Site_Primary',
       'Site_Subtype1', 'Site_Subtype2', 'Site_Subtype3', 'Histology',
       'Hist_Subtype1', 'Hist_Subtype2', 'Hist_Subtype3', 'Gender',
       'Life_Stage', 'Age', 'Race', 'Geo_Loc', 'inferred_ethnicity',
       'Site_Of_Finding', 'Disease', 'Annotation_Source',
       'Original.Source.of.Cell.Line', 'Characteristics', 'Growth.Medium',
       'Supplements', 'Freezing.Medium', 'Doubling.Time.from.Vendor',
       'Doubling.Time.Calculated.hrs', 'type', 'type_refined',
       'PATHOLOGIST_ANNOTATION', 'mutRate', 'tcga_code'],
      dtype='object')


## 7. Annotate the samples with disease

### 7.1 Add annotation from Cellosaurus

In [344]:
# import pandas as pd
# import requests

# url = 'https://ftp.expasy.org/databases/cellosaurus/cellosaurus.obo'
# response = requests.get(url)

# with open('./data/raw_data/cellosaurus.obo', 'wb') as f:
#     f.write(response.content)

# # Path to the downloaded OBO file
# obo_path = './data/raw_data/cellosaurus.obo'

# entries = []
# current_entry = {}

# # Read and parse
# with open(obo_path, 'r', encoding='utf-8') as file:
#     for line in file:
#         line = line.strip()

#         if line == "[Term]":
#             if current_entry.get("xref") and any("NCBI_TaxID:9606" in x for x in current_entry["xref"]):
#                 # Separate NCIt and ORDO xrefs
#                 ncit_refs = [x for x in current_entry["xref"] if x.startswith("NCIt:")]
#                 ordo_refs = [x for x in current_entry["xref"] if x.startswith("ORDO:")]
#                 current_entry["xref_NCIt"] = "; ".join(ncit_refs)
#                 current_entry["xref_ORDO"] = "; ".join(ordo_refs)
#                 entries.append({
#                     "id": current_entry.get("id", ""),
#                     "name": current_entry.get("name", ""),
#                     "synonym": "; ".join(current_entry.get("synonym", [])),
#                     "xref_NCIt": current_entry.get("xref_NCIt", ""),
#                     "xref_ORDO": current_entry.get("xref_ORDO", "")
#                 })
#             current_entry = {"synonym": [], "xref": []}

#         elif line.startswith("id:"):
#             current_entry["id"] = line.split("id:")[1].strip()

#         elif line.startswith("name:"):
#             current_entry["name"] = line.split("name:")[1].strip()

#         elif line.startswith("synonym:"):
#             synonym = line.split("synonym:")[1].split("RELATED")[0].strip().strip('"')
#             current_entry["synonym"].append(synonym)

#         elif line.startswith("xref:"):
#             current_entry["xref"].append(line.split("xref:")[1].strip())

# # Handle last entry
# if current_entry.get("xref") and any("NCBI_TaxID:9606" in x for x in current_entry["xref"]):
#     ncit_refs = [x for x in current_entry["xref"] if x.startswith("NCIt:")]
#     ordo_refs = [x for x in current_entry["xref"] if x.startswith("ORDO:")]
#     entries.append({
#         "id": current_entry.get("id", ""),
#         "name": current_entry.get("name", ""),
#         "synonym": "; ".join(current_entry.get("synonym", [])),
#         "xref_NCIt": "; ".join(ncit_refs),
#         "xref_ORDO": "; ".join(ordo_refs)
#     })

# # Create DataFrame
# cellosaurus_parsed_df = pd.DataFrame(entries)
# cellosaurus_parsed_df = cellosaurus_parsed_df[["id", "name", "synonym", "xref_NCIt", "xref_ORDO"]]
# cellosaurus_parsed_df.to_csv('./data/raw_data/cellosaurus_parsed.csv', index=False)
# # Show preview
# display(cellosaurus_parsed_df)

In [345]:
# # Step 1: Keep the desired columns
# dti_overlapped_samples_desc_df = dti_overlapped_samples_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code']]
# display(dti_overlapped_samples_desc_df)

# # Step 2: Try direct merge on Name == name using left join
# matched_df = pd.merge(
#     dti_overlapped_samples_desc_df,
#     cellosaurus_parsed_df,
#     left_on='Name',
#     right_on='name',
#     how='left'
# )

# # Step 3: Find rows where no match occurred (name is NaN)
# unmatched_df = matched_df[matched_df['name'].isna()].copy()

# # Step 4: Drop the Cellosaurus columns to prep for synonym match
# cols_to_drop = [col for col in matched_df.columns if col in cellosaurus_parsed_df.columns]
# unmatched_df = unmatched_df.drop(columns=cols_to_drop)

# # Step 5: Expand cellosaurus synonyms
# synonym_expanded_df = cellosaurus_parsed_df.copy()
# synonym_expanded_df = synonym_expanded_df.dropna(subset=['synonym'])
# synonym_expanded_df = synonym_expanded_df.assign(
#     synonym=synonym_expanded_df['synonym'].str.split(';')
# ).explode('synonym')
# synonym_expanded_df['synonym'] = synonym_expanded_df['synonym'].str.strip()

# # Step 6: Left join unmatched rows with synonym-expanded Cellosaurus
# synonym_matched_df = pd.merge(
#     unmatched_df,
#     synonym_expanded_df,
#     left_on='Name',
#     right_on='synonym',
#     how='left'
# )

# # Step 7: Combine direct-matched (non-NaN name) and synonym-matched rows
# all_match_df = pd.concat(
#     [matched_df[matched_df['name'].notna()], synonym_matched_df],
#     ignore_index=True
# )

# # Optional: Drop duplicates based on depMapID if needed
# all_match_df = all_match_df.drop_duplicates(subset=['depMapID'])
# display(all_match_df)

In [346]:
# # Function to store primary and extra xref matches with better column handling
# def process_xref_columns_with_extras(df):
#     # Create a copy to avoid modifying the original
#     result_df = df.copy()
    
#     # Always create all the columns, even if they might end up empty
#     result_df['xref_NCIt_ID'] = ''
#     result_df['xref_NCIt_name'] = ''
#     result_df['xref_NCIt_extra_ID'] = ''
#     result_df['xref_NCIt_extra_name'] = ''
#     result_df['xref_ORDO_ID'] = ''
#     result_df['xref_ORDO_name'] = ''
#     result_df['xref_ORDO_extra_ID'] = ''
#     result_df['xref_ORDO_extra_name'] = ''
    
#     # Process xref_NCIt if it exists
#     if 'xref_NCIt' in result_df.columns:
#         # Only process rows that have xref_NCIt values
#         mask = result_df['xref_NCIt'].notna() & (result_df['xref_NCIt'] != '')
        
#         for idx in result_df[mask].index:
#             xref_value = result_df.loc[idx, 'xref_NCIt']
            
#             if '; ' in xref_value:  # Multiple entries
#                 entries = xref_value.split('; ')
                
#                 # Process first entry for primary columns
#                 if ' ! ' in entries[0]:
#                     id_val, name_val = entries[0].split(' ! ', 1)
#                     result_df.loc[idx, 'xref_NCIt_ID'] = id_val
#                     result_df.loc[idx, 'xref_NCIt_name'] = name_val
                
#                 # Process additional entries for extra columns
#                 extra_ids = []
#                 extra_names = []
#                 for entry in entries[1:]:
#                     if ' ! ' in entry:
#                         id_val, name_val = entry.split(' ! ', 1)
#                         extra_ids.append(id_val)
#                         extra_names.append(name_val)
                
#                 if extra_ids:
#                     result_df.loc[idx, 'xref_NCIt_extra_ID'] = '; '.join(extra_ids)
#                     result_df.loc[idx, 'xref_NCIt_extra_name'] = '; '.join(extra_names)
                    
#             elif ' ! ' in xref_value:  # Single entry
#                 id_val, name_val = xref_value.split(' ! ', 1)
#                 result_df.loc[idx, 'xref_NCIt_ID'] = id_val
#                 result_df.loc[idx, 'xref_NCIt_name'] = name_val
        
#         # Drop the original column
#         result_df = result_df.drop(columns=['xref_NCIt'])
    
#     # Process xref_ORDO if it exists
#     if 'xref_ORDO' in result_df.columns:
#         # Only process rows that have xref_ORDO values
#         mask = result_df['xref_ORDO'].notna() & (result_df['xref_ORDO'] != '')
        
#         for idx in result_df[mask].index:
#             xref_value = result_df.loc[idx, 'xref_ORDO']
            
#             if '; ' in xref_value:  # Multiple entries
#                 entries = xref_value.split('; ')
                
#                 # Process first entry for primary columns
#                 if ' ! ' in entries[0]:
#                     id_val, name_val = entries[0].split(' ! ', 1)
#                     result_df.loc[idx, 'xref_ORDO_ID'] = id_val
#                     result_df.loc[idx, 'xref_ORDO_name'] = name_val
                
#                 # Process additional entries for extra columns
#                 extra_ids = []
#                 extra_names = []
#                 for entry in entries[1:]:
#                     if ' ! ' in entry:
#                         id_val, name_val = entry.split(' ! ', 1)
#                         extra_ids.append(id_val)
#                         extra_names.append(name_val)
                
#                 if extra_ids:
#                     result_df.loc[idx, 'xref_ORDO_extra_ID'] = '; '.join(extra_ids)
#                     result_df.loc[idx, 'xref_ORDO_extra_name'] = '; '.join(extra_names)
                    
#             elif ' ! ' in xref_value:  # Single entry
#                 id_val, name_val = xref_value.split(' ! ', 1)
#                 result_df.loc[idx, 'xref_ORDO_ID'] = id_val
#                 result_df.loc[idx, 'xref_ORDO_name'] = name_val
        
#         # Drop the original column
#         result_df = result_df.drop(columns=['xref_ORDO'])
    
#     return result_df

# # Apply the function
# all_match_df = process_xref_columns_with_extras(all_match_df)

# # Update the column list to include the extra columns
# dti_overlapped_samples_desc_co_df = all_match_df[['depMapID', 'Name', 'Pathology', 'Histology', 
#                                              'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 
#                                              'xref_NCIt_ID', 'xref_NCIt_name', 'xref_NCIt_extra_ID', 'xref_NCIt_extra_name',
#                                              'xref_ORDO_ID', 'xref_ORDO_name', 'xref_ORDO_extra_ID', 'xref_ORDO_extra_name']]
# display(dti_overlapped_samples_desc_co_df)

### 7.2 Cell line disease match

#### 7.2.1 Cell line disease hard match

In [347]:
# bmg_disease_name_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Disease/BioMedGraphica_Conn_Disease_GUI_Name.csv')
# display(bmg_disease_name_df)

In [348]:
# # Step 1: Expand BMG_Disease_Name (still lowercased and exploded)
# bmg_expanded = bmg_disease_name_df.copy()

# # First, strip any surrounding quotes and fill NaN values with empty strings
# bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].fillna('')
# bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].str.replace('^"|"$', '', regex=True)

# # Split on pipe with spaces
# bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].str.split(' \| ')

# # Apply strip only to non-empty lists
# bmg_expanded['Disease_Name_List'] = bmg_expanded['Disease_Name_List'].apply(
#     lambda x: [name.strip() for name in x] if isinstance(x, list) else []
# )

# # Explode the list into separate rows
# bmg_expanded = bmg_expanded.explode('Disease_Name_List')

# # Remove rows with empty disease names after exploding
# bmg_expanded = bmg_expanded[bmg_expanded['Disease_Name_List'].str.len() > 0]

# # Create lowercase version for easier matching
# bmg_expanded['Disease_Name_List_lower'] = bmg_expanded['Disease_Name_List'].str.lower()

# display(bmg_expanded)

In [349]:
# # Create a lookup list of tuples: (Conn_ID, Original_Name, Lower_Name)
# bmg_records = bmg_expanded[['BioMedGraphica_Conn_ID', 'Disease_Name_List', 'Disease_Name_List_lower']].to_records(index=False)

# # Updated matching function: look for exact full-name match
# def match_bmg_disease_exact(xref_name, bmg_records):
#     if pd.isna(xref_name):
#         return (None, None)
#     xref_name_lower = str(xref_name).strip().lower()
    
#     for conn_id, original_name, bmg_name_lower in bmg_records:
#         if xref_name_lower == bmg_name_lower:
#             return (conn_id, original_name)
    
#     return (None, None)

# # NCIt
# ncit_matches = dti_overlapped_samples_desc_co_df['xref_NCIt_name'].apply(lambda x: match_bmg_disease_exact(x, bmg_records))
# dti_overlapped_samples_desc_co_df['NCIt_BMGC_ID'] = ncit_matches.apply(lambda x: x[0])
# dti_overlapped_samples_desc_co_df['NCIt_BMGC_name'] = ncit_matches.apply(lambda x: x[1])

# # ORDO
# ordo_matches = dti_overlapped_samples_desc_co_df['xref_ORDO_name'].apply(lambda x: match_bmg_disease_exact(x, bmg_records))
# dti_overlapped_samples_desc_co_df['ORDO_BMGC_ID'] = ordo_matches.apply(lambda x: x[0])
# dti_overlapped_samples_desc_co_df['ORDO_BMGC_name'] = ordo_matches.apply(lambda x: x[1])

# display(dti_overlapped_samples_desc_co_df)


In [350]:
# # keep the columns ['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_ID', 'xref_NCIt_name', 'NCIt_BMGC_ID', 'NCIt_BMGC_name', 'xref_ORDO_ID', 'xref_ORDO_name', 'ORDO_BMGC_ID', 'ORDO_BMGC_name']
# dti_overlapped_samples_desc_co_df = dti_overlapped_samples_desc_co_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_ID', 'xref_NCIt_name', 'NCIt_BMGC_ID', 'NCIt_BMGC_name', 'xref_ORDO_ID', 'xref_ORDO_name', 'ORDO_BMGC_ID', 'ORDO_BMGC_name']]
# display(dti_overlapped_samples_desc_co_df)
# dti_overlapped_samples_desc_co_df.to_csv('./data/process_data/dti_overlapped_samples_desc_co.csv', index=False)

In [351]:
# Merge NCIt_BMGC_ID/ORDO_BMGC_ID and NCIt_BMGC_name/ORDO_BMGC_name into BMGC_Matched columns
# Strategy:
# 1. Prioritize NCIt data when both are available
# 2. Use NCIt data if only NCIt is available
# 3. Use ORDO data if only ORDO is available
# 4. Leave as null if both are null

# dti_matched_df = dti_overlapped_samples_desc_co_df.copy()

# # Create new merged ID column
# dti_matched_df['BMGC_Matched_ID'] = (
#     dti_matched_df['NCIt_BMGC_ID'].fillna(
#         dti_matched_df['ORDO_BMGC_ID']
#     )
# )

# # Create new merged name column
# dti_matched_df['BMGC_Matched_name'] = (
#     dti_matched_df['NCIt_BMGC_name'].fillna(
#         dti_matched_df['ORDO_BMGC_name']
#     )
# )

# # Display the dataframe with new columns
# display(dti_matched_df)

# # Check number of null values in new columns
# print(f"Null values in BMGC_Matched_ID: {dti_matched_df['BMGC_Matched_ID'].isna().sum()}")
# print(f"Null values in BMGC_Matched_name: {dti_matched_df['BMGC_Matched_name'].isna().sum()}")

# # only keep the columns where the BMGC_Matched_ID is not null
# dti_matched_df = dti_matched_df[dti_matched_df['BMGC_Matched_ID'].notna()].reset_index(drop=True)
# display(dti_matched_df)

# # Save to CSV
# dti_matched_df.to_csv('./data/process_data/dti_matched.csv', index=False)

#### 7.2.2 Cell line disease soft match

In [352]:
# # check the unmatched samples in the dti_overlapped_samples_desc_co_df by checking the NaN in the NCIt_BMGC_ID and ORDO_BMGC_ID
# unmatched_samples_df = dti_overlapped_samples_desc_co_df[dti_overlapped_samples_desc_co_df['NCIt_BMGC_ID'].isna() & dti_overlapped_samples_desc_co_df['ORDO_BMGC_ID'].isna()]
# # keep the columns ['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_name', , 'xref_ORDO_name']
# unmatched_samples_df = unmatched_samples_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'xref_NCIt_name', 'xref_ORDO_name']]
# display(unmatched_samples_df)

In [353]:
# embed the disease name for bmg_disease_df by bert-based model
import torch
import numpy as np
from tqdm import tqdm
from typing import List, Tuple, Dict
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset

class SentenceDataset(Dataset):
    def __init__(self, sentences: List[str]):
        self.sentences = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        return self.sentences[idx]


class TextEncoder():
    def __init__(self, model_path: str = "dmis-lab/biobert-v1.1", device: str = "cuda"):
        """
        Args:
            model_path (str, optional): Path to the deberta model. Defaults to 'dmis-lab/biobert-v1.1'.
            device (str, optional): Device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.
        """
        self.model_path = model_path
        self.device = device
        self.model = None
        self.tokenizer = None

    def load_model(self):
        """
        Load the deberta model and tokenizer from the specified model path.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        self.model = AutoModel.from_pretrained(self.model_path).to(self.device)

    def generate_embeddings(self, sentences: List[str], batch_size: int = 32, seq_emb_dim: int = 64) -> torch.Tensor:
        """
        Generate a single-dimensional embedding for each sentence.

        Args:
            sentences (List[str]): List of sentences to embed.
            batch_size (int, optional): Batch size for DataLoader. Defaults to 32.

        Returns:
            List[float]: List of single-dimensional embeddings.
        """
        dataset = SentenceDataset(sentences)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        embedding_batches = []
        for batch in tqdm(dataloader, desc="Embedding sentences", unit="batch"):
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Handle single batch case properly
            mean_embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # [batch_size, hidden_dim]
            
            # For adaptive pooling, we need to reshape for 1D adaptive pooling
            # [batch_size, 1, hidden_dim] -> [batch_size, 1, seq_emb_dim] -> [batch_size, seq_emb_dim]
            batch_size = mean_embeddings.size(0)
            reshaped = mean_embeddings.view(batch_size, 1, -1)
            projected = torch.nn.functional.adaptive_avg_pool1d(reshaped, output_size=seq_emb_dim)
            projected = projected.squeeze(1)  # Only squeeze dimension 1, keep batch dimension
            embedding_batches.append(projected)
        return torch.cat(embedding_batches, dim=0)

    def save_embeddings(self, embeddings, output_npy_path):
        """
        Save embeddings to a .npy file.
        
        Args:
            embeddings (torch.Tensor): The embeddings to save.
            output_npy_path (str): Path to save the embeddings file.
        """
        # Move embeddings to CPU before converting to numpy
        embeddings_cpu = embeddings.cpu().numpy()
        np.save(output_npy_path, embeddings_cpu)
        print(f"Embeddings saved at {output_npy_path} with shape {embeddings_cpu.shape}")

In [354]:
# # convert the bmg_disease_name_df to the list of disease names of each rows
# disease_names = list(bmg_disease_name_df['Disease_Name_List'])
# # to make sure each disease name is a string
# disease_names = [str(name) for name in disease_names]
# print(len(disease_names))
# print(disease_names[:5])
# print(disease_names[-1])

In [355]:
#  # Use language model to embed the name and description
# name_sentence_list = disease_names
# text_encoder = TextEncoder()
# text_encoder.load_model()
# name_embeddings = text_encoder.generate_embeddings(name_sentence_list, batch_size=32, seq_emb_dim=768)
# print(name_embeddings.shape)
# # mkdir folder BMG_emb
# if not os.path.exists('./data/BMG_emb'):
#     os.makedirs('./data/BMG_emb')
# text_encoder.save_embeddings(name_embeddings, './data/BMG_emb/disease_name_embeddings.npy')

In [356]:
# # convert each row in the unmatched_samples_df to a sentence like "Name: NIH:OVCAR-3, Pathology: OVARY, Histology: CARCINOMA, type: CANCER, PATHOLOGIST_ANNOTATION: OVARY, tcga_code: OV"
# depmap_desc_sentence_list = []
# for idx, row in unmatched_samples_df.iterrows():
#     depmap_desc_sentence_list.append(f" {row['type']}, {row['PATHOLOGIST_ANNOTATION']}")

# # and create the ncit_desc_sentence_list and ordo_desc_sentence_list
# ncit_desc_sentence_list = list(unmatched_samples_df['xref_NCIt_name'])
# ordo_desc_sentence_list = list(unmatched_samples_df['xref_ORDO_name'])

# # make sure each element in the depmap_desc_sentence_list, ncit_desc_sentence_list and ordo_desc_sentence_list is a string
# depmap_desc_sentence_list = [str(name) for name in depmap_desc_sentence_list]
# ncit_desc_sentence_list = [str(name) for name in ncit_desc_sentence_list]
# ordo_desc_sentence_list = [str(name) for name in ordo_desc_sentence_list]

# print(depmap_desc_sentence_list)
# print(ncit_desc_sentence_list)
# print(ordo_desc_sentence_list)

In [357]:
# # Define the sentence lists to process and their corresponding prefixes
# sentence_list_mapping = {
#     'depmap': depmap_desc_sentence_list,
#     'ncit': ncit_desc_sentence_list,
#     'ordo': ordo_desc_sentence_list
# }

# # Define top_k for this matching task
# top_k = 2

# # Process each sentence list and add the matches to the DataFrame
# for prefix, sentence_list in sentence_list_mapping.items():
#     # Lists to store the matches for each description
#     all_matched_lists = [[] for _ in range(top_k)]
#     all_matched_name_lists = [[] for _ in range(top_k)]
#     all_similarity_lists = [[] for _ in range(top_k)]
    
#     for desc in sentence_list:
#         desc_embeddings = text_encoder.generate_embeddings([desc], batch_size=1, seq_emb_dim=768)
#         # Calculate cosine similarity
#         cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
#         similarity = cos(name_embeddings, desc_embeddings)
#         # Find the index of top k most similar diseases
#         top_k_idx = torch.argsort(similarity, descending=True)[:top_k]
        
#         # Process each of the top k matches
#         for rank, idx in enumerate(top_k_idx):
#             idx_int = idx.item()
#             disease_id = bmg_disease_name_df.iloc[idx_int]['BioMedGraphica_Conn_ID']
#             disease_name = disease_names[idx_int]
#             sim_score = similarity[idx_int].item()
            
#             # Store in the corresponding lists
#             all_matched_lists[rank].append(disease_id)
#             all_matched_name_lists[rank].append(disease_name)
#             all_similarity_lists[rank].append(sim_score)
    
#     # Add the matched data to the DataFrame with the appropriate prefix
#     for i in range(top_k):
#         rank = i + 1
#         unmatched_samples_df[f'{prefix}_match_{rank}_disease'] = all_matched_lists[i]
#         unmatched_samples_df[f'{prefix}_match_{rank}_disease_name'] = all_matched_name_lists[i]
#         unmatched_samples_df[f'{prefix}_match_{rank}_similarity'] = all_similarity_lists[i]
    
#     print(f"Finished processing {prefix} descriptions")

# # Display the updated DataFrame
# display(unmatched_samples_df)

# # Save the updated DataFrame to CSV
# unmatched_samples_df.to_csv('./data/process_data/unmatched_samples_softmatch.csv', index=False)

### 7.3 Manual filter and match

In [358]:
# unmatched_samples_df = pd.read_csv('./data/process_data/unmatched_samples_softmatch.csv')
# # add the 2 empty columnns 'BMGC_manual_ID' and 'BMGC_manual_name' to the unmatched_samples_manual_df
# unmatched_samples_manual_df = unmatched_samples_df.copy()
# unmatched_samples_manual_df['BMGC_manual_ID'] = None
# unmatched_samples_manual_df['BMGC_manual_name'] = None
# # change the order of the columns to make the 'BMGC_manual_ID' and 'BMGC_manual_name' after id
# unmatched_samples_manual_df = unmatched_samples_manual_df[['depMapID', 'Name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'id', 'BMGC_manual_ID', 'BMGC_manual_name', 'xref_NCIt_name', 'xref_ORDO_name', 'depmap_match_1_disease', 'depmap_match_1_disease_name', 'depmap_match_2_disease', 'depmap_match_2_disease_name', 'ncit_match_1_disease', 'ncit_match_1_disease_name', 'ncit_match_2_disease', 'ncit_match_2_disease_name', 'ordo_match_1_disease', 'ordo_match_1_disease_name', 'ordo_match_2_disease', 'ordo_match_2_disease_name']]
# display(unmatched_samples_manual_df)
# unmatched_samples_manual_df.to_csv('./data/process_data/unmatched_samples_manual.csv', index=False)

### 7.4 Combine matched and manual_unmatached

In [359]:
# Matched samples, rename BMGC_Matched_ID and BMGC_Matched_name to BMGC_ID and BMGC_name
dti_matched_df = pd.read_csv('./data/process_data/dti_matched.csv').rename(columns={'BMGC_Matched_ID': 'BMGC_Disease_ID', 'BMGC_Matched_name': 'BMGC_Disease_name'})
# keep the columns ['depMapID', 'Name', 'BMGC_Disease_ID', 'BMGC_Disease_name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code', 'xref_NCIt_ID', 'xref_NCIt_name', 'xref_ORDO_ID', 'xref_ORDO_name']
dti_matched_df = dti_matched_df[['depMapID', 'Name', 'BMGC_Disease_ID', 'BMGC_Disease_name', 'Pathology', 'Histology', 'type', 'PATHOLOGIST_ANNOTATION', 'tcga_code']]
display(dti_matched_df)
# Unmatched samples, rename BMGC_manual_ID and BMGC_manual_name to BMGC_Disease_ID and BMGC_Disease_name
dti_unmatched_manual_df = pd.read_csv('./data/process_data/unmatched_samples_manual.csv').rename(columns={'BMGC_manual_ID': 'BMGC_Disease_ID', 'BMGC_manual_name': 'BMGC_Disease_name'})
display(dti_unmatched_manual_df)

,depMapID,Name,BMGC_Disease_ID,BMGC_Disease_name,Pathology,Histology,type,PATHOLOGIST_ANNOTATION,tcga_code
0,ACH-000002,HL-60,BMGC_DS01407,acute myeloid leukemia,primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML
1,ACH-000004,HEL,BMGC_DS16646,erythroleukemia,primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML
2,ACH-000007,LS513,BMGC_DS08517,cecum adenocarcinoma,primary,carcinoma,colorectal,Colorectal:Carcinoma,COAD/READ
3,ACH-000008,A101D,BMGC_DS01559,melanoma,primary,malignant_melanoma,melanoma,Skin:Melanoma,SKCM
4,ACH-000009,C2BBe1,BMGC_DS05624,colon adenocarcinoma,primary,carcinoma,colorectal,Colorectal:Carcinoma,COAD/READ
...,...,...,...,...,...,...,...,...,...
549,ACH-000976,HuCCT1,BMGC_DS05988,intrahepatic cholangiocarcinoma,primary,carcinoma,bile_duct,Biliary_Tract:Carcinoma,NaN
550,ACH-000979,DU 145,BMGC_DS06988,prostate carcinoma,metastasis,carcinoma,prostate,Prostate:Carcinoma,PRAD
551,ACH-000997,HCT-15,BMGC_DS05624,colon adenocarcinoma,primary,carcinoma,colorectal,Colorectal:Carcinoma,COAD/READ
552,ACH-001021,C3A,BMGC_DS04044,hepatoblastoma,primary,carcinoma,liver,NaN,LIHC


,depMapID,Name,Pathology,Histology,type,PATHOLOGIST_ANNOTATION,tcga_code,id,BMGC_Disease_ID,BMGC_Disease_name,...,depmap_match_2_disease,depmap_match_2_disease_name,ncit_match_1_disease,ncit_match_1_disease_name,ncit_match_2_disease,ncit_match_2_disease_name,ordo_match_1_disease,ordo_match_1_disease_name,ordo_match_2_disease,ordo_match_2_disease_name
0,ACH-000038,EHEB,primary,lymphoid_neoplasm,lymphoma_other,Leukemia:CLL,CLL,CVCL_1194,NaN,NaN,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS20323,Fever,BMGC_DS22241,NaN,BMGC_DS20323,Fever,BMGC_DS22241,NaN
1,ACH-000070,697,primary,lymphoid_neoplasm,leukemia_other,Leukemia:ALL,ALL,CVCL_0079,NaN,NaN,...,BMGC_DS20478,"Leukemia, Megakaryoblastic, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...,BMGC_DS13122,B-cell acute lymphoblastic leukemia with t(1;1...,BMGC_DS06807,acute myeloid leukemia with inv(16)(p13.1q22) ...
2,ACH-000101,KE-37,primary,lymphoid_neoplasm,leukemia_other,Leukemia:ALL,ALL,CVCL_1327,NaN,NaN,...,BMGC_DS20478,"Leukemia, Megakaryoblastic, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS05322,T-cell adult acute lymphocytic leukemia,BMGC_DS20794,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...
3,ACH-000103,Caov-4,metastasis,carcinoma,ovary,NaN,OV,CVCL_0202,NaN,NaN,...,BMGC_DS03249,uterine adnexa cancer,BMGC_DS08979,ovarian serous surface papillary adenocarcinom...,BMGC_DS09417,peritoneal serous adenocarcinoma | primary per...,BMGC_DS20323,Fever,BMGC_DS22241,NaN
4,ACH-000104,Loucy,primary,lymphoid_neoplasm,T-cell_ALL,Leukemia:ALL,ALL,CVCL_1380,NaN,NaN,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS05322,T-cell adult acute lymphocytic leukemia,BMGC_DS20794,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,ACH-000926,HT55,primary,carcinoma,colorectal,Colorectal:Carcinoma,COAD/READ,CVCL_1294,NaN,NaN,...,BMGC_DS08620,colon squamous cell carcinoma | squamous cell ...,BMGC_DS05296,duodenal adenocarcinoma | duodenum adenocarcinoma,BMGC_DS05340,rectum mucinous adenocarcinoma,BMGC_DS20323,Fever,BMGC_DS22241,NaN
91,ACH-000973,639-V,primary,carcinoma,urinary_tract,Bladder:Carcinoma,BLCA,CVCL_1048,NaN,NaN,...,BMGC_DS20762,Urinary Tract Infections | urinary tract infec...,BMGC_DS09400,infiltrating renal pelvis/ureter urothelial ca...,BMGC_DS08797,inverted urothelial papilloma | urinary tract ...,BMGC_DS08797,inverted urothelial papilloma | urinary tract ...,BMGC_DS09400,infiltrating renal pelvis/ureter urothelial ca...
92,ACH-000995,JURKAT,primary,lymphoid_neoplasm,T-cell_ALL,Leukemia:ALL,ALL,CVCL_0065,NaN,NaN,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...,BMGC_DS20794,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...
93,ACH-001064,EB2,metastasis,lymphoid_neoplasm,lymphoma_Burkitt,NaN,NaN,NaN,NaN,NaN,...,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20323,Fever,BMGC_DS22241,NaN,BMGC_DS20323,Fever,BMGC_DS22241,NaN


In [360]:
dti_unmatched_manual_valid_df = pd.read_csv('./data/manual_match_data/unmatched_samples_manual_valid.csv').rename(columns={'BMGC_manual_ID': 'BMGC_Disease_ID', 'BMGC_manual_name': 'BMGC_Disease_name'})
display(dti_unmatched_manual_valid_df)
# concatenate the dti_matched_df and dti_unmatched_manual_df and sort by depMapID
dti_combined_df = pd.concat([dti_matched_df, dti_unmatched_manual_valid_df], ignore_index=True)
# sort the dti_combined_df by depMapID
dti_combined_df = dti_combined_df.sort_values(by='depMapID').reset_index(drop=True)
# check if there is any empty value in the BMGC_ID and BMGC_name columns
print(dti_combined_df[['BMGC_Disease_ID', 'BMGC_Disease_name']].isnull().sum())
# convert all content in 'BMGC_Disease_name' to lower case
dti_combined_df['BMGC_Disease_name'] = dti_combined_df['BMGC_Disease_name'].str.lower()
display(dti_combined_df)
dti_combined_df.to_csv('./data/process_data/dti_combined_samples.csv', index=False)

,depMapID,Name,Pathology,Histology,type,PATHOLOGIST_ANNOTATION,tcga_code,id,BMGC_Disease_ID,BMGC_Disease_name,...,depmap_match_2_disease,depmap_match_2_disease_name,ncit_match_1_disease,ncit_match_1_disease_name,ncit_match_2_disease,ncit_match_2_disease_name,ordo_match_1_disease,ordo_match_1_disease_name,ordo_match_2_disease,ordo_match_2_disease_name
0,ACH-000038,EHEB,primary,lymphoid_neoplasm,lymphoma_other,Leukemia:CLL,CLL,CVCL_1194,BMGC_DS01395,B-cell chronic lymphocytic leukemia | chronic ...,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS20323,Fever,BMGC_DS22241,NaN,BMGC_DS20323,Fever,BMGC_DS22241,NaN
1,ACH-000070,697,primary,lymphoid_neoplasm,leukemia_other,Leukemia:ALL,ALL,CVCL_0079,BMGC_DS13122,B-cell acute lymphoblastic leukemia with t(1;1...,...,BMGC_DS20478,"Leukemia, Megakaryoblastic, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...,BMGC_DS13122,B-cell acute lymphoblastic leukemia with t(1;1...,BMGC_DS06807,acute myeloid leukemia with inv(16)(p13.1q22) ...
2,ACH-000101,KE-37,primary,lymphoid_neoplasm,leukemia_other,Leukemia:ALL,ALL,CVCL_1327,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,...,BMGC_DS20478,"Leukemia, Megakaryoblastic, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS05322,T-cell adult acute lymphocytic leukemia,BMGC_DS20794,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...
3,ACH-000103,Caov-4,metastasis,carcinoma,ovary,NaN,OV,CVCL_0202,BMGC_DS07934,ovarian adenocarcinoma | ovary adenocarcinoma,...,BMGC_DS03249,uterine adnexa cancer,BMGC_DS08979,ovarian serous surface papillary adenocarcinom...,BMGC_DS09417,peritoneal serous adenocarcinoma | primary per...,BMGC_DS20323,Fever,BMGC_DS22241,NaN
4,ACH-000104,Loucy,primary,lymphoid_neoplasm,T-cell_ALL,Leukemia:ALL,ALL,CVCL_1380,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS05322,T-cell adult acute lymphocytic leukemia,BMGC_DS20794,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,ACH-000926,HT55,primary,carcinoma,colorectal,Colorectal:Carcinoma,COAD/READ,CVCL_1294,BMGC_DS08359,colorectal adenocarcinoma,...,BMGC_DS08620,colon squamous cell carcinoma | squamous cell ...,BMGC_DS05296,duodenal adenocarcinoma | duodenum adenocarcinoma,BMGC_DS05340,rectum mucinous adenocarcinoma,BMGC_DS20323,Fever,BMGC_DS22241,NaN
91,ACH-000973,639-V,primary,carcinoma,urinary_tract,Bladder:Carcinoma,BLCA,CVCL_1048,BMGC_DS07053,bladder carcinoma | urinary bladder carcinoma,...,BMGC_DS20762,Urinary Tract Infections | urinary tract infec...,BMGC_DS09400,infiltrating renal pelvis/ureter urothelial ca...,BMGC_DS08797,inverted urothelial papilloma | urinary tract ...,BMGC_DS08797,inverted urothelial papilloma | urinary tract ...,BMGC_DS09400,infiltrating renal pelvis/ureter urothelial ca...
92,ACH-000995,JURKAT,primary,lymphoid_neoplasm,T-cell_ALL,Leukemia:ALL,ALL,CVCL_0065,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...,BMGC_DS20794,Precursor B-Cell Lymphoblastic Leukemia-Lymphoma,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...
93,ACH-001182,RPMI 6666,primary,lymphoblastoid,lymphoma_Hodgkin,NaN,NaN,CVCL_1665,BMGC_DS20405,Hodgkin Disease | Hodgkins lymphoma,...,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20323,Fever,BMGC_DS22241,NaN,BMGC_DS20323,Fever,BMGC_DS22241,NaN


BMGC_Disease_ID      0
BMGC_Disease_name    0
dtype: int64


,depMapID,Name,BMGC_Disease_ID,BMGC_Disease_name,Pathology,Histology,type,PATHOLOGIST_ANNOTATION,tcga_code,id,...,depmap_match_2_disease,depmap_match_2_disease_name,ncit_match_1_disease,ncit_match_1_disease_name,ncit_match_2_disease,ncit_match_2_disease_name,ordo_match_1_disease,ordo_match_1_disease_name,ordo_match_2_disease,ordo_match_2_disease_name
0,ACH-000001,NIH:OVCAR-3,BMGC_DS07934,ovarian adenocarcinoma | ovary adenocarcinoma,metastasis,carcinoma,ovary,Ovary:Carcinoma,OV,CVCL_0465,...,BMGC_DS08982,ovarian transitional cell carcinoma | ovary tr...,BMGC_DS08979,ovarian serous surface papillary adenocarcinom...,BMGC_DS09417,peritoneal serous adenocarcinoma | primary per...,BMGC_DS20323,Fever,BMGC_DS22241,NaN
1,ACH-000002,HL-60,BMGC_DS01407,acute myeloid leukemia,primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACH-000004,HEL,BMGC_DS16646,erythroleukemia,primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACH-000006,MONO-MAC-6,BMGC_DS20800,"leukemia, myeloid, acute",primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML,CVCL_1426,...,BMGC_DS20480,"Leukemia, Myeloid",BMGC_DS05322,T-cell adult acute lymphocytic leukemia,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...
4,ACH-000007,LS513,BMGC_DS08517,cecum adenocarcinoma,primary,carcinoma,colorectal,Colorectal:Carcinoma,COAD/READ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,ACH-001199,SNU-C2B,BMGC_DS08517,cecum adenocarcinoma,primary,carcinoma,colorectal,NaN,COAD/READ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,ACH-001203,SUP-HD1,BMGC_DS20405,hodgkin disease | hodgkins lymphoma,primary,lymphoid_neoplasm,lymphoma_Hodgkin,NaN,NaN,CVCL_2208,...,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20405,Hodgkin Disease | Hodgkins lymphoma,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20405,Hodgkin Disease | Hodgkins lymphoma,BMGC_DS20499,"Lymphoma, Non-Hodgkin"
646,ACH-001233,VMRC-LCD,BMGC_DS02944,lung adenocarcinoma,primary,carcinoma,lung_NSC,NaN,LUAD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,ACH-001306,8305C,BMGC_DS21147,"thyroid carcinoma, anaplastic",primary,carcinoma,thyroid,Thyroid:Carcinoma,THCA,CVCL_1053,...,BMGC_DS21147,"Thyroid Carcinoma, Anaplastic",BMGC_DS04082,follicular thyroid carcinoma | thyroid gland f...,BMGC_DS03290,malignant tumor of parathyroid gland | parathy...,BMGC_DS03290,malignant tumor of parathyroid gland | parathy...,BMGC_DS04082,follicular thyroid carcinoma | thyroid gland f...


## 8. CRISPR/RNAi Biomarkers

In [361]:
os.makedirs('./data/pretrain_plain_data', exist_ok=True)
os.makedirs('./data/pretrain_status_data', exist_ok=True)

In [362]:
# Create the name and desc files
bmgc_promoter_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_gene_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_transcript_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_protein_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])

bmgc_omics_name_id_tmp_df = pd.concat([bmgc_promoter_name_id_df, bmgc_gene_name_id_df, bmgc_transcript_name_id_df, bmgc_protein_name_id_df], axis=0).reset_index(drop=True)
bmgc_omics_name_id_df = pd.merge(bmgc_omics_df[['BioMedGraphica_Conn_ID']], bmgc_omics_name_id_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
# check the null values in the bmgc_omics_name_id_df
print("Null values in bmgc_omics_name_id_df:")
print(bmgc_omics_name_id_df.isnull().sum())
# fill the NaN values in the Names_and_IDs column with empty string
bmgc_omics_name_id_df['Names_and_IDs'] = bmgc_omics_name_id_df['Names_and_IDs'].fillna(' ')
# recheck the null values in the bmgc_omics_name_id_dff
print("Null values in bmgc_omics_name_id_df:")
print(bmgc_omics_name_id_df.isnull().sum())
display(bmgc_omics_name_id_df)
bmgc_omics_name_id_df.to_csv('./data/pretrain_plain_data/bmgc_omics_name.csv', index=False)
bmgc_omics_name_id_df.to_csv('./data/pretrain_status_data/bmgc_omics_name.csv', index=False)

Null values in bmgc_omics_name_id_df:
BioMedGraphica_Conn_ID    0
Names_and_IDs             0
dtype: int64
Null values in bmgc_omics_name_id_df:
BioMedGraphica_Conn_ID    0
Names_and_IDs             0
dtype: int64


,BioMedGraphica_Conn_ID,Names_and_IDs
0,BMGC_PM00001,NCBI Gene ID:1 | HGNC:5 | OMIM ID:138670 | Ref...
1,BMGC_PM00002,N-acetyltransferase 2 Promoter | RefSeq ID:NG_...
2,BMGC_PM00003,RefSeq ID:NC_000020.11 | OMIM ID:608958 | NCBI...
3,BMGC_PM00004,NCBI Gene ID:1000 | HGNC:1759 | RefSeq ID:NC_0...
4,BMGC_PM00005,RefSeq ID:NG_029764.2 | RefSeq ID:NT_187519.1 ...
...,...,...
705929,BMGC_PT121415,ENSP00000520928.1 | ENSP00000520928 | NCBI Gen...
705930,BMGC_PT121416,RefSeq ID:NP_001366093 | NCBI Gene ID:1438.0 |...
705931,BMGC_PT121417,NCBI Gene ID:1438.0 | CSF2RA | RefSeq ID:NP_00...
705932,BMGC_PT121418,GTPBP6 | ENSP00000520931.1 | NCBI Gene ID:8225...


In [363]:
bmgc_promoter_desc_df = bmgc_promoter_name_id_df.drop(columns=['Names_and_IDs'], axis=1).copy()
bmgc_promoter_desc_df['Description'] = np.nan # add the Description column to bmgc_promoter_name_df with NaN values
bmgc_gene_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'], axis=1)
bmgc_transcript_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'], axis=1)
bmgc_protein_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'], axis=1)

# concat and drop BioMedGraphica_ID column
bmgc_omics_desc_tmp_df = pd.concat([bmgc_promoter_desc_df, bmgc_gene_desc_df, bmgc_transcript_desc_df, bmgc_protein_desc_df], axis=0).reset_index(drop=True)
# check the null values in the bmgc_omics_desc_tmp_df
print(bmgc_omics_desc_tmp_df.isnull().sum())
bmgc_omics_desc_df = pd.merge(bmgc_omics_df[['BioMedGraphica_Conn_ID']], bmgc_omics_desc_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
# check the null values in the bmgc_omics_desc_df
print(bmgc_omics_desc_df.isnull().sum())
# fill NaN values with empty string in Description column
bmgc_omics_desc_df['Description'] = bmgc_omics_desc_df['Description'].fillna(' ')
# recheck the null values in the bmgc_omics_desc_df
print(bmgc_omics_desc_df.isnull().sum())
display(bmgc_omics_desc_df)

bmgc_omics_desc_df.to_csv('./data/pretrain_plain_data/bmgc_omics_desc.csv', index=False)
bmgc_omics_desc_df.to_csv('./data/pretrain_status_data/bmgc_omics_desc.csv', index=False)

BioMedGraphica_Conn_ID         0
Description               193184
dtype: int64
BioMedGraphica_Conn_ID         0
Description               193184
dtype: int64
BioMedGraphica_Conn_ID    0
Description               0
dtype: int64


,BioMedGraphica_Conn_ID,Description
0,BMGC_PM00001,
1,BMGC_PM00002,
2,BMGC_PM00003,
3,BMGC_PM00004,
4,BMGC_PM00005,
...,...,...
705929,BMGC_PT121415,
705930,BMGC_PT121416,
705931,BMGC_PT121417,
705932,BMGC_PT121418,


### 8.0 Entity markers

In [364]:
# translation chain converging to the same node
# fetch the promoter, gene, transcript and protein entity alone
promoter_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Promoter'].copy()
gene_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Gene'].copy()
transcript_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Transcript'].copy()
protein_entity_df = bmgc_omics_df[bmgc_omics_df['Type'] == 'Protein'].copy()

display(bmgc_omics_relation_df)
# recheck the null values in bmgc_omics_relation_df
print("Null values in bmgc_omics_relation_df:")
print(bmgc_omics_relation_df.isnull().sum())

# fetch the Promoter-Gene, Gene-Transcript, Transcript-Protein relation alone
promoter_gene_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Promoter-Gene'].copy()
gene_transcript_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Gene-Transcript'].copy()
transcript_protein_relation_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Transcript-Protein'].copy()

,BioMedGraphica_Conn_ID,BioMedGraphica_ID,BMGC_From_ID,BMGC_To_ID,BMG_From_ID,BMG_To_ID,Source,Type
0,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMGC_GN00001,BMG_PM000001,BMG_GN000001,NaN,Promoter-Gene
1,BMGC_ED_PMGN00002,BMG_ED_PMGN00002,BMGC_PM00002,BMGC_GN00002,BMG_PM000002,BMG_GN000002,NaN,Promoter-Gene
2,BMGC_ED_PMGN00003,BMG_ED_PMGN00003,BMGC_PM00003,BMGC_GN00003,BMG_PM000003,BMG_GN000003,NaN,Promoter-Gene
3,BMGC_ED_PMGN00004,BMG_ED_PMGN00004,BMGC_PM00004,BMGC_GN00004,BMG_PM000004,BMG_GN000004,NaN,Promoter-Gene
4,BMGC_ED_PMGN00005,BMG_ED_PMGN00005,BMGC_PM00005,BMGC_GN00005,BMG_PM000005,BMG_GN000005,NaN,Promoter-Gene
...,...,...,...,...,...,...,...,...
17151448,BMGC_ED_PTPT16484816,BMG_ED_PTPT16484816,BMGC_PT121419,BMGC_PT117260,BMG_PT173978,BMG_PT169819,KEGG,Protein-Protein
17151449,BMGC_ED_PTPT16484817,BMG_ED_PTPT16484817,BMGC_PT121419,BMGC_PT120142,BMG_PT173978,BMG_PT172701,KEGG,Protein-Protein
17151450,BMGC_ED_PTPT16484818,BMG_ED_PTPT16484818,BMGC_PT121419,BMGC_PT120825,BMG_PT173978,BMG_PT173384,KEGG,Protein-Protein
17151451,BMGC_ED_PTPT16484819,BMG_ED_PTPT16484819,BMGC_PT121419,BMGC_PT120826,BMG_PT173978,BMG_PT173385,KEGG,Protein-Protein


Null values in bmgc_omics_relation_df:
BioMedGraphica_Conn_ID        0
BioMedGraphica_ID             0
BMGC_From_ID                  0
BMGC_To_ID                    0
BMG_From_ID                   0
BMG_To_ID                     0
Source                    86238
Type                          0
dtype: int64


In [365]:
gene_transcript_entity_df = pd.merge(gene_entity_df, gene_transcript_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BioMedGraphica_Conn_ID', right_on='BMGC_From_ID', how='outer')
gene_transcript_protein_entity_df = pd.merge(gene_transcript_entity_df, transcript_protein_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BMGC_To_ID', right_on='BMGC_From_ID', how='outer')
# drop NaN values in BMGC_From_ID_x	BMGC_To_ID_x BMGC_From_ID_y	BMGC_To_ID_y
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.dropna(subset=['BMGC_From_ID_x', 'BMGC_To_ID_x', 'BMGC_From_ID_y', 'BMGC_To_ID_y']).reset_index(drop=True)
# keep the columns ['BioMedGraphica_Conn_ID', 'BMGC_To_ID_x', 'BMGC_To_ID_y'] and rename the columns to ['BMGC_GN_ID', 'BMGC_TS_ID', 'BMGC_PT_ID']
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df[['BioMedGraphica_Conn_ID', 'BMGC_To_ID_y']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID', 'BMGC_To_ID_y': 'BMGC_PT_ID'}).sort_values(by='BMGC_GN_ID').reset_index(drop=True)
# drop duplicates rows in gene_transcript_protein_entity_df
gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.drop_duplicates().reset_index(drop=True)
display(gene_transcript_protein_entity_df)
# just copy gene_transcript_protein_entity_df as promoter_gene_transcript_protein_entity_df
bmgc_promoter_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
bmgc_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
promoter_gene_transcript_protein_entity_df = gene_transcript_protein_entity_df.copy()
promoter_gene_df = pd.concat([bmgc_promoter_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PM_ID'}), bmgc_gene_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_GN_ID'})], axis=1)
promoter_protein_entity_df = pd.merge(promoter_gene_transcript_protein_entity_df, promoter_gene_df, left_on='BMGC_GN_ID', right_on='BMGC_GN_ID', how='left').drop(columns=['BMGC_GN_ID'])
promoter_protein_entity_df = promoter_protein_entity_df[['BMGC_PM_ID', 'BMGC_PT_ID']]
display(promoter_protein_entity_df)
transcript_protein_entity_df = pd.merge(transcript_entity_df, transcript_protein_relation_df[['BMGC_From_ID', 'BMGC_To_ID']], left_on='BioMedGraphica_Conn_ID', right_on='BMGC_From_ID', how='outer')
# drop NaN rows in the BMGC_From_ID and	BMGC_To_ID
transcript_protein_entity_df = transcript_protein_entity_df.dropna(subset=['BMGC_From_ID', 'BMGC_To_ID']).reset_index(drop=True)
# keep the columns ['BioMedGraphica_Conn_ID', 'BMGC_To_ID'] and rename the columns to ['BMGC_TS_ID', 'BMGC_PT_ID']
transcript_protein_entity_df = transcript_protein_entity_df[['BioMedGraphica_Conn_ID', 'BMGC_To_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_TS_ID', 'BMGC_To_ID': 'BMGC_PT_ID'}).sort_values(by='BMGC_TS_ID').reset_index(drop=True)
# drop duplicates rows in transcript_protein_entity_df
transcript_protein_entity_df = transcript_protein_entity_df.drop_duplicates().reset_index(drop=True)
display(transcript_protein_entity_df)

,BMGC_GN_ID,BMGC_PT_ID
0,BMGC_GN00001,BMGC_PT013542
1,BMGC_GN00001,BMGC_PT091702
2,BMGC_GN00002,BMGC_PT070222
3,BMGC_GN00002,BMGC_PT041299
4,BMGC_GN00002,BMGC_PT038889
...,...,...
126009,BMGC_GN70323,BMGC_PT121177
126010,BMGC_GN70327,BMGC_PT121280
126011,BMGC_GN70329,BMGC_PT121282
126012,BMGC_GN76774,BMGC_PT007612


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4082382977.py:11: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmgc_promoter_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter.csv')
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4082382977.py:12: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmgc_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')


,BMGC_PM_ID,BMGC_PT_ID
0,BMGC_PM00001,BMGC_PT013542
1,BMGC_PM00001,BMGC_PT091702
2,BMGC_PM00002,BMGC_PT070222
3,BMGC_PM00002,BMGC_PT041299
4,BMGC_PM00002,BMGC_PT038889
...,...,...
126009,BMGC_PM70323,BMGC_PT121177
126010,BMGC_PM70327,BMGC_PT121280
126011,BMGC_PM70329,BMGC_PT121282
126012,BMGC_PM76774,BMGC_PT007612


,BMGC_TS_ID,BMGC_PT_ID
0,BMGC_TS000001,BMGC_PT017571
1,BMGC_TS000002,BMGC_PT014978
2,BMGC_TS000003,BMGC_PT014403
3,BMGC_TS000004,BMGC_PT017807
4,BMGC_TS000005,BMGC_PT043262
...,...,...
152580,BMGC_TS411885,BMGC_PT121415
152581,BMGC_TS411886,BMGC_PT121416
152582,BMGC_TS411887,BMGC_PT121417
152583,BMGC_TS412018,BMGC_PT121418


In [366]:
# keep the columns ['BioMedGraphica_Conn_ID'] and rename the columns to ['BMGC_PT_ID']
only_protein_entity_df = protein_entity_df[['BioMedGraphica_Conn_ID']].rename(columns={'BioMedGraphica_Conn_ID': 'BMGC_PT_ID'}).sort_values(by='BMGC_PT_ID').reset_index(drop=True)
display(only_protein_entity_df)

,BMGC_PT_ID
0,BMGC_PT000001
1,BMGC_PT000002
2,BMGC_PT000003
3,BMGC_PT000004
4,BMGC_PT000005
...,...
121414,BMGC_PT121415
121415,BMGC_PT121416
121416,BMGC_PT121417
121417,BMGC_PT121418


### 8.1 CRISPR top 100 gene entities

In [367]:
import pandas as pd
raw_crispr_df = pd.read_csv('./data/raw_data/CRISPRGeneEffect.csv')
display(raw_crispr_df)

,Unnamed: 0,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.121964,0.042647,0.026102,-0.147614,-0.020425,0.184014,-0.076020,-0.097068,0.182759,...,-0.120536,0.107286,0.166390,0.262275,0.163876,-0.249508,0.053725,0.067986,0.136285,-0.438536
1,ACH-000004,-0.016484,-0.088500,-0.088198,-0.041225,-0.028074,-0.157206,0.280101,-0.106904,0.178125,...,-0.192527,-0.324059,0.230377,0.087609,0.074897,0.054335,-0.330343,0.099067,0.274566,0.001871
2,ACH-000005,-0.184847,0.003300,0.160881,0.086224,-0.149315,-0.253837,0.167011,-0.101209,-0.129827,...,-0.312827,-0.338023,-0.039700,-0.055349,-0.000367,-0.205605,-0.066032,-0.054518,0.035579,-0.150486
3,ACH-000007,-0.071921,-0.113717,0.082872,0.099633,-0.008378,-0.022310,0.014416,-0.184977,-0.173739,...,-0.334843,-0.355499,-0.014183,0.230944,0.044628,-0.081863,-0.390748,-0.036547,-0.273129,-0.382723
4,ACH-000009,-0.019163,-0.134669,0.060323,0.076647,0.078922,-0.100243,0.047559,-0.136988,0.037759,...,-0.299593,-0.194427,-0.027365,0.236591,-0.084224,-0.098271,-0.510495,0.052938,0.018623,-0.258353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,ACH-002926,-0.107883,-0.101501,0.046677,0.096868,-0.118650,-0.065258,-0.064944,-0.175912,-0.044707,...,-0.409541,-0.579376,0.038706,0.338610,-0.174379,-0.040264,-0.239763,0.065168,-0.144088,-0.334424
1174,ACH-002928,-0.287606,-0.179101,-0.135824,-0.046579,-0.249888,-0.097376,0.070385,0.049444,-0.244937,...,-0.095328,-0.440452,0.303500,0.097690,0.151790,0.039994,0.098240,-0.266654,-0.115108,-0.080421
1175,ACH-003012,-0.145878,-0.004000,0.091933,0.191332,-0.065385,-0.134152,0.012353,-0.321012,0.001571,...,-0.280119,-0.662673,-0.180936,0.169997,-0.036326,0.158702,-0.213281,-0.017884,-0.195395,-0.309021
1176,ACH-003177,-0.038435,-0.055660,0.018989,0.117182,0.063397,0.049993,0.175520,-0.085752,0.078435,...,-0.080700,-0.476797,-0.158375,-0.008999,-0.030058,-0.103513,-0.119234,-0.153744,0.032584,-0.536071


In [368]:
# get the column names of the raw_crispr_df aside from the first column and convert this to a list
raw_crispr_df_columns = raw_crispr_df.columns[1:].tolist()
print(raw_crispr_df_columns)

['A1BG (1)', 'A1CF (29974)', 'A2M (2)', 'A2ML1 (144568)', 'A3GALT2 (127550)', 'A4GALT (53947)', 'A4GNT (51146)', 'AAAS (8086)', 'AACS (65985)', 'AADAC (13)', 'AADACL2 (344752)', 'AADACL3 (126767)', 'AADACL4 (343066)', 'AADAT (51166)', 'AAGAB (79719)', 'AAK1 (22848)', 'AAMDC (28971)', 'AAMP (14)', 'AANAT (15)', 'AAR2 (25980)', 'AARD (441376)', 'AARS1 (16)', 'AARS2 (57505)', 'AARSD1 (80755)', 'AASDH (132949)', 'AASDHPPT (60496)', 'AASS (10157)', 'AATF (26574)', 'AATK (9625)', 'ABAT (18)', 'ABCA1 (19)', 'ABCA10 (10349)', 'ABCA12 (26154)', 'ABCA13 (154664)', 'ABCA2 (20)', 'ABCA3 (21)', 'ABCA4 (24)', 'ABCA5 (23461)', 'ABCA6 (23460)', 'ABCA7 (10347)', 'ABCA8 (10351)', 'ABCA9 (10350)', 'ABCB1 (5243)', 'ABCB10 (23456)', 'ABCB11 (8647)', 'ABCB4 (5244)', 'ABCB5 (340273)', 'ABCB6 (10058)', 'ABCB7 (22)', 'ABCB8 (11194)', 'ABCB9 (23457)', 'ABCC1 (4363)', 'ABCC10 (89845)', 'ABCC11 (85320)', 'ABCC12 (94160)', 'ABCC2 (1244)', 'ABCC3 (8714)', 'ABCC4 (10257)', 'ABCC5 (10057)', 'ABCC6 (368)', 'ABCC8 (683

In [369]:
# First, set the "Unnamed: 0" column as the index:
raw_crispr_t_df = raw_crispr_df.set_index("Unnamed: 0")
# Then transpose the DataFrame:
raw_crispr_t_df = raw_crispr_t_df.transpose()
# move the index to a column and rename it as the gene_name
raw_crispr_t_df.reset_index(inplace=True)
raw_crispr_t_df.rename(columns={'index': 'gene_name'}, inplace=True)
# column gene_name should be kept with the content by removing "()" and remove and " " content
raw_crispr_t_df['gene_name'] = raw_crispr_t_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Optionally, if you want to view the result:
display(raw_crispr_t_df)

Unnamed: 0,gene_name,ACH-000001,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,ACH-000014,...,ACH-002806,ACH-002834,ACH-002847,ACH-002922,ACH-002925,ACH-002926,ACH-002928,ACH-003012,ACH-003177,ACH-003180
0,A1BG,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,-0.071322,...,-0.051500,-0.072687,-0.057531,-0.050743,-0.163957,-0.107883,-0.287606,-0.145878,-0.038435,-0.007314
1,A1CF,0.042647,-0.088500,0.003300,-0.113717,-0.134669,-0.053814,-0.024648,-0.139422,-0.162024,...,-0.117224,-0.107839,-0.120021,-0.003851,-0.042068,-0.101501,-0.179101,-0.004000,-0.055660,-0.212478
2,A2M,0.026102,-0.088198,0.160881,0.082872,0.060323,0.065476,0.034448,-0.039279,0.109822,...,0.002730,0.034674,0.005217,-0.029880,0.118830,0.046677,-0.135824,0.091933,0.018989,0.129900
3,A2ML1,-0.147614,-0.041225,0.086224,0.099633,0.076647,0.222308,0.060509,0.021942,0.076680,...,0.070590,0.000908,0.027493,0.243014,-0.042774,0.096868,-0.046579,0.191332,0.117182,-0.023112
4,A3GALT2,-0.020425,-0.028074,-0.149315,-0.008378,0.078922,-0.119423,0.046351,-0.089072,-0.013627,...,-0.152683,-0.187834,-0.108456,-0.197960,-0.090302,-0.118650,-0.249888,-0.065385,0.063397,-0.068613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17911,ZYG11A,-0.249508,0.054335,-0.205605,-0.081863,-0.098271,0.031686,0.028423,-0.099067,-0.015455,...,-0.033863,0.011158,-0.091727,-0.095130,-0.046472,-0.040264,0.039994,0.158702,-0.103513,0.040616
17912,ZYG11B,0.053725,-0.330343,-0.066032,-0.390748,-0.510495,-0.200174,-0.059623,-0.062914,-0.117128,...,-0.135263,0.096450,-0.186492,-0.250546,-0.267548,-0.239763,0.098240,-0.213281,-0.119234,-0.378505
17913,ZYX,0.067986,0.099067,-0.054518,-0.036547,0.052938,0.058936,-0.114437,-0.067185,-0.136238,...,-0.202681,-0.311926,-0.188271,-0.115048,-0.240009,0.065168,-0.266654,-0.017884,-0.153744,-0.143920
17914,ZZEF1,0.136285,0.274566,0.035579,-0.273129,0.018623,-0.186168,-0.117129,-0.111276,0.222765,...,-0.087873,-0.094481,-0.148179,-0.032583,-0.041099,-0.144088,-0.115108,-0.195395,0.032584,-0.039985


In [370]:
# Identify columns to convert (all except 'gene_name')
cols_to_convert = raw_crispr_t_df.columns.difference(['gene_name'])
# Convert these columns to numeric (using pd.to_numeric for safety, which can handle errors)
raw_crispr_t_df[cols_to_convert] = raw_crispr_t_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
raw_crispr_t_df = raw_crispr_t_df.fillna(0.0)
display(raw_crispr_t_df)
# groupby the gene_name and calculate the mean
crispr_df = raw_crispr_t_df.groupby('gene_name', as_index=False).mean()
display(crispr_df)

Unnamed: 0,gene_name,ACH-000001,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,ACH-000014,...,ACH-002806,ACH-002834,ACH-002847,ACH-002922,ACH-002925,ACH-002926,ACH-002928,ACH-003012,ACH-003177,ACH-003180
0,A1BG,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,-0.071322,...,-0.051500,-0.072687,-0.057531,-0.050743,-0.163957,-0.107883,-0.287606,-0.145878,-0.038435,-0.007314
1,A1CF,0.042647,-0.088500,0.003300,-0.113717,-0.134669,-0.053814,-0.024648,-0.139422,-0.162024,...,-0.117224,-0.107839,-0.120021,-0.003851,-0.042068,-0.101501,-0.179101,-0.004000,-0.055660,-0.212478
2,A2M,0.026102,-0.088198,0.160881,0.082872,0.060323,0.065476,0.034448,-0.039279,0.109822,...,0.002730,0.034674,0.005217,-0.029880,0.118830,0.046677,-0.135824,0.091933,0.018989,0.129900
3,A2ML1,-0.147614,-0.041225,0.086224,0.099633,0.076647,0.222308,0.060509,0.021942,0.076680,...,0.070590,0.000908,0.027493,0.243014,-0.042774,0.096868,-0.046579,0.191332,0.117182,-0.023112
4,A3GALT2,-0.020425,-0.028074,-0.149315,-0.008378,0.078922,-0.119423,0.046351,-0.089072,-0.013627,...,-0.152683,-0.187834,-0.108456,-0.197960,-0.090302,-0.118650,-0.249888,-0.065385,0.063397,-0.068613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17911,ZYG11A,-0.249508,0.054335,-0.205605,-0.081863,-0.098271,0.031686,0.028423,-0.099067,-0.015455,...,-0.033863,0.011158,-0.091727,-0.095130,-0.046472,-0.040264,0.039994,0.158702,-0.103513,0.040616
17912,ZYG11B,0.053725,-0.330343,-0.066032,-0.390748,-0.510495,-0.200174,-0.059623,-0.062914,-0.117128,...,-0.135263,0.096450,-0.186492,-0.250546,-0.267548,-0.239763,0.098240,-0.213281,-0.119234,-0.378505
17913,ZYX,0.067986,0.099067,-0.054518,-0.036547,0.052938,0.058936,-0.114437,-0.067185,-0.136238,...,-0.202681,-0.311926,-0.188271,-0.115048,-0.240009,0.065168,-0.266654,-0.017884,-0.153744,-0.143920
17914,ZZEF1,0.136285,0.274566,0.035579,-0.273129,0.018623,-0.186168,-0.117129,-0.111276,0.222765,...,-0.087873,-0.094481,-0.148179,-0.032583,-0.041099,-0.144088,-0.115108,-0.195395,0.032584,-0.039985


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\1869562081.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  crispr_df = raw_crispr_t_df.groupby('gene_name', as_index=False).mean()


Unnamed: 0,gene_name,ACH-000001,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,ACH-000014,...,ACH-002806,ACH-002834,ACH-002847,ACH-002922,ACH-002925,ACH-002926,ACH-002928,ACH-003012,ACH-003177,ACH-003180
0,A1BG,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,-0.071322,...,-0.051500,-0.072687,-0.057531,-0.050743,-0.163957,-0.107883,-0.287606,-0.145878,-0.038435,-0.007314
1,A1CF,0.042647,-0.088500,0.003300,-0.113717,-0.134669,-0.053814,-0.024648,-0.139422,-0.162024,...,-0.117224,-0.107839,-0.120021,-0.003851,-0.042068,-0.101501,-0.179101,-0.004000,-0.055660,-0.212478
2,A2M,0.026102,-0.088198,0.160881,0.082872,0.060323,0.065476,0.034448,-0.039279,0.109822,...,0.002730,0.034674,0.005217,-0.029880,0.118830,0.046677,-0.135824,0.091933,0.018989,0.129900
3,A2ML1,-0.147614,-0.041225,0.086224,0.099633,0.076647,0.222308,0.060509,0.021942,0.076680,...,0.070590,0.000908,0.027493,0.243014,-0.042774,0.096868,-0.046579,0.191332,0.117182,-0.023112
4,A3GALT2,-0.020425,-0.028074,-0.149315,-0.008378,0.078922,-0.119423,0.046351,-0.089072,-0.013627,...,-0.152683,-0.187834,-0.108456,-0.197960,-0.090302,-0.118650,-0.249888,-0.065385,0.063397,-0.068613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17911,ZYG11A,-0.249508,0.054335,-0.205605,-0.081863,-0.098271,0.031686,0.028423,-0.099067,-0.015455,...,-0.033863,0.011158,-0.091727,-0.095130,-0.046472,-0.040264,0.039994,0.158702,-0.103513,0.040616
17912,ZYG11B,0.053725,-0.330343,-0.066032,-0.390748,-0.510495,-0.200174,-0.059623,-0.062914,-0.117128,...,-0.135263,0.096450,-0.186492,-0.250546,-0.267548,-0.239763,0.098240,-0.213281,-0.119234,-0.378505
17913,ZYX,0.067986,0.099067,-0.054518,-0.036547,0.052938,0.058936,-0.114437,-0.067185,-0.136238,...,-0.202681,-0.311926,-0.188271,-0.115048,-0.240009,0.065168,-0.266654,-0.017884,-0.153744,-0.143920
17914,ZZEF1,0.136285,0.274566,0.035579,-0.273129,0.018623,-0.186168,-0.117129,-0.111276,0.222765,...,-0.087873,-0.094481,-0.148179,-0.032583,-0.041099,-0.144088,-0.115108,-0.195395,0.032584,-0.039985


In [371]:
bmg_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_gene_df = bmg_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_gene_df)
# merge the biomedgraphica_id with the crispr_df
merged_crispr_df = pd.merge(bmg_gene_df, crispr_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merged_crispr_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merged_crispr_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\2142238278.py:1: DtypeWarning: Columns (4,5,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_gene_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene.csv')


,BioMedGraphica_Conn_ID,HGNC_Symbol
0,BMGC_GN00001,A1BG
1,BMGC_GN00002,NAT2
2,BMGC_GN00003,ADA
3,BMGC_GN00004,CDH2
4,BMGC_GN00005,AKT3
...,...,...
86233,BMGC_GN86234,NaN
86234,BMGC_GN86235,NaN
86235,BMGC_GN86236,NaN
86236,BMGC_GN86237,NaN


,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,ACH-000014,...,ACH-002806,ACH-002834,ACH-002847,ACH-002922,ACH-002925,ACH-002926,ACH-002928,ACH-003012,ACH-003177,ACH-003180
0,BMGC_GN00001,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,-0.071322,...,-0.051500,-0.072687,-0.057531,-0.050743,-0.163957,-0.107883,-0.287606,-0.145878,-0.038435,-0.007314
1,BMGC_GN00002,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,0.300938,...,0.131496,0.165664,0.156277,0.198406,0.209907,0.192162,0.068782,0.143209,0.074829,-0.262441
2,BMGC_GN00003,-0.116327,-0.200383,-0.211542,-0.120459,-0.100498,0.000700,0.084591,0.025797,-0.141183,...,-0.155286,-0.027161,-0.107837,-0.023697,-0.347253,-0.092996,-0.246541,-0.065361,0.092299,-0.089303
3,BMGC_GN00004,-0.019295,0.099193,-0.057229,-0.213060,-0.100742,-0.558269,-0.027933,0.033106,-0.120098,...,-0.303145,-0.125616,-0.520756,-0.194251,0.065170,-0.098423,0.167748,0.053459,-0.204036,-0.120517
4,BMGC_GN00005,-0.100900,0.151491,0.293747,0.191130,0.025839,0.127902,0.055322,0.074227,-0.331538,...,0.079258,0.102221,0.036739,0.099489,0.121007,0.165328,-0.097967,0.239757,0.193184,0.077518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20283,BMGC_GN69808,0.094246,0.021657,0.196685,0.392400,0.293312,0.404930,0.124325,0.173397,-0.021022,...,0.102379,0.197394,0.081715,0.114349,0.180772,0.017811,0.291685,0.200229,0.209375,0.096925
20284,BMGC_GN69841,-0.137149,-0.601468,-0.147378,-0.320952,0.022595,-0.610531,-0.075869,-0.091422,-0.258779,...,-0.100619,-0.022613,0.084076,-0.180020,-0.017108,-0.325764,0.144469,0.009504,-0.118627,-0.348614
20285,BMGC_GN69848,0.000000,0.333310,-0.151258,0.090807,0.130973,0.035430,0.057994,0.127841,-0.073217,...,-0.097166,-0.125821,0.056233,-0.057404,0.140328,0.182520,0.000000,0.198007,0.000000,0.000000
20286,BMGC_GN69876,0.000000,-0.316461,-0.021494,-0.556799,-0.350030,-0.136289,-0.579060,-0.200260,-0.171833,...,-0.288444,-0.376535,-0.083581,-0.059077,-0.246367,-0.165150,0.000000,-0.199642,0.000000,0.000000


In [372]:
# map the gene entity with gene_transcript_protein_entity_df
protein_crispr_df = pd.merge(gene_transcript_protein_entity_df, merged_crispr_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
# and drop nan in any rows from protein_crispr_df
protein_crispr_df = protein_crispr_df.dropna().reset_index(drop=True)
display(protein_crispr_df)

,BMGC_GN_ID,BMGC_PT_ID,ACH-000001,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,...,ACH-002806,ACH-002834,ACH-002847,ACH-002922,ACH-002925,ACH-002926,ACH-002928,ACH-003012,ACH-003177,ACH-003180
0,BMGC_GN00001,BMGC_PT013542,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,...,-0.051500,-0.072687,-0.057531,-0.050743,-0.163957,-0.107883,-0.287606,-0.145878,-0.038435,-0.007314
1,BMGC_GN00001,BMGC_PT091702,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,...,-0.051500,-0.072687,-0.057531,-0.050743,-0.163957,-0.107883,-0.287606,-0.145878,-0.038435,-0.007314
2,BMGC_GN00002,BMGC_PT070222,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,...,0.131496,0.165664,0.156277,0.198406,0.209907,0.192162,0.068782,0.143209,0.074829,-0.262441
3,BMGC_GN00002,BMGC_PT041299,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,...,0.131496,0.165664,0.156277,0.198406,0.209907,0.192162,0.068782,0.143209,0.074829,-0.262441
4,BMGC_GN00002,BMGC_PT038889,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,...,0.131496,0.165664,0.156277,0.198406,0.209907,0.192162,0.068782,0.143209,0.074829,-0.262441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117025,BMGC_GN69841,BMGC_PT120188,-0.137149,-0.601468,-0.147378,-0.320952,0.022595,-0.610531,-0.075869,-0.091422,...,-0.100619,-0.022613,0.084076,-0.180020,-0.017108,-0.325764,0.144469,0.009504,-0.118627,-0.348614
117026,BMGC_GN69841,BMGC_PT120187,-0.137149,-0.601468,-0.147378,-0.320952,0.022595,-0.610531,-0.075869,-0.091422,...,-0.100619,-0.022613,0.084076,-0.180020,-0.017108,-0.325764,0.144469,0.009504,-0.118627,-0.348614
117027,BMGC_GN69848,BMGC_PT104310,0.000000,0.333310,-0.151258,0.090807,0.130973,0.035430,0.057994,0.127841,...,-0.097166,-0.125821,0.056233,-0.057404,0.140328,0.182520,0.000000,0.198007,0.000000,0.000000
117028,BMGC_GN69876,BMGC_PT120255,0.000000,-0.316461,-0.021494,-0.556799,-0.350030,-0.136289,-0.579060,-0.200260,...,-0.288444,-0.376535,-0.083581,-0.059077,-0.246367,-0.165150,0.000000,-0.199642,0.000000,0.000000


In [373]:
overlapped_omics_union_annotation_samples

['ACH-000001',
 'ACH-000002',
 'ACH-000004',
 'ACH-000005',
 'ACH-000006',
 'ACH-000007',
 'ACH-000008',
 'ACH-000009',
 'ACH-000011',
 'ACH-000012',
 'ACH-000013',
 'ACH-000014',
 'ACH-000015',
 'ACH-000016',
 'ACH-000017',
 'ACH-000018',
 'ACH-000019',
 'ACH-000020',
 'ACH-000021',
 'ACH-000022',
 'ACH-000023',
 'ACH-000024',
 'ACH-000026',
 'ACH-000027',
 'ACH-000028',
 'ACH-000030',
 'ACH-000031',
 'ACH-000032',
 'ACH-000035',
 'ACH-000037',
 'ACH-000038',
 'ACH-000039',
 'ACH-000040',
 'ACH-000041',
 'ACH-000042',
 'ACH-000043',
 'ACH-000044',
 'ACH-000045',
 'ACH-000046',
 'ACH-000047',
 'ACH-000048',
 'ACH-000050',
 'ACH-000051',
 'ACH-000052',
 'ACH-000053',
 'ACH-000054',
 'ACH-000055',
 'ACH-000056',
 'ACH-000058',
 'ACH-000059',
 'ACH-000060',
 'ACH-000061',
 'ACH-000062',
 'ACH-000063',
 'ACH-000065',
 'ACH-000066',
 'ACH-000067',
 'ACH-000068',
 'ACH-000069',
 'ACH-000070',
 'ACH-000071',
 'ACH-000072',
 'ACH-000073',
 'ACH-000074',
 'ACH-000075',
 'ACH-000076',
 'ACH-0000

In [374]:
# Keep BioMedGraphica_Conn_ID and columns that are in overlapped_omics_union_annotation_samples
# For CRISPR data
columns_to_keep = ['BMGC_GN_ID', 'BMGC_PT_ID'] + [col for col in protein_crispr_df.columns if col in overlapped_omics_union_annotation_samples]
processed_crispr_df = protein_crispr_df[columns_to_keep]
display(processed_crispr_df)
processed_crispr_df.to_csv('./data/pretrain_plain_data/processed_crispr.csv', index=False)
processed_crispr_df.to_csv('./data/pretrain_status_data/processed_crispr.csv', index=False)

,BMGC_GN_ID,BMGC_PT_ID,ACH-000001,ACH-000004,ACH-000005,ACH-000007,ACH-000009,ACH-000011,ACH-000012,ACH-000013,...,ACH-001151,ACH-001183,ACH-001188,ACH-001190,ACH-001194,ACH-001233,ACH-001239,ACH-001306,ACH-001307,ACH-001318
0,BMGC_GN00001,BMGC_PT013542,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,...,-0.102453,-0.054710,-0.110358,-0.042641,-0.009775,-0.121936,-0.080115,-0.031081,0.012357,-0.084400
1,BMGC_GN00001,BMGC_PT091702,-0.121964,-0.016484,-0.184847,-0.071921,-0.019163,0.023031,-0.139496,0.014120,...,-0.102453,-0.054710,-0.110358,-0.042641,-0.009775,-0.121936,-0.080115,-0.031081,0.012357,-0.084400
2,BMGC_GN00002,BMGC_PT070222,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,...,0.198034,0.123040,0.168877,0.185244,0.180496,0.086880,0.303579,0.250509,0.223775,0.184682
3,BMGC_GN00002,BMGC_PT041299,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,...,0.198034,0.123040,0.168877,0.185244,0.180496,0.086880,0.303579,0.250509,0.223775,0.184682
4,BMGC_GN00002,BMGC_PT038889,-0.022127,0.196378,0.359769,0.310169,0.198303,0.113667,0.157864,0.154384,...,0.198034,0.123040,0.168877,0.185244,0.180496,0.086880,0.303579,0.250509,0.223775,0.184682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117025,BMGC_GN69841,BMGC_PT120188,-0.137149,-0.601468,-0.147378,-0.320952,0.022595,-0.610531,-0.075869,-0.091422,...,-0.289655,0.190473,-0.057121,0.105202,0.051115,-0.141930,-0.141818,-0.048947,-0.077387,-0.068879
117026,BMGC_GN69841,BMGC_PT120187,-0.137149,-0.601468,-0.147378,-0.320952,0.022595,-0.610531,-0.075869,-0.091422,...,-0.289655,0.190473,-0.057121,0.105202,0.051115,-0.141930,-0.141818,-0.048947,-0.077387,-0.068879
117027,BMGC_GN69848,BMGC_PT104310,0.000000,0.333310,-0.151258,0.090807,0.130973,0.035430,0.057994,0.127841,...,0.293546,0.275511,-0.006515,0.276861,0.300316,0.000000,0.151090,-0.006914,0.126152,0.291545
117028,BMGC_GN69876,BMGC_PT120255,0.000000,-0.316461,-0.021494,-0.556799,-0.350030,-0.136289,-0.579060,-0.200260,...,-0.202856,-0.170992,-0.407035,-0.067530,-0.153063,0.000000,-0.361657,-0.110224,-0.190182,-0.046852


### 8.2 RNAi top 100 transcript entities

In [375]:
raw_rna_combined_df = pd.read_csv('./data/raw_data/D2_combined_gene_dep_scores.csv')
# fill NaN values with 0.0
raw_rna_combined_df = raw_rna_combined_df.fillna(0.0)
display(raw_rna_combined_df)
# Move the Unnamed: 0 to a column and rename it as the gene_name
raw_rna_combined_df.rename(columns={'Unnamed: 0': 'gene_name'}, inplace=True)
# Clean the gene_name column by removing parentheses and extra spaces
raw_rna_combined_df['gene_name'] = raw_rna_combined_df['gene_name'].apply(lambda x: x.split('(')[0].strip())
# Display the result
display(raw_rna_combined_df)

,Unnamed: 0,127399_SOFT_TISSUE,1321N1_CENTRAL_NERVOUS_SYSTEM,143B_BONE,184A1_BREAST,184B5_BREAST,22RV1_PROSTATE,2313287_STOMACH,600MPE_BREAST,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,...,WM88_SKIN,WM983B_SKIN,WSUDLCL2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YKG1_CENTRAL_NERVOUS_SYSTEM,YMB1_BREAST,ZR751_BREAST,ZR7530_BREAST,ZR75B_BREAST
0,A1BG (1),0.000000,0.000000,0.146042,-0.190388,0.907063,-0.019331,-0.016734,0.091580,0.035023,...,-0.088267,0.002171,0.000000,0.120294,0.012540,0.111530,0.000000,-0.079313,-0.141559,0.214268
1,NAT2 (10),0.000000,0.000000,0.102854,0.384106,0.403192,0.001925,-0.153933,-0.317969,0.012341,...,-0.003747,-0.321445,0.000000,-0.003256,-0.220472,0.073460,0.000000,-0.130921,0.127358,-0.405974
2,ADA (100),0.000000,0.000000,0.168839,-0.120700,0.004394,-0.188700,-0.060818,-0.755058,0.129770,...,-0.014085,0.039679,0.000000,0.076521,0.106995,0.227977,0.000000,-0.134479,0.083506,-0.404291
3,CDH2 (1000),-0.194962,-0.028171,0.063047,-0.237251,-0.017059,-0.103047,-0.049460,0.130107,0.146864,...,-0.073435,-0.140041,-0.154436,-0.040308,-0.078707,0.000769,-0.139126,0.047022,-0.097644,-0.062622
4,AKT3 (10000),-0.256108,0.100751,-0.008077,0.060267,-0.094749,-0.066591,0.166029,0.149969,-0.053022,...,0.028714,-0.054628,0.450581,0.002932,0.129679,-0.072564,0.017161,0.123615,0.046846,0.125711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17304,SLC12A6 (9990),0.000000,0.000000,0.106165,0.227968,0.088744,-0.060731,0.127754,-0.049485,0.381034,...,0.170658,-0.004165,0.000000,0.118544,0.273135,0.141616,0.000000,0.100142,0.160643,0.089555
17305,KCNE2 (9992),0.088853,-0.110628,0.057286,0.028095,0.159467,0.197974,0.048420,-0.008985,-0.092494,...,-0.149664,-0.262675,-0.209937,-0.110529,-0.196629,0.109206,-0.048763,0.128882,-0.179656,-0.455001
17306,DGCR2 (9993),0.000000,0.000000,0.025596,-0.080611,0.014071,0.141779,-0.044246,0.055804,-0.489630,...,0.170620,-0.012766,0.000000,-0.011414,-0.243745,0.153414,0.000000,0.159781,0.286456,0.047184
17307,CASP8AP2 (9994),-0.843295,-0.307031,-0.413669,-1.849696,-0.414154,-0.457898,-0.372166,-0.037574,-0.468437,...,-1.066150,-0.676503,-0.476991,-0.437871,-0.200516,-0.046700,-0.865486,-1.039110,-0.301415,-0.865502


,gene_name,127399_SOFT_TISSUE,1321N1_CENTRAL_NERVOUS_SYSTEM,143B_BONE,184A1_BREAST,184B5_BREAST,22RV1_PROSTATE,2313287_STOMACH,600MPE_BREAST,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,...,WM88_SKIN,WM983B_SKIN,WSUDLCL2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YKG1_CENTRAL_NERVOUS_SYSTEM,YMB1_BREAST,ZR751_BREAST,ZR7530_BREAST,ZR75B_BREAST
0,A1BG,0.000000,0.000000,0.146042,-0.190388,0.907063,-0.019331,-0.016734,0.091580,0.035023,...,-0.088267,0.002171,0.000000,0.120294,0.012540,0.111530,0.000000,-0.079313,-0.141559,0.214268
1,NAT2,0.000000,0.000000,0.102854,0.384106,0.403192,0.001925,-0.153933,-0.317969,0.012341,...,-0.003747,-0.321445,0.000000,-0.003256,-0.220472,0.073460,0.000000,-0.130921,0.127358,-0.405974
2,ADA,0.000000,0.000000,0.168839,-0.120700,0.004394,-0.188700,-0.060818,-0.755058,0.129770,...,-0.014085,0.039679,0.000000,0.076521,0.106995,0.227977,0.000000,-0.134479,0.083506,-0.404291
3,CDH2,-0.194962,-0.028171,0.063047,-0.237251,-0.017059,-0.103047,-0.049460,0.130107,0.146864,...,-0.073435,-0.140041,-0.154436,-0.040308,-0.078707,0.000769,-0.139126,0.047022,-0.097644,-0.062622
4,AKT3,-0.256108,0.100751,-0.008077,0.060267,-0.094749,-0.066591,0.166029,0.149969,-0.053022,...,0.028714,-0.054628,0.450581,0.002932,0.129679,-0.072564,0.017161,0.123615,0.046846,0.125711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17304,SLC12A6,0.000000,0.000000,0.106165,0.227968,0.088744,-0.060731,0.127754,-0.049485,0.381034,...,0.170658,-0.004165,0.000000,0.118544,0.273135,0.141616,0.000000,0.100142,0.160643,0.089555
17305,KCNE2,0.088853,-0.110628,0.057286,0.028095,0.159467,0.197974,0.048420,-0.008985,-0.092494,...,-0.149664,-0.262675,-0.209937,-0.110529,-0.196629,0.109206,-0.048763,0.128882,-0.179656,-0.455001
17306,DGCR2,0.000000,0.000000,0.025596,-0.080611,0.014071,0.141779,-0.044246,0.055804,-0.489630,...,0.170620,-0.012766,0.000000,-0.011414,-0.243745,0.153414,0.000000,0.159781,0.286456,0.047184
17307,CASP8AP2,-0.843295,-0.307031,-0.413669,-1.849696,-0.414154,-0.457898,-0.372166,-0.037574,-0.468437,...,-1.066150,-0.676503,-0.476991,-0.437871,-0.200516,-0.046700,-0.865486,-1.039110,-0.301415,-0.865502


In [376]:
# get the overlapped cell lines in methy_samples with cell_line_anno_df by merge on cell line id
rna_combined_samples = raw_rna_combined_df.columns[1:].tolist()
# first format the methy_samples as dataframe
rna_combined_samples_df = pd.DataFrame(rna_combined_samples, columns=['CCLE_Name'])
# merge the methy_samples_df with the cell_line_anno_df
rna_combined_samples_anno_df = pd.merge(rna_combined_samples_df, cell_line_anno_df, left_on='CCLE_Name', right_on='CCLE_ID', how='inner')
display(rna_combined_samples_anno_df)

# get the map dictionary from the ccle id to depmap id
rna_combined_samples_map_dict = dict(zip(rna_combined_samples_anno_df['CCLE_Name'], rna_combined_samples_anno_df['depMapID']))
print(rna_combined_samples_map_dict)
maped_rna_combined_df = raw_rna_combined_df.rename(columns=rna_combined_samples_map_dict)
columns_to_keep = ['gene_name'] + sorted([col for col in maped_rna_combined_df.columns if col in overlapped_omics_union_annotation_samples])
filtered_rna_df = maped_rna_combined_df[columns_to_keep]
display(filtered_rna_df)

,CCLE_Name,CCLE_ID,depMapID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,Histology,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,1321N1_CENTRAL_NERVOUS_SYSTEM,1321N1_CENTRAL_NERVOUS_SYSTEM,ACH-001000,1321N1,primary,central_nervous_system,brain,NS,NS,glioma,...,NaN,NaN,NaN,NaN,NaN,glioma,glioma,NaN,NaN,NaN
1,143B_BONE,143B_BONE,ACH-001001,143B,primary,bone,NS,NS,NS,osteosarcoma,...,NaN,NaN,NaN,NaN,NaN,osteosarcoma,osteosarcoma,NaN,NaN,NaN
2,22RV1_PROSTATE,22RV1_PROSTATE,ACH-000956,22Rv1,primary,prostate,NS,NS,NS,carcinoma,...,RPMI-1640+10% FBS,NaN,5%DMSO,NaN,58.4,prostate,prostate,Prostate:Carcinoma,539.830868,PRAD
3,2313287_STOMACH,2313287_STOMACH,ACH-000948,23132/87,primary,stomach,NS,NS,NS,carcinoma,...,90%RPMI 1640+10% h.i FBS,NaN,10%DMSO,50 hrs,97.6,stomach,stomach,Gastric:Carcinoma,521.355253,STAD
4,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000070,697,primary,haematopoietic_and_lymphoid_tissue,NS,NS,NS,lymphoid_neoplasm,...,RPMI + 10 % FBS,NaN,5% DMSO,NaN,46.6,leukemia_other,leukemia_other,Leukemia:ALL,75.699456,ALL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,YD8_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,ACH-000630,YD-8,primary,upper_aerodigestive_tract,mouth,tongue,NS,carcinoma,...,"RPMI 1640 with L-glutamine (300mg/L), 90%; he...",25mM Hepes and 25mM NaHCo3,NaN,29.3 hrs,NaN,upper_aerodigestive,upper_aerodigestive,Upper_Aerodigestive_Tract:Carcinoma,176.583324,HNSC
598,YKG1_CENTRAL_NERVOUS_SYSTEM,YKG1_CENTRAL_NERVOUS_SYSTEM,ACH-000570,YKG1,primary,central_nervous_system,temporal_lobe,NS,NS,glioma,...,NaN,NaN,NaN,NaN,NaN,glioma,glioma,CNS:Glioma_HighGrade,162.475553,GBM
599,YMB1_BREAST,YMB1_BREAST,ACH-001249,YMB-1,primary,breast,NS,NS,NS,carcinoma,...,NaN,NaN,NaN,NaN,NaN,breast,breast,NaN,NaN,BRCA
600,ZR751_BREAST,ZR751_BREAST,ACH-000097,ZR-75-1,metastasis,breast,NS,NS,NS,carcinoma,...,RPMI-1640 + 10% FBS,NaN,5%DMSO,NaN,88.7,breast,breast,Breast:Carcinoma,65.390119,BRCA


{'1321N1_CENTRAL_NERVOUS_SYSTEM': 'ACH-001000', '143B_BONE': 'ACH-001001', '22RV1_PROSTATE': 'ACH-000956', '2313287_STOMACH': 'ACH-000948', '697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000070', '769P_KIDNEY': 'ACH-000411', '786O_KIDNEY': 'ACH-000649', '8305C_THYROID': 'ACH-001306', '8505C_THYROID': 'ACH-001307', 'A101D_SKIN': 'ACH-000008', 'A172_CENTRAL_NERVOUS_SYSTEM': 'ACH-000558', 'A204_SOFT_TISSUE': 'ACH-000201', 'A2058_SKIN': 'ACH-000788', 'A253_SALIVARY_GLAND': 'ACH-000740', 'A2780_OVARY': 'ACH-000657', 'A375_SKIN': 'ACH-000219', 'A3KAW_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000697', 'A498_KIDNEY': 'ACH-000555', 'A4FUK_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000157', 'A549_LUNG': 'ACH-000681', 'A673_BONE': 'ACH-000052', 'A704_KIDNEY': 'ACH-000429', 'ABC1_LUNG': 'ACH-000528', 'ACCMESO1_PLEURA': 'ACH-000086', 'ACHN_KIDNEY': 'ACH-000046', 'AGS_STOMACH': 'ACH-000880', 'AM38_CENTRAL_NERVOUS_SYSTEM': 'ACH-000269', 'AML193_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE': 'ACH-000557', 'AMO1_

,gene_name,ACH-000001,ACH-000002,ACH-000004,ACH-000005,ACH-000006,ACH-000007,ACH-000008,ACH-000009,ACH-000012,...,ACH-001075,ACH-001089,ACH-001107,ACH-001113,ACH-001129,ACH-001190,ACH-001239,ACH-001306,ACH-001307,ACH-001318
0,A1BG,0.036726,0.231996,-0.136186,0.000000,-0.570520,-0.012940,0.000000,-0.664572,-0.084851,...,-0.044965,0.000000,0.000000,0.188873,-0.008834,-0.042931,-0.326934,0.000000,0.000000,0.365715
1,NAT2,0.191694,-0.311668,-0.193766,0.000000,-0.169935,-0.290193,0.000000,-0.086928,-0.018109,...,-0.120774,0.000000,0.000000,-0.193814,-0.079915,0.146653,-0.107795,0.000000,0.000000,0.083265
2,ADA,0.223975,-0.576057,0.131067,0.000000,-0.055516,-0.282557,0.000000,0.164582,0.320907,...,-0.214594,0.000000,0.000000,0.136263,0.135128,-0.075160,0.029538,0.000000,0.000000,-0.183369
3,CDH2,0.025377,0.072624,0.172407,-0.209676,-0.060467,0.136671,0.095038,0.193100,-0.105455,...,-0.070584,-0.286068,-0.017196,-0.024994,-0.024949,0.221152,-0.166268,-0.295108,-0.101788,0.158220
4,AKT3,0.167998,-0.253519,0.114353,0.105309,0.028471,0.140191,0.084754,-0.010944,0.133982,...,0.019461,-0.583573,0.069328,0.210320,0.304147,0.040289,-0.121801,-0.005521,-0.021668,0.183108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17304,SLC12A6,0.111539,0.029433,0.040104,0.000000,0.044931,0.076166,0.000000,0.123799,-0.093000,...,0.147727,0.000000,0.000000,0.038196,-0.007167,0.046325,0.144315,0.000000,0.000000,-0.098176
17305,KCNE2,-0.112684,0.059434,-0.071567,-0.166129,-0.079283,0.042259,-0.250895,0.207326,-0.050655,...,-0.142186,0.139844,0.078532,0.392288,-0.074156,-0.054130,-0.137692,0.114692,0.060095,-0.038330
17306,DGCR2,0.050646,-0.323283,0.120482,0.000000,-0.117080,0.177101,0.000000,0.353728,-0.366567,...,-0.018230,0.000000,0.000000,0.234106,0.115288,-0.212524,0.223209,0.000000,0.000000,0.109311
17307,CASP8AP2,-1.228253,-0.414654,-0.050006,0.076652,-0.360068,-0.868595,-0.461431,-0.301565,-0.543784,...,-0.433340,-0.293064,-0.454495,-0.867408,-0.930363,-0.828188,-0.138393,-0.983701,-0.626359,-0.440665


In [377]:
bmg_transcript_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')
# keep BioMedGraphica_ID and HGNC_Symbol
bmg_transcript_df = bmg_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']]
display(bmg_transcript_df)
# merge the biomedgraphica_id with the raw_transcript_df
merge_rna_df = pd.merge(bmg_transcript_df, filtered_rna_df, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
merge_rna_df.drop(columns=['HGNC_Symbol', 'gene_name'], inplace=True)
display(merge_rna_df)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\2511689865.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_transcript_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')


,BioMedGraphica_Conn_ID,HGNC_Symbol
0,BMGC_TS000001,ARF5
1,BMGC_TS000002,M6PR
2,BMGC_TS000003,ESRRA
3,BMGC_TS000004,FKBP4
4,BMGC_TS000005,CYP26B1
...,...,...
412034,BMGC_TS412035,LINC02968
412035,BMGC_TS412036,LINC03112
412036,BMGC_TS412037,LINC03112
412037,BMGC_TS412038,LINC03112


,BioMedGraphica_Conn_ID,ACH-000001,ACH-000002,ACH-000004,ACH-000005,ACH-000006,ACH-000007,ACH-000008,ACH-000009,ACH-000012,...,ACH-001075,ACH-001089,ACH-001107,ACH-001113,ACH-001129,ACH-001190,ACH-001239,ACH-001306,ACH-001307,ACH-001318
0,BMGC_TS000001,0.123267,-0.184446,-0.072956,0.000381,-0.037266,-0.029936,0.102107,-0.108996,-0.099402,...,0.127145,0.037874,-0.107370,-0.105618,-0.115614,0.031604,-0.080667,-0.181498,0.048788,0.201115
1,BMGC_TS000002,-0.025559,0.113514,0.141620,0.000000,0.137914,-0.179942,0.000000,-0.028500,-0.324958,...,-0.202727,0.000000,0.000000,-0.198776,0.371099,-0.111130,0.308401,0.000000,0.000000,0.245889
2,BMGC_TS000003,-0.047525,-0.535418,0.024752,-0.012339,0.115913,0.135700,0.104687,-0.274726,0.046911,...,0.043630,-0.082785,-0.084723,0.009095,-0.198575,0.065699,-0.078132,-0.152112,0.103321,0.119976
3,BMGC_TS000004,-0.526937,-0.723573,-0.492790,-0.513913,-0.591701,-0.246384,-0.531943,-0.149865,-0.217154,...,-0.463727,-0.367015,-0.232642,-0.409513,-0.587943,-0.119636,-0.400796,-0.118495,-0.052553,-0.383631
4,BMGC_TS000005,0.116400,0.240852,-0.270499,-0.183875,0.023891,-0.204737,-0.102901,-0.053493,-0.095810,...,-0.208952,0.039113,-0.080288,-0.022177,-0.232425,-0.033772,-0.245861,-0.045608,-0.141908,-0.201729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170386,BMGC_TS411885,-0.236575,-0.155037,0.115897,0.000000,-0.168277,0.041007,0.000000,-0.335707,-0.561136,...,0.054384,0.000000,0.000000,0.002146,-0.072585,-0.254140,-0.124046,0.000000,0.000000,-0.364009
170387,BMGC_TS411886,-0.236575,-0.155037,0.115897,0.000000,-0.168277,0.041007,0.000000,-0.335707,-0.561136,...,0.054384,0.000000,0.000000,0.002146,-0.072585,-0.254140,-0.124046,0.000000,0.000000,-0.364009
170388,BMGC_TS411887,-0.236575,-0.155037,0.115897,0.000000,-0.168277,0.041007,0.000000,-0.335707,-0.561136,...,0.054384,0.000000,0.000000,0.002146,-0.072585,-0.254140,-0.124046,0.000000,0.000000,-0.364009
170389,BMGC_TS412018,0.028071,-0.129326,0.091669,-0.004241,0.055351,-0.016189,0.269712,0.139049,-0.242594,...,-0.127993,-0.081723,0.001105,0.048276,-0.008880,-0.076799,-0.011490,0.257041,0.024479,0.076093


In [378]:
# map the transcript entity with transcript_protein_entity_df
protein_rna_df = pd.merge(transcript_protein_entity_df, merge_rna_df, left_on='BMGC_TS_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
# and drop nan in any rows from protein_rna_df
processed_rna_df = protein_rna_df.dropna().reset_index(drop=True)
display(processed_rna_df)
processed_rna_df.to_csv('./data/pretrain_plain_data/processed_rna.csv', index=False)
processed_rna_df.to_csv('./data/pretrain_status_data/processed_rna.csv', index=False)

,BMGC_TS_ID,BMGC_PT_ID,ACH-000001,ACH-000002,ACH-000004,ACH-000005,ACH-000006,ACH-000007,ACH-000008,ACH-000009,...,ACH-001075,ACH-001089,ACH-001107,ACH-001113,ACH-001129,ACH-001190,ACH-001239,ACH-001306,ACH-001307,ACH-001318
0,BMGC_TS000001,BMGC_PT017571,0.123267,-0.184446,-0.072956,0.000381,-0.037266,-0.029936,0.102107,-0.108996,...,0.127145,0.037874,-0.107370,-0.105618,-0.115614,0.031604,-0.080667,-0.181498,0.048788,0.201115
1,BMGC_TS000002,BMGC_PT014978,-0.025559,0.113514,0.141620,0.000000,0.137914,-0.179942,0.000000,-0.028500,...,-0.202727,0.000000,0.000000,-0.198776,0.371099,-0.111130,0.308401,0.000000,0.000000,0.245889
2,BMGC_TS000003,BMGC_PT014403,-0.047525,-0.535418,0.024752,-0.012339,0.115913,0.135700,0.104687,-0.274726,...,0.043630,-0.082785,-0.084723,0.009095,-0.198575,0.065699,-0.078132,-0.152112,0.103321,0.119976
3,BMGC_TS000004,BMGC_PT017807,-0.526937,-0.723573,-0.492790,-0.513913,-0.591701,-0.246384,-0.531943,-0.149865,...,-0.463727,-0.367015,-0.232642,-0.409513,-0.587943,-0.119636,-0.400796,-0.118495,-0.052553,-0.383631
4,BMGC_TS000005,BMGC_PT043262,0.116400,0.240852,-0.270499,-0.183875,0.023891,-0.204737,-0.102901,-0.053493,...,-0.208952,0.039113,-0.080288,-0.022177,-0.232425,-0.033772,-0.245861,-0.045608,-0.141908,-0.201729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132566,BMGC_TS411885,BMGC_PT121415,-0.236575,-0.155037,0.115897,0.000000,-0.168277,0.041007,0.000000,-0.335707,...,0.054384,0.000000,0.000000,0.002146,-0.072585,-0.254140,-0.124046,0.000000,0.000000,-0.364009
132567,BMGC_TS411886,BMGC_PT121416,-0.236575,-0.155037,0.115897,0.000000,-0.168277,0.041007,0.000000,-0.335707,...,0.054384,0.000000,0.000000,0.002146,-0.072585,-0.254140,-0.124046,0.000000,0.000000,-0.364009
132568,BMGC_TS411887,BMGC_PT121417,-0.236575,-0.155037,0.115897,0.000000,-0.168277,0.041007,0.000000,-0.335707,...,0.054384,0.000000,0.000000,0.002146,-0.072585,-0.254140,-0.124046,0.000000,0.000000,-0.364009
132569,BMGC_TS412018,BMGC_PT121418,0.028071,-0.129326,0.091669,-0.004241,0.055351,-0.016189,0.269712,0.139049,...,-0.127993,-0.081723,0.001105,0.048276,-0.008880,-0.076799,-0.011490,0.257041,0.024479,0.076093


In [379]:
processed_crispr_samples = processed_crispr_df.columns[2:].tolist()
processed_rna_samples = processed_rna_df.columns[2:].tolist()

# get the union and intersection samples of the two lists
overlapped_crispr_rna_samples = sorted(list(set(processed_crispr_samples) & set(processed_rna_samples)))
print("The overlapped samples between protein_crispr_samples and protein_rna_samples are:")
print(len(overlapped_crispr_rna_samples))
print(overlapped_crispr_rna_samples)

# get the union set of samples of the two lists
union_crispr_rna_samples = sorted(list(set(processed_crispr_samples) | set(processed_rna_samples)))
print("The union samples between protein_crispr_samples and protein_rna_samples are:")
print(len(union_crispr_rna_samples))
print(union_crispr_rna_samples)

The overlapped samples between protein_crispr_samples and protein_rna_samples are:
489
['ACH-000001', 'ACH-000004', 'ACH-000005', 'ACH-000007', 'ACH-000009', 'ACH-000012', 'ACH-000014', 'ACH-000015', 'ACH-000017', 'ACH-000018', 'ACH-000019', 'ACH-000021', 'ACH-000023', 'ACH-000024', 'ACH-000028', 'ACH-000030', 'ACH-000035', 'ACH-000039', 'ACH-000040', 'ACH-000041', 'ACH-000042', 'ACH-000045', 'ACH-000048', 'ACH-000052', 'ACH-000054', 'ACH-000055', 'ACH-000060', 'ACH-000067', 'ACH-000070', 'ACH-000075', 'ACH-000086', 'ACH-000087', 'ACH-000090', 'ACH-000092', 'ACH-000093', 'ACH-000094', 'ACH-000095', 'ACH-000096', 'ACH-000097', 'ACH-000098', 'ACH-000100', 'ACH-000103', 'ACH-000108', 'ACH-000111', 'ACH-000113', 'ACH-000114', 'ACH-000115', 'ACH-000118', 'ACH-000123', 'ACH-000127', 'ACH-000132', 'ACH-000133', 'ACH-000138', 'ACH-000139', 'ACH-000142', 'ACH-000146', 'ACH-000147', 'ACH-000148', 'ACH-000149', 'ACH-000152', 'ACH-000153', 'ACH-000155', 'ACH-000159', 'ACH-000161', 'ACH-000163', 'A

In [380]:
# get the intersection based on the overlapped over omics not union over omics by overlapped_omics_annotation_samples
overlapped_omics_crispr_rna_samples = sorted(list(set(overlapped_crispr_rna_samples) & set(overlapped_omics_annotation_samples)))
print("The overlapped samples between protein_crispr_samples and protein_rna_samples are:")
print(len(overlapped_omics_crispr_rna_samples))
print(overlapped_omics_crispr_rna_samples)

# get the intersection based on the overlapped over omics not union over omics by overlapped_omics_annotation_samples for union_crispr_rna_samples
overlapped_omics_union_crispr_rna_samples = sorted(list(set(union_crispr_rna_samples) & set(overlapped_omics_annotation_samples)))
print("The overlapped samples between protein_crispr_samples and protein_rna_samples are:")
print(len(overlapped_omics_union_crispr_rna_samples))
print(overlapped_omics_union_crispr_rna_samples)

The overlapped samples between protein_crispr_samples and protein_rna_samples are:
178
['ACH-000001', 'ACH-000012', 'ACH-000014', 'ACH-000019', 'ACH-000023', 'ACH-000028', 'ACH-000035', 'ACH-000039', 'ACH-000042', 'ACH-000052', 'ACH-000087', 'ACH-000090', 'ACH-000097', 'ACH-000111', 'ACH-000114', 'ACH-000115', 'ACH-000118', 'ACH-000132', 'ACH-000138', 'ACH-000147', 'ACH-000153', 'ACH-000155', 'ACH-000159', 'ACH-000161', 'ACH-000164', 'ACH-000219', 'ACH-000222', 'ACH-000223', 'ACH-000235', 'ACH-000239', 'ACH-000243', 'ACH-000247', 'ACH-000248', 'ACH-000250', 'ACH-000255', 'ACH-000264', 'ACH-000278', 'ACH-000288', 'ACH-000291', 'ACH-000304', 'ACH-000311', 'ACH-000318', 'ACH-000330', 'ACH-000343', 'ACH-000348', 'ACH-000356', 'ACH-000361', 'ACH-000364', 'ACH-000374', 'ACH-000378', 'ACH-000390', 'ACH-000393', 'ACH-000404', 'ACH-000407', 'ACH-000410', 'ACH-000411', 'ACH-000414', 'ACH-000416', 'ACH-000420', 'ACH-000425', 'ACH-000427', 'ACH-000430', 'ACH-000433', 'ACH-000438', 'ACH-000441', 'A

## 9. Sample Splits

### 9.1 Pretrain, Drug and Target Sample Splits

In [381]:
dti_overlapped_samples = list(dti_overlapped_samples_df['depMapID'])
print(dti_overlapped_samples)
print("length of dti_overlapped_samples: ", len(set(dti_overlapped_samples)))

['ACH-000001', 'ACH-000002', 'ACH-000004', 'ACH-000006', 'ACH-000007', 'ACH-000008', 'ACH-000009', 'ACH-000012', 'ACH-000015', 'ACH-000018', 'ACH-000019', 'ACH-000020', 'ACH-000021', 'ACH-000023', 'ACH-000024', 'ACH-000030', 'ACH-000035', 'ACH-000038', 'ACH-000040', 'ACH-000042', 'ACH-000045', 'ACH-000046', 'ACH-000047', 'ACH-000048', 'ACH-000050', 'ACH-000052', 'ACH-000053', 'ACH-000054', 'ACH-000055', 'ACH-000056', 'ACH-000058', 'ACH-000059', 'ACH-000060', 'ACH-000061', 'ACH-000062', 'ACH-000065', 'ACH-000067', 'ACH-000070', 'ACH-000072', 'ACH-000073', 'ACH-000074', 'ACH-000075', 'ACH-000078', 'ACH-000081', 'ACH-000082', 'ACH-000087', 'ACH-000090', 'ACH-000091', 'ACH-000092', 'ACH-000094', 'ACH-000096', 'ACH-000098', 'ACH-000099', 'ACH-000100', 'ACH-000101', 'ACH-000102', 'ACH-000103', 'ACH-000104', 'ACH-000105', 'ACH-000106', 'ACH-000107', 'ACH-000108', 'ACH-000111', 'ACH-000112', 'ACH-000113', 'ACH-000114', 'ACH-000115', 'ACH-000116', 'ACH-000117', 'ACH-000118', 'ACH-000120', 'ACH-

#### 9.1.1 Pretrain Samples

In [382]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Your existing code for rest_samples is good
rest_samples = sorted(list(set(overlapped_omics_union_annotation_samples) - set(dti_overlapped_samples)))
print("len(rest_samples):", len(rest_samples))
print(rest_samples[:5])

# check the non-cancerous samples in the rest_samples
non_cancerous_rest_samples = sorted(list(set(non_cancerous_samples_df['Depmap Id']) & set(rest_samples)))
print("len(non_cancerous_rest_samples):", len(non_cancerous_rest_samples))
print(non_cancerous_rest_samples[:5])

cancerous_rest_samples = sorted(list(set(rest_samples) - set(non_cancerous_rest_samples)))
print("len(cancerous_rest_samples):", len(cancerous_rest_samples))
print(cancerous_rest_samples[:5])

# convert the rest_samples to a dataframe with marking the sample cancerous or non-cancerous status
rest_samples_df = pd.DataFrame({
    'depMapID': rest_samples,
    'cancerous_status': ['cancerous' if sample in cancerous_rest_samples else 'non-cancerous' for sample in rest_samples]
})
display(rest_samples_df)

# Split the rest_samples into pretrain_plain_samples and pretrain_status_samples
# Sample 20% of the rest_samples for pretrain_plain_samples while maintaining the cancerous/non-cancerous ratio

def stratified_split_pretrain_samples(rest_samples_df, pretrain_plain_ratio=0.2, random_state=42):
    """
    Split rest samples into pretrain_plain and pretrain_status while maintaining 
    the cancerous/non-cancerous ratio in both splits.
    
    Args:
        rest_samples_df: DataFrame with depMapID and cancerous_status columns
        pretrain_plain_ratio: Ratio of samples to allocate to pretrain_plain (default 0.2)
        random_state: Random seed for reproducibility
    
    Returns:
        tuple: (pretrain_plain_samples, pretrain_status_samples)
    """
    # Use stratified split to maintain the ratio
    pretrain_plain_df, pretrain_status_df = train_test_split(
        rest_samples_df,
        test_size=1-pretrain_plain_ratio,  # 0.8 for pretrain_status
        stratify=rest_samples_df['cancerous_status'],
        random_state=random_state
    )
    
    # Extract sample lists
    pretrain_plain_samples = sorted(pretrain_plain_df['depMapID'].tolist())
    pretrain_status_samples = sorted(pretrain_status_df['depMapID'].tolist())
    
    return pretrain_plain_samples, pretrain_status_samples, pretrain_plain_df, pretrain_status_df

# Perform the split
pretrain_plain_samples, pretrain_status_samples, pretrain_plain_df, pretrain_status_df = stratified_split_pretrain_samples(
    rest_samples_df, 
    pretrain_plain_ratio=0.2, 
    random_state=42
)

# Display results
print(f"\n📊 Pretrain Sample Split Summary:")
print(f"Total rest samples: {len(rest_samples_df)}")
print(f"Pretrain plain samples: {len(pretrain_plain_samples)} ({len(pretrain_plain_samples)/len(rest_samples_df)*100:.1f}%)")
print(f"Pretrain status samples: {len(pretrain_status_samples)} ({len(pretrain_status_samples)/len(rest_samples_df)*100:.1f}%)")

print(f"\n📋 Pretrain Plain Sample Distribution:")
print(pretrain_plain_df['cancerous_status'].value_counts())
print(pretrain_plain_df['cancerous_status'].value_counts(normalize=True))

print(f"\n📋 Pretrain Status Sample Distribution:")
print(pretrain_status_df['cancerous_status'].value_counts())
print(pretrain_status_df['cancerous_status'].value_counts(normalize=True))

# Save the sample lists and detailed DataFrames
pretrain_plain_df.to_csv('./data/pretrain_plain_data/pretrain_plain_samples.csv', index=False)
pretrain_status_df.to_csv('./data/pretrain_status_data/pretrain_status_samples.csv', index=False)

# Also save just the sample IDs for convenience
pd.DataFrame(pretrain_plain_samples, columns=['depMapID']).to_csv('./data/pretrain_plain_data/pretrain_plain_sample_ids.csv', index=False)
pd.DataFrame(pretrain_status_samples, columns=['depMapID']).to_csv('./data/pretrain_status_data/pretrain_status_sample_ids.csv', index=False)

print(f"\n✅ Saved pretrain sample splits to:")
print(f"- ./data/pretrain_plain_data/pretrain_plain_samples.csv ({len(pretrain_plain_samples)} samples)")
print(f"- ./data/pretrain_status_data/pretrain_status_samples.csv ({len(pretrain_status_samples)} samples)")

# Verify the splits don't overlap
overlap_check = set(pretrain_plain_samples) & set(pretrain_status_samples)
print(f"\n🔍 Overlap check: {len(overlap_check)} overlapping samples (should be 0)")

# Show sample cancerous/non-cancerous ratios
pretrain_plain_cancerous = pretrain_plain_df['cancerous_status'].value_counts()
pretrain_status_cancerous = pretrain_status_df['cancerous_status'].value_counts()

print(f"\n📈 Final Cancerous/Non-cancerous Distribution:")
print(f"Pretrain Plain - Cancerous: {pretrain_plain_cancerous.get('cancerous', 0)}, Non-cancerous: {pretrain_plain_cancerous.get('non-cancerous', 0)}")
print(f"Pretrain Status - Cancerous: {pretrain_status_cancerous.get('cancerous', 0)}, Non-cancerous: {pretrain_status_cancerous.get('non-cancerous', 0)}")

# Split the pretrain_status_samples into training and test sets according to the cancerous status
def stratified_split_status_samples(pretrain_status_df, test_size=0.2, random_state=42):
    """
    Split pretrain status samples into training and test sets while maintaining 
    the cancerous/non-cancerous ratio.
    
    Args:
        pretrain_status_df: DataFrame with depMapID and cancerous_status columns
        test_size: Proportion of the dataset to include in the test split (default 0.2)
        random_state: Random seed for reproducibility
    
    Returns:
        tuple: (train_samples, test_samples, train_df, test_df)
    """
    # Use stratified split to maintain the ratio
    train_df, test_df = train_test_split(
        pretrain_status_df,
        test_size=test_size,
        stratify=pretrain_status_df['cancerous_status'],
        random_state=random_state
    )
    
    # Extract sample lists
    train_samples = sorted(train_df['depMapID'].tolist())
    test_samples = sorted(test_df['depMapID'].tolist())
    
    return train_samples, test_samples, train_df, test_df

# Perform the split
pretrain_status_train_samples, pretrain_status_test_samples, pretrain_status_train_df, pretrain_status_test_df = stratified_split_status_samples(
    pretrain_status_df, 
    test_size=0.2, 
    random_state=42
)

# Display results for training and test sets for cancerous and non-cancerous counts
print(f"\n📊 Pretrain Status Sample Split Summary:"
      f"\nTotal pretrain status samples: {len(pretrain_status_df)}")
print(f"Training samples: {len(pretrain_status_train_samples)} ({len(pretrain_status_train_samples)/len(pretrain_status_df)*100:.1f}%)")
print(f"Test samples: {len(pretrain_status_test_samples)} ({len(pretrain_status_test_samples)/len(pretrain_status_df)*100:.1f}%)")
print(f"\n📋 Training Sample Distribution:"
      f"\n{pretrain_status_train_df['cancerous_status'].value_counts()}")
print(pretrain_status_train_df['cancerous_status'].value_counts(normalize=True))
print(f"\n📋 Test Sample Distribution:"
      f"\n{pretrain_status_test_df['cancerous_status'].value_counts()}")

# Save the training and test sample lists and detailed DataFrames
pretrain_status_train_df.to_csv('./data/pretrain_status_data/pretrain_status_train_samples.csv', index=False)
pretrain_status_test_df.to_csv('./data/pretrain_status_data/pretrain_status_test_samples.csv', index=False)


len(rest_samples): 336
['ACH-000005', 'ACH-000011', 'ACH-000013', 'ACH-000014', 'ACH-000016']
len(non_cancerous_rest_samples): 39
['ACH-000043', 'ACH-000063', 'ACH-000071', 'ACH-000079', 'ACH-000083']
len(cancerous_rest_samples): 297
['ACH-000005', 'ACH-000011', 'ACH-000013', 'ACH-000014', 'ACH-000016']


,depMapID,cancerous_status
0,ACH-000005,cancerous
1,ACH-000011,cancerous
2,ACH-000013,cancerous
3,ACH-000014,cancerous
4,ACH-000016,cancerous
...,...,...
331,ACH-001188,cancerous
332,ACH-001194,cancerous
333,ACH-001239,cancerous
334,ACH-001318,cancerous



📊 Pretrain Sample Split Summary:
Total rest samples: 336
Pretrain plain samples: 67 (19.9%)
Pretrain status samples: 269 (80.1%)

📋 Pretrain Plain Sample Distribution:
cancerous_status
cancerous        59
non-cancerous     8
Name: count, dtype: int64
cancerous_status
cancerous        0.880597
non-cancerous    0.119403
Name: proportion, dtype: float64

📋 Pretrain Status Sample Distribution:
cancerous_status
cancerous        238
non-cancerous     31
Name: count, dtype: int64
cancerous_status
cancerous        0.884758
non-cancerous    0.115242
Name: proportion, dtype: float64

✅ Saved pretrain sample splits to:
- ./data/pretrain_plain_data/pretrain_plain_samples.csv (67 samples)
- ./data/pretrain_status_data/pretrain_status_samples.csv (269 samples)

🔍 Overlap check: 0 overlapping samples (should be 0)

📈 Final Cancerous/Non-cancerous Distribution:
Pretrain Plain - Cancerous: 59, Non-cancerous: 8
Pretrain Status - Cancerous: 238, Non-cancerous: 31

📊 Pretrain Status Sample Split Summary:

#### 9.1.2 Target-CRISPR Samples

In [383]:
# Check the overlapped samples between dti_overlapped_samples and overlapped_crispr_rna_samples
overlapped_dti_crispr_rna_samples = sorted(list(set(dti_overlapped_samples) & set(overlapped_crispr_rna_samples)))
print("The overlapped samples between dti_overlapped_samples and overlapped_crispr_rna_samples are:")
print(len(overlapped_dti_crispr_rna_samples))
print(overlapped_dti_crispr_rna_samples)

# Add the sample information to overlapped_dti_crispr_rna_samples with tcga_code, type, and PATHOLOGIST_ANNOTATION
overlapped_dti_crispr_rna_samples_df = dti_overlapped_samples_df[dti_overlapped_samples_df['depMapID'].isin(overlapped_dti_crispr_rna_samples)].reset_index(drop=True)
# fill NaN values in the columns with 'Unknown'
overlapped_dti_crispr_rna_samples_df['tcga_code'] = overlapped_dti_crispr_rna_samples_df['tcga_code'].fillna('Unknown')
display(overlapped_dti_crispr_rna_samples_df)

The overlapped samples between dti_overlapped_samples and overlapped_crispr_rna_samples are:
363
['ACH-000001', 'ACH-000004', 'ACH-000007', 'ACH-000009', 'ACH-000012', 'ACH-000015', 'ACH-000018', 'ACH-000019', 'ACH-000021', 'ACH-000023', 'ACH-000024', 'ACH-000030', 'ACH-000035', 'ACH-000040', 'ACH-000042', 'ACH-000045', 'ACH-000048', 'ACH-000052', 'ACH-000054', 'ACH-000055', 'ACH-000060', 'ACH-000067', 'ACH-000070', 'ACH-000075', 'ACH-000087', 'ACH-000090', 'ACH-000092', 'ACH-000094', 'ACH-000096', 'ACH-000098', 'ACH-000100', 'ACH-000103', 'ACH-000108', 'ACH-000111', 'ACH-000113', 'ACH-000114', 'ACH-000115', 'ACH-000118', 'ACH-000132', 'ACH-000138', 'ACH-000139', 'ACH-000142', 'ACH-000146', 'ACH-000147', 'ACH-000148', 'ACH-000149', 'ACH-000153', 'ACH-000155', 'ACH-000159', 'ACH-000161', 'ACH-000168', 'ACH-000169', 'ACH-000176', 'ACH-000178', 'ACH-000183', 'ACH-000188', 'ACH-000191', 'ACH-000192', 'ACH-000207', 'ACH-000210', 'ACH-000211', 'ACH-000212', 'ACH-000213', 'ACH-000217', 'ACH-0

,depMapID,CCLE_ID,Name,Pathology,Site_Primary,Site_Subtype1,Site_Subtype2,Site_Subtype3,Histology,Hist_Subtype1,...,Growth.Medium,Supplements,Freezing.Medium,Doubling.Time.from.Vendor,Doubling.Time.Calculated.hrs,type,type_refined,PATHOLOGIST_ANNOTATION,mutRate,tcga_code
0,ACH-000001,NIHOVCAR3_OVARY,NIH:OVCAR-3,metastasis,ovary,NS,NS,NS,carcinoma,NS,...,RPMI+20%FBS,NaN,5%DMSO,NaN,NaN,ovary,ovary,Ovary:Carcinoma,83.806951,OV
1,ACH-000004,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,HEL,primary,haematopoietic_and_lymphoid_tissue,NS,NS,NS,haematopoietic_neoplasm,acute_myeloid_leukaemia,...,RPMI1640+10%FBS,NaN,10%DMSO,36 hrs,38.4,AML,AML,Leukemia:AML,98.185377,LAML
2,ACH-000007,LS513_LARGE_INTESTINE,LS513,primary,large_intestine,caecum,NS,NS,carcinoma,adenocarcinoma,...,NaN,NaN,NaN,NaN,NaN,colorectal,colorectal,Colorectal:Carcinoma,112.349539,COAD/READ
3,ACH-000009,C2BBE1_LARGE_INTESTINE,C2BBe1,primary,large_intestine,colon,NS,NS,carcinoma,adenocarcinoma,...,DMEM +10%FBS,NaN,5% DMSO,NaN,110.9,colorectal,colorectal,Colorectal:Carcinoma,115.886197,COAD/READ
4,ACH-000012,HCC827_LUNG,HCC827,primary,lung,NS,NS,NS,carcinoma,adenocarcinoma,...,RPMI-1640 10% FBS,NaN,5%DMSO,NaN,117.6,lung_NSC,lung_NSC,Lung:NSCLC_Adeno,82.926540,LUAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,ACH-001075,NCIH292_LUNG,NCI-H292,metastasis,lung,NS,NS,NS,carcinoma,mucoepidermoid_carcinoma,...,NaN,NaN,NaN,NaN,NaN,lung_NSC,lung_NSC,Lung:Others,117.719142,Unknown
359,ACH-001107,KP1N_PANCREAS,KP-1N,metastasis,pancreas,NS,NS,NS,carcinoma,ductal_carcinoma,...,NaN,NaN,NaN,NaN,NaN,pancreas,pancreas,NaN,NaN,PAAD
360,ACH-001190,SKMEL2_SKIN,SK-MEL-2,metastasis,skin,thigh,NS,NS,malignant_melanoma,NS,...,NaN,NaN,5%DMSO,NaN,98.8,melanoma,melanoma,Skin:Melanoma,NaN,SKCM
361,ACH-001306,8305C_THYROID,8305C,primary,thyroid,NS,NS,NS,carcinoma,anaplastic_carcinoma,...,RPMI1640+20%FBS,NaN,10%DMSO,54hrs,121.6,thyroid,thyroid,Thyroid:Carcinoma,200.908236,THCA


In [384]:
import pandas as pd
import numpy as np
import os

def stratified_tcga_split_target(df, test_ratio=0.2, random_state=42):
    """
    Split samples by TCGA code with special handling for small groups:
    - For TCGA codes with >=5 samples: split according to test_ratio (0.8/0.2)
    - For TCGA codes with 2-4 samples: keep 1 sample for test
    - For TCGA codes with 1 sample: keep that sample for train
    """
    np.random.seed(random_state)
    
    train_samples = []
    test_samples = []
    
    # Group by TCGA code
    tcga_groups = df.groupby('tcga_code')
    
    print("TCGA Code Distribution and Split Strategy for Target Samples:")
    print("-" * 70)
    
    for tcga_code, group in tcga_groups:
        n_samples = len(group)
        samples = group['depMapID'].tolist()
        
        # Shuffle samples within each TCGA group
        np.random.shuffle(samples)
        
        if n_samples == 1:
            # Single sample: keep for train
            train_samples.extend(samples)
            test_samples_for_group = []
            strategy = "1 sample -> train only"
            
        elif n_samples < 5:
            # 2-4 samples: keep 1 for test, rest for train
            test_samples_for_group = samples[:1]
            train_samples_for_group = samples[1:]
            train_samples.extend(train_samples_for_group)
            test_samples.extend(test_samples_for_group)
            strategy = f"{n_samples} samples -> 1 test, {n_samples-1} train"
            
        else:
            # 5+ samples: use the specified ratio
            n_test = max(1, int(n_samples * test_ratio))
            test_samples_for_group = samples[:n_test]
            train_samples_for_group = samples[n_test:]
            train_samples.extend(train_samples_for_group)
            test_samples.extend(test_samples_for_group)
            strategy = f"{n_samples} samples -> {n_test} test, {len(train_samples_for_group)} train"
        
        print(f"TCGA {tcga_code}: {strategy}")
    
    print("-" * 70)
    print(f"Total train samples: {len(train_samples)}")
    print(f"Total test samples: {len(test_samples)}")
    print(f"Train ratio: {len(train_samples)/(len(train_samples)+len(test_samples)):.3f}")
    print(f"Test ratio: {len(test_samples)/(len(train_samples)+len(test_samples)):.3f}")
    
    return train_samples, test_samples

# Prepare the overlapped samples with TCGA annotation
overlapped_dti_crispr_rna_samples_df_annotated = overlapped_dti_crispr_rna_samples_df.copy()
# Fill NaN TCGA codes with 'Unknown' if any exist
overlapped_dti_crispr_rna_samples_df_annotated['tcga_code'] = overlapped_dti_crispr_rna_samples_df_annotated['tcga_code'].fillna('Unknown')

# Perform stratified split based on TCGA codes
target_crispr_train_samples, target_crispr_test_samples = stratified_tcga_split_target(
    overlapped_dti_crispr_rna_samples_df_annotated, 
    test_ratio=0.2, 
    random_state=42
)

# Count the samples
print(f"\n📊 Target Sample Count Summary:")
print(f"Total samples in dataset: {len(overlapped_dti_crispr_rna_samples_df_annotated)}")
print(f"Training samples: {len(target_crispr_train_samples)}")
print(f"Test samples: {len(target_crispr_test_samples)}")
print(f"Training percentage: {len(target_crispr_train_samples)/len(overlapped_dti_crispr_rna_samples_df_annotated)*100:.1f}%")
print(f"Test percentage: {len(target_crispr_test_samples)/len(overlapped_dti_crispr_rna_samples_df_annotated)*100:.1f}%")

# Create directories
if not os.path.exists('./data/TargetQA'):
    os.makedirs('./data/TargetQA')
if not os.path.exists('./data/TargetScreen'):
    os.makedirs('./data/TargetScreen')

# Sort the samples before saving
train_samples_sorted = sorted(target_crispr_train_samples)
test_samples_sorted = sorted(target_crispr_test_samples)

# Save the sorted train and test samples to CSV files
pd.DataFrame(train_samples_sorted, columns=['depMapID']).to_csv('./data/TargetQA/train_samples.csv', index=False)
pd.DataFrame(test_samples_sorted, columns=['depMapID']).to_csv('./data/TargetQA/test_samples.csv', index=False)
pd.DataFrame(train_samples_sorted, columns=['depMapID']).to_csv('./data/TargetScreen/train_samples.csv', index=False)
pd.DataFrame(test_samples_sorted, columns=['depMapID']).to_csv('./data/TargetScreen/test_samples.csv', index=False)

# Create detailed split information DataFrames
train_target_samples_df = overlapped_dti_crispr_rna_samples_df_annotated[
    overlapped_dti_crispr_rna_samples_df_annotated['depMapID'].isin(target_crispr_train_samples)
].reset_index(drop=True)

test_target_samples_df = overlapped_dti_crispr_rna_samples_df_annotated[
    overlapped_dti_crispr_rna_samples_df_annotated['depMapID'].isin(target_crispr_test_samples)
].reset_index(drop=True)

# Display the TCGA distributions in train and test sets
print("\n📋 Train set TCGA distribution:")
print(train_target_samples_df['tcga_code'].value_counts().sort_index())
print("\n📋 Test set TCGA distribution:")
print(test_target_samples_df['tcga_code'].value_counts().sort_index())

# Save detailed sample information
train_target_samples_df.to_csv('./data/TargetQA/train_samples_detailed.csv', index=False)
test_target_samples_df.to_csv('./data/TargetQA/test_samples_detailed.csv', index=False)
train_target_samples_df.to_csv('./data/TargetScreen/train_samples_detailed.csv', index=False)
test_target_samples_df.to_csv('./data/TargetScreen/test_samples_detailed.csv', index=False)

print(f"\n✅ Saved stratified train and test splits to:")
print(f"- ./data/TargetQA/train_samples.csv ({len(train_samples_sorted)} samples)")
print(f"- ./data/TargetQA/test_samples.csv ({len(test_samples_sorted)} samples)")
print(f"- ./data/TargetScreen/train_samples.csv ({len(train_samples_sorted)} samples)")
print(f"- ./data/TargetScreen/test_samples.csv ({len(test_samples_sorted)} samples)")
print(f"- Detailed sample information with TCGA annotations also saved")

TCGA Code Distribution and Split Strategy for Target Samples:
----------------------------------------------------------------------
TCGA ALL: 6 samples -> 1 test, 5 train
TCGA BLCA: 8 samples -> 1 test, 7 train
TCGA BRCA: 31 samples -> 6 test, 25 train
TCGA COAD/READ: 26 samples -> 5 test, 21 train
TCGA DLBC: 6 samples -> 1 test, 5 train
TCGA ESCA: 18 samples -> 3 test, 15 train
TCGA GBM: 18 samples -> 3 test, 15 train
TCGA HNSC: 11 samples -> 2 test, 9 train
TCGA KIRC: 8 samples -> 1 test, 7 train
TCGA LAML: 9 samples -> 1 test, 8 train
TCGA LCML: 1 sample -> train only
TCGA LGG: 5 samples -> 1 test, 4 train
TCGA LIHC: 8 samples -> 1 test, 7 train
TCGA LUAD: 37 samples -> 7 test, 30 train
TCGA LUSC: 10 samples -> 2 test, 8 train
TCGA MB: 2 samples -> 1 test, 1 train
TCGA MESO: 4 samples -> 1 test, 3 train
TCGA MM: 7 samples -> 1 test, 6 train
TCGA NB: 7 samples -> 1 test, 6 train
TCGA OV: 19 samples -> 3 test, 16 train
TCGA PAAD: 23 samples -> 4 test, 19 train
TCGA PRAD: 5 samples ->

#### 9.1.3 Drug Samples

In [385]:
# Keep the left samples dti_overlapped_samples - overlapped_dti_crispr_rna_samples
remaining_dti_samples = sorted(list(set(dti_overlapped_samples) - set(overlapped_dti_crispr_rna_samples)))
print("The remaining samples in dti_overlapped_samples after removing overlapped_dti_crispr_rna_samples are:")
print(len(remaining_dti_samples))

The remaining samples in dti_overlapped_samples after removing overlapped_dti_crispr_rna_samples are:
286


In [394]:
import pandas as pd
import numpy as np
import os

# Get the remaining DTI samples
remaining_dti_samples = sorted(list(set(dti_overlapped_samples) - set(overlapped_dti_crispr_rna_samples)))
print("The remaining samples in dti_overlapped_samples after removing overlapped_dti_crispr_rna_samples are:")
print(len(remaining_dti_samples))

# Create DataFrame with remaining samples and their TCGA annotations
remaining_dti_samples_df = dti_overlapped_samples_df[
    dti_overlapped_samples_df['depMapID'].isin(remaining_dti_samples)
].reset_index(drop=True)

# Fill NaN TCGA codes with 'Unknown' if any exist
remaining_dti_samples_df['tcga_code'] = remaining_dti_samples_df['tcga_code'].fillna('Unknown')

def stratified_tcga_split_dti(df, test_ratio=0.2, random_state=42):
    """
    Split samples by TCGA code with special handling for small groups:
    - For TCGA codes with >=5 samples: split according to test_ratio (0.8/0.2)
    - For TCGA codes with 2-4 samples: keep 1 sample for test
    - For TCGA codes with 1 sample: keep that sample for train
    """
    np.random.seed(random_state)
    
    train_samples = []
    test_samples = []
    
    # Group by TCGA code
    tcga_groups = df.groupby('tcga_code')
    
    print("TCGA Code Distribution and Split Strategy for DTI Drug Samples:")
    print("-" * 70)
    
    for tcga_code, group in tcga_groups:
        n_samples = len(group)
        samples = group['depMapID'].tolist()
        
        # Shuffle samples within each TCGA group
        np.random.shuffle(samples)
        
        if n_samples == 1:
            # Single sample: keep for train
            train_samples.extend(samples)
            test_samples_for_group = []
            strategy = "1 sample -> train only"
            
        elif n_samples < 5:
            # 2-4 samples: keep 1 for test, rest for train
            test_samples_for_group = samples[:1]
            train_samples_for_group = samples[1:]
            train_samples.extend(train_samples_for_group)
            test_samples.extend(test_samples_for_group)
            strategy = f"{n_samples} samples -> 1 test, {n_samples-1} train"
            
        else:
            # 5+ samples: use the specified ratio
            n_test = max(1, int(n_samples * test_ratio))
            test_samples_for_group = samples[:n_test]
            train_samples_for_group = samples[n_test:]
            train_samples.extend(train_samples_for_group)
            test_samples.extend(test_samples_for_group)
            strategy = f"{n_samples} samples -> {n_test} test, {len(train_samples_for_group)} train"
        
        print(f"TCGA {tcga_code}: {strategy}")
    
    print("-" * 70)
    print(f"Total train samples: {len(train_samples)}")
    print(f"Total test samples: {len(test_samples)}")
    print(f"Train ratio: {len(train_samples)/(len(train_samples)+len(test_samples)):.3f}")
    print(f"Test ratio: {len(test_samples)/(len(train_samples)+len(test_samples)):.3f}")
    
    return train_samples, test_samples

# Perform stratified split based on TCGA codes for remaining DTI samples
dti_train_samples, dti_test_samples = stratified_tcga_split_dti(
    remaining_dti_samples_df, 
    test_ratio=0.2, 
    random_state=42
)

# Count the samples
print(f"\n📊 DTI Drug Sample Count Summary:")
print(f"Total remaining DTI samples: {len(remaining_dti_samples_df)}")
print(f"Training samples: {len(dti_train_samples)}")
print(f"Test samples: {len(dti_test_samples)}")
print(f"Training percentage: {len(dti_train_samples)/len(remaining_dti_samples_df)*100:.1f}%")
print(f"Test percentage: {len(dti_test_samples)/len(remaining_dti_samples_df)*100:.1f}%")

# Create directories
if not os.path.exists('./data/DrugQA'):
    os.makedirs('./data/DrugQA')
if not os.path.exists('./data/DrugScreen'):
    os.makedirs('./data/DrugScreen')

# Sort the samples before saving
dti_train_samples_sorted = sorted(dti_train_samples)
dti_test_samples_sorted = sorted(dti_test_samples)

remaining_dti_samples_df.to_csv('./data/DrugQA/remaining_dti_samples.csv', index=False)
remaining_dti_samples_df.to_csv('./data/DrugScreen/remaining_dti_samples.csv', index=False)
# Save the sorted train and test samples to CSV files
pd.DataFrame(dti_train_samples_sorted, columns=['depMapID']).to_csv('./data/DrugQA/train_samples.csv', index=False)
pd.DataFrame(dti_test_samples_sorted, columns=['depMapID']).to_csv('./data/DrugQA/test_samples.csv', index=False)
pd.DataFrame(dti_train_samples_sorted, columns=['depMapID']).to_csv('./data/DrugScreen/train_samples.csv', index=False)
pd.DataFrame(dti_test_samples_sorted, columns=['depMapID']).to_csv('./data/DrugScreen/test_samples.csv', index=False)

# Create detailed split information DataFrames
train_dti_samples_df = remaining_dti_samples_df[
    remaining_dti_samples_df['depMapID'].isin(dti_train_samples)
].reset_index(drop=True)

test_dti_samples_df = remaining_dti_samples_df[
    remaining_dti_samples_df['depMapID'].isin(dti_test_samples)
].reset_index(drop=True)

# Display the TCGA distributions in train and test sets
print("\n📋 DTI Drug Train set TCGA distribution:")
print(train_dti_samples_df['tcga_code'].value_counts().sort_index())
print("\n📋 DTI Drug Test set TCGA distribution:")
print(test_dti_samples_df['tcga_code'].value_counts().sort_index())

# Save detailed sample information
train_dti_samples_df.to_csv('./data/DrugQA/train_samples_detailed.csv', index=False)
test_dti_samples_df.to_csv('./data/DrugQA/test_samples_detailed.csv', index=False)
train_dti_samples_df.to_csv('./data/DrugScreen/train_samples_detailed.csv', index=False)
test_dti_samples_df.to_csv('./data/DrugScreen/test_samples_detailed.csv', index=False)

print(f"\n✅ Saved stratified DTI drug train and test splits to:")
print(f"- ./data/DrugQA/train_samples.csv ({len(dti_train_samples_sorted)} samples)")
print(f"- ./data/DrugQA/test_samples.csv ({len(dti_test_samples_sorted)} samples)")
print(f"- ./data/DrugScreen/train_samples.csv ({len(dti_train_samples_sorted)} samples)")
print(f"- ./data/DrugScreen/test_samples.csv ({len(dti_test_samples_sorted)} samples)")
print(f"- Detailed sample information with TCGA annotations also saved")

# Summary of all splits
print(f"\n📈 Complete Sample Split Summary:")
print(f"Original DTI overlapped samples: {len(dti_overlapped_samples)}")
print(f"Target identification samples (CRISPR/RNA): {len(overlapped_dti_crispr_rna_samples)}")
print(f"Drug screening samples (remaining): {len(remaining_dti_samples)}")
print(f"  - DTI Drug Train: {len(dti_train_samples)}")
print(f"  - DTI Drug Test: {len(dti_test_samples)}")

The remaining samples in dti_overlapped_samples after removing overlapped_dti_crispr_rna_samples are:
286
TCGA Code Distribution and Split Strategy for DTI Drug Samples:
----------------------------------------------------------------------
TCGA ALL: 16 samples -> 3 test, 13 train
TCGA BLCA: 9 samples -> 1 test, 8 train
TCGA BRCA: 15 samples -> 3 test, 12 train
TCGA CLL: 3 samples -> 1 test, 2 train
TCGA COAD/READ: 17 samples -> 3 test, 14 train
TCGA DLBC: 20 samples -> 4 test, 16 train
TCGA ESCA: 4 samples -> 1 test, 3 train
TCGA GBM: 7 samples -> 1 test, 6 train
TCGA HNSC: 5 samples -> 1 test, 4 train
TCGA KIRC: 7 samples -> 1 test, 6 train
TCGA LAML: 14 samples -> 2 test, 12 train
TCGA LCML: 8 samples -> 1 test, 7 train
TCGA LGG: 6 samples -> 1 test, 5 train
TCGA LIHC: 9 samples -> 1 test, 8 train
TCGA LUAD: 21 samples -> 4 test, 17 train
TCGA LUSC: 5 samples -> 1 test, 4 train
TCGA MB: 1 sample -> train only
TCGA MESO: 2 samples -> 1 test, 1 train
TCGA MM: 8 samples -> 1 test, 7 tr

### 9.2 Data Integration

#### 9.2.0 Data Preparation

In [387]:
maped_methy_df = final_merged_methy_df.rename(columns=methy_map_dict)
display(maped_methy_df)
mapped_protein_df = final_merged_protein_df.rename(columns=protein_map_dict)
display(mapped_protein_df)
print("Sum of all values in mapped_protein_df:", mapped_protein_df.iloc[:, 1:].sum().sum())
# check if all values in mapped_protein_df are zero
print("Are all values in mapped_protein_df zero?", (mapped_protein_df.iloc[:, 1:] == 0).all().all())

,BioMedGraphica_Conn_ID,ACH-000698,ACH-000489,ACH-000707,ACH-000509,ACH-000522,ACH-000613,ACH-000557,ACH-000614,ACH-000523,...,ACH-000993,ACH-000544,MOLT3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000861,ACH-000706,ACH-000428,ACH-000655,ACH-000273,ACH-000504,ACH-000825
0,BMGC_PM00001,0.51504,0.50000,0.98288,1.00000,0.55295,0.8701,0.90770,0.86540,0.71970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.50000,0.89135,0.28806,0.50752
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
2,BMGC_PM00003,0.01768,0.00000,0.09933,0.00000,0.04551,0.0000,0.00385,0.00499,0.01187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00306,0.00160,0.00000,0.00375
3,BMGC_PM00004,0.00481,0.85522,0.96297,0.68518,0.11157,0.0000,0.03094,0.00935,0.00457,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00704,0.00862,0.91237
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
86234,BMGC_PM86235,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
86235,BMGC_PM86236,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000
86236,BMGC_PM86237,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.00000


,BioMedGraphica_Conn_ID,ACH-000849,ACH-000441,ACH-000248,ACH-000684,ACH-000856,ACH-000348,ACH-000062,ACH-000650,ACH-000484,...,NCIH2030_LUNG_TenPx18,22RV1_PROSTATE_TenPx24,A172_CENTRAL_NERVOUS_SYSTEM_TenPx24,BT20_BREAST_TenPx24,CALU6_LUNG_TenPx24,FADU_UPPER_AERODIGESTIVE_TRACT_TenPx24,KP4_PANCREAS_TenPx24,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24,OVCAR8_OVARY_TenPx24,THP1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE_TenPx24
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sum of all values in mapped_protein_df: -88617.17531696183
Are all values in mapped_protein_df zero? False


#### 9.2.0 All tasks shared the same edge_index and nodes_index system

In [388]:
# create an index data file for nodes and save it to the DTI_data folder
nodes = bmgc_entity_df['BioMedGraphica_Conn_ID'].tolist()
# create nodes index ranging from 0 to len(nodes)-1
nodes_index = np.arange(len(nodes))
nodes_index_data = pd.DataFrame({'Node': nodes, 'Index': nodes_index})
nodes_index_data = pd.merge(nodes_index_data, bmgc_entity_df[['BioMedGraphica_Conn_ID', 'Type']], left_on='Node', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_Conn_ID'])
display(nodes_index_data)
nodes_index_data.to_csv('./data/pretrain_plain_data/nodes_index.csv', index=False)
nodes_index_data.to_csv('./data/pretrain_status_data/nodes_index.csv', index=False)
nodes_index_data.to_csv('./data/TargetQA/nodes_index.csv', index=False)
nodes_index_data.to_csv('./data/TargetScreen/nodes_index.csv', index=False)
nodes_index_data.to_csv('./data/DrugQA/nodes_index.csv', index=False)
nodes_index_data.to_csv('./data/DrugScreen/nodes_index.csv', index=False)
node_index_dict = dict(zip(nodes_index_data['Node'], nodes_index_data['Index']))
# Print the first 10 items of node_index_dict
print("\nFirst 10 items in node_index_dict:")
print(list(node_index_dict.items())[:10])

# Convert the bmgc_relation_df to numpy array and save it to the DTI_data folder as the edge_index.npy
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
edge_index_df = bmgc_relation_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
edge_index_df['BMGC_From_ID'] = edge_index_df['BMGC_From_ID'].map(node_index_dict)
edge_index_df['BMGC_To_ID'] = edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the edge_index_df
print("Null values in edge_index_df:")
print(edge_index_df.isnull().sum())
display(edge_index_df)
# convert the edge_index_df to numpy array and save it to the DTI_data folder
edge_index_array = edge_index_df.to_numpy().T
print('The shape of edge_index_array is:', edge_index_array.shape)
print(edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./data/pretrain_plain_data/edge_index.npy', edge_index_array)
np.save('./data/pretrain_status_data/edge_index.npy', edge_index_array)
np.save('./data/TargetQA/edge_index.npy', edge_index_array)
np.save('./data/TargetScreen/edge_index.npy', edge_index_array)
np.save('./data/DrugQA/edge_index.npy', edge_index_array)
np.save('./data/DrugScreen/edge_index.npy', edge_index_array)
# generate the internal_edge_index by selecting bmgc_relation_df in Type ['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein']
internal_edge_index_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Promoter-Gene', 'Gene-Transcript', 'Transcript-Protein'])].copy()
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
internal_edge_index_df = internal_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
internal_edge_index_df['BMGC_From_ID'] = internal_edge_index_df['BMGC_From_ID'].map(node_index_dict)
internal_edge_index_df['BMGC_To_ID'] = internal_edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the internal_edge_index_df
print("Null values in internal_edge_index_df:")
print(internal_edge_index_df.isnull().sum())
# convert the internal_edge_index_df to numpy array and save it to the DTI_data folder
internal_edge_index_array = internal_edge_index_df.to_numpy().T
print('The shape of internal_edge_index_array is:', internal_edge_index_array.shape)
print(internal_edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./data/pretrain_plain_data/internal_edge_index.npy', internal_edge_index_array)
np.save('./data/pretrain_status_data/internal_edge_index.npy', internal_edge_index_array)
np.save('./data/TargetQA/internal_edge_index.npy', internal_edge_index_array)
np.save('./data/TargetScreen/internal_edge_index.npy', internal_edge_index_array)
np.save('./data/DrugQA/internal_edge_index.npy', internal_edge_index_array)
np.save('./data/DrugScreen/internal_edge_index.npy', internal_edge_index_array)
# generate the ppi_edge_index by selecting bmgc_relation_df in Type ['Protein-Protein']
ppi_edge_index_df = bmgc_relation_df[bmgc_relation_df['Type'].isin(['Protein-Protein'])].copy()
# keep the columns ['BMGC_From_ID', 'BMGC_To_ID']
ppi_edge_index_df = ppi_edge_index_df[['BMGC_From_ID', 'BMGC_To_ID']].copy()
# Map the BMGC_From_ID and BMGC_To_ID by the node_index_dict
ppi_edge_index_df['BMGC_From_ID'] = ppi_edge_index_df['BMGC_From_ID'].map(node_index_dict)
ppi_edge_index_df['BMGC_To_ID'] = ppi_edge_index_df['BMGC_To_ID'].map(node_index_dict)
# check the null values in the ppi_edge_index_df
print("Null values in ppi_edge_index_df:")
print(ppi_edge_index_df.isnull().sum())
# convert the ppi_edge_index_df to numpy array and save it to the DTI_data folder
ppi_edge_index_array = ppi_edge_index_df.to_numpy().T
print('The shape of ppi_edge_index_array is:', ppi_edge_index_array.shape)
print(ppi_edge_index_array)
# save the numpy array to the DTI_data folder
np.save('./data/pretrain_plain_data/ppi_edge_index.npy', ppi_edge_index_array)
np.save('./data/pretrain_status_data/ppi_edge_index.npy', ppi_edge_index_array)
np.save('./data/TargetQA/ppi_edge_index.npy', ppi_edge_index_array)
np.save('./data/TargetScreen/ppi_edge_index.npy', ppi_edge_index_array)
np.save('./data/DrugQA/ppi_edge_index.npy', ppi_edge_index_array)
np.save('./data/DrugScreen/ppi_edge_index.npy', ppi_edge_index_array)

,Node,Index,Type
0,BMGC_PM00001,0,Promoter
1,BMGC_PM00002,1,Promoter
2,BMGC_PM00003,2,Promoter
3,BMGC_PM00004,3,Promoter
4,BMGC_PM00005,4,Promoter
...,...,...,...
834804,BMGC_DG20914,834804,Drug
834805,BMGC_DG20915,834805,Drug
834806,BMGC_DG20916,834806,Drug
834807,BMGC_DG20917,834807,Drug



First 10 items in node_index_dict:
[('BMGC_PM00001', 0), ('BMGC_PM00002', 1), ('BMGC_PM00003', 2), ('BMGC_PM00004', 3), ('BMGC_PM00005', 4), ('BMGC_PM00006', 5), ('BMGC_PM00007', 6), ('BMGC_PM00008', 7), ('BMGC_PM00009', 8), ('BMGC_PM00010', 9)]
Null values in edge_index_df:
BMGC_From_ID    0
BMGC_To_ID      0
dtype: int64


,BMGC_From_ID,BMGC_To_ID
0,0,86238
1,1,86239
2,2,86240
3,3,86241
4,4,86242
...,...,...
27087966,834808,819233
27087967,834808,833344
27087968,834808,826065
27087969,834808,833596


The shape of edge_index_array is: (2, 27087971)
[[     0      1      2 ... 834808 834808 834808]
 [ 86238  86239  86240 ... 826065 833596 833612]]
Null values in internal_edge_index_df:
BMGC_From_ID    0
BMGC_To_ID      0
dtype: int64
The shape of internal_edge_index_array is: (2, 666633)
[[     0      1      2 ... 584362 584493 584494]
 [ 86238  86239  86240 ... 705931 705932 705933]]
Null values in ppi_edge_index_df:
BMGC_From_ID    0
BMGC_To_ID      0
dtype: int64
The shape of ppi_edge_index_array is: (2, 16484820)
[[584539 584539 584539 ... 705933 705933 705933]
 [594942 596590 596896 ... 705339 705340 705341]]


#### 9.2.0 All tasks shared the same textual description

In [474]:
# Create the name and desc files
bmgc_promoter_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Promoter/BioMedGraphica_Conn_Promoter_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_gene_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_transcript_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_protein_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_pathway_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Pathway/BioMedGraphica_Conn_Pathway_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_metabolite_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Metabolite/BioMedGraphica_Conn_Metabolite_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_microbiota_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Microbiota/BioMedGraphica_Conn_Microbiota_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_exposure_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Exposure/BioMedGraphica_Conn_Exposure_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_phenotype_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Phenotype/BioMedGraphica_Conn_Phenotype_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_disease_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Disease/BioMedGraphica_Conn_Disease_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_drug_name_id_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Drug/BioMedGraphica_Conn_Drug_LLM_Name_ID_Combined.csv').drop(columns=['BioMedGraphica_ID'])
# get the number of rows in each dataframe and sum the number of rows for those dataframes
print("bmgc_promoter_name_id_df:", len(bmgc_promoter_name_id_df))
print("bmgc_gene_name_id_df:", len(bmgc_gene_name_id_df))
print("bmgc_transcript_name_id_df:", len(bmgc_transcript_name_id_df))
print("bmgc_protein_name_id_df:", len(bmgc_protein_name_id_df))
print("bmgc_pathway_name_id_df:", len(bmgc_pathway_name_id_df))
print("bmgc_metabolite_name_id_df:", len(bmgc_metabolite_name_id_df))
print("bmgc_microbiota_name_id_df:", len(bmgc_microbiota_name_id_df))
print("bmgc_exposure_name_id_df:", len(bmgc_exposure_name_id_df))
print("bmgc_phenotype_name_id_df:", len(bmgc_phenotype_name_id_df))
print("bmgc_disease_name_id_df:", len(bmgc_disease_name_id_df))
print("bmgc_drug_name_id_df:", len(bmgc_drug_name_id_df))
# sum the number of rows for those dataframes
print("Total number of rows in all dataframes:", len(bmgc_promoter_name_id_df) + len(bmgc_gene_name_id_df) + len(bmgc_transcript_name_id_df) + len(bmgc_protein_name_id_df) + len(bmgc_pathway_name_id_df) + len(bmgc_metabolite_name_id_df) + len(bmgc_microbiota_name_id_df) + len(bmgc_exposure_name_id_df) + len(bmgc_phenotype_name_id_df) + len(bmgc_disease_name_id_df) + len(bmgc_drug_name_id_df))

bmgc_promoter_name_id_df: 86238
bmgc_gene_name_id_df: 86238
bmgc_transcript_name_id_df: 412039
bmgc_protein_name_id_df: 121419
bmgc_pathway_name_id_df: 1930
bmgc_metabolite_name_id_df: 62364
bmgc_microbiota_name_id_df: 1119
bmgc_exposure_name_id_df: 1037
bmgc_phenotype_name_id_df: 19078
bmgc_disease_name_id_df: 22429
bmgc_drug_name_id_df: 20918
Total number of rows in all dataframes: 834809


In [475]:
# left join those name_df into bmgc_entity_df
bmgc_entity_cp_df = bmgc_entity_df[['BioMedGraphica_Conn_ID']].copy()
display(bmgc_entity_cp_df)
# concatenate all the name_id_df into one dataframe
bmgc_name_id_tmp_df = pd.concat([bmgc_promoter_name_id_df, bmgc_gene_name_id_df, bmgc_transcript_name_id_df, bmgc_protein_name_id_df, bmgc_pathway_name_id_df, bmgc_metabolite_name_id_df, bmgc_microbiota_name_id_df, bmgc_exposure_name_id_df, bmgc_phenotype_name_id_df, bmgc_disease_name_id_df, bmgc_drug_name_id_df], ignore_index=True)
display(bmgc_name_id_tmp_df)
# left join the bmgc_name_id_tmp_df into bmgc_entity_cp_df on BioMedGraphica_Conn_ID
bmgc_entity_cp_df = pd.merge(bmgc_entity_cp_df, bmgc_name_id_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
display(bmgc_entity_cp_df)
bmgc_entity_cp_df.to_csv('./data/pretrain_plain_data/bmgc_name.csv', index=False)
bmgc_entity_cp_df.to_csv('./data/pretrain_status_data/bmgc_name.csv', index=False)
bmgc_entity_cp_df.to_csv('./data/TargetScreen/bmgc_name.csv', index=False)
bmgc_entity_cp_df.to_csv('./data/TargetQA/bmgc_name.csv', index=False)
bmgc_entity_cp_df.to_csv('./data/DrugScreen/bmgc_name.csv', index=False)
bmgc_entity_cp_df.to_csv('./data/DrugQA/bmgc_name.csv', index=False)

,BioMedGraphica_Conn_ID
0,BMGC_PM00001
1,BMGC_PM00002
2,BMGC_PM00003
3,BMGC_PM00004
4,BMGC_PM00005
...,...
834804,BMGC_DG20914
834805,BMGC_DG20915
834806,BMGC_DG20916
834807,BMGC_DG20917


,BioMedGraphica_Conn_ID,Names_and_IDs
0,BMGC_PM00001,NCBI Gene ID:1 | HGNC:5 | OMIM ID:138670 | Ref...
1,BMGC_PM00002,N-acetyltransferase 2 Promoter | RefSeq ID:NG_...
2,BMGC_PM00003,RefSeq ID:NC_000020.11 | OMIM ID:608958 | NCBI...
3,BMGC_PM00004,NCBI Gene ID:1000 | HGNC:1759 | RefSeq ID:NC_0...
4,BMGC_PM00005,RefSeq ID:NG_029764.2 | RefSeq ID:NT_187519.1 ...
...,...,...
834804,BMGC_DG20914,Zoxazolamine | DrugBank ID:DB19372
834805,BMGC_DG20915,Megestrol | DrugBank ID:DB19378
834806,BMGC_DG20916,DrugBank ID:DB19381 | AZACYCLONOL
834807,BMGC_DG20917,DrugBank ID:DB19397 | AZD8630


,BioMedGraphica_Conn_ID,Names_and_IDs
0,BMGC_PM00001,NCBI Gene ID:1 | HGNC:5 | OMIM ID:138670 | Ref...
1,BMGC_PM00002,N-acetyltransferase 2 Promoter | RefSeq ID:NG_...
2,BMGC_PM00003,RefSeq ID:NC_000020.11 | OMIM ID:608958 | NCBI...
3,BMGC_PM00004,NCBI Gene ID:1000 | HGNC:1759 | RefSeq ID:NC_0...
4,BMGC_PM00005,RefSeq ID:NG_029764.2 | RefSeq ID:NT_187519.1 ...
...,...,...
834804,BMGC_DG20914,Zoxazolamine | DrugBank ID:DB19372
834805,BMGC_DG20915,Megestrol | DrugBank ID:DB19378
834806,BMGC_DG20916,DrugBank ID:DB19381 | AZACYCLONOL
834807,BMGC_DG20917,DrugBank ID:DB19397 | AZD8630


In [476]:
bmgc_promoter_desc_df = bmgc_promoter_name_id_df.drop(columns=['Names_and_IDs'], axis=1).copy()
bmgc_promoter_desc_df['Description'] = np.nan # add the Description column to bmgc_promoter_name_df with NaN values
bmgc_gene_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Gene/BioMedGraphica_Conn_Gene_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_transcript_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_protein_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_pathway_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Pathway/BioMedGraphica_Conn_Pathway_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_metabolite_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Metabolite/BioMedGraphica_Conn_Metabolite_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_microbiota_desc_df = bmgc_microbiota_name_id_df.drop(columns=['Names_and_IDs'], axis=1).copy()
bmgc_microbiota_desc_df['Description'] = np.nan # add the Description column to bmgc_microbiota_name_df with NaN values
bmgc_exposure_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Exposure/BioMedGraphica_Conn_Exposure_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_phenotype_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Phenotype/BioMedGraphica_Conn_Phenotype_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_disease_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Disease/BioMedGraphica_Conn_Disease_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])
bmgc_drug_desc_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Drug/BioMedGraphica_Conn_Drug_Description_Combined.csv').drop(columns=['BioMedGraphica_ID'])

In [477]:
bmgc_desc_tmp_df = pd.concat([bmgc_promoter_desc_df, bmgc_gene_desc_df, bmgc_transcript_desc_df, bmgc_protein_desc_df, bmgc_pathway_desc_df, bmgc_metabolite_desc_df, bmgc_microbiota_desc_df, bmgc_exposure_desc_df, bmgc_phenotype_desc_df, bmgc_disease_desc_df, bmgc_drug_desc_df], ignore_index=True)
display(bmgc_desc_tmp_df)
# left join the bmgc_desc_tmp_df into bmgc_entity_cp_df on BioMedGraphica_Conn_ID
bmgc_entity_cp_df = bmgc_entity_df[['BioMedGraphica_Conn_ID']].copy()
bmgc_desc_df = pd.merge(bmgc_entity_cp_df, bmgc_desc_tmp_df, on='BioMedGraphica_Conn_ID', how='left')
# check the null values in the bmgc_desc_df
print("Null values in bmgc_desc_df:")
print(bmgc_desc_df.isnull().sum())
# fill the NaN values in the Description column with empty string
bmgc_desc_df['Description'] = bmgc_desc_df['Description'].fillna(' ')
# recheck the null values in the bmgc_desc_df
print("Null values in bmgc_desc_df:")
print(bmgc_desc_df.isnull().sum())
display(bmgc_desc_df)
bmgc_desc_df.to_csv('./data/pretrain_plain_data/bmgc_desc.csv', index=False)
bmgc_desc_df.to_csv('./data/pretrain_status_data/bmgc_desc.csv', index=False)
bmgc_desc_df.to_csv('./data/TargetScreen/bmgc_desc.csv', index=False)
bmgc_desc_df.to_csv('./data/TargetQA/bmgc_desc.csv', index=False)
bmgc_desc_df.to_csv('./data/DrugScreen/bmgc_desc.csv', index=False)
bmgc_desc_df.to_csv('./data/DrugQA/bmgc_desc.csv', index=False)

,BioMedGraphica_Conn_ID,Description
0,BMGC_PM00001,NaN
1,BMGC_PM00002,NaN
2,BMGC_PM00003,NaN
3,BMGC_PM00004,NaN
4,BMGC_PM00005,NaN
...,...,...
834804,BMGC_DG20914,NaN
834805,BMGC_DG20915,NaN
834806,BMGC_DG20916,NaN
834807,BMGC_DG20917,DrugBank: AZD8630 is an experimental drug unde...


Null values in bmgc_desc_df:
BioMedGraphica_Conn_ID         0
Description               221047
dtype: int64
Null values in bmgc_desc_df:
BioMedGraphica_Conn_ID    0
Description               0
dtype: int64


,BioMedGraphica_Conn_ID,Description
0,BMGC_PM00001,
1,BMGC_PM00002,
2,BMGC_PM00003,
3,BMGC_PM00004,
4,BMGC_PM00005,
...,...,...
834804,BMGC_DG20914,
834805,BMGC_DG20915,
834806,BMGC_DG20916,
834807,BMGC_DG20917,DrugBank: AZD8630 is an experimental drug unde...


#### 9.2.1 Pretrain data integration (plain + status)

In [399]:
print("The overlapped samples dataframe for methylation between pretrain_plain_samples and pretrain_status_samples are:")
# insert the samples that is in the overlapped_samples but not in the maped_methy_df
for sample in pretrain_plain_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the pretraining_samples
pretrain_plain_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + pretrain_plain_samples]
display(pretrain_plain_methy_df)
# insert the samples that is in the overlapped_samples but not in the maped_methy_df
for sample in pretrain_status_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the pretraining_samples
pretrain_status_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + pretrain_status_samples]
display(pretrain_status_methy_df)

print("The overlapped samples dataframe for protein between pretrain_plain_samples and pretrain_status_samples are:")
# insert the samples that is in the pretraining_samples but not in the mapped_protein_df
for sample in pretrain_plain_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the pretraining_samples
pretrain_plain_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + pretrain_plain_samples]
display(pretrain_plain_protein_df)
# insert the samples that is in the pretraining_samples but not in the mapped_protein_df
for sample in pretrain_status_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the pretrain_status_samples
pretrain_status_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + pretrain_status_samples]
display(pretrain_status_protein_df)

The overlapped samples dataframe for methylation between pretrain_plain_samples and pretrain_status_samples are:


,BioMedGraphica_Conn_ID,ACH-000017,ACH-000022,ACH-000026,ACH-000028,ACH-000063,ACH-000066,ACH-000085,ACH-000086,ACH-000119,...,ACH-000844,ACH-000889,ACH-000942,ACH-000993,ACH-001001,ACH-001036,ACH-001098,ACH-001129,ACH-001188,ACH-001239
0,BMGC_PM00001,0.50165,0.49325,0.0,0.84212,0.48636,0.65552,0.09454,0.69810,0.0,...,0.05468,0.86674,0.45000,0.0,0.0,0.0,0.0,0.72600,0.0,0.80595
1,BMGC_PM00002,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
2,BMGC_PM00003,0.00300,0.00649,0.0,0.00141,0.00000,0.00000,0.00943,0.00064,0.0,...,0.02074,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.01409
3,BMGC_PM00004,0.01952,0.00237,0.0,0.03815,0.02166,0.03068,0.01391,0.01534,0.0,...,0.01450,0.87374,0.01292,0.0,0.0,0.0,0.0,0.26234,0.0,0.00195
4,BMGC_PM00005,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
86234,BMGC_PM86235,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
86235,BMGC_PM86236,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
86236,BMGC_PM86237,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


,BioMedGraphica_Conn_ID,ACH-000005,ACH-000011,ACH-000013,ACH-000014,ACH-000016,ACH-000027,ACH-000031,ACH-000032,ACH-000037,...,ACH-001113,ACH-001130,ACH-001145,ACH-001150,ACH-001162,ACH-001171,ACH-001183,ACH-001194,ACH-001318,ACH-001321
0,BMGC_PM00001,0.70874,1.00000,0.66057,0.88330,0.0,0.51282,0.12250,0.70000,0.15385,...,0.0,0.47703,0.0,0.42555,0.47785,0.0,0.0,0.0,0.50254,0.63336
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
2,BMGC_PM00003,0.06977,0.00000,0.00507,0.01536,0.0,0.00260,0.00351,0.01941,0.01643,...,0.0,0.00206,0.0,0.01197,0.00757,0.0,0.0,0.0,0.00453,0.00000
3,BMGC_PM00004,0.59602,0.01294,0.04124,0.01022,0.0,0.01732,0.00056,0.55973,0.00000,...,0.0,0.05561,0.0,0.01421,0.73156,0.0,0.0,0.0,0.01198,0.01604
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
86234,BMGC_PM86235,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
86235,BMGC_PM86236,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
86236,BMGC_PM86237,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000


The overlapped samples dataframe for protein between pretrain_plain_samples and pretrain_status_samples are:


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4084772069.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4084772069.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4084772069.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

,BioMedGraphica_Conn_ID,ACH-000017,ACH-000022,ACH-000026,ACH-000028,ACH-000063,ACH-000066,ACH-000085,ACH-000086,ACH-000119,...,ACH-000844,ACH-000889,ACH-000942,ACH-000993,ACH-001001,ACH-001036,ACH-001098,ACH-001129,ACH-001188,ACH-001239
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4084772069.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4084772069.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\4084772069.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

,BioMedGraphica_Conn_ID,ACH-000005,ACH-000011,ACH-000013,ACH-000014,ACH-000016,ACH-000027,ACH-000031,ACH-000032,ACH-000037,...,ACH-001113,ACH-001130,ACH-001145,ACH-001150,ACH-001162,ACH-001171,ACH-001183,ACH-001194,ACH-001318,ACH-001321
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [400]:
# fill in the missing samples in the final_merged_gene_df for pretrain_plain_samples
for sample in pretrain_plain_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df for pretrain_plain_samples
for sample in pretrain_plain_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the pretrain_plain_samples
pretrain_plain_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + pretrain_plain_samples]
display(pretrain_plain_gene_df)
pretrain_plain_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + pretrain_plain_samples]
display(pretrain_plain_transcript_df)

# fill in the missing samples in the final_merged_gene_df for pretrain_status_samples
for sample in pretrain_status_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df for pretrain_status_samples
for sample in pretrain_status_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the pretrain_status_samples
pretrain_status_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + pretrain_status_samples]
display(pretrain_status_gene_df)
pretrain_status_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + pretrain_status_samples]
display(pretrain_status_transcript_df)

,BioMedGraphica_Conn_ID,ACH-000017,ACH-000022,ACH-000026,ACH-000028,ACH-000063,ACH-000066,ACH-000085,ACH-000086,ACH-000119,...,ACH-000844,ACH-000889,ACH-000942,ACH-000993,ACH-001001,ACH-001036,ACH-001098,ACH-001129,ACH-001188,ACH-001239
0,BMGC_GN00001,1.085323,0.947735,0.827625,0.937181,1.023235,1.504031,1.412782,1.047031,1.000624,...,1.051902,1.073072,1.032968,1.011142,0.781099,1.181581,1.508210,0.980249,0.920990,1.028902
1,BMGC_GN00002,0.587065,0.669167,0.801330,0.594580,0.992253,0.586154,1.169745,1.010447,0.996161,...,0.743053,1.398623,0.992872,0.994860,0.918612,1.093844,0.747614,1.497803,1.027683,0.820273
2,BMGC_GN00003,1.515145,1.318432,1.175619,0.891041,1.002725,0.619833,1.561758,1.595148,1.002671,...,1.219133,1.436852,1.032843,1.009187,1.673403,1.075427,1.130588,1.013485,1.011336,1.254385
3,BMGC_GN00004,0.912424,0.876892,0.815257,0.825214,0.996676,0.595849,0.828851,0.959989,0.984786,...,1.549725,1.356231,0.984896,0.996060,0.925511,1.429757,0.727942,1.388784,1.038347,1.020457
4,BMGC_GN00005,0.755303,0.989199,1.148937,1.178981,1.006351,1.187420,1.183994,0.579938,0.999671,...,1.062868,1.280013,1.010769,1.007511,0.899186,1.372584,1.052046,1.001074,1.498685,1.403284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_GN86234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86234,BMGC_GN86235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86235,BMGC_GN86236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86236,BMGC_GN86237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,BioMedGraphica_Conn_ID,ACH-000017,ACH-000022,ACH-000026,ACH-000028,ACH-000063,ACH-000066,ACH-000085,ACH-000086,ACH-000119,...,ACH-000844,ACH-000889,ACH-000942,ACH-000993,ACH-001001,ACH-001036,ACH-001098,ACH-001129,ACH-001188,ACH-001239
0,BMGC_TS000001,8.472621,7.566681,7.783121,7.873038,7.212185,6.909611,5.979267,7.439952,7.217133,...,7.797436,7.091215,7.324826,6.095544,6.173282,6.530392,7.358607,6.398834,7.977339,7.570824
1,BMGC_TS000002,6.358975,5.671896,6.396962,6.018021,6.562234,6.994483,7.391298,6.544272,6.348819,...,6.839740,6.631997,6.592223,6.615116,6.490956,7.481551,7.472335,7.119044,6.128324,6.493308
2,BMGC_TS000003,6.017817,5.144851,4.399958,5.303545,3.896775,4.751705,6.028516,3.780447,4.241893,...,5.980503,4.763219,3.556028,4.313112,4.900645,4.200231,4.966883,4.304928,3.319582,4.251047
3,BMGC_TS000004,7.169858,4.858171,6.106998,6.679738,4.583381,5.762251,7.064322,5.614309,4.474898,...,6.525218,6.487419,6.429365,4.358429,6.319403,5.946550,5.796640,4.975769,6.356670,4.805903
4,BMGC_TS000005,0.005892,2.260473,3.123043,0.451694,1.853420,0.175017,1.594867,1.618716,0.208682,...,0.175017,1.442286,0.152104,1.013721,1.133261,0.235919,2.935896,-0.059982,0.113665,0.252312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412034,BMGC_TS412035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
412035,BMGC_TS412036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
412036,BMGC_TS412037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
412037,BMGC_TS412038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,BioMedGraphica_Conn_ID,ACH-000005,ACH-000011,ACH-000013,ACH-000014,ACH-000016,ACH-000027,ACH-000031,ACH-000032,ACH-000037,...,ACH-001113,ACH-001130,ACH-001145,ACH-001150,ACH-001162,ACH-001171,ACH-001183,ACH-001194,ACH-001318,ACH-001321
0,BMGC_GN00001,1.128141,1.067416,0.673066,1.131172,0.0,1.041115,0.838548,0.990240,1.078296,...,1.190599,0.967907,1.000632,0.0,0.536252,0.0,0.918424,1.039522,1.116916,1.042680
1,BMGC_GN00002,1.112178,0.766166,0.892887,0.846887,0.0,1.054138,0.555273,0.977889,0.854899,...,0.611758,1.052372,1.033542,0.0,1.029018,0.0,0.527728,0.535935,0.796347,0.524765
2,BMGC_GN00003,0.778284,1.122100,1.388424,1.668880,0.0,1.040417,1.237650,0.995680,1.530764,...,1.618570,1.045554,1.348982,0.0,1.017926,0.0,1.845727,1.309566,1.323527,1.039671
3,BMGC_GN00004,1.123984,0.789885,0.748113,0.875289,0.0,0.541999,1.081287,0.973006,1.391903,...,1.099402,1.063535,0.681409,0.0,0.995275,0.0,1.072174,0.853730,0.863507,1.010830
4,BMGC_GN00005,1.116293,1.175879,1.057079,1.284645,0.0,1.060672,1.125485,1.470936,1.274312,...,1.442654,1.482049,1.035727,0.0,1.013036,0.0,1.035917,1.394770,1.211618,1.542534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_GN86234,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
86234,BMGC_GN86235,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
86235,BMGC_GN86236,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
86236,BMGC_GN86237,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


,BioMedGraphica_Conn_ID,ACH-000005,ACH-000011,ACH-000013,ACH-000014,ACH-000016,ACH-000027,ACH-000031,ACH-000032,ACH-000037,...,ACH-001113,ACH-001130,ACH-001145,ACH-001150,ACH-001162,ACH-001171,ACH-001183,ACH-001194,ACH-001318,ACH-001321
0,BMGC_TS000001,5.923191,7.457561,6.984308,7.446743,7.415651,8.098085,6.863893,7.156174,6.731636,...,7.230242,0.0,7.931638,6.872406,0.0,0.0,7.006642,7.253365,7.269873,7.552740
1,BMGC_TS000002,7.492971,6.289055,6.297061,7.779837,6.180162,6.419696,7.167468,7.090760,6.580654,...,7.524420,0.0,7.138538,7.700997,0.0,0.0,6.815641,6.686938,7.468948,7.706810
2,BMGC_TS000003,4.438237,3.481874,4.677355,4.323129,3.532014,5.170453,5.547268,4.040626,4.047278,...,5.232906,0.0,5.186650,4.237452,0.0,0.0,5.179700,4.215455,4.649932,5.481446
3,BMGC_TS000004,7.091255,5.503175,6.458677,5.507597,4.635526,5.898858,5.635720,1.803751,5.111995,...,6.953174,0.0,7.281816,5.634324,0.0,0.0,6.525825,6.583393,6.262018,6.471680
4,BMGC_TS000005,0.080957,1.473322,0.197552,0.442388,2.800069,2.903703,3.570329,-0.059982,3.119116,...,3.382773,0.0,1.600665,2.903310,0.0,0.0,1.336995,0.909155,1.032021,-0.046550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412034,BMGC_TS412035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
412035,BMGC_TS412036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
412036,BMGC_TS412037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
412037,BMGC_TS412038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [401]:
pretrain_plain_omics_df = pd.concat([pretrain_plain_methy_df, pretrain_plain_gene_df, pretrain_plain_transcript_df, pretrain_plain_protein_df], axis=0).reset_index(drop=True)
display(pretrain_plain_omics_df)
pretrain_plain_feat_df = pd.merge(bmgc_entity_df, pretrain_plain_omics_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_ID', 'Type'])
pretrain_plain_feat_df = pretrain_plain_feat_df.fillna(0.0)
display(pretrain_plain_feat_df)
pretrain_status_omics_df = pd.concat([pretrain_status_methy_df, pretrain_status_gene_df, pretrain_status_transcript_df, pretrain_status_protein_df], axis=0).reset_index(drop=True)
display(pretrain_status_omics_df)
pretrain_status_feat_df = pd.merge(bmgc_entity_df, pretrain_status_omics_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_ID', 'Type'])
pretrain_status_feat_df = pretrain_status_feat_df.fillna(0.0)
display(pretrain_status_feat_df)

,BioMedGraphica_Conn_ID,ACH-000017,ACH-000022,ACH-000026,ACH-000028,ACH-000063,ACH-000066,ACH-000085,ACH-000086,ACH-000119,...,ACH-000844,ACH-000889,ACH-000942,ACH-000993,ACH-001001,ACH-001036,ACH-001098,ACH-001129,ACH-001188,ACH-001239
0,BMGC_PM00001,0.50165,0.49325,0.0,0.84212,0.48636,0.65552,0.09454,0.69810,0.0,...,0.05468,0.86674,0.45000,0.0,0.0,0.0,0.0,0.72600,0.0,0.80595
1,BMGC_PM00002,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
2,BMGC_PM00003,0.00300,0.00649,0.0,0.00141,0.00000,0.00000,0.00943,0.00064,0.0,...,0.02074,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.01409
3,BMGC_PM00004,0.01952,0.00237,0.0,0.03815,0.02166,0.03068,0.01391,0.01534,0.0,...,0.01450,0.87374,0.01292,0.0,0.0,0.0,0.0,0.26234,0.0,0.00195
4,BMGC_PM00005,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705929,BMGC_PT121415,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
705930,BMGC_PT121416,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
705931,BMGC_PT121417,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
705932,BMGC_PT121418,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


,BioMedGraphica_Conn_ID,ACH-000017,ACH-000022,ACH-000026,ACH-000028,ACH-000063,ACH-000066,ACH-000085,ACH-000086,ACH-000119,...,ACH-000844,ACH-000889,ACH-000942,ACH-000993,ACH-001001,ACH-001036,ACH-001098,ACH-001129,ACH-001188,ACH-001239
0,BMGC_PM00001,0.50165,0.49325,0.0,0.84212,0.48636,0.65552,0.09454,0.69810,0.0,...,0.05468,0.86674,0.45000,0.0,0.0,0.0,0.0,0.72600,0.0,0.80595
1,BMGC_PM00002,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
2,BMGC_PM00003,0.00300,0.00649,0.0,0.00141,0.00000,0.00000,0.00943,0.00064,0.0,...,0.02074,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.01409
3,BMGC_PM00004,0.01952,0.00237,0.0,0.03815,0.02166,0.03068,0.01391,0.01534,0.0,...,0.01450,0.87374,0.01292,0.0,0.0,0.0,0.0,0.26234,0.0,0.00195
4,BMGC_PM00005,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834804,BMGC_DG20914,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
834805,BMGC_DG20915,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
834806,BMGC_DG20916,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000
834807,BMGC_DG20917,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,...,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000


,BioMedGraphica_Conn_ID,ACH-000005,ACH-000011,ACH-000013,ACH-000014,ACH-000016,ACH-000027,ACH-000031,ACH-000032,ACH-000037,...,ACH-001113,ACH-001130,ACH-001145,ACH-001150,ACH-001162,ACH-001171,ACH-001183,ACH-001194,ACH-001318,ACH-001321
0,BMGC_PM00001,0.70874,1.00000,0.66057,0.88330,0.0,0.51282,0.12250,0.70000,0.15385,...,0.0,0.47703,0.0,0.42555,0.47785,0.0,0.0,0.0,0.50254,0.63336
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
2,BMGC_PM00003,0.06977,0.00000,0.00507,0.01536,0.0,0.00260,0.00351,0.01941,0.01643,...,0.0,0.00206,0.0,0.01197,0.00757,0.0,0.0,0.0,0.00453,0.00000
3,BMGC_PM00004,0.59602,0.01294,0.04124,0.01022,0.0,0.01732,0.00056,0.55973,0.00000,...,0.0,0.05561,0.0,0.01421,0.73156,0.0,0.0,0.0,0.01198,0.01604
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705929,BMGC_PT121415,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
705930,BMGC_PT121416,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
705931,BMGC_PT121417,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
705932,BMGC_PT121418,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000


,BioMedGraphica_Conn_ID,ACH-000005,ACH-000011,ACH-000013,ACH-000014,ACH-000016,ACH-000027,ACH-000031,ACH-000032,ACH-000037,...,ACH-001113,ACH-001130,ACH-001145,ACH-001150,ACH-001162,ACH-001171,ACH-001183,ACH-001194,ACH-001318,ACH-001321
0,BMGC_PM00001,0.70874,1.00000,0.66057,0.88330,0.0,0.51282,0.12250,0.70000,0.15385,...,0.0,0.47703,0.0,0.42555,0.47785,0.0,0.0,0.0,0.50254,0.63336
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
2,BMGC_PM00003,0.06977,0.00000,0.00507,0.01536,0.0,0.00260,0.00351,0.01941,0.01643,...,0.0,0.00206,0.0,0.01197,0.00757,0.0,0.0,0.0,0.00453,0.00000
3,BMGC_PM00004,0.59602,0.01294,0.04124,0.01022,0.0,0.01732,0.00056,0.55973,0.00000,...,0.0,0.05561,0.0,0.01421,0.73156,0.0,0.0,0.0,0.01198,0.01604
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834804,BMGC_DG20914,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
834805,BMGC_DG20915,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
834806,BMGC_DG20916,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000
834807,BMGC_DG20917,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,...,0.0,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000


In [402]:
# convert the pretrain_plain_df and pretrain_status_df to numpy arrays and transpose them
pretrain_plain_array = pretrain_plain_feat_df.drop(columns=['BioMedGraphica_Conn_ID']).values.T
pretrain_status_array = pretrain_status_feat_df.drop(columns=['BioMedGraphica_Conn_ID']).values.T
print("Shape of pretrain_plain_array:", pretrain_plain_array.shape)
print("Shape of pretrain_status_array:", pretrain_status_array.shape)
# Save the numpy arrays to .npy files
np.save('./data/pretrain_plain_data/pretrain_plain_feature.npy', pretrain_plain_array)
np.save('./data/pretrain_status_data/pretrain_status_feature.npy', pretrain_status_array)

Shape of pretrain_plain_array: (67, 834809)
Shape of pretrain_status_array: (269, 834809)


##### 9.2.1.1 Pretrain Status

In [403]:
# convert pretrain_status_df to numpy array label with  cancerous as 1 and non-cancerous as 0
pretrain_status_label = np.array([1 if status == 'cancerous' else 0 for status in pretrain_status_df['cancerous_status']])
# Save the pretrain_status_label to a .npy file
np.save('./data/pretrain_status_data/pretrain_status_label.npy', pretrain_status_label)
print("Shape of pretrain_status_label:", pretrain_status_label.shape)

# assign the label by pretrain_status_training_samples and pretrain_status_test_samples
pretrain_status_train_label = np.array([1 if status == 'cancerous' else 0 for status in pretrain_status_train_df['cancerous_status']])
pretrain_status_test_label = np.array([1 if status == 'cancerous' else 0 for status in pretrain_status_test_df['cancerous_status']])
# assign the feat by pretrain_status_training_samples and pretrain_status_test_samples
pretrain_status_train_feat = pretrain_status_feat_df[['BioMedGraphica_Conn_ID'] + pretrain_status_train_samples].drop(columns=['BioMedGraphica_Conn_ID']).values.T
pretrain_status_test_feat = pretrain_status_feat_df[['BioMedGraphica_Conn_ID'] + pretrain_status_test_samples].drop(columns=['BioMedGraphica_Conn_ID']).values.T

# Save the training and test features and labels to .npy files
np.save('./data/pretrain_status_data/pretrain_status_train_feature.npy', pretrain_status_train_feat)
np.save('./data/pretrain_status_data/pretrain_status_test_feature.npy', pretrain_status_test_feat)
np.save('./data/pretrain_status_data/pretrain_status_train_label.npy', pretrain_status_train_label)
np.save('./data/pretrain_status_data/pretrain_status_test_label.npy', pretrain_status_test_label)
print("Shape of pretrain_status_train_feat:", pretrain_status_train_feat.shape)
print("Shape of pretrain_status_test_feat:", pretrain_status_test_feat.shape)
print("Shape of pretrain_status_train_label:", pretrain_status_train_label.shape)
print("Shape of pretrain_status_test_label:", pretrain_status_test_label.shape)
# Print the cancerous/non-cancerous distribution in training and test sets
print(f"\n📈 Pretrain Status Training Set Distribution (Numpy Files):\n"
      f"Cancerous: {np.sum(pretrain_status_train_label == 1)}, Non-cancerous: {np.sum(pretrain_status_train_label == 0)}")
print(f"📈 Pretrain Status Test Set Distribution (Numpy Files):\n"
      f"Cancerous: {np.sum(pretrain_status_test_label == 1)}, Non-cancerous: {np.sum(pretrain_status_test_label == 0)}")

Shape of pretrain_status_label: (269,)
Shape of pretrain_status_train_feat: (215, 834809)
Shape of pretrain_status_test_feat: (54, 834809)
Shape of pretrain_status_train_label: (215,)
Shape of pretrain_status_test_label: (54,)

📈 Pretrain Status Training Set Distribution (Numpy Files):
Cancerous: 190, Non-cancerous: 25
📈 Pretrain Status Test Set Distribution (Numpy Files):
Cancerous: 48, Non-cancerous: 6


#### 9.2.2 Target data integration

In [404]:
print("The overlapped samples dataframe for methylation between overlapped_dti_crispr_rna_samples:")
# insert the samples that is in the overlapped_samples but not in the maped_methy_df
for sample in overlapped_dti_crispr_rna_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the overlapped_dti_crispr_rna_samples
target_crispr_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + overlapped_dti_crispr_rna_samples]
display(target_crispr_methy_df)

print("The overlapped samples dataframe for protein between overlapped_dti_crispr_rna_samples are:")
# insert the samples that is in the pretraining_samples but not in the mapped_protein_df
for sample in overlapped_dti_crispr_rna_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the overlapped_dti_crispr_rna_samples
target_crispr_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + overlapped_dti_crispr_rna_samples]
display(target_crispr_protein_df)

The overlapped samples dataframe for methylation between overlapped_dti_crispr_rna_samples:


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\823047421.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  maped_methy_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\823047421.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  maped_methy_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\823047421.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all 

,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000015,ACH-000018,ACH-000019,ACH-000021,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_PM00001,1.00000,0.77900,0.07045,0.92549,0.93426,0.0,0.30908,0.81250,0.0,...,0.0,0.29050,0.73685,0.0,0.0,0.70055,0.0,0.46825,0.67335,0.6964
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
2,BMGC_PM00003,0.00000,0.05796,0.00686,0.00112,0.00286,0.0,0.00000,0.00999,0.0,...,0.0,0.00433,0.02217,0.0,0.0,0.01297,0.0,0.01149,0.00573,0.0000
3,BMGC_PM00004,0.02149,0.59846,0.88734,0.94058,0.44256,0.0,0.02831,0.03987,0.0,...,0.0,0.32492,0.90407,0.0,0.0,0.00000,0.0,0.11109,0.00307,0.0058
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
86234,BMGC_PM86235,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
86235,BMGC_PM86236,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
86236,BMGC_PM86237,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000


The overlapped samples dataframe for protein between overlapped_dti_crispr_rna_samples are:


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\823047421.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\823047421.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\823047421.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000015,ACH-000018,ACH-000019,ACH-000021,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [405]:
# fill in the missing samples in the final_merged_gene_df for overlapped_dti_crispr_rna_samples
for sample in overlapped_dti_crispr_rna_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df for overlapped_dti_crispr_rna_samples
for sample in overlapped_dti_crispr_rna_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the overlapped_dti_crispr_rna_samples
target_crispr_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + overlapped_dti_crispr_rna_samples]
display(target_crispr_gene_df)
target_crispr_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + overlapped_dti_crispr_rna_samples]
display(target_crispr_transcript_df)

,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000015,ACH-000018,ACH-000019,ACH-000021,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_GN00001,0.851056,1.323804,0.937570,0.980206,1.111580,0.660613,0.917698,0.956259,1.020936,...,0.937035,0.967644,0.958509,0.993997,1.091848,1.418260,1.656834,0.873136,1.330477,0.977833
1,BMGC_GN00002,1.157653,1.072377,0.941289,0.972136,0.776816,0.557800,0.881935,0.551096,1.118855,...,1.052398,1.000457,0.992196,0.589237,0.694145,0.587443,0.414771,0.968923,0.697147,0.778931
2,BMGC_GN00003,1.498803,0.777885,0.958520,2.383638,1.209065,1.156748,2.014502,1.279922,1.189335,...,1.069887,0.974388,1.002760,1.457635,0.676150,1.564420,1.091653,1.433025,1.692643,1.575951
3,BMGC_GN00004,0.669074,1.064669,0.942657,0.562637,0.776307,0.813608,0.902359,0.835197,1.514296,...,1.086093,1.011928,0.997424,0.630750,1.424901,1.040305,0.782961,0.964505,1.325709,0.773253
4,BMGC_GN00005,1.172832,1.032660,1.416574,1.064513,1.134180,1.366983,0.923231,1.047466,0.729370,...,1.034191,1.007370,0.998813,1.254152,1.090315,1.011334,1.208036,1.132882,0.938454,0.772272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_GN86234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86234,BMGC_GN86235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86235,BMGC_GN86236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86236,BMGC_GN86237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000015,ACH-000018,ACH-000019,ACH-000021,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_TS000001,6.643393,6.035001,7.372120,7.814309,6.979760,8.022521,6.887456,7.527567,7.001709,...,7.319774,6.999298,7.314172,0.0,0.0,6.735296,0.0,6.801926,7.183322,6.147807
1,BMGC_TS000002,6.959654,7.571516,6.084658,7.783578,6.860788,5.733721,6.814315,5.928845,6.161633,...,6.879119,6.896168,6.657150,0.0,0.0,6.683453,0.0,7.642422,5.817284,6.617196
2,BMGC_TS000003,5.351106,4.982294,5.535330,4.891940,4.700858,4.168149,3.806256,5.054789,3.674823,...,4.787896,5.823450,4.625168,0.0,0.0,5.475867,0.0,4.209387,3.840053,3.613618
3,BMGC_TS000004,5.868467,6.293503,5.687442,6.753958,6.896627,5.677143,5.758296,6.794872,5.548689,...,6.582769,6.423730,7.328425,0.0,0.0,5.867169,0.0,6.223950,4.805016,5.846684
4,BMGC_TS000005,0.186332,-0.033249,0.031376,0.665821,0.186332,2.152987,3.472767,0.976367,0.931453,...,1.989077,0.284142,0.230671,0.0,0.0,3.012828,0.0,2.922400,0.043941,0.175017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412034,BMGC_TS412035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
412035,BMGC_TS412036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
412036,BMGC_TS412037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
412037,BMGC_TS412038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [406]:
target_crispr_omics_df = pd.concat([target_crispr_methy_df, target_crispr_gene_df, target_crispr_transcript_df, target_crispr_protein_df], axis=0).reset_index(drop=True)
display(target_crispr_omics_df)
target_crispr_df = pd.merge(bmgc_entity_df, target_crispr_omics_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_ID', 'Type'])
target_crispr_df = target_crispr_df.fillna(0.0)
display(target_crispr_df)

,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000015,ACH-000018,ACH-000019,ACH-000021,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_PM00001,1.00000,0.77900,0.07045,0.92549,0.93426,0.0,0.30908,0.81250,0.0,...,0.0,0.29050,0.73685,0.0,0.0,0.70055,0.0,0.46825,0.67335,0.6964
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
2,BMGC_PM00003,0.00000,0.05796,0.00686,0.00112,0.00286,0.0,0.00000,0.00999,0.0,...,0.0,0.00433,0.02217,0.0,0.0,0.01297,0.0,0.01149,0.00573,0.0000
3,BMGC_PM00004,0.02149,0.59846,0.88734,0.94058,0.44256,0.0,0.02831,0.03987,0.0,...,0.0,0.32492,0.90407,0.0,0.0,0.00000,0.0,0.11109,0.00307,0.0058
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705929,BMGC_PT121415,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
705930,BMGC_PT121416,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
705931,BMGC_PT121417,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
705932,BMGC_PT121418,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000


,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000015,ACH-000018,ACH-000019,ACH-000021,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_PM00001,1.00000,0.77900,0.07045,0.92549,0.93426,0.0,0.30908,0.81250,0.0,...,0.0,0.29050,0.73685,0.0,0.0,0.70055,0.0,0.46825,0.67335,0.6964
1,BMGC_PM00002,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
2,BMGC_PM00003,0.00000,0.05796,0.00686,0.00112,0.00286,0.0,0.00000,0.00999,0.0,...,0.0,0.00433,0.02217,0.0,0.0,0.01297,0.0,0.01149,0.00573,0.0000
3,BMGC_PM00004,0.02149,0.59846,0.88734,0.94058,0.44256,0.0,0.02831,0.03987,0.0,...,0.0,0.32492,0.90407,0.0,0.0,0.00000,0.0,0.11109,0.00307,0.0058
4,BMGC_PM00005,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834804,BMGC_DG20914,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
834805,BMGC_DG20915,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
834806,BMGC_DG20916,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000
834807,BMGC_DG20917,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0000


##### 9.2.2.1 Target Screen

In [407]:
# Select the features from target_crispr_df by target_crispr_train_samples and target_crispr_test_samples
target_crispr_train_feat = target_crispr_df[['BioMedGraphica_Conn_ID'] + target_crispr_train_samples].drop(columns=['BioMedGraphica_Conn_ID']).values.T
target_crispr_test_feat = target_crispr_df[['BioMedGraphica_Conn_ID'] + target_crispr_test_samples].drop(columns=['BioMedGraphica_Conn_ID']).values.T
# Save the training and test features to .npy files
np.save('./data/TargetScreen/target_crispr_train_feature.npy', target_crispr_train_feat)
np.save('./data/TargetScreen/target_crispr_test_feature.npy', target_crispr_test_feat)
# Print the shapes of the training and test features
print("Shape of target_crispr_train_feat:", target_crispr_train_feat.shape)
print("Shape of target_crispr_test_feat:", target_crispr_test_feat.shape)

Shape of target_crispr_train_feat: (300, 834809)
Shape of target_crispr_test_feat: (63, 834809)


In [417]:
# Select the features from target_crispr_df
target_crispr_feat = target_crispr_df.drop(columns=['BioMedGraphica_Conn_ID']).values.T
# Save the features to .npy files
np.save('./data/TargetScreen/target_crispr_feature.npy', target_crispr_feat)
# Print the shapes of the features
print("Shape of target_crispr_feat:", target_crispr_feat.shape)

Shape of target_crispr_feat: (363, 834809)


In [408]:
# Build up internal relation for KO drop
bmgc_promoter_gene_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Promoter-Gene'].drop(columns=['Type'])
bmgc_gene_transcript_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Gene-Transcript'].drop(columns=['Type'])
bmgc_transcript_protein_df = bmgc_omics_relation_df[bmgc_omics_relation_df['Type'] == 'Transcript-Protein'].drop(columns=['Type'])
# Rename the columns
bmgc_promoter_gene_df.rename(columns={'BMGC_From_ID':'promoterID','BMGC_To_ID':'geneID'}, inplace=True)
bmgc_gene_transcript_df.rename(columns={'BMGC_From_ID':'geneID','BMGC_To_ID':'transcriptID'}, inplace=True)
bmgc_transcript_protein_df.rename(columns={'BMGC_From_ID':'transcriptID','BMGC_To_ID':'proteinID'}, inplace=True)
# Merge gene_transcript and transcript_protein dataframes
bmgc_gene_transcript_protein_df = bmgc_transcript_protein_df.merge(bmgc_gene_transcript_df, on='transcriptID',how='outer')
display(bmgc_gene_transcript_protein_df)
# Merge promoter
bmgc_promoter_gene_transcript_protein_df = bmgc_gene_transcript_protein_df.merge(bmgc_promoter_gene_df, on='geneID',how='outer')
display(bmgc_promoter_gene_transcript_protein_df)
internal_relation_df = bmgc_promoter_gene_transcript_protein_df.dropna(subset=['geneID'])
internal_relation_df = internal_relation_df[['promoterID', 'geneID', 'transcriptID', 'proteinID']].copy()
display(internal_relation_df)

,BioMedGraphica_Conn_ID_x,BioMedGraphica_ID_x,transcriptID,proteinID,BMG_From_ID_x,BMG_To_ID_x,Source_x,BioMedGraphica_Conn_ID_y,BioMedGraphica_ID_y,geneID,BMG_From_ID_y,BMG_To_ID_y,Source_y
0,BMGC_ED_TSPT000001,BMG_ED_TSPT000001,BMGC_TS000001,BMGC_PT017571,BMG_TS000001,BMG_PT042398,Uniprot;Ensembl,BMGC_ED_GNTS186033,BMG_ED_GNTS186033,BMGC_GN29641,BMG_GN173393,BMG_TS000001,Ensembl
1,BMGC_ED_TSPT000002,BMG_ED_TSPT000002,BMGC_TS000002,BMGC_PT014978,BMG_TS000002,BMG_PT039543,Uniprot;Ensembl,BMGC_ED_GNTS197666,BMG_ED_GNTS197666,BMGC_GN31641,BMG_GN175441,BMG_TS000002,Ensembl
2,BMGC_ED_TSPT000003,BMG_ED_TSPT000003,BMGC_TS000003,BMGC_PT014403,BMG_TS000003,BMG_PT038917,Uniprot;Ensembl,BMGC_ED_GNTS136517,BMG_ED_GNTS136517,BMGC_GN23354,BMG_GN166843,BMG_TS000003,Ensembl
3,BMGC_ED_TSPT000004,BMG_ED_TSPT000004,BMGC_TS000004,BMGC_PT017807,BMG_TS000004,BMG_PT042655,Uniprot;Ensembl,BMGC_ED_GNTS140655,BMG_ED_GNTS140655,BMGC_GN23854,BMG_GN167345,BMG_TS000004,Ensembl
4,BMGC_ED_TSPT000005,BMG_ED_TSPT000005,BMGC_TS000005,BMGC_PT004700,BMG_TS000005,BMG_PT015283,RefSeq,BMGC_ED_GNTS235896,BMG_ED_GNTS235896,BMGC_GN35654,BMG_GN179505,BMG_TS000005,RefSeq;Ensembl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
458278,NaN,NaN,BMGC_TS412035,NaN,NaN,NaN,NaN,BMGC_ED_GNTS427772,BMG_ED_GNTS427772,BMGC_GN86221,BMG_GN230295,BMG_TS412320,Ensembl
458279,NaN,NaN,BMGC_TS412036,NaN,NaN,NaN,NaN,BMGC_ED_GNTS042666,BMG_ED_GNTS042666,BMGC_GN08809,BMG_GN010161,BMG_TS412321,Ensembl
458280,NaN,NaN,BMGC_TS412037,NaN,NaN,NaN,NaN,BMGC_ED_GNTS042667,BMG_ED_GNTS042667,BMGC_GN08809,BMG_GN010161,BMG_TS412322,Ensembl
458281,NaN,NaN,BMGC_TS412038,NaN,NaN,NaN,NaN,BMGC_ED_GNTS042668,BMG_ED_GNTS042668,BMGC_GN08809,BMG_GN010161,BMG_TS412323,Ensembl


,BioMedGraphica_Conn_ID_x,BioMedGraphica_ID_x,transcriptID,proteinID,BMG_From_ID_x,BMG_To_ID_x,Source_x,BioMedGraphica_Conn_ID_y,BioMedGraphica_ID_y,geneID,BMG_From_ID_y,BMG_To_ID_y,Source_y,BioMedGraphica_Conn_ID,BioMedGraphica_ID,promoterID,BMG_From_ID,BMG_To_ID,Source
0,BMGC_ED_TSPT007580,BMG_ED_TSPT007580,BMGC_TS003701,BMGC_PT013542,BMG_TS003701,BMG_PT037915,Uniprot;Ensembl,BMGC_ED_GNTS000001,BMG_ED_GNTS000001,BMGC_GN00001,BMG_GN000001,BMG_TS003701,Ensembl,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMG_PM000001,BMG_GN000001,NaN
1,NaN,NaN,BMGC_TS179036,NaN,NaN,NaN,NaN,BMGC_ED_GNTS000002,BMG_ED_GNTS000002,BMGC_GN00001,BMG_GN000001,BMG_TS179160,Ensembl,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMG_PM000001,BMG_GN000001,NaN
2,NaN,NaN,BMGC_TS180139,NaN,NaN,NaN,NaN,BMGC_ED_GNTS000003,BMG_ED_GNTS000003,BMGC_GN00001,BMG_GN000001,BMG_TS180263,Ensembl,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMG_PM000001,BMG_GN000001,NaN
3,NaN,NaN,BMGC_TS180951,NaN,NaN,NaN,NaN,BMGC_ED_GNTS000004,BMG_ED_GNTS000004,BMGC_GN00001,BMG_GN000001,BMG_TS181076,Ensembl,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMG_PM000001,BMG_GN000001,NaN
4,BMGC_ED_TSPT114868,BMG_ED_TSPT114868,BMGC_TS182456,BMGC_PT091702,BMG_TS182582,BMG_PT144261,Ensembl,BMGC_ED_GNTS000005,BMG_ED_GNTS000005,BMGC_GN00001,BMG_GN000001,BMG_TS182582,Ensembl,BMGC_ED_PMGN00001,BMG_ED_PMGN00001,BMGC_PM00001,BMG_PM000001,BMG_GN000001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458340,NaN,NaN,BMGC_TS411932,NaN,NaN,NaN,NaN,BMGC_ED_GNTS427806,BMG_ED_GNTS427806,BMGC_GN86237,BMG_GN230311,BMG_TS412217,Ensembl,BMGC_ED_PMGN86237,BMG_ED_PMGN86237,BMGC_PM86237,BMG_PM230311,BMG_GN230311,NaN
458341,NaN,NaN,BMGC_TS412014,NaN,NaN,NaN,NaN,BMGC_ED_GNTS427807,BMG_ED_GNTS427807,BMGC_GN86238,BMG_GN230312,BMG_TS412299,Ensembl,BMGC_ED_PMGN86238,BMG_ED_PMGN86238,BMGC_PM86238,BMG_PM230312,BMG_GN230312,NaN
458342,NaN,NaN,BMGC_TS412015,NaN,NaN,NaN,NaN,BMGC_ED_GNTS427808,BMG_ED_GNTS427808,BMGC_GN86238,BMG_GN230312,BMG_TS412300,Ensembl,BMGC_ED_PMGN86238,BMG_ED_PMGN86238,BMGC_PM86238,BMG_PM230312,BMG_GN230312,NaN
458343,NaN,NaN,BMGC_TS412016,NaN,NaN,NaN,NaN,BMGC_ED_GNTS427809,BMG_ED_GNTS427809,BMGC_GN86238,BMG_GN230312,BMG_TS412301,Ensembl,BMGC_ED_PMGN86238,BMG_ED_PMGN86238,BMGC_PM86238,BMG_PM230312,BMG_GN230312,NaN


,promoterID,geneID,transcriptID,proteinID
0,BMGC_PM00001,BMGC_GN00001,BMGC_TS003701,BMGC_PT013542
1,BMGC_PM00001,BMGC_GN00001,BMGC_TS179036,NaN
2,BMGC_PM00001,BMGC_GN00001,BMGC_TS180139,NaN
3,BMGC_PM00001,BMGC_GN00001,BMGC_TS180951,NaN
4,BMGC_PM00001,BMGC_GN00001,BMGC_TS182456,BMGC_PT091702
...,...,...,...,...
458340,BMGC_PM86237,BMGC_GN86237,BMGC_TS411932,NaN
458341,BMGC_PM86238,BMGC_GN86238,BMGC_TS412014,NaN
458342,BMGC_PM86238,BMGC_GN86238,BMGC_TS412015,NaN
458343,BMGC_PM86238,BMGC_GN86238,BMGC_TS412016,NaN


In [409]:
raw_crispr_df = pd.read_csv('./data/raw_data/CRISPRGeneEffect.csv')
# get the column names of the raw_crispr_df aside from the first column and convert this to a list
first_column_name = raw_crispr_df.columns[0]
raw_crispr_df.rename(columns={first_column_name: 'Sample'}, inplace=True)
display(raw_crispr_df)

,Sample,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.121964,0.042647,0.026102,-0.147614,-0.020425,0.184014,-0.076020,-0.097068,0.182759,...,-0.120536,0.107286,0.166390,0.262275,0.163876,-0.249508,0.053725,0.067986,0.136285,-0.438536
1,ACH-000004,-0.016484,-0.088500,-0.088198,-0.041225,-0.028074,-0.157206,0.280101,-0.106904,0.178125,...,-0.192527,-0.324059,0.230377,0.087609,0.074897,0.054335,-0.330343,0.099067,0.274566,0.001871
2,ACH-000005,-0.184847,0.003300,0.160881,0.086224,-0.149315,-0.253837,0.167011,-0.101209,-0.129827,...,-0.312827,-0.338023,-0.039700,-0.055349,-0.000367,-0.205605,-0.066032,-0.054518,0.035579,-0.150486
3,ACH-000007,-0.071921,-0.113717,0.082872,0.099633,-0.008378,-0.022310,0.014416,-0.184977,-0.173739,...,-0.334843,-0.355499,-0.014183,0.230944,0.044628,-0.081863,-0.390748,-0.036547,-0.273129,-0.382723
4,ACH-000009,-0.019163,-0.134669,0.060323,0.076647,0.078922,-0.100243,0.047559,-0.136988,0.037759,...,-0.299593,-0.194427,-0.027365,0.236591,-0.084224,-0.098271,-0.510495,0.052938,0.018623,-0.258353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173,ACH-002926,-0.107883,-0.101501,0.046677,0.096868,-0.118650,-0.065258,-0.064944,-0.175912,-0.044707,...,-0.409541,-0.579376,0.038706,0.338610,-0.174379,-0.040264,-0.239763,0.065168,-0.144088,-0.334424
1174,ACH-002928,-0.287606,-0.179101,-0.135824,-0.046579,-0.249888,-0.097376,0.070385,0.049444,-0.244937,...,-0.095328,-0.440452,0.303500,0.097690,0.151790,0.039994,0.098240,-0.266654,-0.115108,-0.080421
1175,ACH-003012,-0.145878,-0.004000,0.091933,0.191332,-0.065385,-0.134152,0.012353,-0.321012,0.001571,...,-0.280119,-0.662673,-0.180936,0.169997,-0.036326,0.158702,-0.213281,-0.017884,-0.195395,-0.309021
1176,ACH-003177,-0.038435,-0.055660,0.018989,0.117182,0.063397,0.049993,0.175520,-0.085752,0.078435,...,-0.080700,-0.476797,-0.158375,-0.008999,-0.030058,-0.103513,-0.119234,-0.153744,0.032584,-0.536071


In [410]:
train_target_samples_id_df = train_target_samples_df[['depMapID']].copy()
train_target_samples_id_df.rename(columns={'depMapID': 'Sample'}, inplace=True)
test_target_samples_id_df = test_target_samples_df[['depMapID']].copy()
test_target_samples_id_df.rename(columns={'depMapID': 'Sample'}, inplace=True)
# Get the samples in the raw_crispr_df
train_crispr_score_df = raw_crispr_df.merge(train_target_samples_id_df, on="Sample", how="inner")
display(train_crispr_score_df)
test_crispr_score_df = raw_crispr_df.merge(test_target_samples_id_df, on="Sample", how="inner")
display(test_crispr_score_df)

# Set Index column to str to keep digit without .00
train_crispr_score_t_df = train_crispr_score_df.set_index("Sample").T.reset_index()
train_crispr_score_t_df.rename(columns={"index": "HGNC_Symbol"}, inplace=True)
display(train_crispr_score_t_df)
test_crispr_score_t_df = test_crispr_score_df.set_index("Sample").T.reset_index()
test_crispr_score_t_df.rename(columns={"index": "HGNC_Symbol"}, inplace=True)
display(test_crispr_score_t_df)

,Sample,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.121964,0.042647,0.026102,-0.147614,-0.020425,0.184014,-0.076020,-0.097068,0.182759,...,-0.120536,0.107286,0.166390,0.262275,0.163876,-0.249508,0.053725,0.067986,0.136285,-0.438536
1,ACH-000004,-0.016484,-0.088500,-0.088198,-0.041225,-0.028074,-0.157206,0.280101,-0.106904,0.178125,...,-0.192527,-0.324059,0.230377,0.087609,0.074897,0.054335,-0.330343,0.099067,0.274566,0.001871
2,ACH-000007,-0.071921,-0.113717,0.082872,0.099633,-0.008378,-0.022310,0.014416,-0.184977,-0.173739,...,-0.334843,-0.355499,-0.014183,0.230944,0.044628,-0.081863,-0.390748,-0.036547,-0.273129,-0.382723
3,ACH-000009,-0.019163,-0.134669,0.060323,0.076647,0.078922,-0.100243,0.047559,-0.136988,0.037759,...,-0.299593,-0.194427,-0.027365,0.236591,-0.084224,-0.098271,-0.510495,0.052938,0.018623,-0.258353
4,ACH-000012,-0.139496,-0.024648,0.034448,0.060509,0.046351,-0.168111,0.166230,-0.094632,-0.044914,...,-0.480750,-0.631959,0.101635,0.032980,-0.086901,0.028423,-0.059623,-0.114437,-0.117129,-0.150634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,ACH-001075,-0.236411,0.045414,0.004463,-0.032736,-0.181405,-0.008243,0.097413,-0.127815,0.057874,...,-0.264254,-0.226804,0.268218,0.435536,-0.007683,-0.013751,-0.239167,0.053218,0.039981,-0.365370
296,ACH-001107,-0.098559,-0.025209,0.027718,0.182246,-0.065604,-0.011558,0.216557,-0.146614,-0.227518,...,-0.136001,0.031411,0.075305,0.091786,-0.132919,0.040862,-0.139992,0.191589,-0.022600,-0.507938
297,ACH-001190,-0.042641,0.010443,0.162840,-0.065062,-0.048498,-0.254546,-0.015924,-0.248677,0.092303,...,-0.191808,-0.282360,0.072387,0.102600,0.043955,-0.141153,-0.266564,0.029695,-0.095837,-0.100670
298,ACH-001306,-0.031081,-0.000523,-0.036076,-0.012972,0.009998,-0.044483,0.027619,-0.017077,0.002215,...,-0.014357,-0.427889,-0.060975,-0.018635,-0.170612,0.036136,-0.130743,-0.092105,-0.462014,-0.268095


,Sample,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000015,-0.164485,-0.062450,-0.042585,0.067254,-0.029782,-0.021019,0.107219,-0.141959,0.017586,...,-0.356899,-0.161527,0.051580,0.083171,0.021982,-0.016426,-0.152085,-0.153389,-0.130675,-0.183547
1,ACH-000030,0.092091,-0.235943,-0.008186,0.068138,-0.089160,-0.211510,0.079939,-0.243709,0.041970,...,0.024061,-0.252046,-0.006676,0.046137,-0.144591,-0.081285,-0.236948,-0.013121,-0.126559,-0.249220
2,ACH-000055,0.023233,0.023197,0.133568,0.097102,-0.165176,0.037308,-0.118947,0.079731,-0.254460,...,-0.343143,-0.239665,0.107491,-0.188749,0.127635,-0.064280,-0.049646,-0.428021,-0.172303,-0.369053
3,ACH-000070,-0.023093,-0.098678,0.055092,0.211111,-0.202141,0.011529,0.030910,0.110055,0.016656,...,-0.321510,-0.045616,0.132882,0.262532,-0.067119,-0.181128,-0.052296,-0.186571,-0.137025,-0.269073
4,ACH-000075,-0.030929,0.173555,0.144853,0.147002,-0.232510,-0.183895,0.099746,0.078998,0.051277,...,-0.190412,-0.830687,0.117279,0.029483,-0.098088,-0.138234,-0.209159,0.083589,-0.108345,-0.496251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,ACH-000950,-0.086066,-0.070168,0.068361,-0.114017,-0.143689,-0.030228,-0.042546,-0.084736,-0.077060,...,-0.668693,-0.964036,0.239076,0.344992,-0.080482,-0.047990,-0.068670,-0.050717,0.021737,-0.174756
59,ACH-000957,-0.211048,-0.137014,0.085481,0.030893,-0.046826,-0.094417,-0.029150,-0.233241,-0.075270,...,-0.965635,-0.945276,0.184207,0.279205,0.064601,-0.054118,-0.140052,-0.014237,-0.022187,-0.287196
60,ACH-000969,0.136178,-0.088978,0.088356,-0.058193,0.071555,-0.124653,-0.067000,-0.142984,-0.004930,...,-0.148537,-0.864126,0.132900,0.133574,-0.106611,0.116770,-0.041387,0.051083,-0.051369,-0.146886
61,ACH-000971,0.133352,-0.038505,0.077262,-0.084336,-0.093124,0.049209,0.037526,-0.126809,-0.033475,...,-0.541773,-1.170882,0.032318,0.280006,0.065551,0.089177,-0.284905,0.000105,-0.083344,-0.227688


Sample,HGNC_Symbol,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000018,ACH-000019,ACH-000021,ACH-000023,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,A1BG (1),-0.121964,-0.016484,-0.071921,-0.019163,-0.139496,-0.089959,-0.000655,-0.144838,-0.044006,...,-0.080952,-0.086003,0.010321,-0.280963,-0.054307,-0.236411,-0.098559,-0.042641,-0.031081,0.012357
1,A1CF (29974),0.042647,-0.088500,-0.113717,-0.134669,-0.024648,-0.025220,0.010486,-0.190572,-0.135335,...,-0.139068,-0.833193,-0.260665,-0.673329,-0.052153,0.045414,-0.025209,0.010443,-0.000523,-0.107587
2,A2M (2),0.026102,-0.088198,0.082872,0.060323,0.034448,-0.247578,0.100446,-0.033688,-0.014753,...,0.024197,-0.230081,0.022584,0.010727,-0.023250,0.004463,0.027718,0.162840,-0.036076,0.108407
3,A2ML1 (144568),-0.147614,-0.041225,0.099633,0.076647,0.060509,0.099575,0.044937,-0.029677,-0.112628,...,0.033499,0.302444,0.202078,0.239772,0.269846,-0.032736,0.182246,-0.065062,-0.012972,0.203668
4,A3GALT2 (127550),-0.020425,-0.028074,-0.008378,0.078922,0.046351,0.000525,-0.073545,-0.067062,-0.043320,...,-0.105600,-0.147492,0.019020,-0.084180,-0.065527,-0.181405,-0.065604,-0.048498,0.009998,-0.006054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17911,ZYG11A (440590),-0.249508,0.054335,-0.081863,-0.098271,0.028423,-0.087202,-0.047248,-0.056765,-0.078996,...,0.075397,0.162973,-0.000810,0.182418,-0.210634,-0.013751,0.040862,-0.141153,0.036136,-0.096073
17912,ZYG11B (79699),0.053725,-0.330343,-0.390748,-0.510495,-0.059623,-0.046460,-0.164099,-0.259650,-0.167981,...,-0.073492,-0.081934,-0.069572,-0.042853,-0.205769,-0.239167,-0.139992,-0.266564,-0.130743,-0.137811
17913,ZYX (7791),0.067986,0.099067,-0.036547,0.052938,-0.114437,-0.003761,0.168613,-0.161969,0.216434,...,-0.022618,0.151163,0.079822,0.079633,0.285485,0.053218,0.191589,0.029695,-0.092105,-0.170982
17914,ZZEF1 (23140),0.136285,0.274566,-0.273129,0.018623,-0.117129,-0.318758,-0.093001,-0.188478,-0.165553,...,-0.176936,-0.007172,-0.148326,-0.095262,-0.011798,0.039981,-0.022600,-0.095837,-0.462014,-0.051311


Sample,HGNC_Symbol,ACH-000015,ACH-000030,ACH-000055,ACH-000070,ACH-000075,ACH-000098,ACH-000100,ACH-000132,ACH-000138,...,ACH-000880,ACH-000885,ACH-000900,ACH-000915,ACH-000919,ACH-000950,ACH-000957,ACH-000969,ACH-000971,ACH-000977
0,A1BG (1),-0.164485,0.092091,0.023233,-0.023093,-0.030929,-0.037187,0.021787,-0.122873,-0.102403,...,-0.210218,-0.276661,-0.073962,-0.037085,-0.099006,-0.086066,-0.211048,0.136178,0.133352,-0.081687
1,A1CF (29974),-0.062450,-0.235943,0.023197,-0.098678,0.173555,-0.137049,0.138818,0.072059,-0.117266,...,0.023075,0.002950,-0.066371,-0.040370,-0.154016,-0.070168,-0.137014,-0.088978,-0.038505,-0.032364
2,A2M (2),-0.042585,-0.008186,0.133568,0.055092,0.144853,0.070069,0.219874,0.038849,-0.053961,...,0.169803,0.115853,0.051874,-0.013556,0.221679,0.068361,0.085481,0.088356,0.077262,0.129437
3,A2ML1 (144568),0.067254,0.068138,0.097102,0.211111,0.147002,0.010128,0.057229,0.039049,-0.071758,...,-0.106526,0.193006,0.005611,0.139576,0.077502,-0.114017,0.030893,-0.058193,-0.084336,0.105632
4,A3GALT2 (127550),-0.029782,-0.089160,-0.165176,-0.202141,-0.232510,-0.112067,-0.167169,-0.209066,0.073647,...,0.077082,-0.010742,0.056990,-0.319709,-0.184296,-0.143689,-0.046826,0.071555,-0.093124,-0.093056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17911,ZYG11A (440590),-0.016426,-0.081285,-0.064280,-0.181128,-0.138234,-0.040847,-0.123101,0.018842,0.037163,...,-0.103918,0.012138,-0.067628,-0.065556,-0.106669,-0.047990,-0.054118,0.116770,0.089177,0.061751
17912,ZYG11B (79699),-0.152085,-0.236948,-0.049646,-0.052296,-0.209159,-0.188712,-0.130245,-0.164656,-0.272664,...,0.109241,-0.131954,-0.224480,0.047063,-0.268030,-0.068670,-0.140052,-0.041387,-0.284905,-0.122563
17913,ZYX (7791),-0.153389,-0.013121,-0.428021,-0.186571,0.083589,-0.078067,-0.109373,-0.155827,0.052268,...,-0.182024,-0.126813,-0.132606,0.034350,-0.205007,-0.050717,-0.014237,0.051083,0.000105,-0.139035
17914,ZZEF1 (23140),-0.130675,-0.126559,-0.172303,-0.137025,-0.108345,-0.093377,-0.230256,-0.115392,-0.072237,...,-0.202608,-0.145749,-0.234658,-0.341694,-0.192169,0.021737,-0.022187,-0.051369,-0.083344,-0.179099


In [411]:
import re
def extract_gene_name(gene):
    return re.sub(r"\s*\(.*?\)", "", str(gene))

train_crispr_score_t_df["HGNC_Symbol"] = train_crispr_score_t_df["HGNC_Symbol"].apply(extract_gene_name)
unique_Gene_map = bmgc_gene_df[['HGNC_Symbol']].drop_duplicates()
unique_omics_Gene = train_crispr_score_t_df[['HGNC_Symbol']].drop_duplicates()
# match
intersection = set(unique_Gene_map['HGNC_Symbol'])&set(unique_omics_Gene['HGNC_Symbol'])
total_Gene = len(unique_Gene_map)
match_rate = len(intersection) / total_Gene if total_Gene > 0 else 0
print(f"match rate:{match_rate:.2%}")
train_crispr_score_bmgc_df = bmgc_gene_df.merge(train_crispr_score_t_df, on="HGNC_Symbol", how="inner")
train_crispr_score_bmgc_df = train_crispr_score_bmgc_df.drop(columns=['BioMedGraphica_ID','NCBI_Gene_ID','Ensembl_Gene_ID_Version','Gene_Start', 'Gene_End','Chromosome','Gene_Type','Ensembl_Gene_ID','HGNC_ID','Gene_Name','RefSeq_ID','OMIM_ID','HGNC_Symbol'])
display(train_crispr_score_bmgc_df)

test_crispr_score_t_df["HGNC_Symbol"] = test_crispr_score_t_df["HGNC_Symbol"].apply(extract_gene_name)
unique_Gene_map = bmgc_gene_df[['HGNC_Symbol']].drop_duplicates()
unique_omics_Gene = test_crispr_score_t_df[['HGNC_Symbol']].drop_duplicates()
# match
intersection = set(unique_Gene_map['HGNC_Symbol'])&set(unique_omics_Gene['HGNC_Symbol'])
total_Gene = len(unique_Gene_map)
match_rate = len(intersection) / total_Gene if total_Gene > 0 else 0
print(f"match rate:{match_rate:.2%}")
test_crispr_score_bmgc_df = bmgc_gene_df.merge(test_crispr_score_t_df, on="HGNC_Symbol", how="inner")
test_crispr_score_bmgc_df = test_crispr_score_bmgc_df.drop(columns=['BioMedGraphica_ID','NCBI_Gene_ID','Ensembl_Gene_ID_Version','Gene_Start', 'Gene_End','Chromosome','Gene_Type','Ensembl_Gene_ID','HGNC_ID','Gene_Name','RefSeq_ID','OMIM_ID','HGNC_Symbol'])
display(test_crispr_score_bmgc_df)

match rate:39.52%


,BioMedGraphica_Conn_ID,ACH-000001,ACH-000004,ACH-000007,ACH-000009,ACH-000012,ACH-000018,ACH-000019,ACH-000021,ACH-000023,...,ACH-000988,ACH-000991,ACH-000997,ACH-001039,ACH-001065,ACH-001075,ACH-001107,ACH-001190,ACH-001306,ACH-001307
0,BMGC_GN00001,-0.121964,-0.016484,-0.071921,-0.019163,-0.139496,-0.089959,-0.000655,-0.144838,-0.044006,...,-0.080952,-0.086003,0.010321,-0.280963,-0.054307,-0.236411,-0.098559,-0.042641,-0.031081,0.012357
1,BMGC_GN00002,-0.022127,0.196378,0.310169,0.198303,0.157864,0.208419,0.269541,0.137374,0.163614,...,0.119221,0.213694,0.118167,-0.273918,-0.017122,0.105633,0.140576,0.185244,0.250509,0.223775
2,BMGC_GN00003,-0.116327,-0.200383,-0.120459,-0.100498,0.084591,-0.021049,-0.246973,-0.034359,0.011776,...,-0.082360,-0.009491,0.026029,-0.157190,-0.023168,-0.131286,-0.083327,-0.123697,-0.086892,0.059173
3,BMGC_GN00004,-0.019295,0.099193,-0.213060,-0.100742,-0.027933,-0.842505,0.063977,-0.117045,0.070139,...,-0.117608,0.007976,-0.028504,0.233251,-0.176635,-0.057680,-0.250286,0.086460,-0.275844,-0.517292
4,BMGC_GN00005,-0.100900,0.151491,0.191130,0.025839,0.055322,-0.128477,0.418205,0.128353,-0.115753,...,0.160051,0.066251,0.009697,-0.162483,0.139784,0.011788,0.164420,-0.058207,-0.069053,0.237172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20283,BMGC_GN69808,0.094246,0.021657,0.392400,0.293312,0.124325,0.442315,0.151060,0.259185,0.030048,...,0.130424,-0.118179,-0.030500,0.167938,0.084445,0.069296,0.211061,0.254713,0.086061,0.421384
20284,BMGC_GN69841,-0.137149,-0.601468,-0.320952,0.022595,-0.075869,-0.160712,-0.142266,0.055720,0.048859,...,-0.112633,-0.344776,-0.322098,-0.197280,0.084078,-0.194447,-0.160895,0.105202,-0.048947,-0.077387
20285,BMGC_GN69848,NaN,0.333310,0.090807,0.130973,0.057994,0.016015,-0.088375,0.013533,NaN,...,-0.129761,NaN,NaN,NaN,NaN,0.068046,NaN,0.276861,-0.006914,0.126152
20286,BMGC_GN69876,NaN,-0.316461,-0.556799,-0.350030,-0.579060,-0.448828,0.418644,-0.102821,NaN,...,0.017046,NaN,NaN,NaN,NaN,-0.329794,NaN,-0.067530,-0.110224,-0.190182


match rate:39.52%


,BioMedGraphica_Conn_ID,ACH-000015,ACH-000030,ACH-000055,ACH-000070,ACH-000075,ACH-000098,ACH-000100,ACH-000132,ACH-000138,...,ACH-000880,ACH-000885,ACH-000900,ACH-000915,ACH-000919,ACH-000950,ACH-000957,ACH-000969,ACH-000971,ACH-000977
0,BMGC_GN00001,-0.164485,0.092091,0.023233,-0.023093,-0.030929,-0.037187,0.021787,-0.122873,-0.102403,...,-0.210218,-0.276661,-0.073962,-0.037085,-0.099006,-0.086066,-0.211048,0.136178,0.133352,-0.081687
1,BMGC_GN00002,-0.043958,0.172081,0.113231,0.022171,0.085891,0.185540,0.196876,0.175082,0.122638,...,0.228929,0.139567,0.140669,0.246247,0.277374,-0.045897,-0.063168,0.252195,-0.280817,0.013595
2,BMGC_GN00003,-0.140311,-0.022852,-0.158881,-0.039262,-0.119558,0.017364,-0.098256,-0.067811,-0.084381,...,-0.177668,-0.061922,-0.080560,-0.058143,-0.163444,-0.070590,-0.174459,-0.155545,-0.063573,0.023034
3,BMGC_GN00004,-0.159200,0.064357,0.530061,0.185567,-0.217684,-0.469302,0.015246,-0.974951,-0.178604,...,0.034351,-0.029154,-0.263409,-0.195348,0.023198,-0.123158,-0.136271,0.040931,-0.057478,0.046085
4,BMGC_GN00005,0.041047,0.104015,-0.039690,0.017025,-0.044021,-0.066938,-0.113157,-0.099366,0.036075,...,0.083539,0.103063,0.236837,-0.253602,0.166021,0.223567,0.049261,0.121447,0.098172,-0.000221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20283,BMGC_GN69808,0.114893,0.128794,0.111258,0.103258,0.219192,0.305259,0.141067,0.129172,0.344026,...,0.368076,0.280522,0.031917,0.075625,0.261597,0.212018,0.311237,0.120037,0.131675,0.031412
20284,BMGC_GN69841,-0.272400,-0.258718,-0.354407,-0.494304,-0.058402,0.033533,-0.300633,-0.207052,-0.207051,...,-0.165977,-0.125010,-0.279584,-0.035788,-0.531473,-0.140367,-0.390618,-0.023741,-0.142910,-0.418786
20285,BMGC_GN69848,0.098828,0.129801,-0.039741,0.201172,0.294755,-0.018549,0.237250,0.045203,0.198364,...,0.407689,0.525714,0.018922,0.061899,0.002737,0.050819,0.404756,0.111930,NaN,0.190287
20286,BMGC_GN69876,-0.830064,-0.138660,-0.584080,-0.421905,-0.332492,-0.213516,-0.474673,-0.259733,0.044880,...,-0.482228,-0.334915,-0.142542,-0.045426,0.168132,-0.620618,0.146192,-0.242320,NaN,-0.172861


In [412]:
train_crispr_score_label = train_crispr_score_bmgc_df.copy()
train_crispr_score_label_melted = train_crispr_score_label.melt(id_vars=["BioMedGraphica_Conn_ID"], var_name="ACH_ID", value_name="Value")
train_crispr_score_label_melted = train_crispr_score_label_melted.dropna(subset=['Value'])
display(train_crispr_score_label_melted)
test_crispr_score_label = test_crispr_score_bmgc_df.copy()
test_crispr_score_label_melted = test_crispr_score_label.melt(id_vars=["BioMedGraphica_Conn_ID"], var_name="ACH_ID", value_name="Value")
test_crispr_score_label_melted = test_crispr_score_label_melted.dropna(subset=['Value'])
display(test_crispr_score_label_melted)

,BioMedGraphica_Conn_ID,ACH_ID,Value
0,BMGC_GN00001,ACH-000001,-0.121964
1,BMGC_GN00002,ACH-000001,-0.022127
2,BMGC_GN00003,ACH-000001,-0.116327
3,BMGC_GN00004,ACH-000001,-0.019295
4,BMGC_GN00005,ACH-000001,-0.100900
...,...,...,...
6086395,BMGC_GN69808,ACH-001307,0.421384
6086396,BMGC_GN69841,ACH-001307,-0.077387
6086397,BMGC_GN69848,ACH-001307,0.126152
6086398,BMGC_GN69876,ACH-001307,-0.190182


,BioMedGraphica_Conn_ID,ACH_ID,Value
0,BMGC_GN00001,ACH-000015,-0.164485
1,BMGC_GN00002,ACH-000015,-0.043958
2,BMGC_GN00003,ACH-000015,-0.140311
3,BMGC_GN00004,ACH-000015,-0.159200
4,BMGC_GN00005,ACH-000015,0.041047
...,...,...,...
1278139,BMGC_GN69808,ACH-000977,0.031412
1278140,BMGC_GN69841,ACH-000977,-0.418786
1278141,BMGC_GN69848,ACH-000977,0.190287
1278142,BMGC_GN69876,ACH-000977,-0.172861


##### Merge into internal relation

In [413]:
# Based on dti_sample_index, convert this to a dict
target_crispr_sample_index_dict = dict(zip(overlapped_dti_crispr_rna_samples_df['depMapID'], overlapped_dti_crispr_rna_samples_df.index))
# Map columns ['ACH_ID', 'BioMedGraphica_Conn_ID'] by the crispr_sample_index_dict and node_index_dict
train_crispr_score_label_melted['ACH_ID'] = train_crispr_score_label_melted['ACH_ID'].map(target_crispr_sample_index_dict)
train_crispr_score_label_melted['BioMedGraphica_Conn_ID'] = train_crispr_score_label_melted['BioMedGraphica_Conn_ID'].map(node_index_dict)
test_crispr_score_label_melted['ACH_ID'] = test_crispr_score_label_melted['ACH_ID'].map(target_crispr_sample_index_dict)
test_crispr_score_label_melted['BioMedGraphica_Conn_ID'] = test_crispr_score_label_melted['BioMedGraphica_Conn_ID'].map(node_index_dict)

# Map internal_relation_df columns ['promoterID', 'geneID', 'transcriptID', 'proteinID'] by the node_index_dict
internal_relation_map_df = internal_relation_df.copy()
internal_relation_map_df['promoterID'] = internal_relation_map_df['promoterID'].map(node_index_dict)
internal_relation_map_df['geneID'] = internal_relation_map_df['geneID'].map(node_index_dict)
internal_relation_map_df['transcriptID'] = internal_relation_map_df['transcriptID'].map(node_index_dict)
internal_relation_map_df['proteinID'] = internal_relation_map_df['proteinID'].map(node_index_dict)

ko_internal_relation_map_df = internal_relation_map_df.copy()
display(ko_internal_relation_map_df)
ko_internal_relation_index_df = ko_internal_relation_map_df.groupby(["promoterID", "geneID"], as_index=False).agg({
    "transcriptID": lambda x: list(set(x.dropna())), 
    "proteinID": lambda x: list(set(x.dropna()))  
})

,promoterID,geneID,transcriptID,proteinID
0,0,86238,176176.0,598056.0
1,0,86238,351511.0,NaN
2,0,86238,352614.0,NaN
3,0,86238,353426.0,NaN
4,0,86238,354931.0,676216.0
...,...,...,...,...
458340,86236,172474,584407.0,NaN
458341,86237,172475,584489.0,NaN
458342,86237,172475,584490.0,NaN
458343,86237,172475,584491.0,NaN


In [414]:
# Define a function to remove .0 suffix from numbers in lists
def remove_decimal_suffix(list_with_decimals):
    if isinstance(list_with_decimals, list):
        return [int(x) if not pd.isna(x) else x for x in list_with_decimals]
    return list_with_decimals

# Apply the function to both columns
ko_internal_relation_index_df['transcriptID'] = ko_internal_relation_index_df['transcriptID'].apply(remove_decimal_suffix)
ko_internal_relation_index_df['proteinID'] = ko_internal_relation_index_df['proteinID'].apply(remove_decimal_suffix)

# Display the result to verify
display(ko_internal_relation_index_df.head())

,promoterID,geneID,transcriptID,proteinID
0,0,86238,"[352614, 176176, 353426, 354931, 351511]","[598056, 676216]"
1,1,86239,"[294338, 178291]","[654736, 625813, 605858, 605859, 605860, 60586..."
2,2,86240,"[436480, 436481, 436482, 436483, 436484, 43648...","[700171, 597776, 700181, 700183, 700184, 60740..."
3,3,86241,"[177089, 215868, 418536, 224555, 418861, 41784...","[588673, 647555, 604196, 692323, 599402, 58839..."
4,4,86242,"[317453, 423832, 415271, 422836, 415671, 41631...","[691556, 691046, 623102, 602826, 630187, 69124..."


In [415]:
train_crispr_score_label_melted_ko = train_crispr_score_label_melted.merge(ko_internal_relation_index_df, left_on='BioMedGraphica_Conn_ID',right_on='geneID' ,how='left')
train_crispr_score_label_melted_ko = train_crispr_score_label_melted_ko.drop(columns=['BioMedGraphica_Conn_ID'])
display(train_crispr_score_label_melted_ko)

# Create a new column 'merged_ids' by combining the columns into lists
train_crispr_score_label_melted_ko['merged_ids'] = train_crispr_score_label_melted_ko.apply(
    lambda row: [row['promoterID'], row['geneID']] + 
                (row['transcriptID'] if isinstance(row['transcriptID'], list) else []) +
                (row['proteinID'] if isinstance(row['proteinID'], list) else []),
    axis=1
)

# Display the result
train_crispr_score_label_melted_ko = train_crispr_score_label_melted_ko.drop(columns=['promoterID', 'geneID', 'transcriptID', 'proteinID'])
display(train_crispr_score_label_melted_ko)

# reorder the columns to ['ACH_ID', 'merged_ids', 'Value']
train_crispr_score_label_melted_ko = train_crispr_score_label_melted_ko[['ACH_ID', 'merged_ids', 'Value']].copy()
display(train_crispr_score_label_melted_ko)
# Convert train_crispr_score_label_melted_ko to numpy array and save it to the CRISPR-Graph folder
train_crispr_score_label_melted_ko_array = train_crispr_score_label_melted_ko.to_numpy()
print('The shape of train_crispr_score_label_melted_ko_array is:', train_crispr_score_label_melted_ko_array.shape)
print(train_crispr_score_label_melted_ko_array[:10])  # Print first 10 rows as a sample
# Save the numpy array to the CRISPR-Graph folder
np.save('./data/TargetScreen/train_crispr_score_label_melted_ko.npy', train_crispr_score_label_melted_ko_array)

,ACH_ID,Value,promoterID,geneID,transcriptID,proteinID
0,0,-0.121964,0,86238,"[352614, 176176, 353426, 354931, 351511]","[598056, 676216]"
1,0,-0.022127,1,86239,"[294338, 178291]","[654736, 625813, 605858, 605859, 605860, 60586..."
2,0,-0.116327,2,86240,"[436480, 436481, 436482, 436483, 436484, 43648...","[700171, 597776, 700181, 700183, 700184, 60740..."
3,0,-0.019295,3,86241,"[177089, 215868, 418536, 224555, 418861, 41784...","[588673, 647555, 604196, 692323, 599402, 58839..."
4,0,-0.100900,4,86242,"[317453, 423832, 415271, 422836, 415671, 41631...","[691556, 691046, 623102, 602826, 630187, 69124..."
...,...,...,...,...,...,...
6041319,362,0.421384,69807,156045,[448476],[704608]
6041320,362,-0.077387,69840,156078,"[448865, 448866, 448867, 448868, 448869]","[704699, 704700, 704701, 704702]"
6041321,362,0.126152,69847,156085,[388361],[688824]
6041322,362,-0.190182,69875,156113,[449038],[704769]


,ACH_ID,Value,merged_ids
0,0,-0.121964,"[0, 86238, 352614, 176176, 353426, 354931, 351..."
1,0,-0.022127,"[1, 86239, 294338, 178291, 654736, 625813, 605..."
2,0,-0.116327,"[2, 86240, 436480, 436481, 436482, 436483, 436..."
3,0,-0.019295,"[3, 86241, 177089, 215868, 418536, 224555, 418..."
4,0,-0.100900,"[4, 86242, 317453, 423832, 415271, 422836, 415..."
...,...,...,...
6041319,362,0.421384,"[69807, 156045, 448476, 704608]"
6041320,362,-0.077387,"[69840, 156078, 448865, 448866, 448867, 448868..."
6041321,362,0.126152,"[69847, 156085, 388361, 688824]"
6041322,362,-0.190182,"[69875, 156113, 449038, 704769]"


,ACH_ID,merged_ids,Value
0,0,"[0, 86238, 352614, 176176, 353426, 354931, 351...",-0.121964
1,0,"[1, 86239, 294338, 178291, 654736, 625813, 605...",-0.022127
2,0,"[2, 86240, 436480, 436481, 436482, 436483, 436...",-0.116327
3,0,"[3, 86241, 177089, 215868, 418536, 224555, 418...",-0.019295
4,0,"[4, 86242, 317453, 423832, 415271, 422836, 415...",-0.100900
...,...,...,...
6041319,362,"[69807, 156045, 448476, 704608]",0.421384
6041320,362,"[69840, 156078, 448865, 448866, 448867, 448868...",-0.077387
6041321,362,"[69847, 156085, 388361, 688824]",0.126152
6041322,362,"[69875, 156113, 449038, 704769]",-0.190182


The shape of train_crispr_score_label_melted_ko_array is: (6041324, 3)
[[0
  list([0, 86238, 352614, 176176, 353426, 354931, 351511, 598056, 676216])
  -0.121963874711081]
 [0
  list([1, 86239, 294338, 178291, 654736, 625813, 605858, 605859, 605860, 605861, 605862, 605863, 605864, 623402, 623403, 603184, 615867, 615868, 595795, 622430, 586590, 605161, 596210, 598899, 596211, 612213])
  -0.0221269150315085]
 [0
  list([2, 86240, 436480, 436481, 436482, 436483, 436484, 436485, 436486, 436487, 309640, 436360, 436488, 436489, 436490, 436367, 436368, 436471, 436510, 436511, 248104, 308397, 436397, 436398, 436399, 436400, 436401, 436402, 436403, 310456, 436409, 436410, 308923, 436411, 436412, 436413, 436414, 436415, 436416, 314180, 308678, 436423, 436304, 273233, 436305, 436306, 436440, 436441, 436442, 436443, 436444, 436445, 195688, 436464, 436465, 436338, 436466, 436467, 436468, 436469, 436470, 436478, 436479, 700171, 597776, 700181, 700183, 700184, 607403, 700206, 700207, 700208, 700209, 

In [416]:
test_crispr_score_label_melted_ko = test_crispr_score_label_melted.merge(ko_internal_relation_index_df, left_on='BioMedGraphica_Conn_ID',right_on='geneID' ,how='left')
test_crispr_score_label_melted_ko = test_crispr_score_label_melted_ko.drop(columns=['BioMedGraphica_Conn_ID'])
display(test_crispr_score_label_melted_ko)

# Create a new column 'merged_ids' by combining the columns into lists
test_crispr_score_label_melted_ko['merged_ids'] = test_crispr_score_label_melted_ko.apply(
    lambda row: [row['promoterID'], row['geneID']] + 
                (row['transcriptID'] if isinstance(row['transcriptID'], list) else []) +
                (row['proteinID'] if isinstance(row['proteinID'], list) else []),
    axis=1
)

# Display the result
test_crispr_score_label_melted_ko = test_crispr_score_label_melted_ko.drop(columns=['promoterID', 'geneID', 'transcriptID', 'proteinID'])
display(test_crispr_score_label_melted_ko)

# reorder the columns to ['ACH_ID', 'merged_ids', 'Value']
test_crispr_score_label_melted_ko = test_crispr_score_label_melted_ko[['ACH_ID', 'merged_ids', 'Value']].copy()
display(test_crispr_score_label_melted_ko)
# Convert test_crispr_score_label_melted_ko to numpy array and save it to the CRISPR-Graph folder
test_crispr_score_label_melted_ko_array = test_crispr_score_label_melted_ko.to_numpy()
print('The shape of test_crispr_score_label_melted_ko_array is:', test_crispr_score_label_melted_ko_array.shape)
print(test_crispr_score_label_melted_ko_array[:10])  # Print first 10 rows as a sample
# Save the numpy array to the CRISPR-Graph folder
np.save('./data/TargetScreen/test_crispr_score_label_melted_ko.npy', test_crispr_score_label_melted_ko_array)

,ACH_ID,Value,promoterID,geneID,transcriptID,proteinID
0,5,-0.164485,0,86238,"[352614, 176176, 353426, 354931, 351511]","[598056, 676216]"
1,5,-0.043958,1,86239,"[294338, 178291]","[654736, 625813, 605858, 605859, 605860, 60586..."
2,5,-0.140311,2,86240,"[436480, 436481, 436482, 436483, 436484, 43648...","[700171, 597776, 700181, 700183, 700184, 60740..."
3,5,-0.159200,3,86241,"[177089, 215868, 418536, 224555, 418861, 41784...","[588673, 647555, 604196, 692323, 599402, 58839..."
4,5,0.041047,4,86242,"[317453, 423832, 415271, 422836, 415671, 41631...","[691556, 691046, 623102, 602826, 630187, 69124..."
...,...,...,...,...,...,...
1269552,347,0.031412,69807,156045,[448476],[704608]
1269553,347,-0.418786,69840,156078,"[448865, 448866, 448867, 448868, 448869]","[704699, 704700, 704701, 704702]"
1269554,347,0.190287,69847,156085,[388361],[688824]
1269555,347,-0.172861,69875,156113,[449038],[704769]


,ACH_ID,Value,merged_ids
0,5,-0.164485,"[0, 86238, 352614, 176176, 353426, 354931, 351..."
1,5,-0.043958,"[1, 86239, 294338, 178291, 654736, 625813, 605..."
2,5,-0.140311,"[2, 86240, 436480, 436481, 436482, 436483, 436..."
3,5,-0.159200,"[3, 86241, 177089, 215868, 418536, 224555, 418..."
4,5,0.041047,"[4, 86242, 317453, 423832, 415271, 422836, 415..."
...,...,...,...
1269552,347,0.031412,"[69807, 156045, 448476, 704608]"
1269553,347,-0.418786,"[69840, 156078, 448865, 448866, 448867, 448868..."
1269554,347,0.190287,"[69847, 156085, 388361, 688824]"
1269555,347,-0.172861,"[69875, 156113, 449038, 704769]"


,ACH_ID,merged_ids,Value
0,5,"[0, 86238, 352614, 176176, 353426, 354931, 351...",-0.164485
1,5,"[1, 86239, 294338, 178291, 654736, 625813, 605...",-0.043958
2,5,"[2, 86240, 436480, 436481, 436482, 436483, 436...",-0.140311
3,5,"[3, 86241, 177089, 215868, 418536, 224555, 418...",-0.159200
4,5,"[4, 86242, 317453, 423832, 415271, 422836, 415...",0.041047
...,...,...,...
1269552,347,"[69807, 156045, 448476, 704608]",0.031412
1269553,347,"[69840, 156078, 448865, 448866, 448867, 448868...",-0.418786
1269554,347,"[69847, 156085, 388361, 688824]",0.190287
1269555,347,"[69875, 156113, 449038, 704769]",-0.172861


The shape of test_crispr_score_label_melted_ko_array is: (1269557, 3)
[[5
  list([0, 86238, 352614, 176176, 353426, 354931, 351511, 598056, 676216])
  -0.1644853469607449]
 [5
  list([1, 86239, 294338, 178291, 654736, 625813, 605858, 605859, 605860, 605861, 605862, 605863, 605864, 623402, 623403, 603184, 615867, 615868, 595795, 622430, 586590, 605161, 596210, 598899, 596211, 612213])
  -0.0439583570818749]
 [5
  list([2, 86240, 436480, 436481, 436482, 436483, 436484, 436485, 436486, 436487, 309640, 436360, 436488, 436489, 436490, 436367, 436368, 436471, 436510, 436511, 248104, 308397, 436397, 436398, 436399, 436400, 436401, 436402, 436403, 310456, 436409, 436410, 308923, 436411, 436412, 436413, 436414, 436415, 436416, 314180, 308678, 436423, 436304, 273233, 436305, 436306, 436440, 436441, 436442, 436443, 436444, 436445, 195688, 436464, 436465, 436338, 436466, 436467, 436468, 436469, 436470, 436478, 436479, 700171, 597776, 700181, 700183, 700184, 607403, 700206, 700207, 700208, 700209, 

##### 9.2.2.2 TargetQA

##### Omic Feature

In [ ]:
# Select the features from target_crispr_df
target_crispr_feat = target_crispr_df.drop(columns=['BioMedGraphica_Conn_ID']).values.T
# Save the features to .npy files
np.save('./data/TargetQA/target_crispr_feature.npy', target_crispr_feat)
# Print the shapes of the features
print("Shape of target_crispr_feat:", target_crispr_feat.shape)

##### Omic Information

In [ ]:
# select the columns in the overlapped_dti_crispr_rna_samples for gene_df
dti_crispr_rna_gene_df = gene_df[['gene_name'] + sorted(overlapped_dti_crispr_rna_samples)].copy()
display(dti_crispr_rna_gene_df)
bmgc_protein_llmnameid_combined_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
display(bmgc_protein_llmnameid_combined_df)

def extract_gn_info(dti_crispr_rna_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_crispr_rna_gene_df columns
    if sample_ach_name not in dti_crispr_rna_gene_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_genes = dti_crispr_rna_gene_df.nlargest(k, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top k genes by their values in descending order
    top_k_genes = top_k_genes.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_gene_hgnc_name_list = top_k_genes['gene_name'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_gene_df = pd.merge(bmgc_gene_df, top_k_genes, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_k_bmgc_gene_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_k_bmgc_gene_protein_info_df = pd.merge(top_k_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_gene_protein_bmgc_id_list = top_k_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_k_gene_protein_bmgc_llmnameid_list = top_k_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k=10
top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list = extract_gn_info(dti_crispr_rna_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Gene HGNC Names:", top_k_gene_hgnc_name_list)
print(f"Top {k} Gene Protein BMGC IDs:", top_k_gene_protein_bmgc_id_list)
print(f"Top {k} Gene Protein BMGC LLM Name IDs:", top_k_gene_protein_bmgc_llmnameid_list)

In [ ]:
# select the columns in the overlapped_dti_crispr_rna_samples for raw_transcript_df
transcript_overlapped_dti_crispr_rna_samples = sorted(list(set(raw_transcript_df.columns[1:]) & set(overlapped_dti_crispr_rna_samples)))
dti_crispr_rna_transcript_df = raw_transcript_df[['gene_name'] + transcript_overlapped_dti_crispr_rna_samples].copy()
display(dti_crispr_rna_transcript_df)
bmgc_transcript_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Transcript/BioMedGraphica_Conn_Transcript.csv')

def extract_ts_info(dti_crispr_rna_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_crispr_rna_transcript_df columns
    if sample_ach_name not in dti_crispr_rna_transcript_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_transcripts = dti_crispr_rna_transcript_df.nlargest(k, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top k transcripts by their values in descending order
    top_k_transcripts = top_k_transcripts.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_transcript_hgnc_name_list = top_k_transcripts['gene_name'].tolist()
    # Merge with the bmgc_transcript_df to get the BioMedGraphica_Conn_ID
    bmgc_transcript_df = bmgc_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_transcript_df = pd.merge(bmgc_transcript_df, top_k_transcripts, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_transcript_protein_df = pd.merge(transcript_protein_entity_df, top_k_bmgc_transcript_df, left_on='BMGC_TS_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_k_bmgc_transcript_protein_info_df = pd.merge(top_k_bmgc_transcript_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_transcript_protein_bmgc_id_list = top_k_bmgc_transcript_protein_info_df['BMGC_PT_ID'].tolist()
    top_k_transcript_protein_bmgc_llmnameid_list = top_k_bmgc_transcript_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k=10
top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list = extract_ts_info(dti_crispr_rna_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Transcript HGNC Names:", top_k_transcript_hgnc_name_list)
print(f"Top {k} Transcript Protein BMGC IDs:", top_k_transcript_protein_bmgc_id_list)
print(f"Top {k} Transcript Protein BMGC LLM Name IDs:", top_k_transcript_protein_bmgc_llmnameid_list)

In [ ]:
bmg_protein_all_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
bmg_protein_all_df = bmg_protein_all_df[['BioMedGraphica_Conn_ID', 'Uniprot_ID', 'HGNC_Symbol']].copy()
display(bmg_protein_all_df)

# Rename columns in raw_protein_df using the provided mapping
raw_protein_map_df = raw_protein_df.rename(columns=protein_map_dict)
# Merge with bmg_protein_all_df to get HGNC symbols
symbol_protein_map_df = pd.merge(raw_protein_map_df, bmg_protein_all_df, left_on='Uniprot_Acc', right_on='Uniprot_ID', how='inner')
# Reorder columns: keep Uniprot IDs and protein expression values
symbol_protein_map_df = symbol_protein_map_df[['Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol'] + sorted(set(symbol_protein_map_df.columns) - {'Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol'})]
# Identify overlapping samples between protein data and the provided sample list
protein_overlapped_samples = sorted(set(symbol_protein_map_df.columns) & set(overlapped_dti_crispr_rna_samples))
# Select only HGNC symbol and overlapping sample columns
dti_crispr_rna_protein_df = symbol_protein_map_df[['HGNC_Symbol'] + protein_overlapped_samples].copy()
# Split multiple HGNC symbols by ";" and expand into multiple rows
dti_crispr_rna_protein_df = dti_crispr_rna_protein_df.assign(HGNC_Symbol=dti_crispr_rna_protein_df['HGNC_Symbol'].str.split(';')).explode('HGNC_Symbol')
# Remove leading/trailing whitespace in gene symbols
dti_crispr_rna_protein_df['HGNC_Symbol'] = dti_crispr_rna_protein_df['HGNC_Symbol'].str.strip()
# Drop rows with empty or missing gene symbols
dti_crispr_rna_protein_df = dti_crispr_rna_protein_df[dti_crispr_rna_protein_df['HGNC_Symbol'].notna() & (dti_crispr_rna_protein_df['HGNC_Symbol'] != '')].reset_index(drop=True)
# Display final dataframe
display(dti_crispr_rna_protein_df)

bmgc_protein_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')

def extract_pt_info(dti_crispr_rna_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_crispr_rna_protein_df columns
    if sample_ach_name not in dti_crispr_rna_protein_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_proteins = dti_crispr_rna_protein_df.nlargest(k, sample_ach_name)[['HGNC_Symbol', sample_ach_name]]
    # Sort the top k proteins by their values in descending order
    top_k_proteins = top_k_proteins.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_protein_hgnc_name_list = top_k_proteins['HGNC_Symbol'].tolist()
    # Merge with the bmgc_protein_df to get the BioMedGraphica_Conn_ID
    bmgc_protein_df = bmgc_protein_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_protein_df = pd.merge(bmgc_protein_df, top_k_proteins, left_on='HGNC_Symbol', right_on='HGNC_Symbol', how='inner')
    # Get the corresponding protein information
    top_k_bmgc_protein_info_df = pd.merge(top_k_bmgc_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_ID', sample_ach_name])
    top_k_protein_bmgc_id_list = top_k_bmgc_protein_info_df['BioMedGraphica_Conn_ID'].tolist()
    # Replace both "|" and ";" with " or "
    top_k_protein_bmgc_llmnameid_list = top_k_bmgc_protein_info_df['Names_and_IDs'].replace([r' \| ', r';'], ' or ', regex=True).tolist()
    return top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
k = 10
top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list = extract_pt_info(dti_crispr_rna_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Protein HGNC Names:", top_k_protein_hgnc_name_list)
print(f"Top {k} Protein BMGC IDs:", top_k_protein_bmgc_id_list)
print(f"Top {k} Protein BMGC LLM Name IDs:", top_k_protein_bmgc_llmnameid_list)

##### Related Proteins

In [ ]:
def bmgc_pt_id_to_hgnc(bmgc_id_list, bmgc_protein_df):
    """
    Convert a list of BioMedGraphica IDs to their corresponding HGNC symbols.
    
    Args:
        bmgc_id_list (list): List of BioMedGraphica IDs
        bmgc_protein_df (pd.DataFrame): DataFrame with BioMedGraphica IDs and HGNC symbols
        
    Returns:
        tuple: (
            dict: Dictionary mapping each BioMedGraphica ID to its list of HGNC symbols,
            list: Combined list of all HGNC symbols
        )
    """
    # Ensure bmgc_id_list is actually a list
    if not isinstance(bmgc_id_list, list):
        bmgc_id_list = [bmgc_id_list]
    results = {}
    all_hgnc_symbols = []
    for bmgc_id in bmgc_id_list:
        # Filter the DataFrame for the given BioMedGraphica ID
        filtered_df = bmgc_protein_df[bmgc_protein_df['BioMedGraphica_Conn_ID'] == bmgc_id]
        # Skip if no match found
        if filtered_df.empty:
            results[bmgc_id] = []
            continue
        # Get the HGNC symbols
        hgnc_value = filtered_df['HGNC_Symbol'].values[0]
        # Skip if HGNC symbol is NaN
        if pd.isna(hgnc_value):
            results[bmgc_id] = []
            continue
        # Process valid HGNC symbols
        hgnc_list = list(set(hgnc_value.split(';')))
        hgnc_list = [hgnc.strip() for hgnc in hgnc_list if hgnc.strip() != '']
        results[bmgc_id] = hgnc_list
        all_hgnc_symbols.extend(hgnc_list)
    # Remove duplicates from the combined list
    all_hgnc_symbols = list(set(all_hgnc_symbols))
    return results, all_hgnc_symbols

def hgnc_to_bmgc_pt_id(hgnc_list, bmgc_protein_df):
    """
    Convert a list of HGNC symbols to their corresponding BioMedGraphica IDs.
    
    Args:
        hgnc_list (list): List of HGNC symbols
        bmgc_protein_df (pd.DataFrame): DataFrame with BioMedGraphica IDs and HGNC symbols
        
    Returns:
        tuple: (
            dict: Dictionary mapping each HGNC symbol to its list of BioMedGraphica IDs,
            list: Combined list of all BioMedGraphica IDs
        )
    """
    # Ensure hgnc_list is actually a list
    if not isinstance(hgnc_list, list):
        hgnc_list = [hgnc_list]
    results = {}
    all_bmgc_ids = []
    for hgnc in hgnc_list:
        # Filter the DataFrame for the given HGNC symbol
        filtered_df = bmgc_protein_df[bmgc_protein_df['HGNC_Symbol'] == hgnc]
        # Skip if no match found
        if filtered_df.empty:
            results[hgnc] = []
            continue
        # Get the BioMedGraphica IDs
        bmgc_value = filtered_df['BioMedGraphica_Conn_ID'].values[0]
        # Skip if BioMedGraphica ID is NaN
        if pd.isna(bmgc_value):
            results[hgnc] = []
            continue
        # Process valid BioMedGraphica IDs
        bmgc_list = list(set(bmgc_value.split(';')))
        bmgc_list = [bmgc.strip() for bmgc in bmgc_list if bmgc.strip() != '']
        results[hgnc] = bmgc_list
        all_bmgc_ids.extend(bmgc_list)
    # Remove duplicates from the combined list
    all_bmgc_ids = list(set(all_bmgc_ids))
    return results, all_bmgc_ids

# Example usage
bmgc_ids = ['BMGC_PT000001', 'BMGC_PT013541']
hgnc_dict, all_hgnc_symbols = bmgc_pt_id_to_hgnc(bmgc_ids, bmgc_protein_df)
print(hgnc_dict)
print(all_hgnc_symbols)
# Example usage
hgnc_list = ['BRCA1', 'TP53']
bmgc_dict, all_bmgc_ids = hgnc_to_bmgc_pt_id(hgnc_list, bmgc_protein_df)
print(bmgc_dict)
print(all_bmgc_ids)

In [ ]:
def extract_disease_protein(selected_sample_disease_bmgc_id, edge_index, 
                    node_index_df, nodeid_index_dict, index_nodeid_dict):
    # Extract the index based on the selected disease BMGC ID
    sample_disease_bmgc_id_index = nodeid_index_dict[selected_sample_disease_bmgc_id]
    # Find incoming edges (source nodes that point to the disease)
    incoming_mask = edge_index[1, :] == sample_disease_bmgc_id_index
    incoming_source_nodes = edge_index[0, incoming_mask]
    # Find outgoing edges (target nodes that the disease points to)
    outgoing_mask = edge_index[0, :] == sample_disease_bmgc_id_index
    outgoing_target_nodes = edge_index[1, outgoing_mask]
    # Combine all neighbor nodes (both incoming and outgoing)
    disease_related_nodes = np.concatenate([incoming_source_nodes, outgoing_target_nodes])
    unique_disease_related_nodes = np.unique(disease_related_nodes)
    # Get protein node index
    protein_node_index_df = node_index_df[node_index_df['Type'] == 'Protein']
    protein_node_index_list = protein_node_index_df['Index'].tolist()
    # Filter to get only protein nodes directly connected to the disease
    disease_protein_index = sorted(
        list(set(unique_disease_related_nodes) & set(protein_node_index_list))
    )
    # Map protein index to BMGC id
    disease_protein_bmgc_id = [index_nodeid_dict[i] for i in disease_protein_index]
    return disease_protein_index, disease_protein_bmgc_id

def extract_ppi_nodes(disease_protein_index, edge_index, node_index_df, index_nodeid_dict):
    # Get protein node index
    protein_node_index_df = node_index_df[node_index_df['Type'] == 'Protein']
    protein_node_index_list = protein_node_index_df['Index'].tolist()
    # Get all nodes related to the identified protein neighbors (second hop)
    protein_related_nodes = []
    # Iterate through each protein neighbor node index
    for protein_node_idx in disease_protein_index:
        # Find incoming edges (nodes that point to this protein)
        protein_incoming_mask = edge_index[1, :] == protein_node_idx
        protein_incoming_sources = edge_index[0, protein_incoming_mask]
        # Find outgoing edges (nodes that this protein points to)
        protein_outgoing_mask = edge_index[0, :] == protein_node_idx
        protein_outgoing_targets = edge_index[1, protein_outgoing_mask]
        # Add these connected nodes to our list
        protein_related_nodes.extend(protein_incoming_sources)
        protein_related_nodes.extend(protein_outgoing_targets)
    # Convert to numpy array and get unique nodes
    protein_related_nodes = np.array(protein_related_nodes)
    unique_protein_related_nodes = np.unique(protein_related_nodes)
    # Remove any protein nodes themselves from this list to avoid duplication
    unique_protein_related_nodes = np.setdiff1d(
        unique_protein_related_nodes, disease_protein_index
    )
    # Filter to only keep protein nodes among the second-hop neighbors
    ppi_nodes_index = sorted(
        list(set(unique_protein_related_nodes) & set(protein_node_index_list))
    )
    # Map PPI node index to BMGC id
    ppi_nodes_bmgc_id = [index_nodeid_dict[i] for i in ppi_nodes_index]
    return ppi_nodes_index, ppi_nodes_bmgc_id

def extract_kg_related_proteins(selected_sample_disease_bmgc_id, edge_index,  # './data/DTI_data/edge_index.npy'
                               node_index_df, nodeid_index_dict, index_nodeid_dict):
    """
    Extract disease-related proteins and their interactions from the knowledge graph.
    
    Args:
        selected_sample_disease_bmgc_id (str): BMGC ID of the selected disease
        edge_index (str): Edge index file
        node_index_df (pd.DataFrame): DataFrame with node type information
        nodeid_index_dict (dict): Mapping from node ID to index
        index_nodeid_dict (dict): Mapping from index to node ID
    
    Returns:
        tuple: (disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id)
    """
    
    # Extract disease-protein connections
    disease_protein_index, disease_protein_bmgc_id = extract_disease_protein(
        selected_sample_disease_bmgc_id=selected_sample_disease_bmgc_id,
        edge_index=edge_index,
        node_index_df=node_index_df,
        nodeid_index_dict=nodeid_index_dict,
        index_nodeid_dict=index_nodeid_dict
    )

    # Extract protein-protein interactions (can replace this with LLM to generate PPI, may need NER and mapping to BMGC id)
    ppi_nodes_index, ppi_nodes_bmgc_id = extract_ppi_nodes(
        disease_protein_index=disease_protein_index,
        edge_index=edge_index,
        node_index_df=node_index_df,
        index_nodeid_dict=index_nodeid_dict
    )

    # Convert the BMGC id into HGNC symbol

    return disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id

# Example usage
selected_sample_disease_bmgc_id = 'BMGC_DS07934'
edge_index = np.load('./data/TargetQA/edge_index.npy')
index_node_dict = dict(zip(nodes_index_data['Index'], nodes_index_data['Node']))

disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id = extract_kg_related_proteins(
    selected_sample_disease_bmgc_id=selected_sample_disease_bmgc_id,
    edge_index=edge_index,
    node_index_df=nodes_index_data,
    nodeid_index_dict=node_index_dict,
    index_nodeid_dict=index_node_dict
)
print("Disease Protein Index:", disease_protein_index)
print("Disease Protein BMGC ID:", disease_protein_bmgc_id)

##### CRISPR Answer

In [ ]:
answer_crispr_df = crispr_df[['gene_name'] + sorted(overlapped_dti_crispr_rna_samples)].copy()
display(answer_crispr_df)

def extract_answer(answer_crispr_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm=100):
    # Extract the lowest top_bm values for the given sample name
    top_bm_genes = answer_crispr_df.nsmallest(top_bm, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top bm genes by their values in ascending order
    top_bm_genes = top_bm_genes.sort_values(by=sample_ach_name, ascending=True).reset_index(drop=True)
    top_bm_gene_hgnc_name_list = top_bm_genes['gene_name'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_bm_bmgc_gene_df = pd.merge(bmgc_gene_df, top_bm_genes, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_bm_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_bm_bmgc_gene_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_bm_bmgc_gene_protein_info_df = pd.merge(top_bm_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_bm_gene_protein_bmgc_id_list = top_bm_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_bm_gene_protein_bmgc_llmnameid_list = top_bm_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
top_bm = 10
bmgc_protein_llmnameid_combined_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list = extract_answer(answer_crispr_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm)
print(f"Top {top_bm} Gene HGNC Names:", top_bm_gene_hgnc_name_list)
print(f"Top {top_bm} Gene Protein BMGC IDs:", top_bm_gene_protein_bmgc_id_list)
print(f"Top {top_bm} Gene Protein BMGC LLM Name IDs:", top_bm_gene_protein_bmgc_llmnameid_list)

##### Knowledge Graph Information

In [ ]:
def find_protein_relationships(hgnc_symbols, bmgc_protein_df, bmgc_relation_df):
    """
    Find relationships between a list of proteins based on HGNC symbols.
    
    Parameters:
    -----------
    hgnc_symbols : list
        List of HGNC symbols to find relationships between
    bmgc_protein_df : pandas DataFrame
        DataFrame containing BioMedGraphica_Conn_ID and HGNC_Symbol columns
    bmgc_relation_df : pandas DataFrame
        DataFrame containing BMGC_From_ID and BMGC_To_ID columns
        
    Returns:
    --------
    tuple (pandas DataFrame, list)
        - DataFrame with source_symbol, target_symbol and their relationship
        - List of text descriptions of relationships in "A -> B" format
    """
    # Filter the protein DataFrame to only include the proteins we care about
    filtered_proteins = bmgc_protein_df[bmgc_protein_df['HGNC_Symbol'].isin(hgnc_symbols)]
    # Create a mapping from HGNC symbol to BMGC ID
    hgnc_to_bmgc = dict(zip(filtered_proteins['HGNC_Symbol'], filtered_proteins['BioMedGraphica_Conn_ID']))
    bmgc_to_hgnc = dict(zip(filtered_proteins['BioMedGraphica_Conn_ID'], filtered_proteins['HGNC_Symbol']))
    # Get all BMGC IDs of our proteins
    bmgc_ids = list(hgnc_to_bmgc.values())
    # Filter the relationship DataFrame to only include relationships between our proteins
    protein_relations = bmgc_relation_df[
        bmgc_relation_df['BMGC_From_ID'].isin(bmgc_ids) & 
        bmgc_relation_df['BMGC_To_ID'].isin(bmgc_ids)
    ]
    # Map the BMGC IDs back to HGNC symbols
    result_data = []
    text_descriptions = []
    for _, row in protein_relations.iterrows():
        source_bmgc = row['BMGC_From_ID']
        target_bmgc = row['BMGC_To_ID']
        if source_bmgc in bmgc_to_hgnc and target_bmgc in bmgc_to_hgnc:
            source_symbol = bmgc_to_hgnc[source_bmgc]
            target_symbol = bmgc_to_hgnc[target_bmgc]
            # Create text description
            text_description = f"{source_symbol} -> {target_symbol}"
            text_descriptions.append(text_description)
            # If relation_type column exists, include it in the description and data
            relation_info = {
                'source_symbol': source_symbol,
                'target_symbol': target_symbol
            }
            # Add relation type if it exists in the DataFrame
            if 'relation_type' in bmgc_relation_df.columns:
                relation_type = row['relation_type']
                relation_info['relation_type'] = relation_type
                text_descriptions[-1] = f"{source_symbol} -{relation_type}-> {target_symbol}"
            result_data.append(relation_info)
    # Create a DataFrame from the results
    result_df = pd.DataFrame(result_data)
    return result_df, text_descriptions

# Define the HGNC symbols
hgnc_symbols = ['SNRPD3', 'RAN', 'RPS8', 'UBL5', 'SMU1', 'RRM1', 'PSMA6', 'PSMB3', 'WEE1', 
                'PHB1', 'BANF1', 'KIF11', 'SNRPD1', 'PSMA3', 'PSMD11', 'PRPF19', 'SNRPF', 
                'RPS29', 'CDC27', 'SRSF3', 'TUBGCP2', 'ECD', 'RPS20', 'PCNA', 'PSMA7', 'CDC7', 
                'RPL17', 'GINS1', 'PHB2', 'SRSF2', 'MAD2L1', 'MED14']

# Call the function to find relationships
relationships_df, relationship_texts = find_protein_relationships(hgnc_symbols, bmgc_protein_df, bmgc_relation_df)

# You can now use both the DataFrame and text descriptions
# Example usage:
print(f"Found {len(relationship_texts)} relationships between the proteins")
print("Example relationships:")
print(relationship_texts[:5])  # Print first 5 relationships

##### Formulate QA JSON

In [ ]:
def qa_sample_info(sample_ach_name, selected_sample_disease_bmgc_id, k=100, top_bm=100):
    # LLM Info
    print(f"Sample ACH Name: {sample_ach_name}")
    print(f"Extracting top {k} gene information for {sample_ach_name}...")
    top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list = extract_gn_info(dti_crispr_rna_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    print(f"Extracting top {k} transcript information for {sample_ach_name}...")
    top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list = extract_ts_info(dti_crispr_rna_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    print(f"Extracting top {k} protein information for {sample_ach_name}...")
    top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list = extract_pt_info(dti_crispr_rna_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    # KG Info
    edge_index = np.load('./data/TargetQA/edge_index.npy')
    print(f"Extracting disease-related proteins index and bmgc id for {selected_sample_disease_bmgc_id} ({sample_ach_name}) ...")
    disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id = extract_kg_related_proteins(selected_sample_disease_bmgc_id, edge_index, nodes_index_data, node_index_dict, index_node_dict)
    print(f"Knowledge Graph Info: Found {len(disease_protein_index)} disease-related proteins directly connected to {selected_sample_disease_bmgc_id} and {len(ppi_nodes_index)} proteins in their PPI network")
    print(f"Mapping disease-related proteins to HGNC symbols...")
    disease_protein_hgnc_dict, disease_protein_hgnc_list = bmgc_pt_id_to_hgnc(disease_protein_bmgc_id, bmgc_protein_df)
    print(f"Mapping PPI-related proteins to HGNC symbols...")
    ppi_hgnc_dict, ppi_hgnc_list = bmgc_pt_id_to_hgnc(ppi_nodes_bmgc_id, bmgc_protein_df)
    # LLM Used KG Info
    print(f"Extracting protein relationships from BMGC...")
    # Convert the any non-existed string in top_k_gene_hgnc_name_list + top_k_transcript_hgnc_name_list + top_k_protein_hgnc_name_list + disease_protein_hgnc_list into empty list []
    if top_k_gene_hgnc_name_list == "non-existed": top_k_gene_hgnc_name_list = []
    if top_k_transcript_hgnc_name_list == "non-existed": top_k_transcript_hgnc_name_list = []
    if top_k_protein_hgnc_name_list == "non-existed": top_k_protein_hgnc_name_list = []
    if disease_protein_hgnc_list == "non-existed": disease_protein_hgnc_list = []
    # Combine all the HGNC symbols into a single list for relationship extraction
    omics_disease_protein_hgnc_list = list(set(top_k_gene_hgnc_name_list + top_k_transcript_hgnc_name_list + top_k_protein_hgnc_name_list + disease_protein_hgnc_list))
    relationships_df, relationship_texts = find_protein_relationships(omics_disease_protein_hgnc_list, bmgc_protein_df, bmgc_relation_df)
    print(f"Knowledge Graph Info: Found {len(omics_disease_protein_hgnc_list)} unique proteins and {len(relationship_texts)} relationships between them")
    # Answer Info
    print(f"Extracting top {top_bm} CRISPR gene information for {sample_ach_name}...")
    top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list = extract_answer(answer_crispr_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, top_bm)
    return top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list, \
              top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list, \
                top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list, \
                disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id, \
                    disease_protein_hgnc_dict, disease_protein_hgnc_list, ppi_hgnc_dict, ppi_hgnc_list, relationship_texts, \
                        top_bm_gene_hgnc_name_list, top_bm_gene_protein_bmgc_id_list, top_bm_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000001'
selected_sample_disease_bmgc_id = 'BMGC_DS07934'
k = 100
top_bm = 100
return_tuples = qa_sample_info(sample_ach_name, selected_sample_disease_bmgc_id, k=k, top_bm=top_bm)

In [ ]:
import json
import os
from tqdm import tqdm

# Parameters
k = 10
top_bm = 100
save_every_n = 10

# Output folder and filename
output_dir = "./data/TargetQA"
os.makedirs(output_dir, exist_ok=True)
output_filename = f"target_qa_k{k}_bm{top_bm}.json"
output_path = os.path.join(output_dir, output_filename)

# Load existing JSON file if it exists
if os.path.exists(output_path):
    with open(output_path, "r") as f:
        multi_sample_qa_json = json.load(f)
    print(f"Loaded existing JSON with {len(multi_sample_qa_json)} processed samples")
else:
    multi_sample_qa_json = {}
    print("No existing JSON found, starting fresh")

# Load sample info data
dti_sample_info_index = pd.read_csv('./data/process_data/dti_combined_samples.csv')
target_sample_info_index = pd.merge(dti_sample_info_index, overlapped_dti_crispr_rna_samples_df['depMapID'], how='inner', on='depMapID')
target_sample_info_index = target_sample_info_index[target_sample_info_index['depMapID'].isin(overlapped_dti_crispr_rna_samples)].reset_index(drop=True)
target_sample_info_index['BMGC_Disease_name'] = target_sample_info_index['BMGC_Disease_name'].replace(r' \| ', ' or ', regex=True)
# Insert a new column "Index" in the first position
target_sample_info_index.insert(0, 'Index', range(1, len(target_sample_info_index) + 1))
display(target_sample_info_index)

In [ ]:
# Get the list of samples that have already been processed
processed_samples = set(multi_sample_qa_json.keys())
print(f"Found {len(processed_samples)} already processed samples")

count = len(processed_samples)
total_to_process = len(target_sample_info_index)
remaining = total_to_process - count
print(f"Total samples to process: {total_to_process}, already processed: {count}, remaining: {remaining}")

# Iterate through the sample info dataframe, skipping already processed samples
for idx, row_tuple in tqdm(enumerate(target_sample_info_index.iterrows()), total=len(target_sample_info_index)):
    _, row = row_tuple  # Unpack the tuple - index and row data

    sample_ach_name = row["depMapID"]
    
    # Skip if already processed
    if sample_ach_name in processed_samples:
        continue
        
    count += 1
    target_sample_index = row["Index"]
    cell_line_name = row["Name"]
    disease = row["BMGC_Disease_name"]
    disease_bmgc_id = row["BMGC_Disease_ID"]

    print(f"Processing sample {count}/{total_to_process}: {sample_ach_name} ({cell_line_name})")
    print(f"Sample Index: {target_sample_index}")
    print(f"Sample Disease: {disease}")
    print(f"Sample Disease BMGC ID: {disease_bmgc_id}")

    try:
        (top_k_gene_hgnc, top_k_gene_bmgc, top_k_gene_llm,
        top_k_ts_hgnc, top_k_ts_bmgc, top_k_ts_llm,
        top_k_pt_hgnc, top_k_pt_bmgc, top_k_pt_llm,
        dis_pt_idx, dis_pt_bmgc, ppi_idx, ppi_bmgc,
        dis_pt_hgnc_dict, dis_pt_hgnc, ppi_hgnc_dict, ppi_hgnc, relationship_texts,
        ans_hgnc, ans_bmgc, ans_llm) = qa_sample_info(sample_ach_name, disease_bmgc_id, k=k, top_bm=top_bm)

        multi_sample_qa_json[sample_ach_name] = {
            "cell_line_name": cell_line_name,
            "sample_index": target_sample_index,
            "disease": disease,
            "disease_bmgc_id": disease_bmgc_id,
            "input": {
                "top_k_gene": {
                    "hgnc_symbols": top_k_gene_hgnc,
                    "protein_bmgc_ids": top_k_gene_bmgc,
                    "protein_llmname_ids": top_k_gene_llm
                },
                "top_k_transcript": {
                    "hgnc_symbols": top_k_ts_hgnc,
                    "protein_bmgc_ids": top_k_ts_bmgc,
                    "protein_llmname_ids": top_k_ts_llm
                },
                "top_k_protein": {
                    "hgnc_symbols": top_k_pt_hgnc,
                    "protein_bmgc_ids": top_k_pt_bmgc,
                    "protein_llmname_ids": top_k_pt_llm
                },
                "knowledge_graph": {
                    "disease_protein": {
                        "bmgc_ids": dis_pt_bmgc,
                        "hgnc_symbols": dis_pt_hgnc,
                        "indices": dis_pt_idx
                    },
                    "ppi_neighbors": {
                        "bmgc_ids": ppi_bmgc,
                        "hgnc_symbols": ppi_hgnc,
                        "indices": ppi_idx
                    },
                    "protein_relationships": relationship_texts,
                }
            },
            "ground_truth_answer": {
                "top_bm_gene": {
                    "hgnc_symbols": ans_hgnc,
                    "protein_bmgc_ids": ans_bmgc,
                    "protein_llmname_ids": ans_llm
                }
            }
        }

    except Exception as e:
        print(f"⚠️ Error processing {sample_ach_name}: {e}")
        continue

    # Periodic save every N samples
    if count % save_every_n == 0:
        with open(output_path, "w") as f:
            json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)
        print(f"💾 Auto-saved JSON at {count}/{total_to_process} samples to: {output_path}")
        print(f"Last processed sample: {sample_ach_name}")
        processed = len(multi_sample_qa_json)
        remaining = total_to_process - processed
        print(f"Progress: {processed}/{total_to_process} ({processed/total_to_process*100:.1f}%), Remaining: {remaining}")

# Final save after loop
with open(output_path, "w") as f:
    json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)

print(f"✅ Final JSON saved to: {output_path}")
print(f"Total samples processed: {len(multi_sample_qa_json)}")

##### Split QA as Train and Test

In [482]:
import json
import pandas as pd

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def separate_data(data):
    # Get the training and testing list
    target_tr_df = pd.read_csv('./data/TargetQA/train_samples.csv')
    target_te_df = pd.read_csv('./data/TargetQA/test_samples.csv')
    target_tr_list = target_tr_df['depMapID'].tolist()
    target_te_list = target_te_df['depMapID'].tolist()
    # Separate the data into two lists based on tr/te list with (for sample_id, sample_info in data.items())
    tr_data = {}
    te_data = {}
    for sample_id, sample_info in data.items():
        if sample_id in target_tr_list:
            tr_data[sample_id] = sample_info
        elif sample_id in target_te_list:
            te_data[sample_id] = sample_info
    # Check if the separated data is correct
    tr_data_count = len(tr_data)
    te_data_count = len(te_data)
    print(f"Training data count: {tr_data_count}")
    print(f"Testing data count: {te_data_count}")
    # Check if the training and testing data are mutually exclusive
    if set(tr_data.keys()).intersection(te_data.keys()):
        print("Error: Training and testing data are not mutually exclusive.")
    else:
        print("Training and testing data are mutually exclusive.")
    return tr_data, te_data

def save_to_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=2)
    print(f"Data saved to {file_path}")

In [483]:
# Read the JSON file
data = read_json('./data/TargetQA/target_qa_k10_bm100.json')

# Separate the data into training and testing sets
tr_data, te_data = separate_data(data)
# Save the separated data to new JSON files
save_to_json(tr_data, './data/TargetQA/target_qa_k10_bm100_tr.json')
save_to_json(te_data, './data/TargetQA/target_qa_k10_bm100_te.json')

Training data count: 300
Testing data count: 63
Training and testing data are mutually exclusive.
Data saved to ./data/TargetQA/target_qa_k10_bm100_tr.json
Data saved to ./data/TargetQA/target_qa_k10_bm100_te.json


##### Formulate QA text

#### 9.2.3 Drug data integration

In [389]:
# insert the samples that is in the remaining_dti_samples but not in the maped_methy_df
for sample in remaining_dti_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the remaining_dti_samples
dti_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_methy_df)

# insert the samples that is in the remaining_dti_samples but not in the mapped_protein_df
for sample in remaining_dti_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the remaining_dti_samples
dti_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_protein_df)

,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PM00001,0.83335,0.00,0.88351,1.00000,0.94738,1.0,0.50982,0.56520,0.0,...,0.93319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PM00002,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PM00003,0.04959,0.00,0.00129,0.02137,0.01066,0.0,0.01239,0.00000,0.0,...,0.00958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PM00004,0.05386,0.25,0.00000,0.33075,0.07825,0.0,0.72705,0.00761,0.0,...,0.92687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PM00005,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86234,BMGC_PM86235,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86235,BMGC_PM86236,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86236,BMGC_PM86237,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\3891391275.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\3891391275.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mapped_protein_df[sample] = 0.0
C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\3891391275.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider 

,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [390]:
# fill in the missing samples in the final_merged_gene_df
for sample in remaining_dti_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df
for sample in remaining_dti_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the remaining_dti_samples
dti_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_gene_df)
dti_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_transcript_df)
dti_drug_overlap_df = final_drug_df[final_drug_df['ARXSPAN_ID'].isin(remaining_dti_samples)].reset_index(drop=True)
dti_drug_overlap_df = pd.merge(dti_drug_overlap_df, dti_combined_df, left_on='ARXSPAN_ID', right_on='depMapID', how='left')
dti_drug_overlap_df = pd.merge(dti_drug_overlap_df, final_merged_drug_df, on='DRUG_NAME', how='left').rename(columns={'BMGC_Disease_name': 'BMGC_Disease_Name', 'DRUG_NAME': 'BMGC_Drug_Name', 'BioMedGraphica_Conn_ID': 'BMGC_Drug_ID', 'Name': 'Cell_Line_Name', 'AUC_PUBLISHED': 'AUC'})
# only keep columns ['depMapID', 'Cell_Line_Name', 'BMGC_Drug_ID', 'BMGC_Drug_Name', 'BMGC_Disease_ID', 'BMGC_Disease_Name', 'AUC']
dti_drug_overlap_df = dti_drug_overlap_df[['depMapID', 'Cell_Line_Name', 'BMGC_Drug_ID', 'BMGC_Drug_Name', 'BMGC_Disease_ID', 'BMGC_Disease_Name', 'AUC']]
# check if there is null values in the dti_drug_overlap_df
print(dti_drug_overlap_df.isnull().sum())
display(dti_drug_overlap_df)

,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_GN00001,1.039342,1.081979,1.069887,1.021275,1.019950,0.873542,1.954398,0.476005,0.498743,...,0.930320,1.280913,0.816260,0.996102,1.018304,1.050028,1.005951,1.040949,1.409362,0.863441
1,BMGC_GN00002,0.915792,1.032808,0.843013,0.959549,1.009547,0.971925,1.188910,1.511536,1.047491,...,1.009008,1.076599,0.906371,1.057301,0.828024,0.445954,1.003790,1.021203,0.519903,1.112888
2,BMGC_GN00003,0.978356,0.909618,1.676606,0.990115,0.993623,0.953061,1.229717,1.566409,1.034708,...,1.075723,1.279596,1.212607,1.399931,0.986576,1.268384,0.997014,0.996969,0.995337,0.380086
3,BMGC_GN00004,1.514518,1.024754,0.868152,1.921635,1.010449,0.986412,1.231253,1.062672,1.010414,...,1.063356,0.750272,0.880448,0.750693,1.368224,0.465270,0.989083,0.969761,1.023638,0.410329
4,BMGC_GN00005,1.032720,1.165655,1.251083,0.987747,1.006196,1.438704,1.172893,1.024139,1.042507,...,1.019288,1.084336,0.968458,0.988188,1.071999,0.924170,0.999373,1.046953,1.001543,1.245752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_GN86234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86234,BMGC_GN86235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86235,BMGC_GN86236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86236,BMGC_GN86237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_TS000001,6.522291,5.854020,7.104063,6.875601,6.986633,7.254845,0.0,6.293006,6.912981,...,7.422634,0.0,0.0,0.0,0.0,6.220747,0.0,0.0,0.0,0.0
1,BMGC_TS000002,6.729154,6.338599,7.897515,6.917430,7.218023,7.815267,0.0,5.232725,6.944961,...,6.139888,0.0,0.0,0.0,0.0,6.795080,0.0,0.0,0.0,0.0
2,BMGC_TS000003,4.419503,4.986058,4.621525,3.843395,4.675843,4.200734,0.0,5.066766,4.064158,...,4.260150,0.0,0.0,0.0,0.0,4.455935,0.0,0.0,0.0,0.0
3,BMGC_TS000004,5.213352,5.151831,5.766890,5.052784,5.001630,6.215894,0.0,5.329526,5.736236,...,5.175159,0.0,0.0,0.0,0.0,5.969341,0.0,0.0,0.0,0.0
4,BMGC_TS000005,0.043941,-0.073548,0.335595,0.005892,-0.059982,1.853420,0.0,0.031376,-0.059982,...,-0.046550,0.0,0.0,0.0,0.0,2.881209,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412034,BMGC_TS412035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
412035,BMGC_TS412036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
412036,BMGC_TS412037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
412037,BMGC_TS412038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


depMapID             0
Cell_Line_Name       0
BMGC_Drug_ID         0
BMGC_Drug_Name       0
BMGC_Disease_ID      0
BMGC_Disease_Name    0
AUC                  0
dtype: int64


,depMapID,Cell_Line_Name,BMGC_Drug_ID,BMGC_Drug_Name,BMGC_Disease_ID,BMGC_Disease_Name,AUC
0,ACH-000327,NCI-H1395,BMGC_DG04654,ERLOTINIB,BMGC_DS02944,lung adenocarcinoma,0.981350
1,ACH-000644,COLO 829,BMGC_DG04654,ERLOTINIB,BMGC_DS02930,cutaneous melanoma,0.954785
2,ACH-000137,8-MG-BA,BMGC_DG04654,ERLOTINIB,BMGC_DS00965,glioblastoma,0.980213
3,ACH-000733,NCI-H1838,BMGC_DG04654,ERLOTINIB,BMGC_DS02944,lung adenocarcinoma,0.931183
4,ACH-000530,DMS 114,BMGC_DG04654,ERLOTINIB,BMGC_DS21085,small cell lung carcinoma,0.988258
...,...,...,...,...,...,...,...
51507,ACH-000818,BT-483,BMGC_DG20688,JQ1,BMGC_DS07814,breast adenocarcinoma,0.812820
51508,ACH-000196,HCC1599,BMGC_DG20688,JQ1,BMGC_DS08035,breast ductal carcinoma,0.811327
51509,ACH-000902,CAL-148,BMGC_DG20688,JQ1,BMGC_DS07015,breast carcinoma,0.505332
51510,ACH-000930,HCC1569,BMGC_DG20688,JQ1,BMGC_DS08035,breast ductal carcinoma,0.843211


In [391]:
dti_omics_df = pd.concat([dti_methy_df, dti_gene_df, dti_transcript_df, dti_protein_df], axis=0).reset_index(drop=True)
display(dti_omics_df)
dti_feat_df = pd.merge(bmgc_entity_df, dti_omics_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_ID', 'Type'])
dti_feat_df = dti_feat_df.fillna(0.0)
display(dti_feat_df)

# convert dti_feat_df to numpy array and transpose it
dti_array = dti_feat_df.drop(columns=['BioMedGraphica_Conn_ID']).values.T
print("Shape of dti_array:", dti_array.shape)
# Save the numpy array to .npy file
np.save('./data/DrugQA/dti_feature.npy', dti_array)
np.save('./data/DrugScreen/dti_feature.npy', dti_array)

,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PM00001,0.83335,0.00,0.88351,1.00000,0.94738,1.0,0.50982,0.56520,0.0,...,0.93319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PM00002,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PM00003,0.04959,0.00,0.00129,0.02137,0.01066,0.0,0.01239,0.00000,0.0,...,0.00958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PM00004,0.05386,0.25,0.00000,0.33075,0.07825,0.0,0.72705,0.00761,0.0,...,0.92687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PM00005,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705929,BMGC_PT121415,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705930,BMGC_PT121416,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705931,BMGC_PT121417,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705932,BMGC_PT121418,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PM00001,0.83335,0.00,0.88351,1.00000,0.94738,1.0,0.50982,0.56520,0.0,...,0.93319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PM00002,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PM00003,0.04959,0.00,0.00129,0.02137,0.01066,0.0,0.01239,0.00000,0.0,...,0.00958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PM00004,0.05386,0.25,0.00000,0.33075,0.07825,0.0,0.72705,0.00761,0.0,...,0.92687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PM00005,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834804,BMGC_DG20914,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834805,BMGC_DG20915,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834806,BMGC_DG20916,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834807,BMGC_DG20917,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shape of dti_array: (286, 834809)


In [428]:
# filter out the drug screen by training and test with condition that meet both cell line test samples and drug test samples for test
final_training_drug_screen_df = dti_drug_overlap_df[dti_drug_overlap_df['depMapID'].isin(dti_train_samples)].reset_index(drop=True)
display(final_training_drug_screen_df)
# and calculate the final training drug ids and cell line ids
training_drug_ids = list(set(final_training_drug_screen_df['BMGC_Drug_ID']))
training_cell_line_ids = list(set(final_training_drug_screen_df['depMapID']))
print("len(training_drug_ids):", len(training_drug_ids))
print("len(training_cell_line_ids):", len(training_cell_line_ids))

# the rest of the drug screen is the test drug screen
final_test_drug_screen_df = dti_drug_overlap_df[dti_drug_overlap_df['depMapID'].isin(dti_test_samples)].reset_index(drop=True)
display(final_test_drug_screen_df)
# and calculate the final test drug ids and cell line ids
test_drug_ids = list(set(final_test_drug_screen_df['BMGC_Drug_ID']))
test_cell_line_ids = list(set(final_test_drug_screen_df['depMapID']))
print("len(test_drug_ids):", len(test_drug_ids))
print("len(test_cell_line_ids):", len(test_cell_line_ids))

,depMapID,Cell_Line_Name,BMGC_Drug_ID,BMGC_Drug_Name,BMGC_Disease_ID,BMGC_Disease_Name,AUC
0,ACH-000327,NCI-H1395,BMGC_DG04654,ERLOTINIB,BMGC_DS02944,lung adenocarcinoma,0.981350
1,ACH-000137,8-MG-BA,BMGC_DG04654,ERLOTINIB,BMGC_DS00965,glioblastoma,0.980213
2,ACH-000733,NCI-H1838,BMGC_DG04654,ERLOTINIB,BMGC_DS02944,lung adenocarcinoma,0.931183
3,ACH-000530,DMS 114,BMGC_DG04654,ERLOTINIB,BMGC_DS21085,small cell lung carcinoma,0.988258
4,ACH-000514,NCI-H1092,BMGC_DG04654,ERLOTINIB,BMGC_DS02887,lung small cell carcinoma,0.983326
...,...,...,...,...,...,...,...
42061,ACH-000643,HDQ-P1,BMGC_DG20688,JQ1,BMGC_DS08035,breast ductal carcinoma,0.873306
42062,ACH-000117,EFM-192A,BMGC_DG20688,JQ1,BMGC_DS07015,breast carcinoma,0.899328
42063,ACH-000818,BT-483,BMGC_DG20688,JQ1,BMGC_DS07814,breast adenocarcinoma,0.812820
42064,ACH-000930,HCC1569,BMGC_DG20688,JQ1,BMGC_DS08035,breast ductal carcinoma,0.843211


len(training_drug_ids): 172
len(training_cell_line_ids): 234


,depMapID,Cell_Line_Name,BMGC_Drug_ID,BMGC_Drug_Name,BMGC_Disease_ID,BMGC_Disease_Name,AUC
0,ACH-000644,COLO 829,BMGC_DG04654,ERLOTINIB,BMGC_DS02930,cutaneous melanoma,0.954785
1,ACH-000729,NCI-H1963,BMGC_DG04654,ERLOTINIB,BMGC_DS02887,lung small cell carcinoma,0.988450
2,ACH-000196,HCC1599,BMGC_DG04654,ERLOTINIB,BMGC_DS08035,breast ductal carcinoma,0.978046
3,ACH-000703,DMS 79,BMGC_DG04654,ERLOTINIB,BMGC_DS02887,lung small cell carcinoma,0.943754
4,ACH-000353,TE-15,BMGC_DG04654,ERLOTINIB,BMGC_DS05334,esophageal squamous cell carcinoma,0.877464
...,...,...,...,...,...,...,...
9441,ACH-000360,NCI-H508,BMGC_DG20225,BMS-754807,BMGC_DS08517,cecum adenocarcinoma,0.439161
9442,ACH-000489,SW1116,BMGC_DG20225,BMS-754807,BMGC_DS05624,colon adenocarcinoma,0.813459
9443,ACH-000236,SW1417,BMGC_DG20225,BMS-754807,BMGC_DS05624,colon adenocarcinoma,0.785710
9444,ACH-000196,HCC1599,BMGC_DG20688,JQ1,BMGC_DS08035,breast ductal carcinoma,0.811327


len(test_drug_ids): 172
len(test_cell_line_ids): 52


In [396]:
print("The overlapped samples dataframe for methylation between remaining_dti_samples:")
# insert the samples that is in the overlapped_samples but not in the maped_methy_df
for sample in remaining_dti_samples:
    if sample not in maped_methy_df.columns:
        maped_methy_df[sample] = 0.0
# filter out the samples that is not in the remaining_dti_samples
dti_methy_df = maped_methy_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_methy_df)

print("The overlapped samples dataframe for protein between remaining_dti_samples are:")
# insert the samples that is in the pretraining_samples but not in the mapped_protein_df
for sample in remaining_dti_samples:
    if sample not in mapped_protein_df.columns:
        mapped_protein_df[sample] = 0.0
# filter out the samples that is not in the remaining_dti_samples
dti_protein_df = mapped_protein_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_protein_df)

The overlapped samples dataframe for methylation between remaining_dti_samples:


,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PM00001,0.83335,0.00,0.88351,1.00000,0.94738,1.0,0.50982,0.56520,0.0,...,0.93319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PM00002,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PM00003,0.04959,0.00,0.00129,0.02137,0.01066,0.0,0.01239,0.00000,0.0,...,0.00958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PM00004,0.05386,0.25,0.00000,0.33075,0.07825,0.0,0.72705,0.00761,0.0,...,0.92687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PM00005,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_PM86234,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86234,BMGC_PM86235,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86235,BMGC_PM86236,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86236,BMGC_PM86237,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The overlapped samples dataframe for protein between remaining_dti_samples are:


,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PT000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PT000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PT000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PT000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PT000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121414,BMGC_PT121415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121415,BMGC_PT121416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121416,BMGC_PT121417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121417,BMGC_PT121418,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [397]:
# fill in the missing samples in the final_merged_gene_df for remaining_dti_samples
for sample in remaining_dti_samples:
    if sample not in final_merged_gene_df.columns:
        final_merged_gene_df[sample] = 0.0
# fill in the missing samples in the final_merged_transcript_df for remaining_dti_samples
for sample in remaining_dti_samples:
    if sample not in final_merged_transcript_df.columns:
        final_merged_transcript_df[sample] = 0.0
# get the final gene, transcript, drug dataframe by filtering the remaining_dti_samples
dti_gene_df = final_merged_gene_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_gene_df)
dti_transcript_df = final_merged_transcript_df[['BioMedGraphica_Conn_ID'] + remaining_dti_samples]
display(dti_transcript_df)

,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_GN00001,1.039342,1.081979,1.069887,1.021275,1.019950,0.873542,1.954398,0.476005,0.498743,...,0.930320,1.280913,0.816260,0.996102,1.018304,1.050028,1.005951,1.040949,1.409362,0.863441
1,BMGC_GN00002,0.915792,1.032808,0.843013,0.959549,1.009547,0.971925,1.188910,1.511536,1.047491,...,1.009008,1.076599,0.906371,1.057301,0.828024,0.445954,1.003790,1.021203,0.519903,1.112888
2,BMGC_GN00003,0.978356,0.909618,1.676606,0.990115,0.993623,0.953061,1.229717,1.566409,1.034708,...,1.075723,1.279596,1.212607,1.399931,0.986576,1.268384,0.997014,0.996969,0.995337,0.380086
3,BMGC_GN00004,1.514518,1.024754,0.868152,1.921635,1.010449,0.986412,1.231253,1.062672,1.010414,...,1.063356,0.750272,0.880448,0.750693,1.368224,0.465270,0.989083,0.969761,1.023638,0.410329
4,BMGC_GN00005,1.032720,1.165655,1.251083,0.987747,1.006196,1.438704,1.172893,1.024139,1.042507,...,1.019288,1.084336,0.968458,0.988188,1.071999,0.924170,0.999373,1.046953,1.001543,1.245752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86233,BMGC_GN86234,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86234,BMGC_GN86235,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86235,BMGC_GN86236,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
86236,BMGC_GN86237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_TS000001,6.522291,5.854020,7.104063,6.875601,6.986633,7.254845,0.0,6.293006,6.912981,...,7.422634,0.0,0.0,0.0,0.0,6.220747,0.0,0.0,0.0,0.0
1,BMGC_TS000002,6.729154,6.338599,7.897515,6.917430,7.218023,7.815267,0.0,5.232725,6.944961,...,6.139888,0.0,0.0,0.0,0.0,6.795080,0.0,0.0,0.0,0.0
2,BMGC_TS000003,4.419503,4.986058,4.621525,3.843395,4.675843,4.200734,0.0,5.066766,4.064158,...,4.260150,0.0,0.0,0.0,0.0,4.455935,0.0,0.0,0.0,0.0
3,BMGC_TS000004,5.213352,5.151831,5.766890,5.052784,5.001630,6.215894,0.0,5.329526,5.736236,...,5.175159,0.0,0.0,0.0,0.0,5.969341,0.0,0.0,0.0,0.0
4,BMGC_TS000005,0.043941,-0.073548,0.335595,0.005892,-0.059982,1.853420,0.0,0.031376,-0.059982,...,-0.046550,0.0,0.0,0.0,0.0,2.881209,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412034,BMGC_TS412035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
412035,BMGC_TS412036,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
412036,BMGC_TS412037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
412037,BMGC_TS412038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [398]:
dti_omics_df = pd.concat([dti_methy_df, dti_gene_df, dti_transcript_df, dti_protein_df], axis=0).reset_index(drop=True)
display(dti_omics_df)
dti_df = pd.merge(bmgc_entity_df, dti_omics_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='left').drop(columns=['BioMedGraphica_ID', 'Type'])
dti_df = dti_df.fillna(0.0)
display(dti_df)

,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PM00001,0.83335,0.00,0.88351,1.00000,0.94738,1.0,0.50982,0.56520,0.0,...,0.93319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PM00002,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PM00003,0.04959,0.00,0.00129,0.02137,0.01066,0.0,0.01239,0.00000,0.0,...,0.00958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PM00004,0.05386,0.25,0.00000,0.33075,0.07825,0.0,0.72705,0.00761,0.0,...,0.92687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PM00005,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705929,BMGC_PT121415,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705930,BMGC_PT121416,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705931,BMGC_PT121417,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
705932,BMGC_PT121418,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,BioMedGraphica_Conn_ID,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,BMGC_PM00001,0.83335,0.00,0.88351,1.00000,0.94738,1.0,0.50982,0.56520,0.0,...,0.93319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BMGC_PM00002,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BMGC_PM00003,0.04959,0.00,0.00129,0.02137,0.01066,0.0,0.01239,0.00000,0.0,...,0.00958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BMGC_PM00004,0.05386,0.25,0.00000,0.33075,0.07825,0.0,0.72705,0.00761,0.0,...,0.92687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BMGC_PM00005,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834804,BMGC_DG20914,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834805,BMGC_DG20915,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834806,BMGC_DG20916,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834807,BMGC_DG20917,0.00000,0.00,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [423]:
# Select the features from dti_df
dti_feat = dti_df.drop(columns=['BioMedGraphica_Conn_ID']).values.T
# Save the features to .npy files
np.save('./data/DrugQA/dti_feature.npy', dti_feat)
np.save('./data/DrugScreen/dti_feature.npy', dti_feat)
# Print the shapes of the features
print("Shape of dti_feat:", dti_feat.shape)

# Create a dictionary mapping each depMapID to its corresponding row index (default DataFrame index)
dti_sample_index_dict = dict(zip(remaining_dti_samples_df['depMapID'], remaining_dti_samples_df.index))
print(dti_sample_index_dict)

Shape of dti_feat: (286, 834809)
{'ACH-000002': 0, 'ACH-000006': 1, 'ACH-000008': 2, 'ACH-000020': 3, 'ACH-000038': 4, 'ACH-000046': 5, 'ACH-000047': 6, 'ACH-000050': 7, 'ACH-000053': 8, 'ACH-000056': 9, 'ACH-000058': 10, 'ACH-000059': 11, 'ACH-000061': 12, 'ACH-000062': 13, 'ACH-000065': 14, 'ACH-000072': 15, 'ACH-000073': 16, 'ACH-000074': 17, 'ACH-000078': 18, 'ACH-000081': 19, 'ACH-000082': 20, 'ACH-000091': 21, 'ACH-000099': 22, 'ACH-000101': 23, 'ACH-000102': 24, 'ACH-000104': 25, 'ACH-000105': 26, 'ACH-000106': 27, 'ACH-000107': 28, 'ACH-000112': 29, 'ACH-000116': 30, 'ACH-000117': 31, 'ACH-000120': 32, 'ACH-000121': 33, 'ACH-000124': 34, 'ACH-000129': 35, 'ACH-000136': 36, 'ACH-000137': 37, 'ACH-000144': 38, 'ACH-000145': 39, 'ACH-000151': 40, 'ACH-000157': 41, 'ACH-000162': 42, 'ACH-000171': 43, 'ACH-000174': 44, 'ACH-000181': 45, 'ACH-000186': 46, 'ACH-000187': 47, 'ACH-000189': 48, 'ACH-000190': 49, 'ACH-000193': 50, 'ACH-000196': 51, 'ACH-000197': 52, 'ACH-000198': 53, 'ACH

##### 9.2.3.1 Drug Screen Integration

In [429]:
# For final_training_drug_screen_df, only keep the columns ['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']
final_training_drug_screen_dfc = final_training_drug_screen_df[['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']].copy()
# Map depMapID to dti_sample_index, BMGC_Drug_ID to node_index, BMGC_Disease_ID to node_index
final_training_drug_screen_dfc['dti_sample_index'] = final_training_drug_screen_dfc['depMapID'].map(dti_sample_index_dict)
final_training_drug_screen_dfc['BMGC_Drug_ID'] = final_training_drug_screen_dfc['BMGC_Drug_ID'].map(node_index_dict)
final_training_drug_screen_dfc['BMGC_Disease_ID'] = final_training_drug_screen_dfc['BMGC_Disease_ID'].map(node_index_dict)
final_training_drug_screen_dfc = final_training_drug_screen_dfc.drop(columns=['depMapID'])
# reorder the columns to ['dti_sample_index', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']
final_training_drug_screen_dfc = final_training_drug_screen_dfc[['dti_sample_index', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']]
display(final_training_drug_screen_dfc)

# For final_test_drug_screen_df, only keep the columns ['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']
final_test_drug_screen_dfc = final_test_drug_screen_df[['depMapID', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']].copy()
# Map depMapID to dti_sample_index, BMGC_Drug_ID to node_index, BMGC_Disease_ID to node_index
final_test_drug_screen_dfc['dti_sample_index'] = final_test_drug_screen_dfc['depMapID'].map(dti_sample_index_dict)
final_test_drug_screen_dfc['BMGC_Drug_ID'] = final_test_drug_screen_dfc['BMGC_Drug_ID'].map(node_index_dict)
final_test_drug_screen_dfc['BMGC_Disease_ID'] = final_test_drug_screen_dfc['BMGC_Disease_ID'].map(node_index_dict)
final_test_drug_screen_dfc = final_test_drug_screen_dfc.drop(columns=['depMapID'])
# reorder the columns to ['dti_sample_index', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']
final_test_drug_screen_dfc = final_test_drug_screen_dfc[['dti_sample_index', 'BMGC_Drug_ID', 'BMGC_Disease_ID', 'AUC']]
display(final_test_drug_screen_dfc)

,dti_sample_index,BMGC_Drug_ID,BMGC_Disease_ID,AUC
0,80,818544,794405,0.981350
1,37,818544,792426,0.980213
2,197,818544,794405,0.931183
3,139,818544,812546,0.988258
4,137,818544,794348,0.983326
...,...,...,...,...
42061,174,834578,799496,0.873306
42062,31,834578,798476,0.899328
42063,222,834578,799275,0.812820
42064,248,834578,799496,0.843211


,dti_sample_index,BMGC_Drug_ID,BMGC_Disease_ID,AUC
0,175,818544,794391,0.954785
1,195,818544,794348,0.988450
2,51,818544,799496,0.978046
3,191,818544,794348,0.943754
4,90,818544,796795,0.877464
...,...,...,...,...
9441,96,834115,799978,0.439161
9442,132,834115,797085,0.813459
9443,63,834115,797085,0.785710
9444,51,834578,799496,0.811327


In [430]:
# Convert final_training_drug_screen_dfc and final_test_drug_screen_dfc to numpy arrays and save them
final_training_drug_screen_array = final_training_drug_screen_dfc.values
final_test_drug_screen_array = final_test_drug_screen_dfc.values
# Save the numpy arrays to .npy files
np.save('./data/DrugScreen/final_training_drug_screen.npy', final_training_drug_screen_array)
np.save('./data/DrugScreen/final_test_drug_screen.npy', final_test_drug_screen_array)
print("Shape of final_training_drug_screen_array:", final_training_drug_screen_array.shape)
print("Shape of final_test_drug_screen_array:", final_test_drug_screen_array.shape)

Shape of final_training_drug_screen_array: (42066, 4)
Shape of final_test_drug_screen_array: (9446, 4)


##### 9.2.3.2 DrugQA Integration

In [456]:
# select the columns in the remaining_dti_samples for gene_df
# Only include samples that actually exist in gene_df columns
available_samples = [sample for sample in remaining_dti_samples if sample in gene_df.columns]
dti_gene_df = gene_df[['gene_name'] + sorted(available_samples)].copy()
print(f"Total remaining_dti_samples: {len(remaining_dti_samples)}")
print(f"Available samples in gene_df: {len(available_samples)}")
print(f"Missing samples: {len(remaining_dti_samples) - len(available_samples)}")
display(dti_gene_df)
bmgc_protein_llmnameid_combined_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein_LLM_Name_ID_Combined.csv')
display(bmgc_protein_llmnameid_combined_df)

def extract_dti_gn_info(dti_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_gene_df columns
    if sample_ach_name not in dti_gene_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_genes = dti_gene_df.nlargest(k, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top k genes by their values in descending order
    top_k_genes = top_k_genes.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_gene_hgnc_name_list = top_k_genes['gene_name'].tolist()
    # Merge with the bmgc_gene_df to get the BioMedGraphica_Conn_ID
    bmgc_gene_df = bmgc_gene_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_gene_df = pd.merge(bmgc_gene_df, top_k_genes, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_gene_protein_df = pd.merge(gene_transcript_protein_entity_df, top_k_bmgc_gene_df, left_on='BMGC_GN_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_k_bmgc_gene_protein_info_df = pd.merge(top_k_bmgc_gene_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_gene_protein_bmgc_id_list = top_k_bmgc_gene_protein_info_df['BMGC_PT_ID'].tolist()
    top_k_gene_protein_bmgc_llmnameid_list = top_k_bmgc_gene_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000002'
k=10
top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list = extract_dti_gn_info(dti_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Gene HGNC Names:", top_k_gene_hgnc_name_list)
print(f"Top {k} Gene Protein BMGC IDs:", top_k_gene_protein_bmgc_id_list)
print(f"Top {k} Gene Protein BMGC LLM Name IDs:", top_k_gene_protein_bmgc_llmnameid_list)

Total remaining_dti_samples: 286
Available samples in gene_df: 284
Missing samples: 2


Unnamed: 0,gene_name,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000047,ACH-000050,ACH-000053,...,ACH-001106,ACH-001125,ACH-001126,ACH-001127,ACH-001138,ACH-001151,ACH-001182,ACH-001199,ACH-001203,ACH-001233
0,A1BG,1.039342,1.081979,1.069887,1.021275,1.019950,0.873542,1.954398,0.476005,0.498743,...,0.930320,1.280913,0.816260,0.996102,1.018304,1.050028,1.005951,1.040949,1.409362,0.863441
1,A1BG-AS1,1.039342,1.081979,1.069887,1.021275,1.019950,0.873542,1.954398,0.476005,0.498743,...,0.930320,1.280913,0.816260,0.996102,1.018304,1.050028,1.005951,1.040949,1.409362,0.863441
2,A1CF,1.010572,1.032794,0.857506,0.984673,1.007580,1.472495,1.205183,0.518170,0.515374,...,1.103581,0.760629,0.815531,0.981722,0.694711,0.917085,0.994378,1.004767,0.995919,0.815412
3,A2M,0.999607,0.576030,1.225703,0.988541,1.005250,1.956878,1.189136,0.519018,1.009731,...,1.030032,0.825321,0.981129,0.948925,1.486665,0.913336,1.003539,1.038488,1.479791,1.223223
4,A2M-AS1,0.999607,0.576030,1.225703,0.988541,1.005250,1.956878,1.189136,0.519018,1.009731,...,1.030032,0.825321,0.981129,0.948925,1.486665,0.913336,1.003539,1.038488,1.479791,1.223223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38585,ZYG11B,1.016271,0.928239,1.120232,0.983290,1.006196,0.893845,1.190278,0.987717,0.979038,...,0.973301,1.231011,1.368282,0.986425,1.055755,0.906142,0.999373,0.985555,0.998989,1.229548
38586,ZYX,1.015564,0.991367,2.116413,0.970735,1.011754,1.870285,1.173788,1.019697,1.021887,...,1.028176,1.574313,1.112932,1.514654,1.311404,0.893903,0.998849,0.966974,0.995876,1.185762
38587,ZYXP1,0.971470,1.032808,0.961416,0.959549,1.009547,0.986482,1.623606,0.540793,1.495777,...,1.020167,1.076599,1.237886,1.057301,1.114824,1.960846,1.003790,1.021203,0.980589,1.099648
38588,ZZEF1,0.560010,0.597288,0.936882,1.013471,0.994021,1.299603,0.716645,0.978837,0.506384,...,0.982988,0.789985,1.117091,0.830545,0.760735,0.813964,1.000523,0.981905,1.010208,0.962021


,BioMedGraphica_Conn_ID,BioMedGraphica_ID,Names_and_IDs
0,BMGC_PT000001,BMG_PT000006,UniProt ID:A0A023T787 | RefSeq ID:NP_005096.1
1,BMGC_PT000002,BMG_PT000012,RefSeq ID:NP_006570.1 | UniProt ID:A0A024QYX0
2,BMGC_PT000003,BMG_PT000014,UniProt ID:A0A024QYX5 | RefSeq ID:XP_024308127...
3,BMGC_PT000004,BMG_PT000022,UniProt ID:A0A024QZ37 | RefSeq ID:NP_859054.1 ...
4,BMGC_PT000005,BMG_PT000024,UniProt ID:A0A024QZ45 | RefSeq ID:XP_047292848...
...,...,...,...
121414,BMGC_PT121415,BMG_PT173974,ENSP00000520928.1 | ENSP00000520928 | NCBI Gen...
121415,BMGC_PT121416,BMG_PT173975,RefSeq ID:NP_001366093 | NCBI Gene ID:1438.0 |...
121416,BMGC_PT121417,BMG_PT173976,NCBI Gene ID:1438.0 | CSF2RA | RefSeq ID:NP_00...
121417,BMGC_PT121418,BMG_PT173977,GTPBP6 | ENSP00000520931.1 | NCBI Gene ID:8225...


Top 10 Gene HGNC Names: ['CASC11', 'CASC19', 'CASC8', 'CCAT2', 'KNOP1P5', 'LINC00861', 'LINC02964', 'LRATD2', 'MIR1204', 'MYC']
Top 10 Gene Protein BMGC IDs: ['BMGC_PT037862', 'BMGC_PT013286', 'BMGC_PT000209', 'BMGC_PT000096', 'BMGC_PT102081', 'BMGC_PT102076', 'BMGC_PT106022', 'BMGC_PT048597', 'BMGC_PT003978', 'BMGC_PT009616', 'BMGC_PT019176', 'BMGC_PT013287', 'BMGC_PT070896']
Top 10 Gene Protein BMGC LLM Name IDs: ['Protein LRATD2 (Breast cancer membrane protein 101) (LRAT domain-containing 2) (Protein FAM84B) (Protein NSE2) or LRATD2 or NCBI Gene ID:157638 or NCBI Gene ID:157638.0 or ENSP00000302578.3;ENSP00000498944.1 or UniProt ID:Q96KN1 or RefSeq ID:NP_777571 or RefSeq ID:NP_777571.1 or RefSeq ID:XP_016868596.1 or ENSP00000302578;ENSP00000498944', 'ENSP00000367207.3;ENSP00000430235.2;ENSP00000478887.2;ENSP00000499105.1;ENSP00000516742.1;ENSP00000516743.1;ENSP00000516744.1;ENSP00000516745.1 or MYC or Myc proto-oncogene protein (Class E basic helix-loop-helix protein 39) (bHLHe39) (

In [457]:
# select the columns in the remaining_dti_samples for raw_transcript_df
# Only include samples that actually exist in raw_transcript_df columns
available_transcript_samples = [sample for sample in remaining_dti_samples if sample in raw_transcript_df.columns]
dti_transcript_df = raw_transcript_df[['gene_name'] + sorted(available_transcript_samples)].copy()
print(f"Total remaining_dti_samples: {len(remaining_dti_samples)}")
print(f"Available samples in raw_transcript_df: {len(available_transcript_samples)}")
print(f"Missing samples: {len(remaining_dti_samples) - len(available_transcript_samples)}")
display(dti_transcript_df)

def extract_dti_ts_info(dti_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_transcript_df columns
    if sample_ach_name not in dti_transcript_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_transcripts = dti_transcript_df.nlargest(k, sample_ach_name)[['gene_name', sample_ach_name]]
    # Sort the top k transcripts by their values in descending order
    top_k_transcripts = top_k_transcripts.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_transcript_hgnc_name_list = top_k_transcripts['gene_name'].tolist()
    # Merge with the bmgc_transcript_df to get the BioMedGraphica_Conn_ID
    bmgc_transcript_df = bmgc_transcript_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_transcript_df = pd.merge(bmgc_transcript_df, top_k_transcripts, left_on='HGNC_Symbol', right_on='gene_name', how='inner')
    # Get the corresponding proteins
    top_k_bmgc_transcript_protein_df = pd.merge(transcript_protein_entity_df, top_k_bmgc_transcript_df, left_on='BMGC_TS_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'gene_name'])
    top_k_bmgc_transcript_protein_info_df = pd.merge(top_k_bmgc_transcript_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BMGC_PT_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_Conn_ID', 'BioMedGraphica_ID', sample_ach_name])
    top_k_transcript_protein_bmgc_id_list = top_k_bmgc_transcript_protein_info_df['BMGC_PT_ID'].tolist()
    top_k_transcript_protein_bmgc_llmnameid_list = top_k_bmgc_transcript_protein_info_df['Names_and_IDs'].replace(r' \| ', ' or ', regex=True).tolist()
    return top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000002'
k=10
top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list = extract_dti_ts_info(dti_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Transcript HGNC Names:", top_k_transcript_hgnc_name_list)
print(f"Top {k} Transcript Protein BMGC IDs:", top_k_transcript_protein_bmgc_id_list)
print(f"Top {k} Transcript Protein BMGC LLM Name IDs:", top_k_transcript_protein_bmgc_llmnameid_list)

Total remaining_dti_samples: 286
Available samples in raw_transcript_df: 268
Missing samples: 18


Unnamed: 0,gene_name,ACH-000002,ACH-000006,ACH-000008,ACH-000020,ACH-000038,ACH-000046,ACH-000050,ACH-000053,ACH-000056,...,ACH-000983,ACH-000987,ACH-000989,ACH-000995,ACH-000998,ACH-000999,ACH-001016,ACH-001064,ACH-001106,ACH-001151
0,TFE3,2.459987,4.565798,5.006504,3.953176,4.085061,4.537537,3.445636,4.692890,3.348080,...,4.531120,3.921175,3.918550,4.275754,3.876828,3.926410,6.058822,4.963694,3.780212,4.562473
1,MMADHC,4.976372,7.099962,7.215527,5.775586,6.602355,6.713138,7.550829,6.560098,6.256188,...,6.511325,6.596016,6.834165,6.209605,6.542109,5.822229,7.804443,6.842440,6.222557,7.275786
2,PABPC1L,2.698399,4.968360,5.269459,6.012143,4.754882,3.956434,3.737370,6.679576,4.340483,...,4.726988,4.532650,5.912492,4.707263,2.743210,4.269397,4.833559,6.823880,3.655975,5.336246
3,RELL2,1.632580,3.826824,3.005733,3.198194,3.433501,3.470365,4.186973,2.589190,4.706444,...,2.115141,2.948216,3.306925,5.048803,2.485572,3.184126,2.765293,2.385043,2.336957,3.394496
4,GPT,-0.098142,1.047510,0.548131,1.006289,0.712799,0.160307,0.568305,0.658276,1.284211,...,0.973237,0.357728,1.352736,0.968441,0.463925,0.857868,0.480651,0.416134,0.022363,1.274494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19133,NAA60,4.218123,5.351137,3.682937,4.602184,4.857301,4.895785,3.901231,6.073190,4.149113,...,4.931943,4.624226,5.047877,5.095091,5.239022,5.414767,5.465068,5.064381,5.255766,4.935989
19134,MYH4,-0.001724,-0.001724,0.010412,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,-0.001724,...,-0.001724,-0.001724,-0.001724,-0.001724,0.168735,-0.001724,0.000741,0.000741,-0.001724,0.000741
19135,MTAP,3.500397,5.335251,0.024972,1.292930,3.635648,0.296753,4.336532,0.011324,0.065164,...,4.489114,5.192278,4.174660,-0.058907,4.539859,4.890952,4.126143,5.831513,4.535595,0.102115
19136,C14orf93,2.310291,3.067836,2.388417,3.621152,2.834791,2.425974,1.876662,2.999576,3.387713,...,2.852722,2.673945,3.561302,3.231129,3.128145,3.101646,2.039440,3.055175,2.985173,2.577928


Top 10 Transcript HGNC Names: ['MT-ATP8', 'MT-CO2', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-CO1', 'MT-ND4', 'MT-ATP6', 'MT-ND2', 'RPS18']
Top 10 Transcript Protein BMGC IDs: ['BMGC_PT017217', 'BMGC_PT013491', 'BMGC_PT013492', 'BMGC_PT013493', 'BMGC_PT013490', 'BMGC_PT013229', 'BMGC_PT013230', 'BMGC_PT013496', 'BMGC_PT013263', 'BMGC_PT013231', 'BMGC_PT017217', 'BMGC_PT017217', 'BMGC_PT055005', 'BMGC_PT017217', 'BMGC_PT017217', 'BMGC_PT074382', 'BMGC_PT072563', 'BMGC_PT075799', 'BMGC_PT072395', 'BMGC_PT074349', 'BMGC_PT072934', 'BMGC_PT097348']
Top 10 Transcript Protein BMGC LLM Name IDs: ['NCBI Gene ID:6222 or ENSP00000211372.5;ENSP00000393241.2;ENSP00000403175.2;ENSP00000412583.2;ENSP00000416110.2 or UniProt ID:P62269 or NCBI Gene ID:6222.0 or Small ribosomal subunit protein uS13 (40S ribosomal protein S18) (Ke-3) (Ke3) or ENSP00000211372;ENSP00000393241;ENSP00000403175;ENSP00000412583;ENSP00000416110 or RPS18 or RefSeq ID:NP_072045 or RefSeq ID:NP_072045.1', 'NCBI Gene ID:4537.0 or NADH-ub

In [460]:
# Load protein mapping data with BioMedGraphica IDs, Uniprot IDs, and HGNC symbols
bmg_protein_all_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')
bmg_protein_all_df = bmg_protein_all_df[['BioMedGraphica_Conn_ID', 'Uniprot_ID', 'HGNC_Symbol']].copy()
display(bmg_protein_all_df)

# Map sample IDs in raw protein data to standardized format
raw_protein_map_df = raw_protein_df.rename(columns=protein_map_dict)

# Join protein expression data with HGNC symbol annotations
symbol_protein_map_df = pd.merge(raw_protein_map_df, bmg_protein_all_df, left_on='Uniprot_Acc', right_on='Uniprot_ID', how='inner')

# Reorganize columns: protein identifiers first, then sample expression values
identifier_cols = ['Uniprot_ID', 'Uniprot_Acc', 'HGNC_Symbol']
expression_cols = sorted(set(symbol_protein_map_df.columns) - set(identifier_cols))
symbol_protein_map_df = symbol_protein_map_df[identifier_cols + expression_cols]

# Only include samples that actually exist in symbol_protein_map_df columns
available_protein_samples = [sample for sample in remaining_dti_samples if sample in symbol_protein_map_df.columns]
print(f"Total remaining_dti_samples: {len(remaining_dti_samples)}")
print(f"Available samples in symbol_protein_map_df: {len(available_protein_samples)}")
print(f"Missing samples: {len(remaining_dti_samples) - len(available_protein_samples)}")

# Extract protein data for DTI samples with HGNC symbols
dti_protein_df = symbol_protein_map_df[['HGNC_Symbol'] + sorted(available_protein_samples)].copy()

# Handle multiple HGNC symbols per protein (semicolon-separated)
dti_protein_df = dti_protein_df.assign(
    HGNC_Symbol=dti_protein_df['HGNC_Symbol'].str.split(';')
).explode('HGNC_Symbol')

# Clean up HGNC symbols: remove whitespace and filter out empty entries
dti_protein_df['HGNC_Symbol'] = dti_protein_df['HGNC_Symbol'].str.strip()
dti_protein_df = dti_protein_df[
    dti_protein_df['HGNC_Symbol'].notna() & 
    (dti_protein_df['HGNC_Symbol'] != '')
].reset_index(drop=True)

display(dti_protein_df)

def extract_dti_pt_info(dti_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k=10):
    # Check if sample_ach_name exists in dti_protein_df columns
    if sample_ach_name not in dti_protein_df.columns:
        return "non-existed", "non-existed", "non-existed"
    # Extract the top k highest values for the given sample name
    top_k_proteins = dti_protein_df.nlargest(k, sample_ach_name)[['HGNC_Symbol', sample_ach_name]]
    # Sort the top k proteins by their values in descending order
    top_k_proteins = top_k_proteins.sort_values(by=sample_ach_name, ascending=False).reset_index(drop=True)
    top_k_protein_hgnc_name_list = top_k_proteins['HGNC_Symbol'].tolist()
    # Merge with the bmgc_protein_df to get the BioMedGraphica_Conn_ID
    bmgc_protein_df = bmgc_protein_df[['BioMedGraphica_Conn_ID', 'HGNC_Symbol']].copy()
    top_k_bmgc_protein_df = pd.merge(bmgc_protein_df, top_k_proteins, left_on='HGNC_Symbol', right_on='HGNC_Symbol', how='inner')
    # Get the corresponding protein information
    top_k_bmgc_protein_info_df = pd.merge(top_k_bmgc_protein_df, bmgc_protein_llmnameid_combined_df, left_on='BioMedGraphica_Conn_ID', right_on='BioMedGraphica_Conn_ID', how='inner').drop(columns=['BioMedGraphica_ID', sample_ach_name])
    top_k_protein_bmgc_id_list = top_k_bmgc_protein_info_df['BioMedGraphica_Conn_ID'].tolist()
    # Replace both "|" and ";" with " or "
    top_k_protein_bmgc_llmnameid_list = top_k_bmgc_protein_info_df['Names_and_IDs'].replace([r' \| ', r';'], ' or ', regex=True).tolist()
    return top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list

# Example usage
sample_ach_name = 'ACH-000008'
k = 10
top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list = extract_dti_pt_info(dti_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
print(f"Top {k} Protein HGNC Names:", top_k_protein_hgnc_name_list)
print(f"Top {k} Protein BMGC IDs:", top_k_protein_bmgc_id_list)
print(f"Top {k} Protein BMGC LLM Name IDs:", top_k_protein_bmgc_llmnameid_list)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_33876\1700773702.py:2: DtypeWarning: Columns (2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  bmg_protein_all_df = pd.read_csv('./data/BioMedGraphica-Conn/Entity/Protein/BioMedGraphica_Conn_Protein.csv')


,BioMedGraphica_Conn_ID,Uniprot_ID,HGNC_Symbol
0,BMGC_PT000001,A0A023T787,NaN
1,BMGC_PT000002,A0A024QYX0,NaN
2,BMGC_PT000003,A0A024QYX5,NaN
3,BMGC_PT000004,A0A024QZ37,NaN
4,BMGC_PT000005,A0A024QZ45,NaN
...,...,...,...
121414,BMGC_PT121415,NaN,CSF2RA
121415,BMGC_PT121416,NaN,CSF2RA
121416,BMGC_PT121417,NaN,CSF2RA
121417,BMGC_PT121418,NaN,GTPBP6


Total remaining_dti_samples: 286
Available samples in symbol_protein_map_df: 37
Missing samples: 249


,HGNC_Symbol,ACH-000008,ACH-000062,ACH-000091,ACH-000117,ACH-000309,ACH-000335,ACH-000347,ACH-000349,ACH-000351,...,ACH-000670,ACH-000678,ACH-000727,ACH-000730,ACH-000750,ACH-000752,ACH-000755,ACH-000852,ACH-000857,ACH-000987
0,SLC12A2,-0.168840,0.729167,-0.907412,-1.532994,-1.218412,1.472404,-1.492391,1.035254,1.350658,...,-0.398706,0.285981,-1.903259,1.262304,-1.509770,-1.119232,-0.074979,-2.049783,-0.402035,-1.059660
1,HOXD13,0.000000,0.000000,0.000000,-0.525421,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.075547,-0.648070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,KDM1A,0.876906,0.410168,-0.401211,0.561969,-0.059550,-0.279095,-0.421580,-0.058377,-0.385244,...,-0.039246,-0.202710,-0.498443,0.513282,-0.503426,-0.758587,0.272818,0.060652,-0.234683,0.382719
3,SOX1,0.000000,0.000000,-4.317090,-1.344490,0.000000,0.000000,-1.791036,5.416105,0.000000,...,-3.238624,0.000000,0.000000,0.000000,-0.749516,-0.988544,0.614960,0.000000,0.000000,0.000000
4,SOX2,0.000000,1.882953,-1.004530,-1.993823,-1.452967,0.000000,-1.164757,4.897920,-0.974087,...,-1.642244,2.046123,-0.308938,3.730812,-0.492513,-1.067185,0.906425,-1.490469,-1.891108,-1.496319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10623,CCL26,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10624,ALOX5AP,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.118748,...,0.000000,-6.872745,3.007516,-1.455220,0.000000,0.000000,1.263190,1.389608,0.000000,-1.181452
10625,CYSTM1,1.478774,0.298349,0.000000,1.335349,-3.053117,1.929118,1.388368,-0.450275,0.000000,...,-0.740274,-2.308188,-0.997777,0.552372,-0.555743,0.557652,0.000000,0.000000,0.000000,0.982198
10626,MGST2,2.248389,0.000000,-0.092565,1.019450,0.000000,-2.845178,0.382543,-0.548249,-0.168199,...,0.272329,0.000000,0.000000,0.000000,0.000000,0.000000,1.166740,-0.481567,0.000000,0.000000


Top 10 Protein HGNC Names: ['CD177', 'PEAR1', 'FOXC2', 'CRYAB', 'PSG6', 'COL11A1', 'CNN1', 'SUSD5', 'GPC3', 'CCDC8']
Top 10 Protein BMGC IDs: ['BMGC_PT011490', 'BMGC_PT013430', 'BMGC_PT014441', 'BMGC_PT016585', 'BMGC_PT016638', 'BMGC_PT017666', 'BMGC_PT026179', 'BMGC_PT033081', 'BMGC_PT039265', 'BMGC_PT041113', 'BMGC_PT052721', 'BMGC_PT052813', 'BMGC_PT054064', 'BMGC_PT057023', 'BMGC_PT060207', 'BMGC_PT061016', 'BMGC_PT061565', 'BMGC_PT072419', 'BMGC_PT072688', 'BMGC_PT074012', 'BMGC_PT074354', 'BMGC_PT074526', 'BMGC_PT074762', 'BMGC_PT075119', 'BMGC_PT077222', 'BMGC_PT089610', 'BMGC_PT090148', 'BMGC_PT090387', 'BMGC_PT090530', 'BMGC_PT090551', 'BMGC_PT090573', 'BMGC_PT090842', 'BMGC_PT091132', 'BMGC_PT091507', 'BMGC_PT092525', 'BMGC_PT094911', 'BMGC_PT096554', 'BMGC_PT099706', 'BMGC_PT102483', 'BMGC_PT102715', 'BMGC_PT102986', 'BMGC_PT103193', 'BMGC_PT105753', 'BMGC_PT113331', 'BMGC_PT114711', 'BMGC_PT114837', 'BMGC_PT116582']
Top 10 Protein BMGC LLM Name IDs: ['NCBI Gene ID:26032.0 o

In [461]:
def extract_drug_protein(selected_sample_drug_bmgc_id, edge_index, 
                    node_index_df, nodeid_index_dict, index_nodeid_dict):
    # Extract the index based on the selected drug BMGC ID
    sample_drug_bmgc_id_index = nodeid_index_dict[selected_sample_drug_bmgc_id]
    # Find incoming edges (source nodes that point to the drug)
    incoming_mask = edge_index[1, :] == sample_drug_bmgc_id_index
    incoming_source_nodes = edge_index[0, incoming_mask]
    # Find outgoing edges (target nodes that the drug points to)
    outgoing_mask = edge_index[0, :] == sample_drug_bmgc_id_index
    outgoing_target_nodes = edge_index[1, outgoing_mask]
    # Combine all neighbor nodes (both incoming and outgoing)
    drug_related_nodes = np.concatenate([incoming_source_nodes, outgoing_target_nodes])
    unique_drug_related_nodes = np.unique(drug_related_nodes)
    # Get protein node index
    protein_node_index_df = node_index_df[node_index_df['Type'] == 'Protein']
    protein_node_index_list = protein_node_index_df['Index'].tolist()
    # Filter to get only protein nodes directly connected to the drug
    drug_protein_index = sorted(
        list(set(unique_drug_related_nodes) & set(protein_node_index_list))
    )
    # Map protein index to BMGC id
    drug_protein_bmgc_id = [index_nodeid_dict[i] for i in drug_protein_index]
    return drug_protein_index, drug_protein_bmgc_id

# Example usage
selected_sample_drug_bmgc_id = 'BMGC_DG00001'

drug_protein_index, drug_protein_bmgc_id = extract_drug_protein(
    selected_sample_drug_bmgc_id=selected_sample_drug_bmgc_id,
    edge_index=edge_index,
    node_index_df=nodes_index_data,
    nodeid_index_dict=node_index_dict,
    index_nodeid_dict=index_node_dict
)

print("Drug Protein Index:", drug_protein_index)
print("Drug Protein BMGC ID:", drug_protein_bmgc_id)

Drug Protein Index: [603176]
Drug Protein BMGC ID: ['BMGC_PT018662']


##### Formulate the drug related protein json

In [436]:
import json
from tqdm import tqdm

dti_drug_ids = sorted(list(set(dti_drug_overlap_df['BMGC_Drug_ID'])))

drug_kg_protein_bmgc_dict = {}
print_count = 0
max_prints = 5

for i, drug_bmgc_id in enumerate(tqdm(dti_drug_ids, desc="Processing drug-protein relationships")):
    drug_protein_index, drug_protein_bmgc_id = extract_drug_protein(
        selected_sample_drug_bmgc_id=drug_bmgc_id,
        edge_index=edge_index,
        node_index_df=nodes_index_data,
        nodeid_index_dict=node_index_dict,
        index_nodeid_dict=index_node_dict
    )
    hgnc_dict, drug_protein_hgnc_list = bmgc_pt_id_to_hgnc(drug_protein_bmgc_id, bmgc_protein_df)
    
    # Only print first 5 entries
    if print_count < max_prints:
        print(f"Drug BMGC ID: {drug_bmgc_id}, Protein Index: {drug_protein_index}, Protein BMGC ID: {drug_protein_bmgc_id}")
        print_count += 1
    elif print_count == max_prints:
        print("... (remaining entries processed silently)")
        print_count += 1
    
    # Convert NumPy integers to Python integers for JSON serialization
    drug_kg_protein_bmgc_dict[drug_bmgc_id] = {
        'drug_protein_index': [int(x) for x in drug_protein_index],  # Convert numpy ints to Python ints
        'drug_protein_bmgc_id': drug_protein_bmgc_id
    }
    
    # Save every 10 iterations
    if (i + 1) % 10 == 0:
        output_path = "./data/DrugQA/drug_kg_protein_relationships.json"
        with open(output_path, "w") as f:
            json.dump(drug_kg_protein_bmgc_dict, f, indent=2)
        print(f"💾 Auto-saved after processing {i + 1}/{len(dti_drug_ids)} drugs")

# Final save after loop completion
output_path = "./data/DrugQA/drug_kg_protein_relationships.json"
with open(output_path, "w") as f:
    json.dump(drug_kg_protein_bmgc_dict, f, indent=2)
    
print(f"✅ Final save completed - processed {len(drug_kg_protein_bmgc_dict)} drugs and saved to: {output_path}")

Processing drug-protein relationships:   1%|          | 1/172 [00:00<00:36,  4.71it/s]

Drug BMGC ID: BMGC_DG00030, Protein Index: [595431], Protein BMGC ID: ['BMGC_PT010917']


Processing drug-protein relationships:   1%|          | 2/172 [00:02<04:47,  1.69s/it]

Drug BMGC ID: BMGC_DG00141, Protein Index: [594482, 594534, 594581, 594618, 594741, 594837, 594937, 594951, 595044, 595057, 595075, 595084, 595153, 595177, 595206, 595232, 595271, 595371, 595372, 595431, 595586, 595593, 595614, 595628, 596008, 596038, 596224, 596445, 596593, 596641, 596723, 596770, 596793, 596798, 596969, 597078, 597080, 597149, 597367, 597541, 597584, 597593, 597696, 597700, 597755, 597756, 598028, 598158, 598194, 598195, 598196, 598270, 598271, 598309, 598310, 598311, 598321, 598372, 598379, 598406, 598459, 598473, 598501, 598505, 598851, 598906, 598936, 598996, 599105, 599139, 599185, 599228, 599255, 599326, 599349, 599430, 599438, 599509, 599533, 599571, 599576, 599601, 599612, 599623, 599633, 599680, 599681, 599752, 599762, 599849, 599857, 599858, 599918, 599951, 599952, 599953, 599954, 599964, 599980, 600018, 600144, 600217, 600263, 600362, 600395, 600396, 600408, 600423, 600425, 600444, 600559, 600594, 600595, 600596, 600607, 600608, 600609, 600610, 600611, 6006

Processing drug-protein relationships:   2%|▏         | 3/172 [00:05<05:53,  2.09s/it]

...

💾 Auto-saved after processing 160/172 drugs


Processing drug-protein relationships:  99%|█████████▉| 171/172 [01:54<00:00,  5.41it/s]

💾 Auto-saved after processing 170/172 drugs


Processing drug-protein relationships: 100%|██████████| 172/172 [01:54<00:00,  1.50it/s]

✅ Final save completed - processed 172 drugs and saved to: ./data/DrugQA/drug_kg_protein_relationships.json


##### Formulate the sample related json

In [472]:
def drug_qa_sample_info(sample_ach_name, selected_sample_disease_bmgc_id, k=10):
    # LLM Info
    print(f"Sample ACH Name: {sample_ach_name}")
    print(f"Extracting top {k} gene information for {sample_ach_name}...")
    top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list = extract_dti_gn_info(dti_gene_df, bmgc_gene_df, gene_transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    print(f"Extracting top {k} transcript information for {sample_ach_name}...")
    top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list = extract_dti_ts_info(dti_transcript_df, bmgc_transcript_df, transcript_protein_entity_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    print(f"Extracting top {k} protein information for {sample_ach_name}...")
    top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list = extract_dti_pt_info(dti_protein_df, bmgc_protein_df, bmgc_protein_llmnameid_combined_df, sample_ach_name, k)
    # KG Info
    edge_index = np.load('./data/DrugQA/edge_index.npy')
    print(f"Extracting disease-related proteins index and bmgc id for {selected_sample_disease_bmgc_id} ({sample_ach_name}) ...")
    disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id = extract_kg_related_proteins(selected_sample_disease_bmgc_id, edge_index, nodes_index_data, node_index_dict, index_node_dict)
    print(f"Knowledge Graph Info: Found {len(disease_protein_index)} disease-related proteins directly connected to {selected_sample_disease_bmgc_id} and {len(ppi_nodes_index)} proteins in their PPI network")
    print(f"Mapping disease-related proteins to HGNC symbols...")
    disease_protein_hgnc_dict, disease_protein_hgnc_list = bmgc_pt_id_to_hgnc(disease_protein_bmgc_id, bmgc_protein_df)
    print(f"Mapping PPI-related proteins to HGNC symbols...")
    ppi_hgnc_dict, ppi_hgnc_list = bmgc_pt_id_to_hgnc(ppi_nodes_bmgc_id, bmgc_protein_df)
    # LLM Used KG Info
    print(f"Extracting protein relationships from BMGC...")
    # Convert the any non-existed string in top_k_gene_hgnc_name_list + top_k_transcript_hgnc_name_list + top_k_protein_hgnc_name_list + disease_protein_hgnc_list into empty list []
    if top_k_gene_hgnc_name_list == "non-existed": top_k_gene_hgnc_name_list = []
    if top_k_transcript_hgnc_name_list == "non-existed": top_k_transcript_hgnc_name_list = []
    if top_k_protein_hgnc_name_list == "non-existed": top_k_protein_hgnc_name_list = []
    if disease_protein_hgnc_list == "non-existed": disease_protein_hgnc_list = []
    # Combine all the HGNC symbols into a single list for relationship extraction
    omics_disease_protein_hgnc_list = list(set(top_k_gene_hgnc_name_list + top_k_transcript_hgnc_name_list + top_k_protein_hgnc_name_list + disease_protein_hgnc_list))
    relationships_df, relationship_texts = find_protein_relationships(omics_disease_protein_hgnc_list, bmgc_protein_df, bmgc_relation_df)
    print(f"Knowledge Graph Info: Found {len(omics_disease_protein_hgnc_list)} unique proteins and {len(relationship_texts)} relationships between them")
    return top_k_gene_hgnc_name_list, top_k_gene_protein_bmgc_id_list, top_k_gene_protein_bmgc_llmnameid_list, \
              top_k_transcript_hgnc_name_list, top_k_transcript_protein_bmgc_id_list, top_k_transcript_protein_bmgc_llmnameid_list, \
                top_k_protein_hgnc_name_list, top_k_protein_bmgc_id_list, top_k_protein_bmgc_llmnameid_list, \
                disease_protein_index, disease_protein_bmgc_id, ppi_nodes_index, ppi_nodes_bmgc_id, \
                    disease_protein_hgnc_dict, disease_protein_hgnc_list, ppi_hgnc_dict, ppi_hgnc_list, relationship_texts

# Example usage
sample_ach_name = 'ACH-000002'
selected_sample_disease_bmgc_id = 'BMGC_DS07934'
k = 10
return_tuples = drug_qa_sample_info(sample_ach_name, selected_sample_disease_bmgc_id, k)

Sample ACH Name: ACH-000002
Extracting top 10 gene information for ACH-000002...
Extracting top 10 transcript information for ACH-000002...
Extracting top 10 protein information for ACH-000002...
Extracting disease-related proteins index and bmgc id for BMGC_DS07934 (ACH-000002) ...
Knowledge Graph Info: Found 5 disease-related proteins directly connected to BMGC_DS07934 and 4915 proteins in their PPI network
Mapping disease-related proteins to HGNC symbols...
Mapping PPI-related proteins to HGNC symbols...
Extracting protein relationships from BMGC...
Knowledge Graph Info: Found 22 unique proteins and 73 relationships between them


In [470]:
import json
import os
from tqdm import tqdm

# Parameters
k = 10
save_every_n = 10

# Output folder and filename
output_dir = "./data/DrugQA"
os.makedirs(output_dir, exist_ok=True)
output_filename = f"drug_qa_k{k}_bm{top_bm}.json"
output_path = os.path.join(output_dir, output_filename)

# Load existing JSON file if it exists
if os.path.exists(output_path):
    with open(output_path, "r") as f:
        multi_sample_qa_json = json.load(f)
    print(f"Loaded existing JSON with {len(multi_sample_qa_json)} processed samples")
else:
    multi_sample_qa_json = {}
    print("No existing JSON found, starting fresh")

# Load sample info data
dti_sample_info_index = pd.read_csv('./data/process_data/dti_combined_samples.csv')
dti_sample_info_index = pd.merge(dti_sample_info_index, remaining_dti_samples_df['depMapID'], how='inner', on='depMapID').reset_index(drop=True)
dti_sample_info_index['BMGC_Disease_name'] = dti_sample_info_index['BMGC_Disease_name'].replace(r' \| ', ' or ', regex=True)
# Insert a new column "Index" in the first position
dti_sample_info_index.insert(0, 'Index', range(1, len(dti_sample_info_index) + 1))
display(dti_sample_info_index)

Loaded existing JSON with 0 processed samples


,Index,depMapID,Name,BMGC_Disease_ID,BMGC_Disease_name,Pathology,Histology,type,PATHOLOGIST_ANNOTATION,tcga_code,...,depmap_match_2_disease,depmap_match_2_disease_name,ncit_match_1_disease,ncit_match_1_disease_name,ncit_match_2_disease,ncit_match_2_disease_name,ordo_match_1_disease,ordo_match_1_disease_name,ordo_match_2_disease,ordo_match_2_disease_name
0,1,ACH-000002,HL-60,BMGC_DS01407,acute myeloid leukemia,primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,ACH-000006,MONO-MAC-6,BMGC_DS20800,"leukemia, myeloid, acute",primary,haematopoietic_neoplasm,AML,Leukemia:AML,LAML,...,BMGC_DS20480,"Leukemia, Myeloid",BMGC_DS05322,T-cell adult acute lymphocytic leukemia,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...,BMGC_DS09029,acute lymphoblastic leukemia | precursor lymph...,BMGC_DS07386,adult acute lymphoblastic leukemia | adult acu...
2,3,ACH-000008,A101D,BMGC_DS01559,melanoma,primary,malignant_melanoma,melanoma,Skin:Melanoma,SKCM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,ACH-000020,MHH-CALL-2,BMGC_DS06157,precursor b-cell acute lymphoblastic leukemia,primary,lymphoid_neoplasm,B-cell_ALL,Leukemia:ALL,ALL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,ACH-000038,EHEB,BMGC_DS01395,b-cell chronic lymphocytic leukemia or chronic...,primary,lymphoid_neoplasm,lymphoma_other,Leukemia:CLL,CLL,...,BMGC_DS20800,"Leukemia, Myeloid, Acute",BMGC_DS20323,Fever,BMGC_DS22241,NaN,BMGC_DS20323,Fever,BMGC_DS22241,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,282,ACH-001151,OVCAR-5,BMGC_DS07934,ovarian adenocarcinoma or ovary adenocarcinoma,primary,carcinoma,ovary,Ovary:Carcinoma,OV,...,BMGC_DS08982,ovarian transitional cell carcinoma | ovary tr...,BMGC_DS08979,ovarian serous surface papillary adenocarcinom...,BMGC_DS09417,peritoneal serous adenocarcinoma | primary per...,BMGC_DS20323,Fever,BMGC_DS22241,NaN
282,283,ACH-001182,RPMI 6666,BMGC_DS20405,hodgkin disease or hodgkins lymphoma,primary,lymphoblastoid,lymphoma_Hodgkin,NaN,NaN,...,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20323,Fever,BMGC_DS22241,NaN,BMGC_DS20323,Fever,BMGC_DS22241,NaN
283,284,ACH-001199,SNU-C2B,BMGC_DS08517,cecum adenocarcinoma,primary,carcinoma,colorectal,NaN,COAD/READ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284,285,ACH-001203,SUP-HD1,BMGC_DS20405,hodgkin disease or hodgkins lymphoma,primary,lymphoid_neoplasm,lymphoma_Hodgkin,NaN,NaN,...,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20405,Hodgkin Disease | Hodgkins lymphoma,BMGC_DS20499,"Lymphoma, Non-Hodgkin",BMGC_DS20405,Hodgkin Disease | Hodgkins lymphoma,BMGC_DS20499,"Lymphoma, Non-Hodgkin"


In [473]:
# Get the list of samples that have already been processed
processed_samples = set(multi_sample_qa_json.keys())
print(f"Found {len(processed_samples)} already processed samples")

count = len(processed_samples)
total_to_process = len(dti_sample_info_index)
remaining = total_to_process - count
print(f"Total samples to process: {total_to_process}, already processed: {count}, remaining: {remaining}")

# Iterate through the sample info dataframe, skipping already processed samples
for idx, row_tuple in tqdm(enumerate(dti_sample_info_index.iterrows()), total=len(dti_sample_info_index)):
    _, row = row_tuple  # Unpack the tuple - index and row data

    sample_ach_name = row["depMapID"]
    
    # Skip if already processed
    if sample_ach_name in processed_samples:
        continue
        
    count += 1
    target_sample_index = row["Index"]
    cell_line_name = row["Name"]
    disease = row["BMGC_Disease_name"]
    disease_bmgc_id = row["BMGC_Disease_ID"]

    print(f"Processing sample {count}/{total_to_process}: {sample_ach_name} ({cell_line_name})")
    print(f"Sample Index: {target_sample_index}")
    print(f"Sample Disease: {disease}")
    print(f"Sample Disease BMGC ID: {disease_bmgc_id}")

    try:
        (top_k_gene_hgnc, top_k_gene_bmgc, top_k_gene_llm,
        top_k_ts_hgnc, top_k_ts_bmgc, top_k_ts_llm,
        top_k_pt_hgnc, top_k_pt_bmgc, top_k_pt_llm,
        dis_pt_idx, dis_pt_bmgc, ppi_idx, ppi_bmgc,
        dis_pt_hgnc_dict, dis_pt_hgnc, 
        ppi_hgnc_dict, ppi_hgnc, relationship_texts) = drug_qa_sample_info(sample_ach_name, disease_bmgc_id, k)

        multi_sample_qa_json[sample_ach_name] = {
            "cell_line_name": cell_line_name,
            "sample_index": target_sample_index,
            "disease": disease,
            "disease_bmgc_id": disease_bmgc_id,
            "input": {
                "top_k_gene": {
                    "hgnc_symbols": top_k_gene_hgnc,
                    "protein_bmgc_ids": top_k_gene_bmgc,
                    "protein_llmname_ids": top_k_gene_llm
                },
                "top_k_transcript": {
                    "hgnc_symbols": top_k_ts_hgnc,
                    "protein_bmgc_ids": top_k_ts_bmgc,
                    "protein_llmname_ids": top_k_ts_llm
                },
                "top_k_protein": {
                    "hgnc_symbols": top_k_pt_hgnc,
                    "protein_bmgc_ids": top_k_pt_bmgc,
                    "protein_llmname_ids": top_k_pt_llm
                },
                "knowledge_graph": {
                    "disease_protein": {
                        "bmgc_ids": dis_pt_bmgc,
                        "hgnc_symbols": dis_pt_hgnc,
                        "indices": dis_pt_idx
                    },
                    "ppi_neighbors": {
                        "bmgc_ids": ppi_bmgc,
                        "hgnc_symbols": ppi_hgnc,
                        "indices": ppi_idx
                    },
                    "protein_relationships": relationship_texts,
                }
            }
        }

    except Exception as e:
        print(f"⚠️ Error processing {sample_ach_name}: {e}")
        continue

    # Periodic save every N samples
    if count % save_every_n == 0:
        with open(output_path, "w") as f:
            json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)
        print(f"💾 Auto-saved JSON at {count}/{total_to_process} samples to: {output_path}")
        print(f"Last processed sample: {sample_ach_name}")
        processed = len(multi_sample_qa_json)
        remaining = total_to_process - processed
        print(f"Progress: {processed}/{total_to_process} ({processed/total_to_process*100:.1f}%), Remaining: {remaining}")

# Final save after loop
with open(output_path, "w") as f:
    json.dump(multi_sample_qa_json, f, indent=2, default=lambda o: int(o) if hasattr(o, 'item') else o)

print(f"✅ Final JSON saved to: {output_path}")
print(f"Total samples processed: {len(multi_sample_qa_json)}")

Found 0 already processed samples
Total samples to process: 286, already processed: 0, remaining: 286


  0%|          | 0/286 [00:00<?, ?it/s]

Processing sample 1/286: ACH-000002 (HL-60)
Sample Index: 1
Sample Disease: acute myeloid leukemia
Sample Disease BMGC ID: BMGC_DS01407
Sample ACH Name: ACH-000002
Extracting top 10 gene information for ACH-000002...
Extracting top 10 transcript information for ACH-000002...
Extracting top 10 protein information for ACH-000002...
Extracting disease-related proteins index and bmgc id for BMGC_DS01407 (ACH-000002) ...
Knowledge Graph Info: Found 377 disease-related proteins directly connected to BMGC_DS01407 and 25307 proteins in their PPI network
Mapping disease-related proteins to HGNC symbols...
Mapping PPI-related proteins to HGNC symbols...
Extracting protein relationships from BMGC...


  0%|          | 1/286 [03:56<18:41:28, 236.10s/it]

Knowledge Graph Info: Found 224 unique proteins and 583 relationships between them
Processing sample 2/286: ACH-000006 (MONO-MAC-6)
Sample Index: 2
Sample Disease: leukemia, myeloid, acute
Sample Disease BMGC ID: BMGC_DS20800
Sample ACH Name: ACH-000006
Extracting top 10 gene information for ACH-000006...
Extracting top 10 transcript information for ACH-000006...
Extracting top 10 protein information for ACH-000006...
Extracting disease-related proteins index and bmgc id for BMGC_DS20800 (ACH-000006) ...
Knowledge Graph Info: Found 0 disease-related proteins directly connected to BMGC_DS20800 and 0 proteins in their PPI network
Mapping disease-related proteins to HGNC symbols...
Mapping PPI-related proteins to HGNC symbols...
Extracting protein relationships from BMGC...


...

100%|█████████▉| 285/286 [8:57:03<00:35, 35.35s/it]

Knowledge Graph Info: Found 10 unique proteins and 0 relationships between them
Processing sample 286/286: ACH-001233 (VMRC-LCD)
Sample Index: 286
Sample Disease: lung adenocarcinoma
Sample Disease BMGC ID: BMGC_DS02944
Sample ACH Name: ACH-001233
Extracting top 10 gene information for ACH-001233...
Extracting top 10 transcript information for ACH-001233...
Extracting top 10 protein information for ACH-001233...
Extracting disease-related proteins index and bmgc id for BMGC_DS02944 (ACH-001233) ...
Knowledge Graph Info: Found 532 disease-related proteins directly connected to BMGC_DS02944 and 27288 proteins in their PPI network
Mapping disease-related proteins to HGNC symbols...
Mapping PPI-related proteins to HGNC symbols...
Extracting protein relationships from BMGC...


100%|██████████| 286/286 [9:01:07<00:00, 113.52s/it]

Knowledge Graph Info: Found 300 unique proteins and 655 relationships between them


✅ Final JSON saved to: ./data/DrugQA\drug_qa_k10_bm100.json
Total samples processed: 286
